# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 1.1. 필요 라이브러리 설치 및 불러오기

In [1]:
# !wget https://raw.githubusercontent.com/BusyCod/4th_SBS/main/requirements.txt
# !pip install pip==20.1.1
# !pip install -r requirements.txt

In [2]:
# # !pip install pandas_profiling --upgrade
# !pip install tqdm
# !pip install xgboost==1.7.3
# !pip install lightgbm==3.3.5
# !pip install catboost==1.2.3
# !pip install tensorflow==2.13.0
# !pip install optuna==3.5.0
# !pip install pandas==1.5.3
# !pip install scikit-learn==1.2.2
# !pip install googletrans

In [94]:
import os
import re
import joblib  
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter
from googletrans import Translator


from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder

import optuna
from optuna.samplers import TPESampler

import tensorflow as tf
tf.random.set_seed(1015) 
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

### 1.2. 데이터 셋 읽어오기

In [4]:
df_train = pd.read_csv('./train.csv').drop_duplicates().reset_index(drop = True)        # 학습용 데이터
df_test = pd.read_csv('./submission.csv').drop_duplicates().reset_index(drop = True)        # 테스트 데이터(제출파일의 데이터)

df_train.head()     # 학습용 데이터 살펴보기

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True


### 1.3. 데이터 결측치 및 특성 확인

1.3.1. 결측치 확인

In [5]:
df_train.isnull().sum()     # 학습 데이터 확인

bant_submit                    0
customer_country             974
business_unit                  0
com_reg_ver_win_rate       41666
customer_idx                   0
customer_type              41358
enterprise                     0
historical_existing_cnt    43410
id_strategic_ver           52441
it_strategic_ver           54672
idit_strategic_ver         51333
customer_job               16804
lead_desc_length               0
inquiry_type                 891
product_category           17174
product_subcategory        46774
product_modelname          46749
customer_country.1           974
customer_position              0
response_corporate             0
expected_timeline          28292
ver_cus                        0
ver_pro                        0
ver_win_rate_x             37993
ver_win_ratio_per_bu       40896
business_area              37993
business_subarea           50600
lead_owner                     0
is_converted                   0
dtype: int64

In [6]:
df_test.isnull().sum()      # 테스트 데이터 확인

id                            0
bant_submit                   0
customer_country              0
business_unit                 0
com_reg_ver_win_rate       3483
customer_idx                  0
customer_type              1457
enterprise                    0
historical_existing_cnt    3996
id_strategic_ver           4678
it_strategic_ver           5218
idit_strategic_ver         4625
customer_job               1439
lead_desc_length              0
inquiry_type               1292
product_category           1858
product_subcategory        4478
product_modelname          4709
customer_country.1            0
customer_position             0
response_corporate            0
expected_timeline          2408
ver_cus                       0
ver_pro                       0
ver_win_rate_x             2898
ver_win_ratio_per_bu       3365
business_area              2898
business_subarea           3455
lead_owner                    0
is_converted                  0
dtype: int64

1.3.2. 데이터 특성 확인

In [7]:
df_train.info()     # 학습 데이터 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55780 entries, 0 to 55779
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              55780 non-null  float64
 1   customer_country         54806 non-null  object 
 2   business_unit            55780 non-null  object 
 3   com_reg_ver_win_rate     14114 non-null  float64
 4   customer_idx             55780 non-null  int64  
 5   customer_type            14422 non-null  object 
 6   enterprise               55780 non-null  object 
 7   historical_existing_cnt  12370 non-null  float64
 8   id_strategic_ver         3339 non-null   float64
 9   it_strategic_ver         1108 non-null   float64
 10  idit_strategic_ver       4447 non-null   float64
 11  customer_job             38976 non-null  object 
 12  lead_desc_length         55780 non-null  int64  
 13  inquiry_type             54889 non-null  object 
 14  product_category      

In [8]:
df_test.info()      # 테스트 데이터 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5271 entries, 0 to 5270
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       5271 non-null   int64  
 1   bant_submit              5271 non-null   float64
 2   customer_country         5271 non-null   object 
 3   business_unit            5271 non-null   object 
 4   com_reg_ver_win_rate     1788 non-null   float64
 5   customer_idx             5271 non-null   int64  
 6   customer_type            3814 non-null   object 
 7   enterprise               5271 non-null   object 
 8   historical_existing_cnt  1275 non-null   float64
 9   id_strategic_ver         593 non-null    float64
 10  it_strategic_ver         53 non-null     float64
 11  idit_strategic_ver       646 non-null    float64
 12  customer_job             3832 non-null   object 
 13  lead_desc_length         5271 non-null   int64  
 14  inquiry_type            

### 1.4. 데이터 특성 확인을 통한 열 분류

In [9]:
# 전처리 필요 O
cols_cat_need_pre = ['customer_country', 'customer_type', 'customer_job', 'product_modelname', 'expected_timeline',
            'inquiry_type', 'product_category', 'product_subcategory','customer_position', 'business_subarea']
cols_num_need_pre = ['com_reg_ver_win_rate', 'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver', 
            'ver_win_rate_x', 'ver_win_ratio_per_bu']

# 전처리 필요 X
cols_cat_no_pre = ['business_unit', 'customer_idx', 'enterprise', 'response_corporate', 'ver_cus', 'ver_pro']
cols_num_no_pre = ['lead_desc_length', 'bant_submit', 'lead_owner']

# customer_country.1 는 customer_country 과 중복되는 데이터 많아서 데이터 구성에서 제외

## 2. 데이터 전처리

### 2.1. 전처리를 위한 데이터 통합

In [10]:
# 전체 정보 합친 데이터프레임 생성
df_all = pd.concat([df_train.drop('is_converted', axis = 1), df_test.drop(['id', "is_converted"], axis = 1)], axis = 0).reset_index(drop = True)

# train / test 데이터 프레임 나누기 위한 인덱스 길이값 저장
idx_train = len(df_train)

# 전처리 완료한 데이터 넣을 데이터프레임 생성
df_train_prepare = pd.DataFrame()       # 학습용 데이터
df_test_prepare = pd.DataFrame()        # 테스트 데이터

### 2.2. 수치형( Numeric ) 변수 전처리

2.2.1. 수치형이나, 범주형인 변수

In [11]:
###################
### bant_submit ###
###################

# 라벨 생성 후 매핑
label_dic = {k : v for v, k in enumerate(df_all['bant_submit'].value_counts(sort = False).keys())} 
label_dic[0] = label_dic[0.25]  
df_all['bant_submit'] = df_all['bant_submit'].map(label_dic)

# 데이터 넣기
df_train_prepare['bant_submit'] = df_all['bant_submit'].values[: idx_train]     # 학습용 데이터
df_test_prepare['bant_submit'] = df_all['bant_submit'].values[idx_train : ]     # 테스트 데이터


##################
### lead_owner ###
##################

# 데이터 넣기
df_train_prepare['lead_owner'] = df_all["lead_owner"].values[:idx_train]        # 학습용 데이터
df_test_prepare['lead_owner'] = df_all["lead_owner"].values[idx_train:]         # 테스트 데이터

2.2.2. 이상치 제거(IQR) and 스케일러 적용(SDS)하는 변수 ('lead_desc_length')

In [12]:
########################
### lead_desc_length ###
########################

# 데이터 분포 변환을 위해 로그 변환 실시
df_all['lead_desc_length'] = np.log(df_all['lead_desc_length'])

# 스케일러 적용하여 데이터 넣기
scaler = StandardScaler()
df_train_prepare['lead_desc_length']  = scaler.fit_transform(df_all[['lead_desc_length']].iloc[: idx_train])        # 학습용 데이터
df_test_prepare['lead_desc_length']  = scaler.transform(df_all[['lead_desc_length']].iloc[idx_train : ])            # 테스트 데이터


2.2.3. 결측치 0으로 만들고 범주형(binary)인 변수 ('id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver')

In [13]:
########################
### id_strategic_ver ###
########################

# 결측치 0으로 채우기
df_train_prepare['id_strategic_ver'] = df_train['id_strategic_ver'].fillna(0).astype(int)       # 학습용 데이터
df_test_prepare['id_strategic_ver'] = df_test['id_strategic_ver'].fillna(0).astype(int)         # 테스트 데이터


########################
### it_strategic_ver ###
########################

# 결측치 0으로 채우기
df_train_prepare['it_strategic_ver'] = df_train['it_strategic_ver'].fillna(0).astype(int)       # 학습용 데이터
df_test_prepare['it_strategic_ver'] = df_test['it_strategic_ver'].fillna(0).astype(int)         # 테스트 데이터


##########################
### idit_strategic_ver ###
##########################

# 결측치 0으로 채우기
df_train_prepare['idit_strategic_ver'] = df_train['idit_strategic_ver'].fillna(0).astype(int)       # 학습용 데이터
df_test_prepare['idit_strategic_ver'] = df_test['idit_strategic_ver'].fillna(0).astype(int)         # 테스트 데이터

2.2.4. 결측치 0으로 만들고, 이상치 제거(IQR)한 후, 스케일러(SDS)적용하는 변수 ('ver_win_rate_x', 'ver_win_ratio_per_bu')

In [14]:
######################
### ver_win_rate_x ###
######################

# 결측치 0으로 채우기
df_all['ver_win_rate_x'] = df_all['ver_win_rate_x'].fillna(0)

# 데이터 분포 변환을 위해 로그 변환 실시
df_all['ver_win_rate_x'] = np.log(df_all['ver_win_rate_x'] + 1e10)

# 스케일러 적용하여 데이터 넣기
scaler = StandardScaler()
df_train_prepare['ver_win_rate_x']  = scaler.fit_transform(df_all[['ver_win_rate_x']].iloc[: idx_train])        # 학습용 데이터
df_test_prepare['ver_win_rate_x']  = scaler.transform(df_all[['ver_win_rate_x']].iloc[idx_train : ])            # 테스트 데이터


############################
### ver_win_ratio_per_bu ###
############################

# 결측치 0으로 채우기
df_all['ver_win_ratio_per_bu'] = df_all['ver_win_ratio_per_bu'].fillna(0)

# 스케일러 적용하여 데이터 넣기
scaler = StandardScaler()
df_train_prepare['ver_win_ratio_per_bu']  = scaler.fit_transform(df_all[['ver_win_ratio_per_bu']].iloc[: idx_train])        # 학습용 데이터
df_test_prepare['ver_win_ratio_per_bu']  = scaler.transform(df_all[['ver_win_ratio_per_bu']].iloc[idx_train : ])            # 테스트 데이터


2.2.5. 결측치 많은 변수 ('com_reg_ver_win_rate', 'historical_existing_cnt')

In [15]:
############################
### com_reg_ver_win_rate ###
############################

# 결측치 0으로 채워넣기
df_all['com_reg_ver_win_rate'] = df_all['com_reg_ver_win_rate'].fillna(0)

# 스케일러 적용하기
scaler = StandardScaler()
df_train_prepare['com_reg_ver_win_rate']  = scaler.fit_transform(df_all[['com_reg_ver_win_rate']].iloc[: idx_train])        # 학습용 데이터
df_test_prepare['com_reg_ver_win_rate']  = scaler.transform(df_all[['com_reg_ver_win_rate']].iloc[idx_train : ])            # 테스트 데이터


###############################
### historical_existing_cnt ###
###############################

# IQR을 이용한 이상치 처리 ( 값을 << Q1 - iqr * 1.5 ~ Q1 + iqr * 1.5 >> 로 만들기 )
q1, q3 = np.quantile(df_all['historical_existing_cnt'], .25), np.quantile(df_all['historical_existing_cnt'], .75)     # 1, 3사분위 수 구하기
iqr = q3 - q1       # IQR 구하기
ood_tail, ood_top = q1 - 1.5 * iqr, q3 + 1.5 * iqr      # 이상치 값의 threhold 구하기
df_all['historical_existing_cnt'] = df_all['historical_existing_cnt'].map(lambda x : ood_tail if x < ood_tail else ood_top if x > ood_top else x)      # 데이터에 매핑

# 결측치 0으로 채워넣기
df_all['historical_existing_cnt'] = df_all['historical_existing_cnt'].fillna(0)

# 스케일러 적용하기
scaler = StandardScaler()
df_train_prepare['historical_existing_cnt']  = scaler.fit_transform(df_all[['historical_existing_cnt']].iloc[: idx_train])      # 학습용 데이터
df_test_prepare['historical_existing_cnt']  = scaler.transform(df_all[['historical_existing_cnt']].iloc[idx_train : ])          # 테스트 데이터

### 2.3. 문자형( String ) 변수 처리

2.3.1. 결측치 없으며 분류( 라벨링 )만 필요한 변수

In [16]:
#####################
### business_unit ###
#####################

# 라벨링 진행 및 데이터 넣기
le = LabelEncoder()
df_all['business_unit'] = le.fit_transform(df_all['business_unit'])
df_train_prepare['business_unit'] = df_all['business_unit'].values[ : idx_train]        # 학습용 데이터
df_test_prepare['business_unit'] = df_all['business_unit'].values[idx_train :]          # 테스트 데이터


##################
### enterprise ###
##################

# 이진분류 잘 되어있으므로 라벨링 진행 및 데이터 넣기
le = LabelEncoder()
df_all['enterprise'] = le.fit_transform(df_all['enterprise'])
df_train_prepare['enterprise'] = df_all['enterprise'].values[ : idx_train]      # 학습용 데이터
df_test_prepare['enterprise'] = df_all['enterprise'].values[idx_train :]        # 테스트 데이터


###############
### ver_cus ###
###############

# 이진분류 잘 되어있으므로 라벨링 진행 및 데이터 넣기
le = LabelEncoder()
df_all['ver_cus'] = le.fit_transform(df_all['ver_cus'])
df_train_prepare['ver_cus'] = df_all['ver_cus'].values[ : idx_train]        # 학습용 데이터
df_test_prepare['ver_cus'] = df_all['ver_cus'].values[idx_train :]          # 테스트 데이터


###############
### ver_pro ###
###############

# 이진분류 잘 되어있으므로 라벨링 진행 및 데이터 넣기
le = LabelEncoder()
df_all['ver_pro'] = le.fit_transform(df_all['ver_pro'])
df_train_prepare['ver_pro'] = df_all['ver_pro'].values[ : idx_train]        # 학습용 데이터
df_test_prepare['ver_pro'] = df_all['ver_pro'].values[idx_train :]          # 테스트 데이터


2.3.2. 결측치는 없으나 카테고리 수의 조정이 필요한 변수

In [17]:
##########################
### response_corporate ###
##########################

# 빈도수 50회 이상인 문자들만 남기고 나머지를 'Ohters'로 변환
df_all['response_corporate'] = df_all['response_corporate'].map(lambda x : x[3:])       # 뒤에 2글자만 가져오기 ( 국가코드 )
upper_50 = [val[0] for val, cnt in df_all[['response_corporate']].value_counts().items() if cnt >= 50]      # 국가별 count 후, 50개 이상인 것만 가져오기
df_all['response_corporate'] = df_all['response_corporate'].map(lambda x : x if x in upper_50 else 'Ohters')

# 라벨링 진행 및 데이터 넣기
le = LabelEncoder()
df_all['response_corporate'] = le.fit_transform(df_all['response_corporate'])
df_train_prepare['response_corporate'] = df_all['response_corporate'].values[ : idx_train]      # 학습용 데이터
df_test_prepare['response_corporate'] = df_all['response_corporate'].values[idx_train :]        # 테스트 데이터


####################
### customer_idx ###
####################

# 빈도수 5회 초과 문자들만 남기고 나머지를 'others'로 변환
tem_li = [x for x, y in df_all['customer_idx'].value_counts().items() if y > 5]
df_all['customer_idx'] = df_all['customer_idx'].map(lambda x : str(x) if x in tem_li else 'others')

# 라벨링 진행 및 데이터 넣기
le = LabelEncoder()
df_all['customer_idx'] = le.fit_transform(df_all['customer_idx'])
df_train_prepare['customer_idx'] = df_all['customer_idx'].values[ : idx_train]          # 학습용 데이터
df_test_prepare['customer_idx'] = df_all['customer_idx'].values[idx_train :]            # 테스트 데이터

2.3.4. 결측치가 존재하며 동의어 처리 및 카테고리 수 조정이 필요한 변수

In [18]:
#########################
### product_modelname ###
#########################

# product_modelname :null값 유무 정도 구분하는 것이 좋을 듯 함 (0 or 1)
df_all["product_modelname"] = ~df_all["product_modelname"].isnull()
df_all["product_modelname"] = df_all["product_modelname"].astype(int)

# 라벨링 진행 및 데이터 넣기
le = LabelEncoder()
df_all["product_modelname"] = le.fit_transform(df_all["product_modelname"])
df_train_prepare["product_modelname"] = df_all["product_modelname"].values[ : idx_train]        # 학습용 데이터
df_test_prepare["product_modelname"] = df_all["product_modelname"].values[idx_train :]          # 테스트 데이터

2.3.4. 결측치가 존재하며 동의어 처리 및 카테고리 수 조정이 필요한 변수

In [19]:
# ## 영어 이외의 자연어 번역을 위한 코드

# # 번역 객체 선언
# translator = Translator()

# # 번역 위한 변수 선언
# error_li, trans_dic, max_len = [], {}, 5000

# # 번역 필요 열 지정하여 번역 실행
# needs_trans_col = ['inquiry_type', 'customer_job', 'customer_country']
# for c in needs_trans_col :

#     # 전처리 진행
#     df_all[c] = df_all[c].map(lambda x : x.lower().replace("-", " ").replace("_", " ").strip() if type(x) == str else x)

#     # 만약 문자열에 '/' 존재한다면 분할 실행하는 전처리
#     data_li = []
#     for d in set(df_all[c].dropna()) :
#         if "/" in d : [data_li.append(s.strip()) for s in d.strip().split("/") if s]
#         else : data_li.append(d.lower().strip())

#     # 임시 저장할 단어 사전 생성하여 번역 실행
#     trans_dic_tem, cnt = {}, 1
#     for d in tqdm(set([x for x in data_li if x]), desc = f'{(needs_trans_col.index(c) + 1)}/{len(needs_trans_col)} {c}') :

#         # 글자 선언
#         str_tem = d.strip().lower()
#         trans_str = None
        
#         try :   # CASE 1 : 제대로 번역이 되는 경우
#             trans_str = translator.translate(str_tem).text.lower()
#             if cnt % 100 == 0 : time.sleep(int(np.random.rand() * 20)) 
            
#         except : # CASE 2 : 제대로 번역이 되지 않은 경우 -> 따로 저장하여 한번 더 실행
#             time.sleep(30)
#             try : trans_str = translator.translate(str_tem).text.lower()
#             except : error_li.append([c, str_tem])

#         cnt += 1

#         # 변화가 존재하는 경우만 사전에 넣기
#         if trans_str and str_tem != trans_str : trans_dic_tem[str_tem] = trans_str     

#     # 열 최종 결과물을 사전에 넣기
#     trans_dic[c] = trans_dic_tem

# # 오류난 경우 추가 번역 진행
# for c, str_tem in error_li :
#     try :   # CASE 1 : 제대로 번역이 되는 경우
#         trans_dic[c][str_tem] = translator.translate(str_tem).text.lower()
#         if cnt % 100 == 0 : time.sleep(int(np.random.rand() * 20)) 
            
#     except : # CASE 2 : 제대로 번역이 되지 않은 경우 -> 따로 저장하여 한번 더 실행
#         time.sleep(30)
#         trans_dic[c][str_tem] = translator.translate(str_tem).text.lower()

In [20]:
# 위의 코드로 얻은 단어 사전 데이터
trans_dic = {}

trans_dic['inquiry_type'] = {
    "denduluru" : "delluder",
    "waukegan" : "waoi cancer",
    "cotonou" : "coton",
    "pundri" : "pundari",
    "bresso" : "brex",
    "armação dos búzios" : "buzios frame",
    "ramgarh cantt" : "ramgarh santat",
    "acharnai" : "relevant",
    "criciuma" : "cricium",
    "junagadh gujarat" : "junagarh gujarat",
    "ấp hòa hữu  xã long đức tptv" : "hoa huu hamlet, long duc commune, tptv",
    "valle de bravo" : "bravo valley",
    "กรุงเทพฯ" : "bangkok",
    "rancho queimado" : "burnt ranch",
    "abha" : "river",
    "villahermosa, tabasco" : "villahermosa tabasco",
    "presidente getulio sc" : "president getulio sc",
    "1 kv2 p.trà nóc" : "1 kv2 p.trà noc",
    "paço do lumiar" : "lumiar paço",
    "aracaju   se" : "aracaju if",
    "morando em são paulo" : "living in são paulo",
    "ayankig" : "youyankig",
    "hatimiyyah street   muwaqqar p. o. box 141351" : "hatimiyyh street muwaqar p.o.box 141351",
    "merut" : "meerut",
    "góra kalwaria" : "mount kalwaria",
    "long hựu tây cần đước long an" : "long huu tay can long an",
    "tuchorza" : "tuchorz",
    "mumvai" : "mumwai",
    "nalagonda" : "disagree",
    "53 e 1st ave" : "೫೩ this ೧ st",
    "maringa pr" : "on mringa",
    "fortaleza ce" : "fortaleza what",
    "2 sengol nagar 3 rd street old vilangudi" : "௨ scepter city ௩ street old",
    "jonadi" : "jondi",
    "châu đôc" : "chau dac",
    "القطيف" : "qatif",
    "quảng ngãi" : "quang ngai",
    "campos rj" : "rj fields",
    "antique philippines" : "ancient philippines",
    "ekangersarai" : "ecadesrai",
    "tp hồ chí minh" : "ho chi minh city",
    "ribeirão das neves" : "stream of snow",
    "raposos" : "fox",
    "ma   timon" : "my drawbar",
    "ottawa ontario" : "i followed ontario",
    "cd de mexico" : "mexico cd",
    "coronel fabriciano mg" : "colonel fabriciano mg",
    "berhampore" : "in berhampur",
    "cossato" : "sock",
    "gaziabadh" : "ghaziabad",
    "dias d'avila ba" : "days d'avila ba",
    "durban , kwa zulu natal" : "durban, to zulu natal",
    "elkhart" : "garden",
    "maumbai" : "mumbai",
    "morbi" : "soccer",
    "via di s. erasmo 12" : "via di s.erasmus 12",
    "novia" : "bride",
    "6252 e grant rd suite 150 tucson, az 85712" : "6252 is a grant rd swing 150 tutzon, i 85712",
    "hồ chí minh" : "ho chi minh",
    "hata" : "even",
    "1602 e yandell dr" : "1602 e yandell dr dr.",
    "arroyo hondo viejo" : "hondo viejo arroyo",
    "juliaca   san roman   puno" : "juliaca san roman puno",
    "pondichéry" : "pondicherry",
    "itapema sc" : "it will spit sc",
    "phường tân phong biên hoà đồng nai" : "tan phong bien hoa ward dong nai",
    "885 haleakala hwy" : "889 videos",
    "kishtwar" : "tendering",
    "las parejas" : "the couples",
    "platón sánchez" : "plato sánchez",
    "palhoça" : "hazel",
    "la misma" : "the same",
    "seria" : "he would be",
    "સુરત" : "surat",
    "nuxis (su)   via dante n. 34" : "nuxis (your) dante n.3. 4",
    "lampa" : "lamp",
    "baquba" : "the bacquuba",
    "bacolod city nergros occidental" : "bacolod city negros occidental",
    "السويس" : "suez",
    "tambopata" : "tamopata",
    "290 thành công, q. tân phú" : "290 success, q.tan phu",
    "vijayapura" : "vijaypura",
    "cordba" : "cordoba",
    "santo andre   sao paulo" : "santo andre sao paulo",
    "buenos aires   azul" : "buenos aires blue",
    "1 penn plz ste 1400 fl 14" : "1 pen plz rose 1400 fl 14",
    "al riyad" : "riyadh",
    "ecatepec , edo mex" : "ecatepec, edo mex",
    "hapur" : "open",
    "guarne" : "cheek",
    "mntpellier" : "montpellier",
    "una" : "a",
    "partappur" : "pratappur",
    "colegiales" : "schoolboys",
    "koribori 8g" : "corbari 1 c",
    "brasilia   águas claras" : "brasilia águas claras",
    "arain rajasthan" : "rajasthan",
    "1850 m. h. del pilar st,  malate" : "1850 m.h.of the pillar st, malate",
    "tapira mg" : "tap mg",
    "santa ines" : "holy ines",
    "palmas   pr" : "palmas pr",
    "hindamata" : "unstated",
    "gyál" : "gate",
    "số 7 đường 3419 phường 7 quận 8" : "no. 7 street 3419 ward 7 district 8",
    "jacareí" : "jacarei",
    "mazowsze" : "mazovia",
    "1605 ave. ponce de león, suite 400 san juan, 00909, puerto rico" : "1605 ave.ponce de león, suite 400 san juan, 00909, puerto rico",
    "purnia" : "purania",
    "pato branco pr" : "ties on the bronze",
    "นนทบุรี" : "nonthaburi",
    "capivari de baixo" : "downstream",
    "auragabad" : "aura",
    "حفر الباطن" : "hafar al -batin",
    "15 phan huy thực, tân kiểng, q7" : "15 phan huy thuc, tan kieng, district 7",
    "cali   valle del cuaca" : "cali valle del cauca",
    "koura" : "gold",
    "الشرقية" : "eastern",
    "gubin" : "burn",
    "huyện chợ gạo  tỉnh tiền giang" : "cho gao district, tien giang province",
    "jatara" : "jatra",
    "sheoganj" : "sheyganj",
    "carrera 66b # 6   26 805006050" : "carrera 66b # 6 26 805006050",
    "almirqab" : "alminqab",
    "ktx khu b dhqg tp.hcm" : "dormitory of building b, ho chi minh city",
    "hydrabad" : "hyderabad",
    "rajkot    gujarat" : "rajkot gujarat",
    "são paulo  sp" : "sao paulo-sp",
    "phước long b" : "phuoc long b",
    "chile santiago" : "santiago chile",
    "quận 2, tp. hcm" : "district 2, tp.hcm",
    "dhrol" : "drol",
    "perintalmanna" : "pond",
    "sumatera selatan" : "south sumatra",
    "bulandshahar" : "bulandshahr",
    "goiania   go" : "goiania go",
    "tân an  ,càng long, trà vinh" : "tan an, lau long, tra vinh",
    "aurngabad" : "aurangabad",
    "sete lagoas   mg" : "sete lagoas mg",
    "cra 29 a # 5 sur 125 valle" : "cra 29 a # 5 of 125 valle",
    "wakas nort" : "end nort",
    "浙江省杭州市" : "hangzhou, zhejiang",
    "mumbay" : "mumbai",
    "נתיבות" : "netivot",
    "al ayat   giza" : "the giza",
    "garhshankar" : "gadhashankar",
    "hoà bình trà ôn vĩnh long" : "hoa binh tra wen vinh long",
    "dhamnod dhar m.p" : "dhamnod dhar mp",
    "yirka" : "the umbri",
    "formello" : "fork",
    "uruli kanchan" : "urali kanchan",
    "台南市" : "tainan city",
    "uran" : "career",
    "14058 mirabella dr  education" : "14058 mirabella dr education",
    "muriae" : "muria",
    "mundra kutch" : "mundra inch",
    "talafar" : "fiction",
    "ciudad del este" : "eastern city",
    "qalubia" : "heartfelt",
    "registro sp" : "sp registration",
    "funza" : "learn",
    "venturosa pe" : "venturous pe",
    "bremerhavcen" : "bremerhaven",
    "são paulo   sp" : "sao paulo-sp",
    "dumaguete city, negrosd oriental" : "dumaguete city, negros oriental",
    "18 nguyễn văn lượng, p6 gò vấp" : "18 nguyen van luong, p6 go vap",
    "kajang" : "brim",
    "sheridan" : "lion",
    "senador canedo" : "senator canedo",
    "kabupaten sanggau, kalimantan barat" : "sanggau regency, west kalimantan",
    "estuar" : "estuary",
    "migo das cruzes" : "mogi das cruzes",
    "thiruvanthapuram" : "thiruvantapuram",
    "chikmagalore" : "the chikamagore",
    "ciudad autonoma de buenos aires" : "autonomous city of buenos aires",
    "porto nacional" : "national port",
    "coonoor tamilnadu" : "coonor tamilnadu",
    "via fontana rutola,22" : "via fontana rutola, 22",
    "al ayat" : "a",
    "ibira" : "pull",
    "avare" : "stingy",
    "canoinhas" : "canoiinhas",
    "الديوانية" : "diwaniyah",
    "pavoorchatram" : "pavoric chatram",
    "nova mutum" : "noa man",
    "manab ray" : "manab re",
    "joelformcheck magnit" : "joelformcheck magnet",
    "lucko" : "lutsk",
    "via spiaggia,328" : "via beach, 328",
    "papanduva" : "papoda",
    "hòa thuận châu thành tỉnh trà vinh" : "harmony of tra vinh province",
    "tasocagi cad. no:64" : "tasocagi cad.no: 64",
    "piazzale s. giuseppe, 10" : "square s.giuseppe, 10",
    "osório" : "osorium",
    "مدينة الجلود بدر" : "the city of leather badr",
    "yahoud  monoson" : "monson jews",
    "thạch bàn long biên hà nội" : "thach ban long bien hanoi",
    "namkhana" : "name",
    "espírito santo do pinhal" : "holy spirit of pinhal",
    "pilkhuwa, distt hapur, west up" : "pilkhuwa, distt hapur, west sub",
    "bhayandar" : "bayander",
    "santo antônio do tauá   pará" : "saint anthony of tauá pará",
    "paranaque  city" : "paranaque city",
    "viale  odino  4" : "viale odin 4",
    "kothara abdasa 370645" : "the statement of the word 1",
    "peixe   tocantins" : "tocantins fish",
    "lan o lakes" : "lan the lakis",
    "la paz" : "peace",
    "navegantes   sc" : "sc navigators",
    "sp são paulo" : "sp sao paulo",
    "jakarta utara, kelapa gading" : "north jakarta, kelapa gading",
    "makka" : "mecca",
    "nova scotia" : "new scotia",
    "amargosa valley" : "bitter",
    "gzira" : "young",
    "partapur" : "paratapur",
    "cascina" : "farmhouse",
    "venda nova do imigrante" : "new sale do immigrant",
    "toronto on" : "toronto is",
    "rio de janeiro e região" : "rio de janeiro and region",
    "terra nova do norte mt" : "northern terr mt",
    "miraj" : "mirage",
    "de leon" : "by leon",
    "brześce" : "brzesce",
    "los andes" : "the andes",
    "saúde ba" : "ba health",
    "3125 avenida topanga" : "to topan 3125 avenue",
    "广东省深圳市" : "shenzhen city, guangdong province",
    "dois vizinhos. pr" : "two neighbors.pr",
    "etah" : "fsl",
    "burla parit" : "burla couples",
    "112 sol building amorsolo st. legaspi village, makati city manila, philippines" : "112 sol building amorsolo st.legaspi village, makati city manila, philippines",
    "carrera 7a # 69 99 d.c." : "career 7a # 69 99 a.c.",
    "la pampa   santa rosa" : "la pampa santa rosa",
    "rio grande" : "big river",
    "niwali" : "preached",
    "al ques" : "at any",
    "guarujá são paulo" : "guarujá sao paulo",
    "siek" : "sick",
    "castello d argile" : "castle of swimming pool",
    "odrano wola" : "wola was rebuke",
    "3300 e.guasti rd  medical" : "3300 e.guasti rd medical",
    "viale della navigazione interna, 48" : "viale della internal navigation, 48",
    "nanaimo" : "seven places",
    "karur" : "some",
    "building 18a 3rd nangang industrial park tangtou shiyantown baoan district  commercial display" : "building 18a 3rd nangang industrial park tangtou shiyan town baoan district commercial display",
    "são paulo, pinheiros" : "sao paulo, pinheiros",
    "matinhos" : "pound",
    "594 howard stsan franciscoca94106" : "594 howard st san francisco ca 94106",
    "bishrampur" : "restor",
    "مكة" : "mecca",
    "la spezia" : "spice",
    "bol" : "he was",
    "maarab" : "arab",
    "kuchina" : "to china",
    "גבעת זאב" : "ze'ev hill",
    "londrina pr" : "at laundrina",
    "terdal" : "consistently",
    "alvorada" : "dawn",
    "baripada" : "range",
    "icó" : "icon",
    "mavelikara" : "mavelikkara",
    "santa catarina" : "st. catarina",
    "sombrio" : "dingy",
    "barueri   são paulo" : "barueri sao paulo",
    "bhubanewar" : "bhubnewar",
    "ghazizbad" : "scope",
    "birghman or london" : "birmingham or london",
    "bergkamen" : "mountain ranges",
    "cao lãnh" : "cao lanh",
    "nishi shinjuku, shinjuku, tokyo" : "nishi -shinjuku, shinjuku, and",
    "bhucho mandi" : "bhucho took a shower",
    "rodeville" : "roseville",
    "1000 primera blvd" : "1000 first blvd",
    "cantton, ohio" : "canton, ohio",
    "153 210, illuppapattu village rajakulam post , kanchipuram" : "೨೧೦ ೨೧೦, illuppapattu village rajakulam post, kanchipuram",
    "otorowo" : "you are aological",
    "pr londrina" : "laundrina on",
    "khorda" : "chord",
    "purifiacacion tolima" : "tolima purification",
    "i̇stanbul (anadolu)" : "istanbul (anatolia)",
    "avenida hostos 410 ave hostos medical" : "hostos avenue 410 ave hostos medical",
    "kasur punjab" : "punjab kasur",
    "الغردقة" : "hurghada",
    "baliguda ,district.. kandhamala, orisa" : "baliguda, district .. kandhamal, orissa",
    "krishnagar" : "black",
    "major izidoro   alagoas" : "major izidoro alagoas",
    "dolnosląskie" : "lower silesian",
    "gevrai" : "gavarai",
    "thủ đức" : "thu duc",
    "vellore,tamil nadu" : "vellore, tamil nadu",
    "amour( baisi)" : "love (baisi)",
    "józefów k. otwocka" : "józefów near otwock",
    "ujazd" : "utille",
    "las pinas city" : "the city pinas",
    "es cariacica" : "cariacica",
    "attiki" : "attica",
    "brasília df" : "brasilia df",
    "9 âu cơ tân bình" : "9 au co tan binh",
    "maceió   alagoas" : "maceio alagoas",
    "القصيم بريدة" : "qassim buraidah",
    "حائل" : "barrier",
    "alipurduar" : "alipurdar",
    "tuxtla gutiérrez" : "tuxtla gutierrez",
    "campos dos goytacazes" : "goytacazes fields",
    "singapoer" : "singapore",
    "calle del trigal 255, quilicura" : "255 trigal street, quilicura",
    "5900 balcones ste 5170" : "5900 ste 5170 balconies",
    "2101 nuuanu ave" : "210170 aflost airline around",
    "senador canedo go" : "senator canedo go",
    "betivoglio" : "bentivoglio",
    "lunavada" : "ransom",
    "gangtok" : "gangts",
    "badaun" : "adults",
    "passos mg" : "mg steps",
    "metropolitana buin" : "metropolitan buin",
    "jakarta pusat" : "central jakarta",
    "lima peru" : "lima's name",
    "ahmedbad" : "ahmed",
    "bijainagar" : "bijayanagar",
    "2414 ถนนเพชรบุรึตัดใหม่" : "2414 new phetchaburi road",
    "dghgfdh" : "dghfh",
    "พระนครศรีอยุธยา" : "ayutthaya",
    "los alamitos" : "the alamitos",
    "são bento do sul" : "sao bento do sul",
    "cra 20 f # 61 b   19 sur" : "cro 20 f # 61 b 19 sur",
    "governador celso ramos" : "governor celso ramos",
    "świętochłowice (woj. śląskie)" : "świętochłowice (śląskie voivodeship)",
    "avcılar" : "hunters",
    "minterrey" : "monterrey",
    "macapá   amapá" : "macapá amapá",
    "sc  itapema" : "sc will shoot",
    "ca' zusto 52" : "ca 'zusto 52",
    "triunfo" : "triumph",
    "airoli" : "aeroli",
    "θέση πέτσα βακαλοπούλου βιοπα παλλήνης  153 51" : "peta peta vakalopoulou biopa pallini 153 51",
    "costa del este" : "east coast",
    "cty tnhh kỹ thuật hts" : "hts technical company limited",
    "santo andre" : "santo other",
    "passa e fica" : "passes and stays",
    "bari, italy" : "barry, italian",
    "ponte nova" : "new bridge",
    "cedula de ciudadania" : "citizenship card",
    "la liberfad" : "freedom",
    "kondotty" : "conduit",
    "mato grosso   sorriso" : "mato grosso smile",
    "ras al khimah" : "ras al khaimah",
    "privato" : "private",
    "prata" : "talk",
    "salto guaira" : "jump guaira",
    "ереван" : "yerevan",
    "nuevo león" : "new lion",
    "jammuand kashmir" : "jammu and kashmir",
    "cali   valle" : "valley",
    "passo fundo   rs" : "passo fundo rs",
    "itaira" : "consuming",
    "21 lạc long quânp8 qtân bình" : "21 lac long quanp8 qtan binh",
    "la grange" : "barn",
    "kannur keray" : "kannur",
    "go luziania" : "him",
    "npida" : "distributing",
    "marinha grande" : "large navy",
    "gramado" : "lawn",
    "campinas   sp" : "campinas, sp",
    "udaipur g 2. ankur complex opp narayan seva sansthan sex.5" : "udaipur c 2. ankur complex o narayan seva sansthan sex. 5",
    "telengana" : "telenganana",
    "930 fairway dr ne" : "930 fairway",
    "panama oeste" : "west panama",
    "suchy las" : "dry forest",
    "bécsi út 3 5" : "vienna road 3 5",
    "277 ave coll y toste" : "277 ave coll and toste",
    "caruaru pe" : "caruaru on",
    "osie" : "axles",
    "orai" : "weather",
    "tupanatinga" : "tupaniatation",
    "kudan kulam" : "kudon pond",
    "cambé   pr" : "camps pr",
    "assandh" : "blessed",
    "cra. 51 #12 sur 75, san fernando, itagüi, medellín, guayabal, medellín, antioquia, colombia" : "cra.51 #12 sur 75, san fernando, itagüi, medellin, guayabal, medellin, antioquia, colombia, colombia, colombia, colombia",
    "gustavo a. madero" : "gustavo a.log",
    "mas grenier" : "more grenier",
    "القنفذة" : "cumulus",
    "chicoloapan de juárez" : "chicololoapan by juarez",
    "melmaruvathur" : "malamaruvathur",
    "khaldeh" : "khalida",
    "850 lincoln ave, suite 11 bohemia ny 11716" : "850 lincoln ave, suite 11 bohemia no. 11716",
    "ain khaled" : "where is khalid",
    "p10 q6 tphcm" : "p10 q6 hcmc",
    "balneário camburiu" : "balneário camboriú",
    "sremska 4a" : "srems' cha",
    "san ramon" : "3 ramon",
    "gusyaquil" : "guayaquil",
    "newton" : "newtonto",
    "ahemdnagar" : "ahmednagar",
    "ajah   lagos" : "ajah lagos",
    "ara" : "we buy",
    "portblair" : "port blair",
    "brusque  sc" : "brusque sc",
    "dedham" : "dadham",
    "leandro n alen 734" : "leandro n alem 734",
    "delhi cant" : "delhi cantt",
    "al nadha" : "the club",
    "rajapalayam" : "rajpalayam",
    "santa ctuz de la sierra" : "santa cruz de la sierra",
    "schaafheim" : "collapheim",
    "são josé dos campos  sp" : "são josé dos campos sp",
    "naxalbari" : "nahalari",
    "charummoodu" : "chaummood",
    "via cav. vitt. veneto" : "via cav.vit.veneto",
    "28 michael adekoya street ilupeju" : "28 michael adekaya street start",
    "gladwyne" : "smooth wines",
    "glenndale" : "glendale",
    "rua rui barbosa n 1095 bairro goiabeiras mato grosso" : "rui barbosa street n 1095 bairro goiabeiras mato grosso",
    "kozhikode" : "leathercode",
    "vadoara" : "vadodara",
    "mimbai" : "branded",
    "dum dum" : "stupid",
    "2520 marsha sharp fwy  apt481" : "2520 marsha sharp more apt481",
    "sorriso   mato grosso" : "smile mato grosso",
    "p1 , tp.cao lãnh , đồng tháp" : "p1, cao lanh city, dong thap",
    "rampurhat   731224" : "rampurhat 1222",
    "กรุงเทพมหานคร" : "bangkok",
    "calle 51#8g 55" : "51#8g 55 street",
    "theni" : "why",
    "nova scctia" : "nova scotia",
    "turmalina" : "tourmaline",
    "nikol,ahmedabad  gujarat" : "nicole, ahmed gogarat",
    "nagercoil" : "nagercoil temple",
    "2124 s michigan" : "2124 s road cancer",
    "sorriso  mt" : "smile mt",
    "la unión" : "the union",
    "klein windhoek klein windhoek" : "small wind corner small wind corner",
    "mandi adampur" : "bathing adix",
    "montréal, qc" : "montreal, qc",
    "via rastrello 22" : "via rastal 22",
    "são joão de meriti" : "st. johns wood",
    "tilhar" : "strain",
    "davao coty" : "gave coty",
    "po tankua ,dist koraput" : "by tankua, dist",
    "688 hưương lộ 2" : "688 huang lo 2",
    "joao pessoa" : "joao person",
    "giza, 6th october" : "giza, october 5th",
    "سلطنة عمان" : "sultanate of oman",
    "simão dias  se" : "simão dias se",
    "bebedouro" : "drinking fountain",
    "las palmas, grand canaria" : "las palmas, gran canaria",
    "al khobar   saudi arabia" : "a saudi arab news",
    "19  giang võ văn kiệt" : "19 giang vo van kiet",
    "assuncion" : "asuncion",
    "ciudad de buenos aires" : "buenos aires city",
    "ave. ricardo alfaro, ph plaza aventura piso 4 ofic 423 panam?" : "bird.ricardo alfaro, ph plaza aventura 4th office 423 panam?",
    "hải dương" : "hai duong",
    "renukoot" : "renucotus",
    "vridhachalam" : "old age",
    "korba" : "to age",
    "león guanajuato" : "guanajuato león",
    "west bandung" : "west ribbon",
    "santa maria   rs" : "santa maria rs",
    "hajipur ,vaishali, bihar" : "hajipur, vaishali, bihar",
    "taigore nagar near veer loric stadium" : "tagore nagar near veer loric stadium",
    "nrw, deutschland" : "nrw, germany",
    "cachoeira paulista" : "paulista waterfall",
    "1000 navaho drive  hotel" : "1000 navaho drive hotel",
    "kout" : "short",
    "490 l'enfant plz, sw" : "490 the child plz, sw",
    "são luís   ma" : "são luís ma",
    "puttur" : "putter",
    "łódzkie" : "łódź",
    "ile ife" : "the love of love",
    "são josé do cedro  sc" : "são josé do cedro sc",
    "bengalooru" : "bangalore",
    "jakarta selatan" : "south jakarta",
    "ابها" : "abha",
    "dheradun" : "dheeradun",
    "channai" : "milk production",
    "rio claro" : "clear river",
    "via g. di vittorio 5 a" : "via g.of vittorio 5 a",
    "1051 calle 3 se ste lc1a 3003 pr, san juan, 00921, puerto rico" : "1051 street 3 sete lc1a 3003 pr, san juan, 00921, puerto rico",
    "bauru   sp" : "bauru sp",
    "celle ligure" : "that ligure",
    "uttarpara kotrung" : "northern",
    "independencia esq heroes de luperon" : "independence esq heroes de luperon",
    "cư xá c . đại học nông lâm tp hcm" : "cu xa c.ho chi minh city university of agriculture and forestry",
    "espeŕanza" : "hope",
    "nh 27 deesa highway badarpura" : "no 27 disa highway badarpur",
    "qracaju" : "grocaju",
    "gen. trias cavite" : "gen.trias cavite",
    "taubaté   são paulo" : "taubaté são paulo",
    "bertioga   sp" : "bertioga sp",
    "hafar al batin" : "drill the clay",
    "barasat" : "rain",
    "nova serrana" : "new serrana",
    "niterói   rj" : "nitero rj",
    "masjid bander" : "bander mosque",
    "pichari" : "backward",
    "chhatapur" : "umbrella",
    "naina tikker" : "naina is ticking",
    "bangaloru" : "bangalore",
    "bezerros" : "calves",
    "juárez chihuahua méxico" : "juárez chihuahua mexico",
    "belo horizinte" : "belo horizonte",
    "dharuhera" : "darhera",
    "1001 camino ct" : "1001 ct path",
    "bucuresti" : "bucharest",
    "софия" : "sofia",
    "nowa wola" : "new wola",
    "rio grande do sul   erechim" : "rio grande do sul erechim",
    "west des moines" : "west monks",
    "terra boa" : "land",
    "viramgam" : "veeranga",
    "presidente juscelino" : "president juscelino",
    "ấp mỹ tân  thị trấn mỹ luông" : "my tan hamlet, my luong town",
    "phagwara" : "fagwara",
    "maloposkie" : "lizposki",
    "niterói   rio de janeiro" : "niterói rio de janeiro",
    "تيماء" : "tayma",
    "san andres islas" : "san andres islands",
    "bodh gaya" : "gone",
    "centro de produção audiovisual   sesc são paulo" : "sesc são paulo audiovisual production center",
    "perumanallur" : "advertise",
    "capacio paestum" : "paestum",
    "شارجه" : "sharjah",
    "karaikudi" : "go from",
    "marau" : "curricular",
    "são josé dos campos   sp" : "são josé dos campos sp",
    "shkoat" : "skoat",
    "debashis chakraborty" : "debashish chakraborty",
    "via mar della cina 254" : "via mar della china 254",
    "stary wiśnicz" : "old wiśnicz",
    "librebille" : "libreville",
    "1855 folsom st ste 601  medical" : "1855 folsom st you 601 medical",
    "paramaribo suriname" : "paramaribo, suriname",
    "mahrajganj" : "maharajganj",
    "alecrim  rs" : "rosemary rs",
    "medan" : "while",
    "artena" : "clay",
    "ertert" : "is",
    "pederneiras" : "bane",
    "würselen" : "sewer",
    "5808 socorro dr" : "5808 dr socorro",
    "ambato" : "hint",
    "tekulasomaram" : "techulsamaram",
    "doctor andrade  63 1er. piso  colonia doctores ciudad de m?xico" : "doctor andrade 63 1st.colonia doctors city of m? xico",
    "centro comercial asturias de ovideo  cra 10 # 1   50   local 10 cundinamarca" : "asturias shopping center of oviedo cra 10 # 1 50 local 10 cundinamarca",
    "atalaya" : "watchtower",
    "jericho" : "jeri",
    "kothrud,pune" : "kothrud, pune",
    "dharmaragar" : "dharmagar",
    "jangaon" : "the jangaoon",
    "belo horizonte   minas gerais" : "belo horizonte minas gerais",
    "3815 faraon st ste e" : "3815 pharaon st you e",
    "baruero" : "barueri",
    "medellín colombia" : "medellin colombia",
    "مسقط" : "muscat",
    "جازان" : "jizan",
    "ciudad satelite" : "ciudad satellite",
    "irara" : "breathy",
    "waukesha" : "you up",
    "pulivendla" : "pulivendula",
    "miagao iloilo" : "misama expections",
    "trà ôn vĩnh long" : "tra wen vinh long",
    "senhora" : "madam",
    "satuba" : "one",
    "ramanagar" : "ramnagar",
    "amk" : "bachelor",
    "fortaleza, ce" : "fortaleza, what",
    "3030 maingate ln" : "3030 mangate lan",
    "barreiros" : "barriers",
    "salmiya" : "peaceful",
    "04, prasadi gali , kotla mubarakpur delhi  delhi" : "04, prasadi gali, mubarakpur delhi delhi boiler",
    "la paz bolivia" : "la paz, bolivia",
    "margao,goa" : "margao, goa",
    "arame   ma" : "aram",
    "latur" : "lazy",
    "alta floresta" : "rainforest",
    "تموز (يوليو)" : "july (july)",
    "paraíso do norte pr" : "north paradise pr",
    "beberibe" : "beeribe",
    "banaso, hazaribagh" : "country, hazaribagh",
    "kochi, kozhikode" : "this, this ← iko",
    "rio branco" : "white river",
    "احد رفيده   الواديين" : "one of the valleys of the valley",
    "عجمان" : "ajman",
    "25c nguyễn bỉnh khiêm, q1, tp.hcm" : "25c nguyen binh khiem, district 1, ho chi minh city",
    "goiás   anápolis" : "goiás anápolis",
    "hessen   meinhard" : "hessen meinhard",
    "negros oriental" : "eastern blacks",
    "manser" : "male",
    "lima perú" : "lima peru",
    "piripiri" : "adhesive",
    "passo fundo" : "deep step",
    "fanna" : "us to",
    "bhongir" : "bhogir",
    "santiago de cali" : "santiago from cali",
    "armação de buzios" : "buzios frame",
    "mombasa,kenya" : "mombasa, kenya",
    "capão da canoa" : "canoa capão",
    "jais" : "them",
    "baki" : "mouth",
    "cambrideshire" : "cambridgeshire",
    "sitamadhi" : "sitamarhi",
    "paraná   francisco beltrão" : "paraná francisco beltrão",
    "budaun" : "budhon",
    "moskova" : "moscow",
    "shendurni" : "stubborn",
    "guwhati" : "gauwh",
    "uberlândia   mg" : "uberlândia mg",
    "małopolska" : "lesser poland",
    "toledo pr" : "on toledo",
    "ixtalhuaca de rayon" : "istalhuacon",
    "amapá   macapá" : "amapá macapá",
    "nova veneza" : "new venice",
    "nandyala" : "nandala",
    "getúlio vargas" : "getulio vargas",
    "barra bonita" : "pretty bar",
    "khober" : "hover",
    "cantagalo" : "sink",
    "bahia   salvador" : "bahia salvador",
    "mérida yucatán" : "merida yucatan",
    "mondai" : "problem",
    "mahesana" : "mehsana",
    "i̇zmir" : "mirzmir",
    "ananthapur" : "anantapur",
    "kepulauan riau" : "riau islands",
    "سوهاج" : "sohag",
    "francisco i. madero mz. 28 lt. 37 m?xico, d.f." : "francisco i.madero mz.28 lt.37 m? xico, d.f.",
    "sahaswan=243638" : "sahawan = 243638",
    "chaguanas" : "selectnas",
    "seleccionar comunapunitaqui" : "select comunapunitaqui",
    "saopaulo" : "baby be drowered",
    "da nang city, viet nam  vi?t nam" : "da nang city, viet nam vi? t nam",
    "foz de iguaçu   pr avenida tancredo neves 6731 jardim itaipu" : "foz de iguaçu pr avenida tancredo neves 6731 jardim itaipu",
    "שריגים (לי און)" : "shss (li on)",
    "i̇stanbul hadımköy" : "istanbul hadımköy",
    "alkhobar" : "the news",
    "reyadh" : "riyadh",
    "neyveli" : "unsheated",
    "medicina" : "medicine",
    "singen" : "to sing",
    "hei longjiang" : "heilongjiang",
    "poznam" : "i know",
    "los ángeles" : "the angels",
    "ahmednagar" : "ahmednaagar",
    "neyba provincia bahoruco" : "neyba bahoruco province",
    "bhawani mandi" : "bhavani people",
    "dĩ an, bình dương" : "di an, binh duong",
    "أنقرة" : "ankara",
    "cancún" : "cancun",
    "metropolitana de santiago" : "metropolitan of santiago",
    "al najaf" : "al najaf",
    "és" : "and",
    "punei" : "place",
    "pará" : "for",
    "campos dos goytqcazes" : "dos goytacazes fields",
    "cobquecura" : "chasing",
    "rj   rio de janeiro" : "rj rio de janeiro",
    "recife  pe" : "recife on",
    "ciudad autonoma de bs. as" : "autonomous city of bs.ace",
    "litleton" : "littleton",
    "sharq" : "east",
    "guidoval" : "he gurid",
    "marechal deodoro" : "marshal deodoro",
    "las piñas   muntinlupa" : "las piñas muntinlupa",
    "paudalho pe" : "padalho pe",
    "chatra" : "student",
    "alriyadh" : "riyadh",
    "76 78 bạch đằng, p. hải châu i" : "76 78 bach dang, p.hai chau i",
    "rs   caxias do sul" : "rs caxias do sul",
    "สงขลา" : "songkhla",
    "capelinha" : "chapel",
    "satata" : "stata",
    "novi beograd" : "new belgrade",
    "kerala" : "cherry",
    "playas de rosarito  baja california" : "rosarito baja california beaches",
    "são paio de oleiros" : "st.",
    "اربد" : "irbid",
    "mg   araxá" : "mg araxá",
    "sete lagoas" : "seven lagoons",
    "gujrat.godhra" : "gujarat. godhra",
    "greatfalls" : "great falls",
    "joinvielle" : "joinville",
    "4 tp thủ đức tphcm" : "4 thu duc city ho chi minh city",
    "siquirres limón" : "limón siquirres",
    "f6" : "timing",
    "lauscha" : "lollipotia",
    "ambalappuzha" : "ambalappuzh",
    "العين" : "eye",
    "أبها" : "abha",
    "komarapalayam" : "comrapalayam",
    "goiania  go" : "goiania go",
    "ktx khu c   đại học nông lâm tp hcm" : "ktx area c, agriculture and forestry university of ho chi minh city",
    "búzios" : "shells",
    "sadhoganj kharawan baragaon vns." : "sadhganj kharwan bargaon ones.",
    "bạc liêu" : "bac lieu",
    "betim   mg" : "betim mg",
    "engativa" : "deceit",
    "turvo" : "cloudy",
    "elkmont" : "quantity",
    "la vega" : "the vega",
    "dique de caxias" : "dike",
    "teresina pi" : "teeasia pi",
    "via oratorio,5" : "via oratory, 5",
    "três barras santa catarina" : "three santa catarina bars",
    "ulu tiram" : "the coast",
    "wiskitki" : "tarpaulin",
    "số 7 đường 3419 phường 7 quận 8." : "no. 7, street 3419 ward 7 district 8.",
    "la palma" : "the palm",
    "5n  ấp đông lân xã bà điểm" : "5n dong lan hamlet, ba diem commune",
    "hydarabaad" : "hysradons",
    "5b tân chánh hiệp 35" : "5b tan chanh hiep 35",
    "m'sila" : "me",
    "28 to ngọc van" : "28 to ngoc van",
    "الدمام" : "dammam",
    "campina verde" : "green campina",
    "rohini" : "expense",
    "são  paulo" : "são paulo",
    "safwa" : "greed",
    "aracaju   sergipe" : "aracaju - sergipe",
    "hauz khas,delhi" : "hauz special, delhi",
    "contagem minas gerais" : "count minas gerais",
    "hadapsar" : "front",
    "pančevo" : "pancevo",
    "davao del sur, davao city" : "gave del sur, gave city",
    "barão de cotegipe" : "baron de cotegipe",
    "5 lorong kemaris 1 bukit bandaraya" : "5 lorong lorong 1 bukit city",
    "salvador  bahia" : "salvador, bahia",
    "klinikum nürnberg (standort nord)" : "clinic nuremberg (north location)",
    "lalmatia" : "lacquer",
    "sumatra barat" : "west sumatra",
    "วังน้อย" : "wang noi",
    "bbsr" : "bubbling",
    "dandeli" : "capable",
    "tarn taran" : "taran taran",
    "13 ryad street shoubra, cairo" : "૧૩ rayal street shoubra, cairo",
    "san isidro davao oriental" : "like an equid davo oriental",
    "dubai invesyments park 1" : "dubai investments park 1",
    "ب" : "b",
    "airwa katra" : "airwa each",
    "balneário  camboriú" : "balneário camboriú",
    "mersin" : "myrtle",
    "araria" : "arriya",
    "la lucila, vicente lopez" : "lucila, vicente lopez",
    "السالمية" : "salmiya",
    "пловдив" : "plovdiv",
    "guanambi" : "guanhabi",
    "puttaparthi" : "puttparthi",
    "1 va ctr," : "1 ctr,",
    "ha noi" : "ha we",
    "antiguo cuscatlan" : "ancient cuscatlan",
    "tabira" : "toy",
    "mousl" : "mosul",
    "via pizzinardi  64" : "via pizzinardi 64",
    "ระยอง" : "rayong",
    "s73w16773 briagate ln  medical monitors" : "s73w16773 briargate ln medical monitors",
    "vinh   nghệ an" : "vinh nghe an",
    "vitória da conquista   ba" : "vitória da conquista ba",
    "joão pessoa:pb" : "joão pessoa: pb",
    "thalavadi" : "thalwadi",
    "160 calle primera" : "160 first street",
    "الجيزه" : "giza",
    "nagpur ,kanhan" : "nagpur, kanhan",
    "103 d tate circle  education" : "103 d tate circle  education",
    "boaco" : "reindeer",
    "calle gala 8" : "street gala 8",
    "test34" : "test 4",
    "mundo novo  ms" : "new world ms",
    "عسير" : "difficult",
    "av 531 no 6 ciudad de mexico" : "av 531 no 6 mexico city",
    "al seeb" : "the sep",
    "52  ต.ป่าตอง อ.กะทู้ จ.ภูเก็ต" : "52 patong subdistrict, kathu district, phuket province",
    "9540 hamburg rd" : "9540 hamburg rd",
    "biñan city" : "biñan cyty",
    "lagos de moreno" : "moreno lakes",
    "hydeeabad" : "hyderabad",
    "san jose del cabo" : "san jose of cabo",
    "minneola" : "memory",
    "ammanabrolu" : "amamanballu",
    "el cove" : "the cove",
    "huixquiliucan" : "huixquilucan",
    "hubali" : "hubli",
    "viçosa" : "lurid",
    "pará canaã dos carajás" : "para canaã dos carajás",
    "موصل" : "insulated",
    "raibareli" : "rai barre",
    "gandu" : "tribute",
    "río tercero   córdoba" : "río tercero córdoba",
    "st julians" : "st. julians",
    "nova lima" : "new lime",
    "la quiaca" : "the quiaca",
    "shikaripura" : "plastic",
    "pelotas" : "pellets",
    "vespasiano   minas gerais" : "vespasian minas gerais",
    "zielona góra" : "green mountain",
    "curitiba e são paulo" : "curitiba and são paulo",
    "podlaskie" : "podlasie",
    "são thome das letras" : "saint thome of letters",
    "bhabra" : "emotional",
    "176 thủ khoa huân phan thiết bt" : "176 valedictorian huan phan thiet bt",
    "12 phạm van bach tân bình" : "12 pham van bach tan binh",
    "434 bình quới, phường 28" : "434 binh quoi, ward 28",
    "bhachau kutch" : "bachau kutch",
    "karwar" : "increase",
    "ภูเก็ต" : "phuket",
    "số7,đường 3419,phạm thế hiển ,q8," : "7, 3419, pham the hien, q8,",
    "maceió  al" : "macea al",
    "λαρισα" : "larissa",
    "guayaquil   ecuador" : "guayaquil, ecuador",
    "ashtapur" : "ashtpur",
    "shakopee" : "shakopi",
    "são joão da boa vista" : "sao joao da boa vista",
    "nabatieh" : "nabaiah",
    "2c trần hưng đạo, p1, q5" : "2c tran hung dao, p1, q5",
    "أبوظبي" : "abu dhabi",
    "cân rhow" : "weigh",
    "hai dương" : "two -positive",
    "santa izabel do oeste" : "santa izabel do west",
    "boituva" : "boituvian",
    "stary żabieniec" : "old żabieniec",
    "hampden" : "hem",
    "新竹縣" : "hsinchu county",
    "strada statale 182 c.da nasari snc" : "state road 182 c.da nasari snc",
    "cachoeiro de itapemirim   es" : "cachoeiro de itapemirim es",
    "7985 kentshire dr  medical" : "7985 i'm",
    "quận 10" : "district 10",
    "1380 fondo de la legua piso 1 â°  commercial display" : "1380 legua fund floor 1 ° commercial display",
    "las condes, santiagom, chile" : "las condes, santiago, chile",
    "bento gonçalves. rs" : "bento gonçalves. lol",
    "hà tiên" : "ha tien",
    "gualeguaychu" : "gualeguare",
    "elmansoura" : "mansoura",
    "sitio de abadia go" : "abbot site",
    "marechal cândido rondon   paraná" : "marshal cândido rondon paraná",
    "vaniyambadi" : "vaniya -enambadi",
    "house no 663 chopal wali gali alipur garhi delhi  110036" : "house no 663 chopal gali alipur garhi delhi 110036",
    "thrithallore" : "trianalore",
    "96 moo 1 klongnueng klongluang" : "96 where 1 klongnueng klongluang",
    "10 zonara st" : "10 zonara st.",
    "roddi" : "give",
    "bilasipara" : "feipy",
    "63 trương định" : "63 truong dinh",
    "a17 12 chung cư samland" : "a17 12 samland apartment",
    "tio hugo rio grande do sul" : "uncle hugo rio grande do sul",
    "plot #35,phase iv,malur  malur taluk  kolar district" : "plot #35, phase iv, malur malur taluk kolar district",
    "durgapur bhiringi" : "durgapur bhirangi",
    "la reina" : "the queen",
    "canningvale" : "canning",
    "governador valadares" : "valadares governor",
    "303 gò dầu phường tân quý" : "303 go dau ward tan quy",
    "seri kembangan" : "kembangan series",
    "sa’ad, riyadh dammam highway,  ksa" : "sa'ad, riyadh dammam highway, ksa",
    "belém e região" : "belém and region",
    "2777 n stemmons fwy  healthcare" : "2777 n stemmons more healthcare",
    "house no. 375, road no. 28, new dohs, mohakhali, dhaka cantonment, dhaka 1206, bangladesh bangladesh" : "house no.1, road no.28, new das, mohakhali, dhaka cantonment, dhaka 1 to 3, bangladesh bangladesh",
    "sp brasil" : "sp brazil",
    "agudos" : "acute",
    "pathanamthitta" : "pathanamtta",
    "via gramsci angolo via de gasperi" : "via gramsci corner via de gasperi",
    "tiruvallur" : "thiruvallur",
    "vadodra" : "vadodara",
    "29222 rancho viejo rd ste 108  medical" : "29222 rancho viejo rd ste 108 medical",
    "mahua" : "mahu",
    "89 hoàng quốc việt,q7" : "89 hoang quoc viet, district 7",
    "الرس" : "alrass",
    "loja" : "store",
    "begunia" : "beguni",
    "nasik (nashik)" : "nasik (we doubt)",
    "sta. cruz" : "stay.cruz",
    "surbiton" : "surbs",
    "lagoa da prata   mg" : "mg silver lagoa",
    "heroica puebla de zaragoza" : "heroic puebla of zaragoza",
    "baga calangute" : "lung calangute",
    "315 alameda blvd ne" : "315 alamed blvd not",
    "palitana" : "glory",
    "anoop shahr" : "anoop city",
    "mianwali colony gurugram" : "miavali colony gurugram",
    "las piñas city" : "the city pineapples",
    "resistencia" : "endurance",
    "capao da canoa" : "canoe",
    "barakar" : "vanity",
    "tukwila" : "us",
    "sanriago" : "after sanria",
    "trà cú trà vinh" : "tra cu tra vinh",
    "tubarao" : "scrapping",
    "dharamapuri" : "dharampuri",
    "barueri" : "barley",
    "sląskie" : "silesian",
    "duque de caxias" : "duke of caxias",
    "chatrapati sambhajnagar" : "chhatrapati",
    "rio grande do sul  it" : "rio grande do sul it",
    "cornélio procópio" : "cornelius procopio",
    "thanh xuân" : "youth",
    "ateli mandi" : "mandy atels",
    "bucaramanda" : "bukramanda",
    "mozzate" : "severed",
    "skawina" : "skaw",
    "wielkopolski" : "wielkopolska",
    "campo bom" : "good camp",
    "naucalpan , estado de mexico" : "naucalpan, state of mexico",
    "nova venécia" : "new venecia",
    "macherla" : "machers",
    "225 baldwin aveneu  healthcare" : "225 baldwin avenue healthcare",
    "oklahoma city" : "ololama city",
    "1759 wehrle dr  it" : "1759 wehrle dr it",
    "kihei" : "bay",
    "saocha" : "laboru",
    "sao paulo   sp" : "sao paulo-sp",
    "cabaceiras do paraguaçu" : "paraguaçu cabaceiras",
    "curitiba pr" : "on karitiba",
    "patos de minas" : "mining ducks",
    "nova iguaçu" : "new iguaçu",
    "brasil" : "brazil",
    "tp.hcn" : "tp.hcm",
    "benagluru" : "bengaluru",
    "ilhabela" : "beautiful island",
    "boa vista   rr" : "boa vista rr",
    "via san martino ,12  fraz. eremo" : "via san martino, 12 fraz.hermitage",
    "hévíz" : "heat water",
    "kupwara" : "to blow",
    "autopista medella. bogotá, colombia" : "medella highway.bogota colombia",
    "duque de caxias   rio de janeiro" : "duque de caxias rio de janeiro",
    "9716 mcfarring drfort worthtx76244" : "9716 mcfarring dr fort worth tx 76244",
    "bielsko biała" : "bielsko-biala",
    "tinsukia" : "chinsquia",
    "wólka kozodawska" : "wólka kozodska",
    "shri madhopur" : "shri madhur",
    "contagem mg" : "count mg",
    "17902 kellas ct" : "17902 clock ct",
    "jatai" : "expressed",
    "menicucci 3" : "menicccci 3",
    "kayseri̇" : "kayseri",
    "pouso alegre" : "happy landing",
    "otro (latinoamérica)" : "another (latin america)",
    "28 ngô quyen" : "28 ngo quyen",
    "sao pauo" : "sao make",
    "5560 katella ave" : "5560 cover ave",
    "30 mân quang 6, sơn trà, đà nẵng" : "30 man quang 6, son tra, da nang",
    "ventanilla   callao" : "callao window",
    "garki" : "forth",
    "al ahsa" : "i count",
    "nandurbar" : "invested",
    "2 2 5 toranomon, minato ku, tokyo" : "2 2 5 toranomon, minato ward, and",
    "25c nguyễn bỉnh khiêm, q1" : "25c nguyen binh khiem, district 1",
    "ينبع" : "stems",
    "nandyal" : "nandala",
    "san martino in rio    re" : "san martino in rio re",
    "deoghar" : "deo -house",
    "cariacica" : "cariacic",
    "pirangi" : "like",
    "nova iguaçu   rio de janeiro" : "nova iguaçu rio de janeiro",
    "avram iancu 20a  it" : "avram iancu 20a it",
    "tulua" : "twinkle",
    "tlalnepantla estado de mexico" : "tlalnepantla state of mexico",
    "hargeisa" : "harge",
    "santo andre são paulo" : "santo andre sao paulo",
    "curitiba   pr" : "on karitiba",
    "são paulo tatui" : "sao paulo tatui",
    "ri9 de.janeiro" : "rurayd d.",
    "impasugong bukidnon" : "impasuto bukidnon",
    "zamboanga city" : "vegeta city",
    "palma decmallorca" : "palma de mallorca",
    "merida yucatán" : "merida yucatan",
    "juárez" : "juare",
    "chandausi" : "check",
    "kennebunkport" : "kenkeybunkport",
    "kareli" : "plaid",
    "tshimaku,tokyo" : "t film, tokyo",
    "ugar" : "fallow",
    "ปัอมปราบฯ" : "pork suppression",
    "kouhoku ku, yokohama" : "kohoku ward, yokohama",
    "פתח תיקווה" : "petah tikva",
    "9647701084310 sulemaniyah" : "9647701084310 sulemanya:",
    "the colonty" : "the colony",
    "dariba" : "remark",
    "świętokrzyskie skarżysko  kamienna" : "świętokrzyskie skarżysko kamienna",
    "caolanh" : "a winter",
    "la mesa" : "table",
    "gran canarias playa del ingles" : "gran canarias playa del english",
    "geebung" : "vomiting",
    "volta redonda   rj" : "volta redonda rj",
    "cam sur" : "cam on",
    "via v. volterra, 9" : "via v.volterra, 9",
    "kraków" : "cracow",
    "17 νοεμβρη 79 πυλαια" : "17 november 79 pylaia",
    "taboão da srra  sp" : "taboão da serra sp",
    "digha" : "dagh",
    "bandung, kota." : "bandung, city.",
    "doveton 3177" : "covton 3177",
    "panabo" : "bailer",
    "avan" : "before",
    "كلباء" : "all",
    "dhi qar" : "ruild",
    "goleta" : "schooner",
    "piedade" : "pity",
    "lachhmangarh sikar" : "lachmangarh sikar",
    "habra" : "there will be",
    "8 ấp 4 xã nhơn đức" : "8 hamlet 4, nhon duc commune",
    "gzb" : "calamity",
    "primavera do leste" : "spring of the east",
    "una, himachal" : "one, himachal",
    "b5 lưu văn liệt, phường 2, tpvl" : "b5 luu van liet, ward 2, tpvl",
    "gola gokaran nath" : "circle gokharan nath",
    "安徽省芜湖市" : "wuhu city, anhui province",
    "caucaia ce" : "caucaia what",
    "barao de cotegipe" : "cotegipe barao",
    "barreiras  ba" : "ba barriers",
    "campos de julio mt" : "julio mt fields",
    "eylül" : "september",
    "margao" : "cursing",
    "itapema   sc" : "it will spit sc",
    "กทม" : "bangkok",
    "norte de santander, cucuta" : "north of santander, cucuta",
    "mooca   sp" : "mooca sp",
    "propriá   sergipe" : "propriá sergipe",
    "alger centre" : "algiers center",
    "constanta" : "constant",
    "prol corregidora norte 1116 of 301 quer?taro" : "north correcting prol 1116 of 301 want? taro",
    "carrera 71 # 50   57 off 201" : "carrera 71 # 50 57 off 201",
    "bagnan" : "garden",
    "pune   kharadi" : "hardy",
    "ชลบุรี" : "chonburi",
    "la florida" : "florida",
    "kasaragod" : "discovery",
    "falakata" : "fatal",
    "formosa do sul   sc" : "formosa do on the sc",
    "otyń" : "obesin",
    "115 trần bá giao f5" : "115 tran ba giao f5",
    "paulista  pernambuco" : "paulista pernambuco",
    "seropédica   rj" : "seropédica rj",
    "chandugarh" : "chandgarh",
    "carrera 11a 94 46 edificio chico 3000 piso 3 bogota" : "carrera 11a 94 46 buy chico 3000 floor 3 bogota",
    "corso canale 32a" : "channel 32a course",
    "ponta grossa   pr" : "ponta groro pr",
    "sâo paulo" : "são paulo",
    "varodrada" : "varodarada",
    "badung, kuta selatan" : "badung, south kuta",
    "via jardins" : "via gardens",
    "chapadinha   maranhão" : "chapadinha maranhão",
    "jakrta" : "jakarta",
    "istanbul   beyoglu" : "istanbul beyoglu",
    "recufe" : "reefs",
    "quatre bornes" : "four terminals",
    "malavalli" : "malwalli",
    "1655 e arlingtonblvd" : "1655 e arlington blvd",
    "buzet sur tarn" : "buzet on tarn",
    "vapi" : "coat of arms",
    "中山區" : "zhongshan area",
    "tiruchengodu" : "we have a cleanliness",
    "małopolks" : "lesser poland",
    "al madinah" : "city",
    "itaberaba   ba" : "plate club",
    "zławieś mała" : "zławieś little",
    "palakonda" : "plakonda",
    "merta city dist nagaur" : "merrata city dist nagaur",
    "venezia mestre" : "venice mestre",
    "botucatu" : "booted",
    "الموصل" : "mosul",
    "tp thủ đức" : "thu duc city",
    "butibori" : "bump",
    "aparecida   sp" : "aparecida sp",
    "16604 arminta st, van nuys, ca 91406" : "16604 arminta sat, van nayas, 91406",
    "fhfu" : "jar",
    "thị xã bình minh" : "binh minh town",
    "buzios" : "bunks",
    "vassouras" : "broom",
    "padra  vadodara" : "bench",
    "baleares" : "whaarse",
    "pernambuco  petrolandia" : "pernambuco petrolandia",
    "van horn" : "horn",
    "8 phan văn hớn" : "8 phan van hon",
    "başakşehi̇r" : "başakşehi",
    "lajeado" : "slab",
    "port bouet rue des caraibes" : "bouet port rue des caraibes",
    "pauri" : "on you",
    "gudalur" : "guddur",
    "cl 20 sur 45 70 ap 904" : "cl 20 of 45 70 ap 904",
    "bình dương" : "binh duong",
    "gangakher" : "gangakhed",
    "barnala" : "childhood",
    "di̇di̇m aydın" : "deep",
    "guadalajara, jalisco" : "guadalajara jalisco",
    "corpus christi" : "body of christ",
    "hunasagi" : "gaid",
    "16 lam sơn, p2, tân bình" : "16 lam son, p2, tan binh",
    "tornesch" : "tornorous",
    "kab. bandung barat" : "regency. west bandung",
    "redenção" : "redemption",
    "50675 yukon rd po box 537" : "50675 oil now rd popular x 537",
    "via provinciale sud, 2" : "provincial south via, 2",
    "espírito santo, es" : "holy espírito, is",
    "obrassano" : "work",
    "carrera 18 n. 48 52" : "race 18 n.48 52",
    "são gabriel" : "saint gabriel",
    "paleo faliro" : "old faliro",
    "sulaimanya" : "solomon",
    "cascavel" : "rattlesnake",
    "śląskie" : "silesian",
    "ciudad madero" : "madero city",
    "tremblay en france" : "tremblay in france",
    "via delle battaglie" : "via delle battles",
    "sorriso" : "smile",
    "45 west flagge street  medical" : "45 west flag street medical",
    "zakopane" : "buried",
    "268 lý thường kiệt quận 10" : "268 ly thuong kiet district 10",
    "مكه" : "mecca",
    "دقهليه" : "dakahleh",
    "samar leyte" : "production of leyte",
    "quatro barras" : "four bars",
    "silvassa" : "silvasa",
    "ulaanbaatar" : "ulbatar",
    "343 s 500 e, apt 118" : "zhz with 500 e, apt 118",
    "muthanna  samawah city" : "muthanna samawah city",
    "puttparthi" : "puttaparthi",
    "praia grande" : "big beach",
    "ciudad de mëxico" : "mexico city",
    "monte verde" : "green mount",
    "kadiri" : "as much as possible",
    "28 tô ngoc vân p thạnh xuan .q12" : "28 to ngoc van p thanh xuan .q12",
    "baturaja" : "king stone",
    "idar" : "id",
    "nouméa" : "noumea",
    "جنوب عبدالله مبارك" : "south abdullah mubarak",
    "sapiranga   rs" : "sapiranga rs",
    "nova ponte" : "new bridge",
    "perote  veracruz" : "perote veracruz",
    "chandigadh" : "chandigarh",
    "baden würtemberg" : "baden württemberg",
    "slask" : "slush",
    "salvador  ba" : "salvador ba",
    "laguna de terminos 221, torre b 604 bis cdmx" : "lagoon of terminnos 221, tower b 604 bis cdmx",
    "台北" : "taipei",
    "marechal candido rondon" : "marshal candido rondon",
    "المحله الكبري  الغربية" : "western major locality",
    "5 av. des hauts fourneaux" : "5 av.high stoves",
    "phường phú thuận, quận 7" : "phu thuan ward, district 7",
    "belo horizonte   mg" : "belo horizonte mg",
    "san mateo rizal" : "san mato rizal",
    "medianeira" : "mediator",
    "alfyom" : "palafume",
    "pest megye" : "pest county",
    "7730 n central dr" : "7730 n central dr.",
    "ktx khu b   đhqg tp.hcm" : "ktx area b vnu hcmc",
    "strovolos" : "stingy",
    "ראש לציון" : "rishon lezion",
    "piauí   teresina" : "piauí teresina",
    "675 đường giải phóng" : "675 liberation road",
    "189 paramus rd. paramus" : "189 paramus rd.paramus",
    "pernambuco   recife" : "pernambuco recife",
    "los olivos" : "the olive trees",
    "ul. łódzka 308" : "ul.łódź 308",
    "nagarnausa" : "carrity",
    "pekanbaru" : "pecanbal",
    "sarangpur" : "rangpur",
    "laxmiganj guna" : "laxmiganj use",
    "sao jose do rio preto   sp" : "sao jose do rio preto sp",
    "karamsad" : "karmasad",
    "santiago de  chile" : "santiago de chile",
    "cao lãnh đồng tháp" : "cao lanh dong thap",
    "lieusaint" : "holy place",
    "mokattam" : "mute",
    "pathanapuram" : "prathanapuram",
    "malapuram" : "malapppuram",
    "campinas shopping móveis" : "campinas shopping mobile",
    "mérida, yucatán, méxico" : "mérida, yucatán, mexico",
    "yucatán" : "yucatan",
    "gharaunda" : "gharonda",
    "sntiago" : "santiago",
    "três rios" : "three rivers",
    "gardabaer" : "keep",
    "são josé do rio preto   sp" : "são josé do rio preto sp",
    "camarines sur" : "camarines on",
    "são caetano" : "sao caetano",
    "chapeco" : "hat",
    "mestre" : "teacher",
    "la follette" : "the madman",
    "via c. battisti, 3b" : "via c.battisti, 3b",
    "lo barnechea" : "barnechea",
    "manhuacu" : "rough comic",
    "foz do iguacu   pr" : "foz do iguacu pr",
    "araçatuba   sp" : "araçatuba sp",
    "कटिहार" : "katihar",
    "paranaíba   ms" : "paranaíba ms",
    "bounsi" : "beyonce",
    ".mpilo and pioneer road maseru" : ".mmp and pioneer road maseru",
    "itajai  sc" : "it will be sc",
    "14 tám danh p4.q8.tphcm" : "14 eight names p4.q8.tphcm",
    "águas formosas" : "famous waters",
    "via de filippis 40" : "way of filippis 40",
    "tangeran" : "tangerang",
    "valleduapar" : "valledupar",
    "san mateo 5 int ala b, boulevares, naucalpan estado de mexico" : "san mateo 5 int ala b, boulevares, naucalpan state of mexico",
    "contagem   minas gerais" : "count minas gerais",
    "bonita ave." : "nice bird.",
    "kiseljak" : "sorrel",
    "cidade" : "city",
    "salfit" : "salt",
    "gretna" : "grena",
    "11100 sepulveda blvd ste 8 pmb 393" : "11100 sepulveda blvd you are 8 pmb zia",
    "barranquilla , colombia" : "barranquilla, colombia",
    "dolnośląskie" : "lower silesian",
    "urb  rio piedras heights 144 wezer" : "urb rio piedras heights 144 wees",
    "cavolo del tomba" : "cabbage of the tomb",
    "nizwa" : "hear",
    "rae bareli" : "raye white",
    "torreón" : "keep",
    "distrito federal" : "federal district",
    "bokakhat" : "bucahat",
    "via circumvallazione esterna 89" : "via external circumvalulation 89",
    "cantabria" : "sneaky",
    "8, lake drive, rajagiriya. na" : "8, lake drive, rajgiria.no",
    "ktx khu a đhqg tp hồ chí minh" : "ktx area a vnu ho chi minh city",
    "juína" : "junior",
    "amapa" : "amapapapapapapapapa about",
    "kanchrapara" : "raw",
    "hải phòng" : "hai phong",
    "calle divino figueroa nº3 bajo c" : "divine street figueroa nº3 under c",
    "ar riyad" : "riyadh",
    "serta" : "as well as",
    "anakapalle" : "unaccounted",
    "tapurah" : "tapur:",
    "al muruj exit5 riyadh" : "the promoter is a feast of riad",
    "località grand chemin 22" : "big chemin location",
    "kuzey kıbrıs" : "north cyprus",
    "ktx khu a đhqg tp hcm" : "ktx area a vnu hcmc",
    "b35, 533" : "akhb, 533",
    "quito ecuador" : "quito, ecuador",
    "sri ganganagar" : "shri ganga nagar",
    "cd.jurez" : "cd.juarez",
    "sas nagar mohali" : "mother -in -law mohali",
    "españa" : "space",
    "via ai ronchi 288, mozzate co." : "away to the ronchi 288, mozzate co.",
    "unanderra" : "anderrera",
    "zduńska wola" : "zdunska wola",
    "bissingen an der teck" : "bissingen on the teck",
    "s.c. de bariloche" : "scby bariloche",
    "wrzeście k" : "september k",
    "belo horizone" : "belo horizonte",
    "śąskie" : "in a good",
    "bauneario" : "road ario",
    "salvado   bahia" : "saved bahia",
    "siddhartha nagar up" : "siddharth nagar sub",
    "hortolandia" : "vegetation",
    "raxaul" : "ragaul",
    "sambhal" : "careful",
    "alto paraiso" : "high paradise",
    "são caetano do sul" : "sao caetano do sul",
    "leh" : "with",
    "2.05 b chung cư mỹ thuận f16 q8" : "2.05 b my thuan apartment f16 q8",
    "смолян" : "smolyan",
    "shrihari kota" : "srihari kota",
    "ciudad de mexico" : "mexico city",
    "asunción" : "assumption",
    "cananda" : "canada",
    "saudi arabia \ al riyad" : "saudi arabia / al riyad",
    "mazowieckie   gabryelin 05 505" : "mazowieckie gabryelin 05 505",
    "ribeirão pires   são paulo" : "ribeirão pires são paulo",
    "bucaramanga   colombia" : "bucaramanga colombia",
    "goianinha" : "gouge",
    "rio grande do sul estação" : "rio grande do sul station",
    "辽宁省大连市" : "dalian, liaoning province",
    "marcham" : "march",
    "kurkheda" : "kurged",
    "haelen" : "hasten",
    "kalimantan selatan" : "south kalimantan",
    "barshi" : "let it go",
    "itabirito   mg" : "it is a mg",
    "quận 11" : "district 11",
    "cần tho" : "can tho",
    "المعادى" : "hostile",
    "רמת השרון" : "ramat hasharon",
    "naugarh siddharth nagar up" : "naugarh siddharth nagar sub",
    "koongal" : "canon",
    "juruti para" : "juruti for",
    "ecuador   loja" : "ecuador loja",
    "edo" : "or",
    "seriate" : "serial",
    "chimalhuacan" : "chimalhuana",
    "huyện tam bình   tĩnh vĩnh long" : "tam calm district vinh long",
    "przysucha" : "priests",
    "kumbakonam" : "kumbakal angle",
    "nellore" : "in",
    "aruppukkottai" : "archae",
    "cao lảnh" : "tall",
    "long xuyên tỉnh an giang" : "long xuyen province an giang",
    "محايل عسير" : "mahayil asir",
    "maadi" : "subject",
    "bhatpar rani" : "bhatpar queen",
    "tekkali" : "indentation",
    "băc giang" : "bác giang",
    "campinas sp" : "campinas, sp",
    "plazoleta la cerámica ste 2 6 ave sanchez vilella esq pr 190 medical" : "pladoleta la ceráma ste 2 6 ave sanchez vilella esq pr 190 medical",
    "sase krstica 43" : "six krstica 43",
    "magé" : "maggish",
    "juiz de fora   mg" : "juiz de fora mg",
    "gautambuddh nagar" : "gautam budh nagar",
    "salvador bahia" : "salvador, bahia",
    "ul. żwirki i wigury 18" : "ul.żwirki i wigury 18",
    "verás" : "you will see",
    "sojat" : "sozat",
    "mubai" : "mumbai",
    "ilocoa norte" : "north ilocos",
    "surat gujarat india" : "indian gujarat letter",
    "itaguaí   rio de janeiro" : "itaguaí rio de janeiro",
    "bulakan" : "bulacan",
    "itiquira" : "itique",
    "มุกดาหาร" : "mukdahan",
    "dokki, giza" : "poisoning, giza",
    "simões filho" : "simão filho",
    "bharuch" : "bharruch",
    "la" : "the",
    "сандански" : "sandanski",
    "rensselaer" : "cleanser",
    "caruaru, pe" : "caruaru, on",
    "macaé   rj" : "macaé rj",
    "sderot" : "deer",
    "imbé   rs" : "imbé rs",
    "godhara" : "godhra",
    "ss129jf" : "ss 129 jaff",
    "beograd" : "white city",
    "al los laureles, el laurel mexico" : "to the laurels, the mexico laurel",
    "douala" : "nation",
    "hạ long" : "ha long",
    "1900 lukken ind dr w" : "1900 luken ind dr w",
    "sesto san giovanni   milano" : "sesto san giovanni milan",
    "valle del cauca" : "cauca's valley",
    "gravataí   rs" : "gravataí rs",
    "tubarão" : "shark",
    "aruppukottai" : "i want to be",
    "santa catarina , sao jose" : "santa catarina, sao jose",
    "peoria" : "peolia",
    "dhbai" : "diani",
    "paragominas   pa" : "paragomin pa",
    "1700 diagonal rd ste 500  commercial display" : "1700 diagonal rd ste 500  commercial display",
    "khaitan" : "khitan",
    "marabá" : "maraba",
    "nova andradina" : "new andradina",
    "av.cra 30 # 17b 25 sur  bogot? d.c" : "av.cra 30 # 17b 25 sur bogot?d.c",
    "kartarpur" : "i'm not sure",
    "via manfredonia km 2,2" : "via manfredonia km 2.2",
    "av. pref. waldemar grubba, 3000 ? vila lalau  sc" : "of.pref.waldemar grubba, 3000?vila lalau sc",
    "norte de santander" : "north of santander",
    "belo jardim" : "beautiful garden",
    "nabadwip" : "ninth",
    "حولي" : "around me",
    "29222 rancho viejo rd ste 108  healthcare" : "29222 rancho vie rd s hand 108 hair lth he",
    "dolnoslaskie" : "donnoslav",
    "arteaga y salazar 108. col. contadero, cuajimalpa cdmx mexico city" : "arteaga y salazar 108. col.contero, cuajimalpa cdmx mexico city",
    "solteiro" : "single",
    "sulaimni" : "sulaimy",
    "senador sá" : "senator sá",
    "aap" : "you",
    "rio das ostras" : "oysters river",
    "shahapur" : "shahpur",
    "sao poaulo" : "sao paulo",
    "padipiranga" : "padipirga",
    "05 520 konstancin,warszawska 21d" : "05 520 konstancin, warszawska 21d",
    "plaghar" : "palathe",
    "châu thành  trà vinh" : "chau thanh tra vinh",
    "barreras" : "barriers",
    "6600 van aalst boulevard  medical" : "6600 van aalst boulevard medical",
    "15 lê đình cẩn, bình tân" : "15 le dinh can, binh tan",
    "pekini ave.41  pekini ave.41" : "pekini ave.41 pekini ave.41",
    "vyara" : "very",
    "liberdastr. 8" : "liberdatr.8",
    "פתח תקווה" : "petah tikva",
    "santa bárbara d’ oeste" : "santa barbara d 'west",
    "chanagancherry, kottayam" : "chanaganceri, kottayam",
    "aracajú sergipe" : "aracaju - sergipe",
    "florencia" : "florence",
    "pimpalgaon baswant" : "baswant pifts",
    "otros" : "others",
    "los alamos" : "poplars",
    "bhabua" : "babua",
    "rizwankhan" : "rizwean",
    "bahia   lauro de freitas" : "bahia lauro de freitas",
    "porto algre" : "porto alegre",
    "gurupi to" : "with gurpi",
    "al obour" : "transit",
    "09 đỗ xuân hợp, bình trưng đông,q2" : "09 do xuan hop, binh trung dong, district 2",
    "mohali , punjab" : "mohali, punjab",
    "sao pedro" : "saint peter",
    "wauwatosa" : "difference from wow wow",
    "kujawsko   pomorskie" : "kujawsko pomorskie",
    "اكتوبر" : "october",
    "13405 stemmons frwy" : "13405 stemmons",
    "rio verde   goiás" : "rio verde goiás",
    "santo andré" : "saint andré",
    "changaramkulam" : "changramakulam",
    "peru   lima" : "the name is lima",
    "monterret" : "monterrey",
    "cai lậy tiền giang" : "cai lay tien giang",
    "sana'a" : "occupation",
    "linh trung, thủ đức, hồ chí minh" : "linh trung, thu duc, ho chi minh",
    "pismo beach" : "beach letter",
    "hyderadbad" : "hyderabad",
    "belagavi" : "belagogi",
    "roca sales" : "sales roca",
    "itu" : "that",
    "união da vitória   pr" : "união da vitória pr",
    "arvada" : "pivot",
    "visconde do rio branco" : "viscount of rio branco",
    "ortigueira" : "ortigue",
    "curitiba   parana" : "curitiba, parana",
    "unite kingdom" : "united kingdom",
    "kaveripakkam" : "frostbite",
    "po box 9897, el paso tx. 79995 1201 kessler" : "po box 9897, el paso tx.79995 1201 kessler",
    "khowai" : "khavai",
    "2a 4a tôn đức thắng, quận 1,  hcm" : "2a 4a ton duc thang, district 1, hcm",
    "shimoga karnataka" : "the bottom is ran",
    "praia grande  sp" : "praia grande sp",
    "saint jean de moirans" : "saint john of moirans",
    "đường số 21 bình chiểu thủ đức" : "road no. 21 binh chieu thu duc",
    "3318 se 6th ave" : "3318 to 6 th",
    "bacood sta mesa manila" : "bacood is mesa manila",
    "khoda colony" : "colonies",
    "65 trương định" : "65 truong dinh",
    "łętownia" : "warehouse",
    "marathovounou 7 latsia" : "marathow 7 latsia",
    "ustronie morskie, pow. kołobrzeski" : "ustronie morskie, areakołobrzeg",
    "nenava" : "random",
    "jastrzębia" : "hawk",
    "abigjan" : "abijan",
    "carrera 29 # 45 45 piso 15 8902130749" : "carrera 29 # 45 45 floor 15 8902130749",
    "newyork parkview apartments 22122" : "new york parkview apartments 22122",
    "kab. lumajang" : "regency.lumajang",
    "payas" : "share",
    "dabra" : "pass",
    "sanand" : "gentleman",
    "turkiye" : "turkey",
    "135 se 6th ave" : "135 to 6 th and",
    "manaus  am" : "manaus am",
    "bangunan jalex, lot 3750, lrg 2d,  kampung melayu subang" : "jalex building, lot 3750, lrg 2d, subang malay village",
    "thaltej" : "taulge",
    "niterói" : "niteroi",
    "modasa" : "outfit",
    "ekangarsarai nalanda bihar" : "ekangarasarai nalanda bihar",
    "新莊區" : "xinzhuang district",
    "ferozepur" : "firozpur",
    "belém do para" : "belém do pará",
    "lộc hòa" : "loc hoa",
    "san josé del cabo" : "san jose del cabo",
    "jid ali" : "good for me",
    "paraiba   patos" : "paraiba patos",
    "geuteng" : "headal",
    "xalle 71d#67 85" : "helle 71d # 67 85",
    "السعوديه" : "saudi arabia",
    "nyiregyhaza" : "corbygougy",
    "biaora" : "baiaora",
    "vize" : "visa",
    "katra" : "every",
    "ive andrica 1, 71250 kiseljak sarajevo" : "this is andrew ೧, ೭೧೨೫೦ kiseljak sare jeevano",
    "cần thơ" : "can tho",
    "joinvile" : "joinville",
    "colombia   tunja" : "colombia tunja",
    "marhaura" : "madhaura",
    "xangri la   rs" : "xangri la rs",
    "pe   serra talhada" : "pe serra talhada",
    "sivasagar" : "sivasagar is the best",
    "dolnyśląsk" : "lower silesia",
    "prbtsar jawla" : "with a tour of a tour",
    "thủ đức, tp hồ chí min" : "thu duc, ho chi minh city",
    "nghệ an" : "nghe an",
    "edo de mex" : "come in edo x",
    "carr 3 oste # 42 45" : "car 3 oste # 42 45",
    "300 las vistas shopping village ave. felisa rincón ste.31" : "300 the views shopping village ave.felisa rincón ste.31",
    "רמת גן" : "ramat gan",
    "gunj basoda" : "ganj basoda",
    "chennau" : "lichen",
    "venda do pinheiro" : "pinheiro sale",
    "lavalle 780" : "on stage 780",
    "fortaleza  ce" : "fortaleza what",
    "ibadan, oyo state" : "ibadan, that state",
    "quận 9" : "district 9",
    "żółwin" : "ywin",
    "บุรีรัมย์" : "buriram",
    "tepeji del rio de ocampo hidalgo" : "tepeji of the river of ocampo hydalgo",
    "abudhabi city" : "abu dhabi city",
    "asdf" : "seashell",
    "lima, lima" : "five, five",
    "bhte minas gerais" : "flowing minas geris",
    "số 04 đường d9, quận tân phú" : "no. 04 d9 street, tan phu district",
    "ashgabat" : "the underum",
    "città della pieve" : "city of the pieve",
    "guadalajara, jal." : "guadlajara, water.",
    "palghat" : "palaghat",
    "tarakeswar" : "tartar",
    "minas gerais   japaraiba" : "minas gerais japaraiba",
    "patos de minas  mg" : "patos de minas mg",
    "khardaha" : "the lines",
    "θηβων 500" : "thebes 500",
    "kanuta" : "kaluuta",
    "dwsa" : "get",
    "mazowieckie warszawa" : "masovian voivodeship warsaw",
    "goias   goiânia" : "goias goiânia",
    "الاقصر" : "the shortest",
    "bangana" : "they are equal",
    "palghar  boisar  mumbai" : "palghar boisar mumbai",
    "gudemaranhalli" : "gudemaranahalli",
    "akola" : "did",
    "594 howard stsan franciscoca94105" : "594 howard st san francisco ca 94105",
    "kashan" : "is the",
    "ấp chợ tập sơn trà cú trà vinh" : "tap son tra cu market hamlet tra vinh",
    "jawa barat" : "west java",
    "edo.mex" : "edo.x",
    "khanouri" : "jerk",
    "lagarto" : "lizard",
    "aquiraz   ceará" : "aquiraz ceará",
    "rosà" : "rose",
    "gridih" : "greedih",
    "montes claros" : "clear montes",
    "al ain" : "where is",
    "khartom" : "khartum",
    "rj   macaé" : "rj macaé",
    "rua j?natas serrano, 182   casa 2   jardim castro alves" : "rua j? natas serrano, 182 house 2 jardim castro alves",
    "929 gessner  healthcare" : "929 gessner healthcare",
    "السنطه" : "eagle",
    "14 đường số 5, đặng văn bi thủ đức" : "14 street no. 5, dang van bi thu duc",
    "plachkovski manastir 7" : "plachkovsky monastery will",
    "santa lucia" : "saint lucia",
    "κ.,κοντου 2, αθηνα, 11255" : "k., kontou 2, athens, 11255",
    "rio de janeiro, rj" : "rio de janeiro - rj",
    "madhopara ,purnia" : "madhopra, puraniya",
    "ciudad apodaca" : "apodaca city",
    "mato grosso do sul   campo grande" : "mato grosso do sul campo grande",
    "kakching" : "i'm not sure",
    "kujawsko pomorskie,nowe" : "kujawsko pomorskie, new",
    "10 nguyễn duy, p12, q8" : "10 nguyen duy, p12, district 8",
    "tirunelveli, tamil nadu 627007" : "tirunelveli, tamil nadu ౬౨౭౦౦౭",
    "rajawali timur no 18 jawa barat" : "rajawali timur no 18 west java",
    "الباحة" : "courtyard",
    "hunsgi" : "hazard",
    "dehradoon" : "dehradun",
    "cd mex" : "cdmx",
    "st. michael im lungau" : "st.michael im lungau",
    "скопје" : "skopje",
    "rasalkhaima" : "ax",
    "mazowieckie" : "masovian",
    "الباجور منوفيه" : "bagor menoufia",
    "山东威海" : "shandong weihai",
    "bolangir" : "collarew",
    "pernambuco   paulista" : "pernambuco paulista",
    "warmińsko mazyrskie" : "warmian maozyrskie",
    "thủ đức, tp.hcm" : "thu duc, ho chi minh city",
    "maceió al" : "macea al",
    "kanhangad" : "canneman",
    "mounofia" : "munophy",
    "ghaziabaad" : "gypsy",
    "monte hermoso" : "monte beautiful",
    "warszawa" : "warsaw",
    "adamantina" : "adamantine",
    "bankura" : "to the bank",
    "santa amrta" : "santa marta",
    "русе" : "ruse",
    "tp thủ đức, tp.hcm" : "thu duc city, ho chi minh city",
    "catalão" : "catalan",
    "premia de mar" : "sea award",
    "dar es salaam tanzania" : "give is salaam tanzania",
    "bhavangar" : "bhavnagar",
    "kakkanad, kochi" : "his excellency d, here",
    "yopal   casanare" : "yopal casanare",
    "bueno aires" : "well airs",
    "ropar" : "call out",
    "herriman" : "faculation",
    "los banos, laguna" : "bans, laguna",
    "khamaria" : "vintage",
    "belait" : "belite",
    "sorriso mt" : "smile mt",
    "srikalahasti" : "sikalahas",
    "bắc giang" : "bac giang",
    "vizianagaram" : "vizayanagaram",
    "cebu,lapu lapu city" : "cebu, lap lap city",
    "curitiba paraná" : "curitiba, parana",
    "jk raod karande mala darewadi krishna house" : "czech rawd kandande mala darevadi krishna house",
    "timarni" : "timar",
    "dolnośłąskie" : "lower silesian",
    "vũng tàu" : "vung tau",
    "la altagracia" : "altagracia",
    "irinjalakuda" : "toolbar",
    "hà nội" : "hanoi",
    "uberaba  mg" : "uberaba mg",
    "ghatal" : "gossip",
    "las palmeras" : "the palm trees",
    "saint paul" : "st.paul",
    "presidente prudente" : "prudent president",
    "sarhochiya" : "sarashia",
    "via a. manzoni" : "away a.manzoni",
    "hạ long quảng ninh" : "ha long quang ninh",
    "sabana perdida" : "lost savanna",
    "москва" : "moscow",
    "libano" : "lebanon",
    "pudukottai" : "i want to be in the prefecture",
    "واسط" : "interface",
    "iguala" : "equal to",
    "región metropolitana de santiago" : "santiago metropolitan region",
    "al khawaneej" : "al khwani",
    "mpumalanga,emalahleni" : "mpumalanga, emalahleni",
    "76 vĩnh hưng" : "76 vinh hung",
    "taubaté   sp" : "taubaté sp",
    "koduvai, tiruppur" : "koduwai, tiruppur",
    "rua comandante costa 3021  cuiab?   mt 3021" : "rua commante costa 3021 cuiab?mt 3021",
    "via 4 novembre n^95" : "via 4 november n^95",
    "manaus  amazonas" : "manaus, amazonas",
    "núcleo bandeirante" : "bandeirante core",
    "القبة" : "dome",
    "தூத்துக்குடி" : "throat",
    "estado de mexico" : "mexico state",
    "nosara" : "and sarah",
    "arlington, ny 12603" : "arlington, 12603",
    "sangotedo" : "three words",
    "kirayan norte miagao ,iloilo" : "kirayan norte miagao ,ilolo",
    "rio verde" : "green river",
    "são luís maranhão" : "sao luis maranhão",
    "ปทุมธานี" : "pathum thani",
    "porto velho, rondônia" : "velho porto, rondônia",
    "364 dương quản hàm" : "364 duong quang ham",
    "tambo grande" : "big tambo",
    "cao lãnh , đồng tháp" : "cao lanh, dong thap",
    "8 phan đình giót, p2, tb" : "8 phan dinh giot, p2, tb",
    "salzburger straße 777" : "salzburger strasse 777",
    "6135 nw 167th st, ste e24 ste e24 cruise" : "6135 nw 167th st, you are e24 you e24 cruise",
    "livramento" : "liberation",
    "malopolskie" : "lesopolskie",
    "şırnak" : "sirnak",
    "taboão da serra   sp" : "taboão da serra sp",
    "台灣台北市" : "taipei city, taiwan",
    "ботевград" : "botevgrad",
    "mehgaon" : "ish",
    "opelousas" : "operator",
    "359 hồ học lãm" : "359 ho hoc lam",
    "sarni" : "sar",
    "guaiba" : "be good",
    "buga" : "print",
    "buenos aires   tigre" : "buenos aires tigre",
    "bangalorr" : "bangalirr",
    "barrio viejo de callosa de segura" : "viejo de callosa de segura",
    "mahemdavad" : "mahmadism",
    "kannad" : "heels",
    "chotomów" : "chotom",
    "salador" : "salter",
    "bagneux" : "prickly",
    "králův dvůr levín" : "the king's court",
    "campo largo pr" : "largo pitch pr",
    "nakhal" : "disgrace",
    "afua" : "originated",
    "purnea" : "poorna",
    "ciudad juarez" : "juarez city",
    "la calorta" : "the carat",
    "ballajura" : "dancing it",
    "calo" : "decline",
    "tirunelveli" : "tirunelevel",
    "rondonópolis   mt" : "rondonópolis mt",
    "612 hangar ln ste. 223, nashville, tn 37217" : "612 hangar ln you are.223, nashville, tn 37217",
    "república dominicana" : "dominican republic",
    "divino" : "divine",
    "poções" : "potions",
    "recife pe" : "recife on",
    "san luis potosí" : "san luis potosi",
    "cuneo" : "wedge",
    "via castelgomberto 96 italia" : "via castelgomberto 96 italy",
    "châu đốc" : "continental director",
    "bhubanerwar" : "bhubnerwar",
    "新北市" : "new taipei city",
    "méxico" : "mexico",
    "via a. manzoni 212" : "away a.manzoni 212",
    "vila velha" : "old village",
    "ayias zonis 60 c limassol" : "of this zone 60 c limassol",
    "thiruppttur" : "tirupur",
    "terrassa barcelona" : "barcelona terrace",
    "offlaga" : "offto",
    "ras al khaima" : "ras al khaimah",
    "guadalajara jalisco méxico" : "guadalajara jalisco mexico",
    "raiadh" : "records",
    "itaberai" : "portion of the board",
    "الرستاق" : "rust",
    "hyde rabad" : "hyderabad",
    "s.d. de rana" : "s.d.of wound",
    "presidente bernardes" : "president bernardes",
    "varca" : "gap",
    "ποσειδωνος 58" : "poseidon 58",
    "aachen   alemanha" : "aachen germany",
    "londrina, pr." : "laundrina, on.",
    "bucaramaga" : "busramga",
    "572, navamin90, klongkum," : "572, navamin 90, kangkum,",
    "gurgaoon" : "gurgaon",
    "michigan" : "road cancer",
    "senador canedo goiás" : "senator canedo goiás",
    "6 ballela road" : "6 read the road",
    "argao cebu city" : "a cream ce is not city",
    "banaglore" : "bangalore",
    "suratgarh" : "photograph",
    "tierra del fuego" : "land of fire",
    "dar es salaam" : "s salam house",
    "zaječar" : "zajecar",
    "bhind" : "bind",
    "deesa" : "goddess",
    "14 đường số 17, p. tân thuận tây" : "14 street no. 17, p.tan thuan tay",
    "imperatriz" : "empress",
    "chikkaballapura" : "chikkallapura",
    "100 first stsan franciscoca94015" : "100 first st san francisco ca 94015",
    "المنوفية" : "menoufia",
    "satana" : "satan",
    "bella vista" : "beautiful view",
    "beit yanai" : "hate love",
    "via lombardia n.8" : "via lombardia ranked # 8",
    "salto sp" : "heel sp",
    "kalaphar" : "black mountain",
    "vijayapur" : "vijaypur",
    "bangalorej" : "bungalore",
    "erith" : "eager",
    "upper pueryo cdo" : "upper puerto cdo",
    "datia (m.p.)" : "data (m.p.)",
    "savadatti" : "conscience",
    "tân đông hiệp, dĩ an, bình dương" : "tan dong hiep, di an, binh duong",
    "chapadinha" : "hat",
    "minas gerais  uberlandia" : "minas gerais uberlandia",
    "negros occidental" : "western blacks",
    "zaragoza, nueva ecija" : "zaragoza, new ecija",
    "eastren area" : "eastern area",
    "merida" : "mary",
    "nawada" : "the rest",
    "338 nguyen van giap ha noi" : "338 nguyen van giap has us",
    "pandikkad" : "bangy wall",
    "رابغ" : "rib",
    "कलकता" : "cadre",
    "dekalb" : "decalb",
    "ηρωων πολυτεχνειου 74" : "heroes people 74",
    "itaú de minas   minas gerais" : "itaú de minas minas gerais",
    "khu phố 1, tân hiệp, biên hòa" : "quarter 1, tan hiep, bien hoa",
    "смедерево" : "smederevo",
    "coxim" : "cushion",
    "sultanpur up" : "sultanpur sub",
    "enna" : "what",
    "novo gama" : "new range",
    "תל אביב" : "tel aviv",
    "kottrakara, kollam" : "kollam, kollam",
    "reibeirão preto sp" : "ribeirão preto sp",
    "bulandhshar" : "bulandshahr",
    "perambalur" : "pounding",
    "asandh" : "blessed",
    "linh trung, tp thủ đức, tp hcm" : "linh trung, thu duc city, ho chi minh city",
    "al khobar" : "the news",
    "jaen, nueva ecija" : "jaen, new ecija",
    "الرحبة" : "splendor",
    "talala gir" : "talala provides",
    "srinsgar" : "srins",
    "dorangla" : "dagang",
    "tihu" : "quiet",
    "elmsford" : "elmford",
    "sidvokodvo" : "sidvoko",
    "ipaba" : "tain",
    "bordon" : "shore",
    "nizgny novgorod" : "nizhny novgorod",
    "la union" : "union",
    "corrientes capital   corrientes" : "corrientes corrientes",
    "kadıköy" : "kadikoy",
    "sangat" : "very",
    "quezon coty" : "quezon city",
    "curitiba, pr" : "karitiba, but",
    "são luís  ma" : "são luís ma",
    "strada provinciale.le sud 101" : "provincial road. south 101",
    "tema" : "her",
    "nairbi" : "nai rujie",
    "hurgada" : "hurghada",
    "dehra du" : "dehra you",
    "igrejinha rs" : "church rs",
    "mettur" : "mector",
    "progreso, yucatán" : "progress, yucatan",
    "santiago, region metropolitana" : "santiago, metropolitan region",
    "hazaribag" : "hazaribagh",
    "shriwardhan" : "scriwarray",
    "pindaré mirim   maranhão" : "pindaré mirim maranhão",
    "via galatina,187   sant'angelo" : "via galatina, 187 sant'angelo",
    "shenzen" : "before shen",
    "גבעתיים" : "givatayim",
    "boituva  sp" : "boituva sp",
    "uberaba   mg" : "uberaba mg",
    "uberlandia   mg" : "uberlandia mg",
    "boirani" : "boyla",
    "canoas" : "canoes",
    "3era seccion lomas verdes" : "3rd lomas verdes section",
    "ak" : "and",
    "buttlar" : "bitch",
    "la cisterna" : "the cistern",
    "sangao" : "three high",
    "mahopac" : "mahoapc",
    "borca" : "a fighter",
    "mazowiecki" : "mazovian",
    "上海" : "shanghai",
    "เมือง" : "city",
    "paschim medinipur" : "west midnapore",
    "via s. anguissola 1" : "way's.anguissola 1",
    "4221 w john carpenter fwy" : "4221 w john carpenter more",
    "mandya" : "cloak",
    "braço do norte" : "northern arm",
    "markapur" : "marakpur",
    "ninavah" : "ninveh",
    "arapongas pr" : "on arpongas",
    "vitória do mearim" : "mearim victory",
    "cancun" : "residual",
    "sheakh zayed" : "the necks drop",
    "الشارقه" : "shark",
    "ilinois" : "illinois",
    "arame" : "wire",
    "kí  túc xá khu b đại học quốc gia" : "dormitory of national university",
    "türki̇ye" : "the turkistan use",
    "ראשון לציון" : "rishon lezion",
    "er" : "is",
    "dharamavaram" : "dharmavaram",
    "6 hẻm số 5 mễ cốc" : "6 alley no. 5 me coc",
    "3 bethesda metro ctr ste 700" : "x better metro str in 700",
    "las piñas" : "the pineapples",
    "bhilwara bapunagar kerali suitings.." : "bhilwara bapunagar kerli suitings ..",
    "praha 9" : "prague 9",
    "quận 3" : "district 3",
    "4880 stevens creek blvdsan joseca95129" : "4880 stevens creek blvd san jose ca 95129",
    "ulwe" : "fight",
    "dos hermanas" : "two sisters",
    "ciudad del carmen" : "carmen city",
    "berhampur" : "delay",
    "shahdol" : "shadol",
    "barauna rn" : "maruna rn",
    "петрич" : "petrich",
    "נתניה" : "netanya",
    "tata hospital actrec  sector 22 kharghar" : "tata hospital actrec seater 22 kharghar",
    "29 nguyễn t minh khai đà nẵng" : "29 nguyen t minh khai da nang",
    "omalur" : "alum",
    "arandu" : "wise",
    "ouagadougou" : "ungaduer",
    "62b kp3 an lạc a bình tân tphcm" : "62b kp3 an lac a binh tan hcmc",
    "extremos" : "extremes",
    "honduras,tegucigalpa" : "honduras, tegucigalpa",
    "marilia  sp" : "marilia sp",
    "los banos" : "the bathrooms",
    "al nakheel, ras al khaimah" : "al nakheel, ras al tent",
    "cheshire" : "chill",
    "hadrmout seyun" : "hadrmout pregnant",
    "itapecerica da serra   sp" : "itapecerica da serra sp",
    "500 chesterfield ctr, ste 355 ste 355" : "500 chesterfield ctr, you are 355 you are 355",
    "pasay city , metro manila" : "pasay city, metro manila",
    "ktx đại học quốc gia khu b" : "national university dormitory area b",
    "العبور" : "transit",
    "saint mitre les remparts" : "saint miter the ramparts",
    "meerut uttar pradesh" : "meerut uttor pradesh",
    "tradate" : "trades",
    "13 kim đồng giáp bát" : "13 kim dong giap bat",
    "aska" : "ash",
    "kanasin" : "cassin",
    "via roma  20" : "via roma 20",
    "corso s. ignazio     88" : "course s.ignazio 88",
    "via degli ippocastani,1" : "via degli ippocastani, 1",
    "678 , chua chu kang cresc #11 622" : "678 , that's the fire cresc #11 622",
    "yverdon les bains" : "yverdon les bains",
    "davao del norte" : "north davao",
    "são sebastião" : "san sebastian",
    "cali  valle" : "valley",
    "शादी" : "marriage",
    "jaru" : "spring",
    "campina grande  pb" : "campina grande pb",
    "מבשרת ציון" : "mevaseret zion",
    "trần đại nghĩa bình tân" : "tran dai nghia binh tan",
    "suite 22, no.4 ajala street (alhaji opeyemi shopping complex), off ikotun idimu road, subol bus stop" : "shuhunte ౨౨, no.౪ alaji street",
    "تبوك" : "tabuk",
    "นครราชสีมา" : "nakhon ratchasima",
    "thiruvalla (tiruvalla)" : "turna (turn)",
    "presidente kennedy" : "president kennedy",
    "jaleswar" : "gap",
    "umm al quain" : "umm al quin",
    "tanajib provence" : "tajiib provence",
    "929 gessner drive  healthcare" : "929 gessner drive healthcare",
    "ariccia" : "habit",
    "goiânia goiás" : "goiania goias",
    "cene" : "prices",
    "prayagraj" : "praisagraj",
    "recifes" : "reef",
    "bollampally veldanda nagarkarnool telangana" : "bollampalay weldand nagarakarnul telangana",
    "shirur" : "shiroor",
    "lac du flambeau" : "flambeau lake",
    "digos city, davao del sur" : "digos city, south davao",
    "chomu" : "why",
    "rewa mp" : "les mp",
    "para abaetetuba" : "to abaetetuba",
    "rio grande do norte" : "large northern river",
    "2777 n stemmons fwy  cloud it" : "2777 n stemmons more cloud it",
    "saihat" : "health",
    "via  tenuta del cavaliere,1" : "via tenuta del cavaliere, 1",
    "metromanila" : "metro manila",
    "tyerl" : "terlle",
    "thoothukudi" : "throat",
    "palams" : "of the palms",
    "chengalpattu" : "excitement",
    "ciudad de panama [panama city]" : "city of panama [paama city]",
    "konopiska" : "hemp",
    "belo orizonte" : "belo horizonte",
    "pargi" : "park",
    "المنيا" : "minya",
    "sta catarina   florianopolis" : "sta catarina florianopolis",
    "são paulo, são bernardo do campo" : "sao paulo, sao bernardo do campo",
    "rinconada del agua 2811 jalisco" : "water rinconada 2811 jalisco",
    "para de minas" : "for mines",
    "via del lavoro artigiano 10" : "via del lavoro craftsman 10",
    "rigadh" : "rigade",
    "gravataí" : "tie",
    "herbertpu, dehradun" : "herbertpur, dehradun",
    "台北市松山區" : "songshan district, taipei city",
    "kapaa" : "softly",
    "buin" : "done",
    "nalgonda" : "phid",
    "ddun" : "tiny",
    "banagalore" : "bangalore",
    "μεσογειων 85" : "mediterranean 85",
    "bogota, colombia." : "bogota colombia.",
    "long vĩnh, gò công tây, tiền giang" : "long vinh, go cong tay, tien giang",
    "rawatbhata" : "his salaries",
    "nueva vizcaya" : "new biscay",
    "tp. hồ chí minh" : "tp.ho chi minh",
    "saudi arabia  riyadh" : "saudi arabia riyadh",
    "bombinhas" : "bombs",
    "puneou" : "puneoula",
    "hueypoxtla" : "caridary",
    "c.da cataniello,12" : "c.da cataniello, 12",
    "viamão   rs" : "viamão rs",
    "koto ku, tokyo" : "ward and residence",
    "calle 52 no. 22   44 bogota" : "52 street no.22 44 bogota",
    "santa fe  rosario" : "santa fe rosario",
    "chakan" : "check",
    "união dos palmares" : "palmares union",
    "8 phan đình giót" : "8 phan dinh giot",
    "ghazipur up" : "ghazipur sub",
    "imbituba sc" : "imbitaba sc",
    "جمصه" : "jamsa",
    "adoni" : "adon",
    "villavicencio, meta" : "villavicencio, goal",
    "pouso redondo" : "round landing",
    "kharepatan" : "weapon",
    "são leopoldo" : "sao leopoldo",
    "visakhapatnm" : "visakhapatnam",
    "cabo frio" : "cabio cabio",
    "vinhedo   são paulo   brasil" : "vineyard são paulo brazil",
    "jagtial" : "hunting",
    "kishinev" : "chisinau",
    "r. catarina eufemia" : "r.catarina eufemia",
    "jabalpu" : "jabalpur",
    "dianopolis" : "intellectual",
    "tiruttani" : "stream",
    "bhubaneswer" : "showerwar",
    "surat kamrej" : "kamrej's letter",
    "karaikkudi" : "karaikudi",
    "edo. de mexico" : "it's from.mexico",
    "são paulo sorocaba" : "sao paulo sorocaba",
    "جده" : "grandmother",
    "demuro, 11" : "room, 11",
    "อุบลราชธานี" : "ubon ratchathani",
    "taiwan , taipei" : "taiwan, taipei",
    "curico" : "curic",
    "woolloomooloo" : "woolboyo",
    "khóm 1 thị trấn càng long" : "pineapple 1 town",
    "kalpetta" : "pale",
    "miraflores   lima" : "miraflores lima",
    "варна" : "varna",
    "jesi" : "you are",
    "ký túc xá khu b" : "dormitory area b",
    "paracatu   mg" : "paracatu mg",
    "jakarta timur, dki jakarta" : "east jakarta, dki jakarta",
    "i̇stanbul (avr)" : "istanbul (avr)",
    "londonamie" : "londonama",
    "tiruvallur district" : "tiruvalur district",
    "nagerkovil" : "nagerkov",
    "miscat" : "moved",
    "гр. софия, област софи" : "sofia, sophie district",
    "jawa timur" : "east java",
    "new delh" : "new delhi",
    "bastos" : "bastly",
    "bjn" : "they make",
    "mazatlán" : "mazatlan",
    "macae" : "macaaye",
    "puñe" : "dagger",
    "costa  mesa" : "costa mesa",
    "أكتوبر" : "october",
    "hòa an" : "hoa an",
    "36, 31 465 kraków" : "36, 31 465 krakow",
    "197 đường 9 phước bình" : "197 street 9 phuoc binh",
    "kharagpur" : "khargpur",
    "cl. 6 sur no. 79 150" : "cl.6 south no.79 150",
    "del mar" : "from sea",
    "puerto colombia" : "colombian port",
    "10,rue 2,lotissement abdelmoumen n 6 immeuble hajjar" : "10, rue 2, abdelmoumen subdivision n 6 hajjar building",
    "الفجيرة" : "fujairah",
    "שוהם" : "onyx",
    "raebareli" : "rae bareli",
    ". mancherial.... telangana" : ".manancherl .... telangana",
    "kopargaon" : "copargaon",
    "tiangua ce" : "hexagram test",
    "bahia   presidente tancredo neves" : "bahia presidente tancredo neves",
    "minhas gerais   araxá" : "my general araxá",
    "la libertad" : "freedom",
    "paraná,  assis chateaubriand" : "paraná, assis chateaubriand",
    "teutônia" : "teutonia",
    "riyadh   central" : "riyadh central",
    "2040 buda?rs, vas?t utca 9." : "2040 buda? rs, vas? t street 9.",
    "kí túc xá khu a" : "dormitory a",
    "বাদসাআনারুললস্কয" : "bang",
    "shahjhanpur" : "shahjahanpur",
    "kuna" : "there are",
    "cancún benito juárez" : "cancun benito juárez",
    "بغداد ." : "baghdad .",
    "65 đường 14" : "65 street 14",
    "anjuna" : "attempt",
    "la verne" : "verne",
    "santa tecla" : "holy button",
    "arni" : "on",
    "đồng xoài bình phước" : "dong xoai binh phuoc",
    "pedra preta" : "black stone",
    "navegantes" : "navigators",
    "haute savoie" : "haute-savoie",
    "shahpura (jaipur dist.)" : "shahpura (jaipur d from)",
    "souhag   maragha city" : "sohag maragha city",
    "1935 silverton rd ne" : "1935 silverton rd no",
    "balanga city" : "they advertise the city",
    "flores da cunha" : "cunha flowers",
    "borgata lavino, 32a" : "borgata developed, 32a",
    "uông bí" : "uong bi",
    "harnatand" : "haranatand",
    "guadalupe nuevo  leon" : "guadalupe nuevo leon",
    "via g. pascoli 39" : "via g.pascoli 39",
    "río de janeiro" : "rio de janeiro",
    "tupã" : "bed",
    "ofibodegas perisur no 8 frente escuela international" : "ofibodegas perisur no 8 front international school",
    "nova crixas" : "new crisles",
    "elsherouk" : "sunrise",
    "via uscioli,114 nocera superiore" : "via usgioli, 114 nocera superiore",
    "p.o. box 411, baysh" : "p.o.box 1 c, age",
    "dzs" : "cbs",
    "kosamba" : "praying",
    "registro" : "record",
    "shimoga" : "below",
    "são joão da boa vista   sp" : "são joão da boa vista sp",
    "francesco laparelli" : "francesco laparchelli",
    "san luis" : "saint louis",
    "santa barbara d’oeste" : "santa barbara d'oeste",
    "vicolo della stufa 14," : "stove 14 alley,",
    "espírito santo" : "holy spirit",
    "مدينه الخبر" : "khobar",
    "barangay alupay rosario batangas" : "barangay alupuds rosario batangas",
    "belém" : "bethlehem",
    "santo ângelo   rs" : "santo ângelo rs",
    "tirupathi" : "tirupati",
    "siddapur" : "sidpur",
    "udarsi" : "generous",
    "tân hiệp, trà cú, trà vinh" : "tan hiep, tra cu, tra vinh",
    "priyanka jhavar" : "priyanka jhawar",
    "52cc khu phố 2 phường tân quy q7" : "52cc quarter 2 tan quy ward district 7",
    "jamkhandi" : "jokhandi",
    "itsitio" : "self -sufficiency",
    "số 2 ngõ 6" : "no. 2 lane 6",
    "kasauli" : "cosmus",
    "pato branco   paraná" : "pato branco paraná",
    "algérie" : "algeria",
    "via dell'informatica 10   37036 san martino buon albergo (veneto), italy" : "via dell'oformatica 10 37036 san martino buon albergo (veneto), italy",
    "brasília" : "brasilia",
    "argyll" : "arch",
    "vitória da conquista." : "victory of the conquest.",
    "ciudad de méxico" : "mexico city",
    "rayachoty" : "rage",
    "ribeião pretp   sp" : "ribeirão preto sp",
    "ras al khaimah, uae" : "the head of the tent, a",
    "mriehel" : "twist",
    "khattar ,navi mumbai" : "khattar, navi mumbai",
    "poá" : "dust",
    "dharmshala (dharamshala)" : "dharamshala (dharamshala)",
    "400 s el camino real  cloud it" : "400 s the royal cloud it",
    "cainta" : "santa",
    "cuernavaca morelos" : "cuernavaca, morelos",
    "can.cun quintanaroo" : "can.cuni quintanaroo",
    "kampli" : "kamplipl",
    "fortaleza   ceará" : "fortaleza, ceara",
    "región" : "region",
    "wielkopolskiej" : "greater poland",
    "sao paulo" : "são paulo",
    "fort belvoir" : "very belvoir",
    "al gudaibiys" : "the jadbis",
    "nahan" : "retain",
    "cancún quintana roo." : "cancun quintana roo.",
    "anápolis   goiás" : "anápolis goiás",
    "chau đốc" : "chau doc",
    "bahia   camaçari" : "bahia camaçari",
    "del" : "of the",
    "omerga" : "to the ommer",
    "raigarh" : "raigad",
    "banediya" : "banadia",
    "pápa" : "pope",
    "yopal  casanare" : "yopal casanare",
    "an de klump 8 47661 issum" : "and the clump 8 47661 issum",
    "doda" : "he adds",
    "cd. lázaro cárdenas" : "cd.cárdenian lazarus",
    "kalinzar" : "calinzar",
    "उदयपुर" : "udaipur",
    "nunamara" : "outer",
    "rusterburg" : "rustenburg",
    "concórdia" : "concord",
    "vereeniging" : "association",
    "deoria" : "rear",
    "katargam" : "abolition",
    "القاهرة‬" : "cairo",
    "general villamil playas" : "general villamil beaches",
    "kendrapara" : "centerpara",
    "alejandro schroeder 6407 uruguay" : "aljandro schroeder 6407 uruguay",
    "المدينه المنورة" : "al madinah al munawwarah",
    "chodecz" : "pedestrian",
    "beed" : "egg",
    "mata roma" : "mata rome",
    "sahibabad, ghaziabad" : "halabad, gas",
    "dolnoślaskie" : "lower silesian",
    "aliso viejo" : "old alder",
    "nova igualu" : "new iguac",
    "tierra blanca" : "white land",
    "518 dungan street" : "518 season street",
    "kavali" : "want to be",
    "nueva ecija" : "nue's ecija",
    "panamá" : "panama",
    "mihijam" : "michiam",
    "sóc trăng" : "soc trang",
    "naucalpan, estado de méxico" : "naucalpan, state of mexico",
    "bình định" : "pacify",
    "fatehnagar distt udaipur" : "fatehanagar distt udipur",
    "amsterdã" : "amsterdam",
    "bolu" : "divided",
    "barranquilla colombia" : "barranquilla, colombia",
    "feijó" : "bee",
    "palwal" : "palval",
    "mexicaltzingo 2352 a y b, col. arcos, guadalajara, jalisco. jalisco" : "mexicantzingo 2352 a and b, col.arcos, guadalajara, jalisco.jalisco",
    "ينبع البحر" : "sea's origin",
    "hẻm 13 . đường sơn thông . k10. p9" : "alley 13.son thong street.k10.p9",
    "jacundá" : "jacket",
    "santa ana 2 mza 24 casa 5" : "santa ana 2 mza 24 house 5",
    "vasai virar" : "slap",
    "são josé do rio preto" : "sao jose do rio preto",
    "siko" : "elbow",
    "são paulo   jundiai" : "sao paulo jundiai",
    "subkowy" : "subdural",
    "islas de la bahia" : "bay islands",
    "caviye" : "poems",
    "קרית מוצקין" : "kiryat motzkin",
    "muntinlupa ciy" : "muntinlupa city",
    "12 a âu cơ , q.tân phú" : "12 a au co, tan phu district",
    "sirhind" : "sirlonging",
    "pasto" : "meal",
    "jaraguá do sul" : "south jaragua",
    "emiliano zapata, ver" : "emiliano zapata, see",
    "caçapava" : "hunt",
    "karawang, west java" : "karang, to go to west",
    "chehalis" : "shalis",
    "warminister" : "warminster",
    "via gregorio ricci curbastro, 29" : "via gregorio ricci curbaster, 29",
    "mundo novo" : "new world",
    "bắc ninh" : "bac ninh",
    "warminsko mazurskie" : "warminsko masurian voivodeship",
    "paraiba" : "paraíba",
    "via garibaldi,9" : "via garibaldi, 9",
    "zagazig" : "titillate",
    "são carlos,sp" : "são carlos, sp",
    "zalec" : "heal",
    "tarwara" : "we are sick",
    "tiruppur" : "twell",
    "lunawada" : "avada",
    "tanda" : "mark",
    "гр. хасково" : "haskovo",
    "jaragua do sul" : "south jaragua",
    "luzern" : "lucerne",
    "fortaleza ceara" : "fortaleza, ceara",
    "k̤o̤l̤k̤a̤t̤a̤" : "k̤oul̤k̤a̤̤̤a",
    "avenida independencia 1297" : "independence avenue 1297",
    "بابل" : "babylon",
    "tân an càng long trà vinh" : "tan an more long tra vinh",
    "udupu" : "stump",
    "madhapur" : "madapur",
    "melgar  tolima" : "melgar tolima",
    "1530 kurunji strt" : "1533",
    "vitoria espírito santo" : "vitoria, espirito santo",
    "espumoso" : "sparkling",
    "plewiska" : "plew",
    "alto taquari" : "high taquari",
    "ceara" : "wax",
    "darsi" : "give up",
    "katihar" : "mytihar",
    "king fahd rd, al olaya,  riyadh 11564, saudi arabia" : "king fahd red, alia, riad 11564, saudi arabia",
    "الحمام" : "the bathroom",
    "alkhobor" : "alhobor",
    "jauru" : "joru",
    "tantipara .barahnagar" : "tantipada.",
    "al wazarat" : "the ministry",
    "puerto deseado   santa cruz" : "puerto deseado santa cruz",
    "lajeado rs" : "laxed rs",
    "ahmed rajabli baku" : "ahmed rajal bey",
    "tambau" : "to sleep",
    "البصره" : "vision",
    "kumbazha" : "digel",
    "1820 chotti bajaria garha" : "1820 chitti bazaria gadha",
    "florianopilis" : "florianopilic",
    "940 redna terrace" : "940 regular terrace",
    "crtagena" : "cartagena",
    "sumidouro" : "sink",
    "dharmapuri" : "dharmapurai",
    "pinheiros" : "pine trees",
    "el nido, palawan" : "the nest, palawan",
    "bengaluru urban" : "bangalore urban",
    "vadodara   390002" : "donarasa 390002",
    "roxana" : "roxane",
    "sao bernardo do campo" : "são bernardo do campo",
    "shivamogga" : "shimoga",
    "san juan. capital" : "san juan.capital",
    "patna , kankarbagh, bihar" : "patna, kankarbagh, bihar",
    "topeka" : "stopy",
    "providencia" : "providence",
    "bhende sk" : "bhende ske",
    "wai" : "water",
    "barranca bermeja" : "barrancabermeja",
    "31 tân trụ, phường 15, q. tân bình" : "31 tan tru, ward 15, district.tan binh",
    "surco" : "groove",
    "não tem" : "it does not have",
    "istanbull" : "istanbul",
    "l'hay les roses" : "l'ho les roses",
    "vĩnh long" : "vinh long",
    "tp thủ  đức , tp hcm" : "thu duc city, ho chi minh city",
    "alvorada do sul" : "southern dawn",
    "pulivendula" : "i'm cleaning up",
    "pimpalner" : "pimpner",
    "hòa bình" : "peace",
    "đồng nai" : "dong nai",
    "warzno" : "brew",
    "rs uruguaiana" : "uruguaiana hospital",
    "siam piwat" : "ciam",
    "bình thuận" : "binh thuan",
    "rio brilhante   ms" : "brilliant river ms",
    "phường tân kiểng, quận 7, tp.hcm" : "tan kieng ward, district 7, ho chi minh city",
    "puerto vallarts" : "puerto vallarta",
    "rio de janeiro rj" : "rio de janeiro - rj",
    "17876 st clair ave.  cloud it" : "17876 st clair ave.  cloud it",
    "khandwa" : "khandva",
    "mindoro" : "mindro",
    "almadinah" : "city",
    "las pinas" : "the pine",
    "الزرقاء" : "blue",
    "lubelskie" : "lublin",
    "ghhv" : "ghahva",
    "กรุงเทพ" : "bangkok",
    "3655 nw 87 ave  cruise" : "3655 nw 87th ave cruise",
    "ponta grossa pr" : "ponta groro pr",
    "الخرج الدلم" : "al -kharj al -dalam",
    "6900 alma dr" : "6900 apples dr",
    "مصر  المنيا  بني مزار" : "egypt minya bani mazar",
    "21 lạc long quânp8tân bình" : "21 lac long quanp8 tan binh",
    "girne (kyrenia)" : "girne (kynie)",
    "friarland" : "freeland",
    "chatrapur" : "chhatarpur",
    "albasrh" : "vision",
    "carrera 69 b no. 98a 31   bogota" : "carrera 69 b no.98a 31 bogota",
    "campo grande   mato grosso do sul" : "campo grande mato grosso do sul",
    "tp quy nhơn, bình định" : "quy nhon city, binh dinh",
    "palari" : "hat",
    "كوم حماده" : "kom hamada",
    "imus, cavite" : "we go, cavite",
    "poznań i okolice" : "poznań and its surroundings",
    "chikhli" : "the subsi",
    "cachoeiras de macacu" : "macacu waterfalls",
    "belçika" : "belgium",
    "são bernardo" : "st bernard",
    "cruzeiro do sul" : "southern cruise",
    "kharghar" : "external",
    "brr alto bosque trv 52 #21d 26" : "brr alto trv 52 #21d 26",
    "sao paull" : "sao paulo",
    "timbauba pe" : "timbauba on",
    "kura place suva" : "kura cries suva",
    "kroi i bardhe no14 prishtine kosove" : "white spring no14 prishtina kosovo",
    "arcoverde" : "green arch",
    "malumichampatty,  coimbatore" : "malumchathatty, codutue",
    "717 general booth blvdvirginia beach, va 23451, usa" : "717 general booth blvd virginia beach, va 23451, usa",
    "sardulgarh" : "sarduhgarh",
    "cuernavaca,  edo. de morelos" : "cuernavaca, edo.of morelos",
    "mỹ phú. mỹ phong mỹ tho tiềng giang" : "my phu.my phong my tho tien giang",
    "51 thống nhất, p10, gò vấp" : "51 thong nhat, p10, go vap",
    "bali , badung" : "bali, badung",
    "taboão da serra" : "taboao da serra",
    "igarassu   pernambuco" : "igarassu pernambuco",
    "jhunjhunun" : "jhunjhunu",
    "7 bình quới" : "7 vase",
    "cogliate" : "they pick up",
    "kfar saba" : "kafr saba",
    "kovvur" : "kovoor",
    "canelones" : "cannelloni",
    "مطروح" : "proportional",
    "narsapur" : "narasapur",
    "via  g  marconi  115" : "via g marconi 115",
    "afyon" : "opium",
    "freiburg im breisgau" : "freiburg in breisgau",
    "pragathi nagar" : "pragati nagar",
    "eusébio" : "eusebius",
    "bangalore , karnataka" : "bangalore, karnataka",
    "бургас" : "burgas",
    "piedras blancas" : "white stones",
    "rillieux la pape" : "rillieux la pope",
    "taoru" : "towel",
    "schriever space force base" : "schlie space force base",
    "minatitlan" : "interest",
    "koregaon" : "the koregaon",
    "5700 lakewright drive suite 300 other" : "5700 lake wright drive suite 300 other",
    "jeddaj" : "jeddah",
    "brovello carpugnino" : "bovello carpugnino",
    "hà nội" : "hanoi",
    "morges" : "morning",
    "bagalkot" : "bang",
    "via dell'industria,3" : "via dell'industria, 3",
    "ciudad de santa fe" : "santa fe city",
    "saint gilles croix de vie" : "saint gilles cross of life",
    "ilha de itamaracá" : "itamaracá island",
    "algeri" : "algiers",
    "mangaratiba rj" : "mangratiba raja",
    "daudnagar." : "daudnagar",
    "مكة المكرمة" : "makkah",
    "từ sơn bắc ninh" : "from son bac ninh",
    "bb@72#biplab" : "bb@72#bbiplab",
    "صنعاء" : "sana'a",
    "września" : "september",
    "marthandam, kanniyakumari" : "marthandam, kanniakumari",
    "repuvlica dominicana" : "dominican republic",
    "diyarbakır" : "diyarbakir",
    "formiga" : "ant",
    "são paulo, são caetano do sul" : "sao paulo, sao caetano do sul",
    "garça" : "heron",
    "lucas do rio verde   mato grosso" : "lucas do rio verde mato grosso",
    "rua turmalina 543" : "tourmaline street 543",
    "olivos" : "olives",
    "15 phan huy thực phường tân kiểng" : "15 phan huy thuc tan kieng ward",
    "15 ấp 6b vĩnh lộc a bình chánh" : "15 hamlet 6b vinh loc a binh chanh",
    "kyrenia" : "souvenir",
    "samdari balotra  badmer" : "sandari balotra barmer",
    "baqdad" : "baghdd",
    "la sarraz" : "unbroke",
    "çameli" : "cameli",
    "kuj.pom." : "also.pom.",
    "chapadão do céu   goias" : "chapadão do céu goias",
    "centro empresarial e industrial de loulé, zona poente, lote 11b" : "loulé business and industrial center, long zone, lot 11b",
    "prueba" : "proof",
    "ras tanura" : "mighted raas",
    "barra do garça mt" : "mt garça bar",
    "ogun state" : "the state war",
    "alwaye" : "he is still",
    "ka gd f" : "the gd f",
    "furrkhabad" : "farrukhabad",
    "cachoeira do piriá" : "piriá waterfall",
    "campo largo" : "long field",
    "recife.    pernambuco" : "recife.pernambuco",
    "san mateo" : "saint matthew",
    "مدينة نصر   القاهرة" : "nasr city, cairo",
    "assis chateaubriand" : "seated chateaubriand",
    "chełm" : "helmet",
    "foz do iguacu   parana" : "foz do iguacu parana",
    "san juan opico" : "san juan monkey",
    "dantiago    recoleta" : "santiago recoleta",
    "3865 e 34th st suite 105, tucson, az 85713" : "3865 is zchtht otse 105, tutzon, i 85713",
    "campinas  sp" : "campinas, sp",
    "جازان محافظة العارضة" : "jazan al -ardah governorate",
    "manzana g casa 2" : "apple g house 2",
    "rua sud?rio maximiano rocha, n 123, bloco 4 ap 202 mg" : "rua maximiano rocha street, n 123, block 4 ap 202 mg",
    "pará." : "for.",
    "thiruvarur" : "tiruvarur",
    "ิกรุงเทพ" : "the gods",
    "nezahualcóyotl  estado de mexico" : "nezahualcóyotl state of mexico",
    "primavera do leste   mt" : "spring of the east mt",
    "ji parana" : "zee parna",
    "ranaghat" : "ranaaghat",
    "birkirkara" : "well",
    "quinto di treviso" : "fifth of treviso",
    "cra32#110 11" : "sharia#110 11",
    "guara   cidade do automovel" : "guara city of automobile",
    "warszaw" : "warsaw",
    "rua santa vitoria 663 minas gerais" : "santa vitoria street 663 minas gerais",
    "القليبوية" : "qalibiya",
    "אזור" : "area",
    "bahía de banderas" : "bay of flags",
    "tphcm" : "hcmc",
    "công ty cdc" : "cdc company",
    "4b mỹ hoà 1,trung chánh, hóc môn" : "4b my hoa 1, trung chanh, hoc mon",
    "h.no 807,lila chowk old abadi lila dharmsaala ,old abadi" : "h.no ౮౦౭, lila chowk old body leela dharamsala, old body",
    "shireer aurangabad" : "shirir aurangabad",
    "itliy" : "lever",
    "bahia blanca" : "white bay",
    "243a  đê la thành" : "243a de la thanh",
    "leverano" : "levers",
    "sor juana ines de la cruz 555 abcd" : "sister juana ines of cruz 555 abcd",
    "baihata chariali" : "behta chariali",
    "البصرة" : "basra",
    "porto seguro" : "safe harbor",
    "benglore" : "bullock",
    "biên hoà" : "bien hoa",
    "tân phú, tp hồ chí minh" : "tan phu, ho chi minh city",
    "niterói rj" : "nitero rj",
    "tuljapur" : "tulajapur",
    "mandideep" : "manitideep",
    "blk iii, cez ii, gen. trias cavite" : "blk iii, via ii, gen.trias cavite",
    "kazhakuttam, trivandrum" : "kazhakutam, trivandrum",
    "teki̇rdağ" : "in",
    "kattappana" : "katappana",
    "se  barra dos coqueiros" : "if barra dos coqueiros",
    "chapra  bihar" : "chhapra bihar",
    "wallaroo" : "pair",
    "bonfinopolis" : "bonfinopol",
    "mazatlan, sin." : "mazatlan, without.",
    "9 landsdowne streetbostonma2215" : "9 lansdowne street boston ma 2215",
    "autoput za novi sad 71" : "novi sad highway 71",
    "sao jose dos campos" : "são josé dos campos",
    "ślaśkie" : "silesian",
    "יד בנימין" : "hand benjamin",
    "santa cruz de sierra  bolivia" : "santa cruz de sierra bolivia",
    "bengalure" : "bangalore",
    "salalah" : "stalemate",
    "ujjain" : "fingers",
    "tanuku" : "tension",
    "capivari de baixo. sc" : "capivari from below.sc",
    "blvr kukulkan zona hotelera km 14.5 jw marriott cancun" : "blvr kukulkan area hotel km 14.5 jw marriott cancun",
    "ceranesi" : "cerane",
    "i̇stanbul" : "istanbul",
    "caoro" : "carro",
    "biharsharif" : "bihar sharif",
    "long biên hà noiij" : "long bien ha noiij",
    "medelin" : "medium",
    "sirkali" : "vermillion",
    "عمان" : "oman",
    "شبين الكوم المنوفيه" : "shebin el -kom menoufia",
    "nabga" : "navaga",
    "mansoura" : "manoura",
    "mazowiecie" : "masovian",
    "ganaur sonipat haryana" : "ganore sonipat haryana",
    "prakasam" : "aura",
    "via g.cesare n.54" : "via g.ceso n.54",
    "jau" : "already",
    "rio brilhante" : "brilliant river",
    "pr cascavel" : "pr rattlesnake",
    "ramat gan" : "ramat gain",
    "presidente getúlio sc" : "president getúlio sc",
    "marica  rio de janeiro" : "marica rio de janeiro",
    "la mirada" : "the look",
    "rytwiany" : "rytwany",
    "pappelweg 4" : "papel road 4",
    "metropolitana" : "metropolitan",
    "via e. de amicis, 23 . 90044 carini (pa)" : "away and.de amicis, 23.90044 carini (pa)",
    "cycleworld, hosa road juncrion" : "sisilebold, jose road junkrin",
    "guasave" : "guasuave",
    "milagro" : "miracle",
    "ديالى" : "diyala",
    "2310 east el segundo blvd" : "2310 east the second blvd",
    "navi mumbai, ghansoli" : "navi mumbai, ghansali",
    "campina grande   pb" : "campina grande pb",
    "sandur" : "sand",
    "navsari" : "nervous",
    "1120 south tyron street, ste 700  medical" : "1120 south tryon street, ste 700 medical",
    "beafort" : "beaufort",
    "6f 1, no.288 5, xinya rd., qianzhen dist., kaohsiung city 80673, taiwan" : "6 rounds 1, no.288 5, new asia tropical. on the third day of the previous array., kaohsiung city 80673, taiwan",
    "浙江省宁波市" : "ningbo city, zhejiang province",
    "h15 tổ 9 kp2 long bình tân biên hòa" : "h15 group 9 kp2 long binh tan bien hoa",
    "kolkatta" : "kolkattat",
    "hubballi" : "giggle",
    "mirjapur" : "mirzapur",
    "jyoraha" : "jyraha",
    "محافظة القاهرة" : "cairo governorate",
    "1305 n holopono st suite 3" : "1305 n juice stuite 3",
    "puttur manalore" : "puttur is ours",
    "narasaraopet" : "narsaraipet",
    "1375 n buena vista dr  commercial display" : "1375 n buena vista dr commercial display",
    "são apulo" : "são paulo",
    "buxar" : "steam",
    "المنصوره" : "asan",
    "estación central" : "central station",
    "صحار" : "sohar",
    "keonjhar" : "odor",
    "orindiuva sp" : "orinduva sp",
    "teresina   pi" : "teeasia pi",
    "chur" : "put",
    "al medhatiya" : "the praise",
    "city of malolos (capital)" : "city of malolos (capatial)",
    "jamtara" : "guess",
    "santa cruz de tenerife" : "santa cruz of tenerife",
    "drabhanga" : "dranska",
    "ikorodu" : "illuminating",
    "karbalaa" : "carquel",
    "kaltim" : "east kalimantan",
    "jacobina" : "jacobin",
    "sonepur" : "sonpur",
    "rialp" : "laugh",
    "குளித்தலை" : "bath head",
    "espirito santo" : "holy spirit",
    "marilandia es" : "marilandia is",
    "lapu lapu city 6015, cebu" : "lapa lap city 6015, cebu",
    "21 calle 7 41 zona 11, mariscal" : "21 street 7 41 zone 11, marshal",
    "attur" : "actor",
    "av. livertador 1285 cochabamba bolivia" : "av.freeder 1285 cochabamba bolivia",
    "auarangabad" : "aurangabad",
    "tapejara  rs" : "tapejara rs",
    "sutton" : "suton",
    "liên chiểu   đà nẵng" : "lien chieu da nang",
    "belem" : "white",
    "puente alto" : "high bridge",
    "itu sp" : "that's sp",
    "222, gurmukh jeevan,11th road, nxt to hinduja hospital,khar (w), maharashtra" : "222, gurmukh jeevan, 11 th road, destroy then hinduja hospital, khar (v), maharashtra",
    "adare manor hotel" : "but the manor hotel",
    "fazenda rio grande (pr)" : "rio grande farm (pr)",
    "tandwa,chatra" : "tandwa, student",
    "aracaju sergipe" : "aracaju - sergipe",
    "bongaon" : "bone",
    "el espinal tolima" : "the tolima espinal",
    "tehuacan  puebla  mexico" : "tehuacan puebla mexico",
    "niterói  rj" : "nitero rj",
    "٦ اكتوبر" : "october 6",
    "florianopois" : "florianop",
    "via deserto 101" : "via desert 101",
    "πατησιων 27" : "patision 27",
    "lima, perú" : "lima peru",
    "sri ganganagarj" : "shri gangnagarz",
    "khu nhà ở phú hồng thịnh, dĩ an, bd" : "phu hong thinh, di an, bd housing area",
    "são miguel do iguaçu   paraná" : "são miguel do iguaçu paraná",
    "murrumbateman" : "murgerumbateman",
    "jakarta barat" : "west jakarta",
    "calle 41 n° 27   79 primer piso" : "41th street no. 27 79 first floor",
    "chaibasa" : "laid",
    "fortaleza ceará" : "fortaleza, ceara",
    "lajas" : "slabs",
    "vinhedo" : "vineyard",
    "la grande" : "the big one",
    "bhadohi, u.p ,india" : "bhadohi, u.p., india",
    "recife   pernambuco" : "recife pernambuco",
    "konstanz" : "consistency",
    "islampur. uttar dinajpur" : "islampur.north dinajpur",
    "via del commercio 9" : "via del commerce 9",
    "diinópolis" : "diinopolis",
    "ratia" : "rat",
    "chennak" : "to chen",
    "hutti" : "incarcerate",
    "la senia" : "no sunnis",
    "uttarpara, hooghly, west bengal" : "north, houghli, west bengal",
    "calle 16a n 55 64 antioquia" : "calle 16a n 55 64 colombia",
    "mossoró" : "mossoro",
    "deogarh" : "dogarah",
    "banten， tangerang selatan" : "banten ， south tangerang",
    "lubeldkie" : "lubeldke",
    "azowieckie" : "masovian",
    "ҭ. ڬܬݬ ѭѭ ҭ.18" : "ҭ.ڬ ܬ ݬ ѭѭ ҭ.18",
    "brotas" : "sprout",
    "são bernardo do campo   sp" : "são bernardo do campo sp",
    "kothaguda, kondapur" : "kottaguda, kondpur",
    "são paulp" : "são paulo",
    "palu" : "hammer",
    "cao lãnh" : "cao lanh",
    "ฉะเชิงเทรา" : "chachoengsao",
    "kasoa" : "kao",
    "tucano bahia" : "bahia toucan",
    "honnudike" : "hount",
    "vitorino freire  ma" : "vitorino freire ma",
    "são vicente" : "saint vincent",
    "santa rosa   rs" : "santa rosa rs",
    "nhị quí cai lậy tiền gianh" : "nhi quy cai lay tien giang",
    "kaohsiung" : "gooohining",
    "144 w 133rd sr apt 1" : "144 w 133rd st apt 1",
    "yamuna nagar" : "yam nagar nagar",
    "contagem" : "score",
    "tarumã   sp" : "tarumã sp",
    "edi̇rne" : "normally",
    "magpur" : "nagpur",
    "bhubaneswar" : "bluewar",
    "vila velha   es" : "vila velha es",
    "picedo" : "piced",
    "washougal" : "washogal",
    "kanpur dehat" : "kanpur countryside",
    "lonavla" : "lonavala",
    "são paulo  ' sp" : "sao paulo-sp",
    "panamá city" : "panama city",
    "yanbu" : "eye",
    "łobodno" : "lobby",
    "sanchez mira" : "sanchez look",
    "dokki" : "poisoning",
    "uthangarai ,krishnagiri" : "utangarai, krishnagiri",
    "nova cruz" : "new cross",
    "viale premuda 12" : "viale pressed 12",
    "עשרת" : "ten",
    "pilibanga" : "the pilot",
    "van nuys" : "van nys",
    "sangli" : "sangli is the best.",
    "são paulo   sp." : "sao paulo-sp.",
    "juarez" : "juerez",
    "ποσειδώνος 4 χαλάνδρι" : "poseidon 4 chalandri",
    "umaria" : "maria",
    "baymont inn 2731 watson bld" : "baymont inn 2731 watson blvd",
    "mundra" : "the man of muman",
    "barreiras" : "barriers",
    "são paulo americana" : "american sao paulo",
    "ras al khaimah gg" : "the head of the tent c",
    "jequié  bahia" : "jequié bahia",
    "cao  lãnh" : "cao lanh",
    "hariharpara" : "hariharpada",
    "damnhour" : "damanhour",
    "sjcampos" : "seacamp",
    "laspinas" : "laspin",
    "arnenia quindío" : "armenia quindio",
    "chattogram" : "chittagong",
    "toluca estado de méxico" : "toluca state of mexico",
    "koraput" : "curtain",
    "جدة" : "grandmother",
    "almodhaibi" : "al -mudhaibi",
    "sunam" : "call",
    "eslam mahdi@hotmail.com" : "islam mahdi",
    "gautang" : "loan",
    "بغداد" : "baghdad",
    "marina del rey" : "navy of the king",
    "kalyoubiyah" : "alkalibi",
    "opa locka" : "opa lock",
    "100 đinh bộ lĩnh p26  q.bình thạnh" : "100 dinh bo linh p26 district binh thanh",
    "merida, yucatan" : "merida, yukatan",
    "shirakawa shi" : "shirakawa",
    "al kharj" : "the outside",
    "la jolla" : "the jolla",
    "varanasí" : "varanza",
    "الجيزة" : "giza",
    "matrouh" : "matrosh",
    "taxila" : "taxilas",
    "moorsville" : "mooresville",
    "via degli allori, 79" : "via degli lapel, 79",
    "shope no 3, mangalwai complex" : "shop no 3, mangalwari complex",
    "zalka" : "zalk",
    "cuerámaro gto méxico" : "gto mexico cuerramaro",
    "bhadrak" : "gentleman",
    "aydın kuşadası" : "aydin kusadasi",
    "chourai" : "shuy",
    "ethachlml" : "somehow",
    "calle larga" : "long street",
    "canela" : "cinnamon",
    "10, ql13, hiệp bình phước" : "10, ql13, hiep binh phuoc",
    "itajaí   santa catarina" : "itajaí santa catarina",
    "rangia" : "rangi",
    "salto" : "heel",
    "serra es" : "serra is",
    "marlloca, espanha" : "mallorca, spain",
    "nala sopara" : "subternair",
    "gadarwara" : "bridge",
    "azamgarh tarwa uttar pradesh 276123" : "azamgarh tarawa uttar pradesh 276123",
    "dheerpura" : "dhirpura",
    "paithan" : "penetration",
    "oaxaca" : "oxaca",
    "puerto vallarta" : "vallarta port",
    "región de coquimbo" : "coquimbo region",
    "centro direzionale milanofiori," : "milanofiori disional center,",
    "beba" : "baby",
    "phulbani" : "fulbani",
    "monclova , coahuila" : "monclova, coahuila",
    "toluca, edo mex" : "toluca, or mex",
    "virgem da lapa" : "virgin da lapa",
    "1175, chiêu liêu bình dương" : "1175, lieu binh duong",
    "al sulay" : "al try",
    "são carlos   sp" : "são carlos sp",
    "des moines" : "monks",
    "pfalzen" : "pfallel",
    "haldibari" : "is l -bali",
    "mossoro" : "mossorus",
    "fonte nuova" : "new source",
    "shahajahanpur" : "shahjahanpur",
    "el obour" : "transit",
    "vitória da conquista" : "victory of the conquest",
    "dozza" : "doness",
    "meteti" : "mete",
    "soro" : "difficult",
    "tiruchirappalli" : "turuchir them",
    "haveri" : "breakdown",
    "cd. júarez" : "cd.júarez",
    "pydibhimavaram" : "paidibhimavaram",
    "mazatlan" : "matlan",
    "isranbul beykoz" : "istanbul beykoz",
    "vengurla" : "vetearla",
    "in 46601, ee. uu." : "in 46601, in.his.",
    "kab.tangerang, gdg serpong" : "kab. tangerang, gading serpong",
    "conselheiro lafaiete" : "counselor lafaiete",
    "akwa ibom" : "immoral",
    "1 travessa francisco pereira, pitua?i bahia" : "1 travessa francisco pereira, pitua? i bahia",
    "shindkheda" : "shirtink",
    "riftvalley" : "rift valley",
    "tabuk" : "taboos",
    "350 ne 24st apt 1105" : "350 not 24th apt 1105",
    "general santos" : "santo",
    "rua keisuke koza, 189 s?o paulo" : "keisuke lua, 189 s?power",
    "sta maria bulacan" : "maria bulacan stays",
    "são josé de ribamar ma" : "sao josé de ribamar ma",
    "468 phan văn trị phường 7 gò vấp" : "468 phan van tri ward 7 go vap",
    "antwerpen" : "antwerp",
    "bengaluru" : "bangalore",
    "campo grande" : "large field",
    "santarem para" : "santarem to",
    "barra de são francisco" : "san francisco barra",
    "villeta" : "villet",
    "maravilha" : "wonder",
    "الكويت" : "kuwait",
    "itarumã" : "istarumã",
    "punjab sailkot" : "punjab salakot",
    "bulchandi.malda" : "bunkanki.al",
    "kesariya" : "saffron",
    "presidente getúlio" : "president getúlio",
    "bharuwa sumerpur" : "bharua sumerpur",
    "mineiros" : "miners",
    "ribeirão pires   sao paulo" : "ribeirão pires sao paulo",
    "mummy ki" : "mummy to",
    "shivmogga" : "to the shivogate",
    "lubelskie lublin" : "lublin lublin",
    "distrito nacional" : "national district",
    "chakan pune" : "check the spectrum",
    "los jagüeyes, namiquipa, chihuahua" : "the jagüeyes, namiquipa, chihuahua",
    "san borja" : "san borje",
    "jubail saudi arabia" : "jubel saudi arabia",
    "keshena" : "cereal",
    "vle roma,4" : "want roma, 4",
    "6 ấp 4, nhơn đức, nhà bè" : "6 hamlet 4, nhon duc, nha be",
    "hanamkonda" : "he hasn't crushed him",
    "murau" : "the region",
    "kenitra" : "chest",
    "manab" : "human",
    "wielbrandowo" : "molbrandowo",
    "radom" : "by working",
    "mardi̇n" : "marḋn",
    "muddebihal" : "mudd is more",
    "campos dos goytacazes,rj" : "campos dos goytacazes, rj",
    "novi ligure" : "new liguers",
    "pontalina" : "pontaline",
    "vitória" : "victory",
    "נהריה" : "nahariya",
    "25 อาคารมูฟอะเมซ ชั้น 4 ห้อง 4b" : "25 movez building, 4th floor 4b",
    "san jose del guaviare" : "san jose of guaviare",
    "الأحساء" : "hasa",
    "cordoba, ver" : "cordoba, see",
    "cd. obregon" : "cd.obregon",
    "puerto montt" : "montt port",
    "marinduqe" : "maruqe",
    "salvador   bahia" : "salvador, bahia",
    "aparecida" : "appeared",
    "san carlos" : "saint charles",
    "vijaywada" : "vijayawada",
    "ontario" : "ontaria",
    "الخبر" : "the news",
    "ciudad de panama" : "city ​​of panama",
    "kahului" : "tickets from kahului",
    "guarulhos   são paulo" : "guarulhos sao paulo",
    "באר שבע" : "beer sheva",
    "jabel" : "jelly",
    "1 skyview drfort worthtx76155" : "1 skyview dr fort worth tx 76155",
    "srungavarapukota, vizianagaram" : "shringwarpukota, vijayanagaram",
    "srinagar j&k" : "srinagar j & k",
    "ابوظيي" : "abuzia",
    "nanded." : "nanded",
    "maricá" : "sissy boy",
    "medchal" : "gambling",
    "quenzon city" : "quezon city",
    "lahar" : "lava",
    "catas altas" : "high tastings",
    "سكاكا" : "sakaka",
    "kudal" : "fences",
    "estado de méxico" : "mexico state",
    "sp franca" : "sp frank",
    "đà nẵng" : "danang",
    "quận 2 sài gòn" : "district 2 saigon",
    "para" : "for",
    "angul" : "corner",
    "via caldaia 17" : "via boiler 17",
    "403 quốc lộ 62  tp tân an long an" : "403 national highway 62 tan an long an city",
    "aydın" : "clear",
    "pueblo libre" : "free town",
    "609 medical center dr. decatur, texas, 76234" : "609 medical center dr. decatur, texas, 76234",
    "ichapuram" : "chick",
    "casa branca" : "white house",
    "ouezon" : "loss of planting",
    "devakottai" : "devakotai",
    "paulinia" : "paulinaia",
    "جيزان" : "jizan",
    "tambogrande" : "tambogging",
    "ourinhos" : "outer",
    "dubay" : "gray",
    "กทม." : "bangkok",
    "اسوان" : "aswan",
    "zakrzew" : "zakhil",
    "carrera 80 # 63b  55 cundinamarca" : "carrera 80 # 63b 55 cundinamarca",
    "viale trento ,20" : "viale trento, 20",
    "vũng liêm" : "vung liem",
    "καποδιστριου 27" : "kapodistriou 27",
    "sete barras" : "seven bars",
    "dharpally dist nizamabad" : "dharpalli dish is nizamabad",
    "sykesville" : "heart rate",
    "via bosco n. 9" : "via bosco n.9",
    "avenida 3c # 45 100" : "3c # 45 100 avenue",
    "bùi xương trạch thanh xuân hà nội" : "bui xuong trach thanh xuan hanoi",
    "kailua" : "halewais",
    "ne" : "it is",
    "partur" : "portion",
    "l'anse" : "the handle",
    "cruzeiro do sul   acre" : "southern cruzeiro acre",
    "piduguralla" : "pigrore",
    "etawah" : "elevator",
    "matamoro" : "math",
    "vadgaon" : "vadga",
    "mpumalanga" : "east",
    "lika" : "equal",
    "são mateus" : "saint matthew",
    "harta" : "assets",
    "katoria" : "android",
    "são paulo, arandu" : "sao paulo, arandu",
    "sobótka" : "bonfire",
    "trà vinh" : "tra vinh",
    "tustin" : "pain",
    "ciudad autónoma de buenos aires" : "autonomous city of buenos aires",
    "sahranpur" : "saharanpur",
    "paraíba joão  pessoa" : "paraíba joão pessoa",
    "aquiraz" : "aquiiraz",
    "mato grosso   cuiaba" : "mato grosso cuiaba",
    "novo cruzeiro   m g" : "new cruise m g",
    "siddipet" : "siddipate",
    "الشرقيه" : "eastern",
    "sc   jaraguá do sul" : "sc jaraguá do sul",
    "台中" : "taichung",
    "sanguem" : "face to face",
    "hawalli" : "around",
    "新骏环路" : "xinjun ring road",
    "101 gosai dr" : "101 later dr",
    "encantado" : "charmed",
    "istanbul   büyükcekmece" : "istanbul büyükcekmece",
    "kogarah" : "kagarah",
    "bijapur   vijayapura" : "bijapur vijaypura",
    "belford roxo" : "purple belford",
    "gurgaoun" : "gurgaon",
    "udumalpet" : "dumalpet",
    "caratinga   mg" : "karatinga mug",
    "vitoria da conquista" : "victory of the conquest",
    "sasthamcotta" : "mammalian",
    "joenvile" : "joinville",
    "99 định công hạ" : "99 dinh cong lower",
    "charqueadas" : "pitty",
    "el paso" : "step",
    "miryalaguda" : "miyalaguda",
    "othon p. blanco" : "othon p.blanch",
    "dadri" : "the",
    "hulhumale" : "humid",
    "ki tuc xa khu a, dhqg ho chi minh" : "ki tuc far from a, dhqg ho chi minh",
    "jr, alemania 2485" : "jr, germany 2485",
    "campo largo   paraná" : "long field paraná",
    "itajobi" : "jobi",
    "maglio" : "mallet",
    "ρουσβελτ 18" : "roussvelt 18",
    "kemaman" : "kember",
    "los lagos" : "the lakes",
    "camarines sur philippines" : "camarines on philippines",
    "dehra dun" : "a good man anton",
    "lagos, surulere" : "lagos, surulate",
    "beirut" : "beir",
    "seoni" : "sony",
    "elazığ" : "elazig",
    "arujá" : "aruja",
    "عدن" : "aden",
    "جيزة" : "giza",
    "ấp vĩnh phú xã vĩnh thạnh" : "vinh phu hamlet, vinh thanh commune",
    "ciudad e méxico" : "mexico city",
    "palompoj" : "palombo",
    "marília" : "marilia",
    "64 nguyễn thị tần p2 q8" : "64 nguyen thi tan p2 q8",
    "mysuru" : "my",
    "macul" : "the brave",
    "eranakulam" : "ernakulam",
    "bagepalli" : "baking palli",
    "barueri   sp" : "barueri sp",
    "são paulo, sp" : "sao paulo-sp",
    "altamont springs" : "altamonte springs",
    "hamilton on" : "hamilton is",
    "dakor" : "significance",
    "united kindgom" : "united kingdom",
    "piotrków trybunalski" : "piotrkow trybunalski",
    "مصر" : "egypt",
    "kuthuparamba" : "to sweat",
    "asturias   gijon" : "asturias gijon",
    "المحلة الكبرى" : "mahalla al -kubra",
    "sindagi" : "inferior",
    "cao lanh" : "kaolin",
    "minamisatsuma" : "southern satsuma",
    "villa maria   cordoba" : "villa maria cordoba",
    "jantral" : "jaralal",
    "دبي" : "dubai",
    "chợ mới" : "new market",
    "uruguaiana" : "uruguayan",
    "khordha" : "khodha",
    "lince   lima" : "lince lima",
    "800 3rd ste nw" : "800 width six nou",
    "balneário camboriú   sc" : "balneário camboriú sc",
    "กม" : "km",
    "goiânia   go" : "goiânia go",
    "via xxv ottobre,  4" : "via xxv october, 4",
    "puerto escondido" : "hidden port",
    "taliparamba" : "respondaramba",
    "phuket" : "fuket",
    "channapatna" : "channapana",
    "ulus mah. manolya cad. no:16" : "ulus mah.magnolia cad.no: 16",
    "bogotá, colombia" : "bogota colombia",
    "nova iguaçu   rj" : "nova iguaçu rj",
    "alangulam" : "alongulam",
    "balangir" : "forbidden",
    "san angelo" : "as an angel",
    "santa catarina em itapema" : "santa catarina in itapema",
    "surar" : "chapter",
    "chikkamagaluru" : "ukuru",
    "cornaredo" : "cornado",
    "الدقهلية" : "dakahlia",
    "concepción" : "conception",
    "jaypur" : "jaipur",
    "piauí   pi" : "piauí pi",
    "candiolo" : "cadlo",
    "الجهراء" : "jahra",
    "méxico city" : "mexico city",
    "draria" : "drarea",
    "สมุทรสาคร" : "samut sakhon",
    "arco (tn)" : "arc (tn)",
    "55 camino aguajito" : "55 aguajito road",
    "banihal" : "bees",
    "jaggayyapeta" : "jaggaiahpet",
    "hosapete" : "assistant pente",
    "calle 167a #5a 4" : "calle 167a #kha 4",
    "al madina" : "city",
    "ganaur" : "ganore",
    "são manuel" : "saint manuel",
    "sonnefeld" : "sunfield",
    "stekene" : "stitch",
    "bangkok (กรุงเทพมหานคร)" : "bangkok (bangkok)",
    "quảng nam" : "quang nam",
    "nguyễn xiển thành phố thủ đức" : "nguyen xien thu duc city",
    "27 phường 17 xô viết nghệ tĩnh" : "27 ward 17 soviet nghe tinh",
    "jangareddygudem" : "jangeddygudem",
    "narkatiya ganj" : "narkatia ganj",
    "undri pune" : "underi puts",
    "havre de grace" : "harbour grace",
    "av. doutor alvaro camargos, 1680  mg" : "av.doctor alvaro camargos, 1680 mg",
    "av. guanabara 95 sa8, jardim s?o vicente, itupeva s?o paulo" : "av.guanabara 95 sa8, jardim s? o vicente, itupeva s? o paulo",
    "تربة" : "soil",
    "slaskie" : "silesian",
    "paruthippara" : "cotton",
    "đồng tháp" : "dong thap",
    "kottakkal malppuram" : "kottakkal malappuram",
    "marigot" : "husband",
    "muheillah" : "timing",
    "الرياض" : "riyadh",
    "chuluota" : "out of him",
    "farropilha" : "farroupilha",
    "ilupeju" : "ilupe",
    "waunakee" : "waukyo",
    "uniao da vitoria" : "victory union",
    "250 e sepulveda blvd #1046" : "250 is sepulveda blvd #1046",
    "hamden" : "hammer",
    "st maarten" : "saint martin",
    "rua nossa senhora de lourdes" : "nossa senhora de lourdes street",
    "חדרה" : "hadera",
    "شربين" : "sherbin",
    "arujá   são paulo" : "arujá sao paulo",
    "strzelno" : "shooting",
    "itaporã ms" : "stone ms",
    "بريدة" : "burial",
    "triprayar (valapad)" : "tripreer (valapad)",
    "laguna sta. cruz" : "santa laguna.cross",
    "via 4 novembre, 2" : "via 4 november, 2",
    "sofija" : "sofia",
    "jumaira" : "jumai",
    "são pedro" : "saint peter",
    "armenia, quindío" : "armenia quindio",
    "binan,laguna" : "binan, laguna",
    "mato grosso   alta floresta" : "mato grosso alta floresta",
    "capela do alto" : "chapel",
    "日本" : "japan",
    "ambelakion 28, 4046 germasogia, limassol, cyprus" : "amblakion 28, 4046 germasogia, limassol, cyprus",
    "الظاهره" : "phenomenon",
    "pedra azul" : "blue stone",
    "ירושלים" : "jerusalem",
    "ahemdabad" : "ahmedabad",
    "mohali" : "local",
    "kosli" : "kosla",
    "como" : "as",
    "governador valadares minas gerais" : "governor valadares minas gerais",
    "sp   pirassununuga" : "sp pirassununuga",
    "río negro" : "black river",
    "lima   san borja" : "five san borja",
    "676 n michigan avenue" : "676 road cancer ave nue",
    "kattumannarkoil" : "overlapping",
    "chamarajanagara" : "chamarajanagar",
    "sta. rosa" : "is.rose",
    "cedry wielkie" : "cedry great",
    "hựu thành, trà ôn, vĩnh long" : "huu thanh, tra on, vinh long",
    "kottur" : "cat",
    "bolivar, magangue" : "boli, maganugu",
    "thiruchangkodu" : "church line",
    "7616 benbrook pwky" : "7616 benbrook parkway",
    "gourav bachan" : "gaurav bachan",
    "233 south beaudry avenue  los angeles ca" : "233 south beaudry avenue los angeles ca",
    "oriximiná   pará" : "oriximiná pará",
    "mednine" : "civilians",
    "strada dei palazzi 5d" : "road of the palazzi 5d",
    "lagoa da prata" : "silver lagoa",
    "rewari" : "leimrit",
    "bhayander" : "bayander",
    "lousada" : "sloe",
    "berwyn" : "pillow",
    "botuporanga" : "bowupounga",
    "hyderbad" : "hyderabad",
    "cuidad de mexico" : "mexico city",
    "addis ababa" : "addis short",
    "pabellon de arteaga" : "pabellon de artaaga",
    "voluntari" : "volunteer",
    "اادوحة" : "idah",
    "manaus am ." : "manaus am.",
    "via sempione 27  italia" : "via sempione 27 italy",
    "brejo santo" : "brejo holy",
    "türkiye cumhuriyeti" : "republic of türkiye",
    "kampala  muyenga" : "kampala muyenga",
    "alhasa" : "whiet",
    "favria" : "favory",
    "الخرج" : "output",
    "davao del sur" : "south davao",
    "secha sadan" : "sikad sika",
    "jatai go" : "expressed",
    "rua 15 de novembro, 1262 rs" : "november 15th street, 1262 rs",
    "abudhabi   western region  ghayathi city" : "abu dhabi and sterene, ghiathi city",
    "mahad" : "thanks",
    "mallapur hyderbad" : "mallapur hyderabad",
    "monterrey, méxico" : "monterrey mexico",
    "florianópolis   sc" : "florianópolis sc",
    "recife   pe" : "recife on",
    "são paulo sp" : "sao paulo-sp",
    "calle 89 no 67 136 811014571" : "street 89 no 67 136 811014571",
    "cd mx" : "cdmx",
    "soufriere" : "sufriere",
    "são paulo   são paulo" : "sao paulo sao paulo",
    "indoren" : "indo",
    "chrompet" : "cromarty",
    "εθνικης αντιστασεως" : "national resistance",
    "imus cavite" : "we go to cavite",
    "colgong" : "colong",
    "rio dr janeiro" : "rio de janeiro",
    "curitiba parana" : "curitiba, parana",
    "alamuru" : "opportunity",
    "el salvador" : "the savior",
    "czeladź" : "czeladz",
    "thủ đức, tp hcm" : "thu duc, ho chi minh city",
    "khorasan razavi" : "khorasan razvi",
    "المدينه المنوره" : "al madinah al munawwarah",
    "neily" : "nile",
    "calle 64 b # 1  71" : "calle 64 b # 1 71",
    "eustis" : "welcome",
    "كربلاء المقدسة" : "holy karbala",
    "via vittorio emanuele,17" : "via vittorio emanuele, 17",
    "jawabarat" : "answer",
    "madinat abqaiq" : "madinat abqiqi",
    "nearambedkar hospet" : "nearambakkar hospet",
    "mohania" : "mohinia",
    "θηβων 500 ιλιον" : "thebes 500 ilion",
    "açailândia   ma" : "açailândia ma",
    "boa esperança" : "good hope",
    "po box 701362  san antonio" : "well box 701362 san antonio",
    "quảng ninh" : "quang ninh",
    "5949 via del cielo  cloud it" : "5949 via del cielo cloud it",
    "blvd. miguel aleman n?m. 1101, san mateo atenco edo. de m?xico estado de m?xico" : "blvdmiguel aleman n? m.1101, san mateo atenco edo.of the same state of music",
    "oshawa" : "shawawa",
    "campo novo do parecis" : "parecis novo campo",
    "buenos aires , florida" : "buenos aires, florida",
    "dourados" : "golden",
    "sindekela" : "sindekelala",
    "kaij" : "hallter",
    "duliajan" : "dulia",
    "2500 menaul blvd ne" : "2500 menal blvd not",
    "اسكندرية" : "alexandria",
    "ตาก" : "dry",
    "anuppur" : "transmitter",
    "gurugam" : "gurgam",
    "santos domingo" : "saints sunday",
    "sancristobal" : "sanctobal",
    "ji paraná" : "her paraded",
    "santa cruz   rio gallegos" : "santa cruz rio gallegos",
    "florida 1 piso: 12 ofic: 48   entre: esquina rivadavia   capital federal (1005) buenos aires" : "florida 1 floor: 12 off: 48 between: corner rivadavia capital federal (1005) buenos aires",
    "kaimur bhabuan" : "kaimur bhabhua",
    "playa del carmen" : "carmen beach",
    "401st afsb" : "401st aff",
    "villa alemana" : "german village",
    "bangaluru" : "bangluru",
    "mancherial telangana" : "manancherl telangana",
    "purwkhn" : "prize",
    "atílio vivacqua" : "atilio vivacqua",
    "99 định công hạ  hoàng mai" : "99 dinh cong ha hoang mai",
    "caieiras   são paulo" : "caieiras são paulo",
    "tekirdağ" : "tekirdag",
    "rosário oeste mt" : "rosario west mt",
    "getulina" : "we have it",
    "no.08, sir chittampalam a gardiner mw," : "no.08, sir chittampam gardin mw,",
    "piro" : "score",
    "tandwa" : "struggle",
    "chinsurah" : "chinasur:",
    "vitória da conquista   bahia" : "vitória da conquista bahia",
    "rio de janeiro   rj" : "rio de janeiro - rj",
    "jognda" : "jogna",
    "kodagu" : "kernel",
    "gurgoan" : "gurgaon",
    "6october" : "6 october",
    "lavras" : "mulch",
    "blairsville" : "blairville",
    "begusaria" : "begusariia",
    "gaurulhos" : "guarulhos",
    "kathumar" : "jelum",
    "juan de acosta atlantico" : "juan de acosta atlantic",
    "motihari  chowk" : "motihari chowk",
    "dasuya" : "put out",
    "balneário piçarras" : "balneario piçarras",
    "القاهرة" : "cairo",
    "100 vestavia parkwaybirminghamal35216" : "100 vestavia parkway birmingham al 35216",
    "atlantico" : "atlantic",
    "pompeia, sao paulo" : "pompeii, sao paulo",
    "ponda goa" : "crush goa",
    "la plata" : "at payment",
    "espinar" : "spine",
    "tocantins   palmas" : "tocantins palmas",
    "volta redonda" : "round",
    "agbara" : "power",
    "olinda pe" : "olinda the",
    "virar east" : "east",
    "mato grosso,  sinop" : "mato grosso, sinop",
    "banglore" : "bengaluru",
    "ابوظبي" : "abu dhabi",
    "kalwa" : "calf",
    "perú lima" : "lima peru",
    "nova resende" : "new resende",
    "mumbai" : "moon",
    "cote d'ivoire" : "ivory coast",
    "διγενή 25" : "digeni 25",
    "na vyhlídce 268 268" : "on the prospect of 268 268",
    "carpina   pernambuco" : "carpina pernambuco",
    "słupska" : "słupsk",
    "bünde" : "fründe",
    "el cerrito" : "the hill",
    "vitória   espirito santo" : "vitoria, espirito santo",
    "khagaria" : "khagariia",
    "النوبارية" : "nubaria",
    "joelformcheck magnit joelformcheck magnit" : "joelformcheck mets joelformcheck magnet",
    "pan india" : "indian bread",
    "colombia c2: solo información" : "colombia c2: only information",
    "via fontana dell'oste 52" : "via fontana dell'osta 52",
    "barranquilla, atlantico" : "barranquilla, atlantic",
    "calle goya, 90" : "goya street, 90",
    "roha" : "steam",
    "aquiraz   ce" : "aquiraz ce",
    "المدينة المنورة" : "al madinah al munawwarah",
    "três corações" : "three hearts",
    "ponda" : "crush",
    "via barone s. ferla" : "via barone s.ferla",
    "sc   maravilha" : "sc wonder",
    "boca raton" : "bottle of raton",
    "raigad" : "rig",
    "mahnar" : "mahanar",
    "ribeirão preto   sp" : "ribeirão preto sp",
    "arsikere" : "arsic",
    "abudhabi" : "abu dhabi",
    "viluppuram" : "developuram",
    "rua piau" : "piau street",
    "carazinho" : "cad",
    "odenton" : "denton",
    "rio das ostras   rj" : "rio das ostras rj",
    "región metropolitana" : "metropolitan region",
    "nagpur india" : "india",
    "台灣" : "taiwan",
    "vadodara" : "let them",
    "correntes" : "chains",
    "gangavathi" : "gangavati",
    "balneário camboriú  sc" : "balneário camboriú sc",
    "barka" : "helloon",
    "jalna" : "burn",
    "kota bandung" : "bandung",
    "cordilheira alta   sc" : "high cordilher sc",
    "portharcourt" : "porthcourt",
    "i̇zmir urla" : "ilkir urla",
    "avenida de europa 4 planta baja, local b" : "avenida de europa 4 ground floor, local b",
    "rua leonor maria barbosa condom?nio village ba" : "rua leonor maria barbosa condom?nicil village i",
    "teste" : "test",
    "frontera hidalgo" : "hidalgo border",
    "avc systemy audiowizualne jakub pacholewicz, ul. widokowa 3" : "avc audiovisual systems jakub pacholewicz, ul.viewing 3",
    "kosagumuda" : "kosa no waste",
    "waraseoni, dist balaghat" : "varasai, district balaghat",
    "kerala  kannur" : "kerala kannur",
    "breaux bridge" : "breaux bridge",
    "piler" : "arrows",
    "chikodi" : "health",
    "stevensville" : "stevensville",
    "amparo" : "protection",
    "mau ranipur" : "it's ranipur",
    "parana , medianeira" : "parana, medianeira",
    "peje" : "pe said",
    "parbhani" : "perbhani",
    "edappal" : "adapal",
    "san jose del monte" : "san jose of monte",
    "lastra a signa" : "slab in signa",
    "bazas" : "bazae",
    "116đường cầu kinh" : "116 cau kinh road",
    "rajam" : "stoning",
    "خميس مشيط" : "khamis mushait",
    "noorsarai" : "joun",
    "sveta nedelja" : "holy sunday",
    "basavanbagewadi" : "basebagewadi",
    "sayoun" : "sion",
    "shahabad markanda distt kurukshetra" : "shahabad markanda distra kurukshetra",
    "benito juárez" : "benito juarez",
    "proszowice, kraków, małopolskie" : "proszowice, kraków, lesser poland",
    "rajashthan" : "rampage",
    "sialkot" : "sigot",
    "la rioja" : "the rioja",
    "piedras negras" : "black stones",
    "amethi" : "to",
    "40 s k bole marg" : "3 s.",
    "tada" : "then",
    "bazpur uttrakhand" : "bajpur uttarakhand",
    "الاسكندرية" : "alexandria",
    "via d.f. rosso" : "via d.f.red",
    "nazaré da mata  pernambuco" : "nazaré da mata pernambuco",
    "la chorrera" : "the chorrera",
    "sao paulo vargen grande do sul" : "sao paulo vargem granding do sul",
    "moshi,pimpri  chinchawad" : "moshi, pimpri chinchwad",
    "umri" : "age",
    "duabi" : "dubai",
    "fort pierre" : "strong stone",
    "manavalanagar" : "manwalnagar",
    "akole" : "we can not",
    "boa vista" : "good view",
    "san carlos," : "saint charles,",
    "partur,jalna" : "part, jalna",
    "dubi" : "look",
    "kodambakkam" : "kodubakkam",
    "league coty" : "league city",
    "barra velha" : "old bar",
    "pinhalzinho" : "pineglass",
    "الرياض السعودية" : "al riyadh, saudi arabia",
    "fraiburgo" : "fraiburg",
    "maharashatra" : "maharashtra",
    "andana" : "story",
    "b0g0tá" : "b0g0g0",
    "dubia" : "doubts",
    "gaziantep i̇slahiye" : "gazi̇antep i̇slahi̇ye",
    "sumatera utara" : "north sumatra",
    "shagamu" : "shop",
    "cachoeiro" : "waterfall",
    "18 r. aquino st., hagdang bato libis, mandaluyong city metro manila" : "18 r.aquino st., hagdang bato libis, mandaluyong city metro manila",
    "jantral vijapur mehsana gujarat india" : "bharanal vijapur mehsana gujarat india",
    "dreieich" : "triad",
    "via tosco romagnola  1422" : "via tosco romagnola 1422",
    "montería" : "hunting",
    "信義區" : "xinyi district",
    "resende" : "reset",
    "النبطية" : "nabatieh",
    "schladming" : "sladming",
    "quitó" : "he removed",
    "san josé" : "saint joseph",
    "são gabriel do oeste   ms" : "são gabriel do oeste ms",
    "ny 11358" : "the 11358",
    "castelo" : "castle",
    "betul" : "proper",
    "rua celso ramos, 145   vila andrade   s?o paulo   sp   cep: 05734 080 sp" : "rua celso ramos, 145 vila andrade são paulo sp zip code: 05734 080 sp",
    "malur" : "anoint",
    "paita" : "shirt",
    "marib" : "maribe",
    "7001 e fish lake rd #200  cloud it" : "7001 e fish lake rd #200  cloud it",
    "cuiabá   mt" : "cuiabá mt",
    "vasai road, mumbai" : "wasi road, mumbai",
    "thủ dầu một, tỉnh bình dương" : "thu dau mot, binh duong province",
    "reasi" : "reaction",
    "yanbu al sinaiyah" : "eye aris resistance marks",
    "chile, santiago" : "santiago chile",
    "contai" : "anyway",
    "paranaque, metro manila" : "paranaque, metro manila",
    "mestre venezia" : "mestre venice",
    "madikeri" : "madatti",
    "haag in oberbayern" : "haag in upper bavaria",
    "700 patroon creek blvdalbanyny12206" : "700 pattern creek blvdalbanyny12206",
    "passos" : "steps",
    "الشارقة" : "sharjah",
    "malerkotla" : "painter kotla",
    "itanhaém  sp" : "itanhaém sp",
    "aracaju se" : "aracaju if",
    "el dorado hills" : "the dorado hills",
    "57 neumann road    capalaba  qld  4157  us qld" : "57 neumann road capalaba qld 4157 us qld",
    "maceió   al" : "macea al",
    "atizapan, estado de méxico" : "atizapan, state of mexico",
    "provinsi banten" : "banten province",
    "sta. rosa city" : "is.rosa city",
    "645 hulet dr ste 200" : "645 hollowed dr ste 200",
    "franca são paulo" : "franca sao paulo",
    "rajghar" : "royal",
    "sqlvador" : "salvador",
    "sjo" : "sea",
    "al qusais" : "al quasais",
    "πάροδος θηβαΐδος 21" : "thebaidis 21",
    "pozzonovo" : "refuse",
    "chakia" : "hugs",
    "neópolis" : "neopolis",
    "القصيم بريده" : "al -qassim is his mail",
    "kí túc xá khu a đhqg tp. hcm" : "dormitory a vnu tp.hcm",
    "ny" : "the",
    "pilani jhunjunu" : "pilani jhhununu",
    "sai paulo" : "you know paulo",
    "tibu" : "chick",
    "brgy 13 naglicuan, pasuquin, in" : "brgy 13 naglikuan, pasuquin, in",
    "wynnum" : "whiten",
    "tadiprru" : "earlierpru",
    "torreblanca" : "white tower",
    "ramnagar" : "ramnagar ramnagar",
    "nizmabad" : "nizamabad",
    "bhestan, surat." : "bhestan, letter.",
    "biloxi" : "comparison",
    "balıkesir" : "balikesir",
    "al jubail" : "to the jubail",
    "kosova" : "chuang",
    "santarém  pará" : "santarém pará",
    "linh  trung,  thủ đức" : "linh trung, thu duc",
    "ghatsila" : "ghatila",
    "joão pessoal" : "joão pessoa",
    "kuj pomorskie" : "kuj pomeranian",
    "新竹" : "hsinchu",
    "taloja" : "houses",
    "joão pessoa   pb" : "joão pessoa pb",
    "videira" : "vine",
    "warmińsko  mazurskie" : "warmian masurian voivodeship",
    "johannesburh" : "johannesburg",
    "tamluk" : "ownership",
    "chennaj" : "chennai",
    "via d. alighieri 13" : "via d.alighieri 13",
    "chengalpattu district" : "district",
    "malkapur" : "malakpur",
    "thatta" : "mockery",
    "wolfschlugen" : "wolf",
    "recife  pernambuco" : "recife pernambuco",
    "san lorenzo" : "saint lawrence",
    "samana" : "the same",
    "tangerang selatan" : "south tangerang",
    "nossa senhora do socorro" : "our lady of socorro",
    "via venezia n. 69" : "via venezia n.69",
    "tezpur" : "tamur",
    "balikesi̇r edremi̇t" : "fish edrememi",
    "175 e. schroon river road" : "175 e.river road schron",
    "boa vista  rr" : "boa vista rr",
    "jharsuguda" : "guard",
    "bình thạnh" : "binh thanh",
    "akluj" : "accuse",
    "santo andré   sp" : "santo andré sp",
    "dolny śląsk" : "lower silesia",
    "merkez" : "centre",
    "são paulo (sp)" : "sao paulo-sp)",
    "aurangabad mh" : "aurangabad maha",
    "pomorskie" : "pomeranian",
    "zahle" : "pay",
    "margoa" : "painting",
    "rahata city." : "live city",
    "doda jammu" : "adds to the jammu",
    "bergdietikon" : "bergdietic",
    "nord rhein" : "north rhine",
    "londres" : "london",
    "accettura" : "accepting",
    "bình chánh, tp hồ chí minh" : "binh chanh, ho chi minh city",
    "las palmas de gran canaria" : "the gran canarian palms",
    "coronel fabriciano" : "colonel fabriciano",
    "bhusawal" : "bhusaval",
    "173 xuân thiy" : "173 xuan thuy",
    "navi mumbai (new mumbai)" : "navi mumbai (new moon)",
    "novo hamburgo" : "new hamburg",
    "camarenis sur" : "south camarines",
    "s.s. 115 avola siracusa km 388,8" : "s.s.115 avola siracusa km 388.8",
    "dhanbad" : "dhanbad dhanbad",
    "sutrapada" : "tomorrow",
    "criciúma" : "criciuma",
    "casado" : "married",
    "hosadurga" : "hosadadugha",
    "niš" : "nis",
    "chittaurgarh" : "chittorgarh",
    "neemrana" : "neemountain",
    "kothagudem" : "newgudem",
    "الاغوار الجنوبيه" : "southern jordan valley",
    "via l. da vinci 12" : "via l.da vinci 12",
    "santiago chile" : "santiago, chile",
    "alterosa" : "alterious",
    "berlin   neukölln" : "berlin neukölln",
    "catigua" : "catiguua",
    "caçador" : "hunter",
    "hathars" : "hathras",
    "kruszyn" : "crumble",
    "rangapara" : "raduate",
    "campi salentina" : "salento campi",
    "76 đường 34, an phú, q.2" : "76 street 34, an phu, district 2",
    "villahermosa" : "beautiful villa",
    "19 picadlly rd hampstead" : "19 piccadilly rd hampstead",
    "estrada das carrascas" : "carrascas road",
    "minas gerais visconde do rio branco" : "minas gerais viscount of rio branco",
    "pusad" : "sweaters",
    "via alba 20   roddi" : "via alba 20 roddi",
    "sao paulo sp" : "sao paulo-sp",
    "rio grande do sul   porto alegre" : "rio grande do sul porto alegre",
    "8795 runamuck pl, unit b  medical" : "8795 runamuck pl, unit b medical",
    "20370 passagio drive  medical" : "20370 passage drive medical",
    "sordio" : "sordy",
    "oswego" : "oestrus",
    "pattukkottai" : "tukkottai",
    "virum, denmark" : "before, dermark",
    "gia tân 2, thống nhất, đồng nai" : "gia tan 2, thong nhat, dong nai",
    "cabo sa lucas" : "cabo in lucas",
    "umhlanga rocks" : "reed rocks",
    "guarulhos são paulo" : "guarulhos sao paulo",
    "marília  sp" : "marília sp",
    "via campo marinaro, 8 italia" : "via campo marinaro, 8 italy",
    "mahdia" : "can",
    "paratwada" : "paste",
    "solteiro(a)" : "single",
    "cabreúva" : "cabreuva",
    "pontotoc" : "pontoc",
    "jebel ali" : "mount on",
    "via dante n. 34   nuxis (su)" : "travel n.34 nuxis (su)",
    "phú quí cai lậy tiền giang" : "phu qui cai lay tien giang",
    "hristo smirnenski blv., 30" : "hristo smirnenski bl., 30",
    "cruzeiro" : "cruise",
    "pr   foz do iguaçu" : "pr foz do iguaçu",
    "curitiba.   paraná" : "curitiba.paraná",
    "113 หมู่ 6 ลาดพร้าว 124" : "113 moo 6 lat phrao 124",
    "bambavde, kolhapur" : "bambavade, kolhapur",
    "riaydh" : "riaeyd",
    "pampangga" : "pampanga",
    "pomona" : "pomon",
    "prishtina" : "pristina",
    "fdzs" : "fed",
    "limeira" : "lime",
    "davao city" : "gave city",
    "av. boyacá n 67 96 813013550" : "av.boyacá n 67 96 813013550",
    "mahi" : "assignment",
    "مكه المكرمه" : "makkah",
    "rura,kanpur dehat" : "rura, kanpur dehat",
    "majri" : "stream",
    "al qatif ash sharqiyah" : "al qatif ash sharqi",
    "ecuador manabí  manta" : "ecuador manabí manta",
    "pelotas rs" : "rs balls",
    "minh khai  hai bà trưng hà nội" : "minh khai hai ba trung hanoi",
    "bairagania" : "bargania",
    "putinga" : "to putin",
    "burien" : "yellowish garden",
    "conselheiro pena" : "pena",
    "becharaji" : "disgraceful",
    "1100 lee ave, lafayette, la 70501," : "1100 lee ave, lafayette, 70501,",
    "rio do sul" : "south river",
    "vitotoria" : "mtotoria",
    "gò công đông tiền giang" : "go cong dong tien giang",
    "18, đường số 12, hiệp bình chánh" : "18, road no. 12, hiep binh chanh",
    "bhanga" : "bank",
    "śląsk" : "silesia",
    "san isidro" : "yese isiddro",
    "56a dương thị mười" : "56a duong thi muoi",
    "pangasinan,pozorrubio" : "pangasinan, pozorrubio",
    "garhmuktesar" : "gadhmuktear",
    "nagpur yatmaval" : "nagpur yatmawal",
    "los angeles" : "the angels",
    "beni mellal" : "bani boredom",
    "barra mansa rj" : "mansa rj bar",
    "hajipur,  phubihar" : "hajipur, fubihar",
    "naigoan" : "went",
    "via dei fiumi" : "via dei riversi",
    "buengkum" : "builder",
    "sindgi" : "are",
    "sanyo antônio do monte" : "santo antônio do monte",
    "ambajogai" : "to the ambition",
    "bhrompur" : "bharompur",
    "te a, 2222 w. grand river ave, okemos, mi, 48864" : "a, 2222 w.grand river ave, ok, mi, 48864",
    "kolkata, india" : "kolka, india",
    "centro de distribucion amelia  it" : "amelia it distribution center",
    "2826 nw 72 av" : "2826 nw 72 of",
    "davanagere" : "davangere",
    "dibai" : "glance",
    "taguatinga sul" : "southern taguatinga",
    "tanta" : "so much",
    "puerto wilches" : "wilches port",
    "colina" : "hill",
    "9110 forest crossingthe woodlandstx77381" : "9110 forest crossing the woodlands tx 77381",
    "mandigobindgarh(amandeepanand786@gmail.com)" : "mandigobindagarha (amandipananda",
    "são paulino" : "sao paulino",
    "cabo frio   rj" : "cabo frio rj",
    "74 molave st. z 1 central signal village, taguig city philippines" : "74 molave st.z 1 central signal village, taguig city philippines",
    "warmińsko mazurskie" : "warmian masurian voivodeship",
    "mankapur gonda" : "manikpur gonda",
    "itajaí   sc" : "itajaí sc",
    "dumka" : "opinion",
    "raichur" : "raycur",
    "governador valadares mg" : "governor valadares mg",
    "hyderabab" : "hyderabad",
    "serra   espírito santo" : "serra espírito santo",
    "vico 2° ginnasio 8" : "vico 2nd gymnasium 8",
    "missoula" : "touched",
    "hailakandi" : "hallakandi",
    "ras alkhaimah" : "ras al khaimah",
    "yeddumailaram" : "bull",
    "butler" : "thighs",
    "rio de janiero" : "rio de janeiro",
    "batala" : "butter",
    "atlanta, ga" : "ah, but",
    "brasília   df" : "brasilia df",
    "อยุธยา" : "ayutthaya",
    "الاحساء" : "hasa",
    "al fujairah" : "the dawn",
    "crisciuma" : "i'm going to",
    "الانبار" : "anbar",
    "ciudad de mexi o" : "mexico city",
    "assis são paulo" : "assis sao paulo",
    "thanh hoá" : "thanh hoa",
    "viale alpi orientali nord, 6 int. 3" : "viale alps orientali nord, 6 int.3",
    "kodikonda check post" : "choose check post",
    "gidderbaha" : "budgebaha",
    "bauneario camburiu" : "balneário camboriú",
    "narre warren" : "fool warren",
    "รพ.สมิติเวช" : "samitivej hospital",
    "mukherjinagar" : "mukherjnagar",
    "terlingua" : "elder",
    "ernakulum" : "ernamum",
    "6300 harry hines blvdste. 1400dallastx75235" : "6300 harry hines blvd ste. 1400 dallas tx 75235",
    "ciudad arce" : "arce city",
    "sta. maria bulacan" : "stay.maria bulacan",
    "colíder" : "collide",
    "new canaan" : "new year",
    "piedras negraa" : "black stones",
    "campo verde mt" : "green field mt",
    "102c đường 79, phước long b," : "102c street 79, phuoc long b,",
    "padre hurtado  central 673las condes santiago" : "father hurtado central 673las condes santiago",
    "salvador   ba" : "salvador ba",
    "milão" : "milan",
    "nürnberg" : "nuremberg",
    "galati" : "galata",
    "biała" : "white",
    "bhndara" : "bhandara",
    "chatrapati sambhajinagar" : "chhatrapati sambhajinagar",
    "la trinidad" : "the trinity",
    "mandvi" : "mandavi",
    "lima lima" : "five five",
    "leicesterhsire" : "leicestershire",
    "den haag" : "the hague",
    "sojatcity" : "sojat city",
    "santa bárbara d'oeste" : "santa barbara d'oeste",
    "104 caldo" : "104 hot",
    "القاهره" : "cairo",
    "611 w. park street fait medical" : "611 w.park street make medical",
    "tiruchirapalli" : "truuchrapalli",
    "luna la union" : "luna laon",
    "odisha baripada" : "odisha barada",
    "parli vaijnath" : "married vijayanath",
    "trà ôn, vĩnh long" : "tra on, vinh long",
    "kovilpatti" : "kovilpat",
    "दिल्ली" : "delhi",
    "pangkal pinang" : "town",
    "जैतारण आनंदपुर कालू" : "jaitaran anandpur kalu",
    "newnan" : "they were new",
    "jawa tengah" : "central java",
    "ahad rofaida ksa" : "sunday rofaida ksa",
    "barraquilla" : "barranquilla",
    "jardim do seridó rn" : "seridó rn garden",
    "مبارك الكبير" : "mubarak al -kabeer",
    "gov. valadares" : "gov.valadares",
    "macae rj" : "macae raj",
    "karaikal" : "stains",
    "tusayan" : "show",
    "hoogbocht 1" : "beam 1",
    "810 n kingston dr peoria, il 61604 2145" : "810 n kingston dr peoria, il",
    "jabel ali" : "mount on",
    "pandhurna" : "bad",
    "thành phố hồ chí minh" : "ho chi minh city",
    "gaouteng" : "karcarian hurts",
    "tân bình" : "tan binh",
    "kapadwanj" : "kapadwanjj",
    "ouro branco" : "white cholocate bonbon",
    "itaboraí   rj" : "itaboraí rj",
    "κ. βαρναλη 130 132" : "k. varnali 130 132",
    "xalapa e ver" : "xalapa and see",
    "imbituva pr" : "pilva pr",
    "rosario   santa fe" : "rosario santa fe",
    "jakarta timur" : "east jakarta",
    "arkport" : "carpet",
    "ichalkaranji" : "akinkaranan",
    "via col vento" : "gone with the wind",
    "nordrhein westfalen   herford" : "north rhine westphalia herford",
    "amarillo" : "yellow",
    "uruara" : "unafare",
    "mancherial" : "mancheril",
    "russas" : "russian",
    "500 valley rd.,  suite 202  it" : "500 valley rd.,  suite 202  it",
    "campo novo" : "new field",
    "tp ho chi minh" : "ho chi minh city",
    "nowe skalmierzyce" : "new skalmierzyce",
    "san nicolas de los garza" : "san nicolas de los heros",
    "9700 maumelle blvd" : "9700 multidio blvd",
    "changanacherry, kottayam" : "changanacheri, kottayam",
    "keshav rai patan" : "keshav rai collapse",
    "ongole" : "credit",
    "bogazkoy" : "bogatskoy",
    "mumbai ( mumbai)" : "mumbai (mumbai)",
    "القليوبيه" : "qaliubiya",
    "เพชรบูรณ์" : "phetchabun",
    "cachoeira do piriá pará" : "piriá pará waterfall",
    "30 ung văn khiêm bình thạnh" : "30 ung van khiem binh thanh",
    "nova andradina   mato grosso do sul" : "nova andradina mato grosso do sul",
    "małopolskie, 34 436 maniowy" : "lesser poland, 34 436 maniowy",
    "baradwar" : "paradise",
    "dhuri" : "the wall",
    "el segundo" : "the second",
    "anjugramam" : "anjugaramam",
    "2 alexfisher drive    burliegh heads  qld  4220  us qld" : "2 alex fisher drive burleigh heads qld 4220 us qld",
    "hai phong" : "two phong",
    "pedro leopoldo" : "pedro leopold",
    "thành phố hồ chí minhp" : "ho chi minh city",
    "arambagh" : "arambagi",
    "11 penns trail bldg. 100" : "11 pennies trail bldg.100th",
    "centro" : "center",
    "salete" : "silet",
    "shivapark" : "shivakarak",
    "9x chu văn an" : "9x is văn",
    "rua doutor álvaro albuquerque" : "rua doctor álvaro albuquerque",
    "184 lê đình can" : "184 le dinh can",
    "małopolskie" : "lesser poland",
    "hưng yên" : "hung yen",
    "jaora" : "loser",
    "franca   sp" : "franca sp",
    "14334 e layton pl  medical" : "14334 e layton pl medical",
    "gaziabad" : "attractive",
    "bakı" : "baku",
    "birsa, balaghat, mp india" : "birsa, balaghat, pump india",
    "kalimantan barat" : "west kalimantan",
    "nashirabad" : "nasirabad",
    "cancún, q. roo" : "cancun, q.roo",
    "distrito capital de bogotá" : "capital district of bogotá",
    "ofibodegas fraijanes bodega 23 km. 18 carretera a san jose  commercial display" : "ofibodegas fraijanes bodega 23 km.18 road to san jose commercial display",
    "الطائف" : "taif",
    "vespasiano" : "vespasian",
    "via irlanda, 12" : "via ireland, 12",
    "vitoria es" : "vitoria is",
    "gulbarga" : "to a floron",
    "gorari" : "lalli",
    "botad" : "cured",
    "φιλολαου 28 ηλιουπολη" : "filololaou 28 ilioupoli",
    "ipatinga  mg" : "ipatinga mg",
    "santa cruz do capibaribe   pe" : "santa cruz do capibaribe pe",
    "se" : "himself",
    "karachi gulshane shamim fb area" : "karachi gulzhane shamim fb area",
    "corso m. d'azeglio 30" : "course m.d'azeglio 30",
    "kodinar" : "codeinar",
    "malappuram, manjeri" : "malappuram, manjari",
    "cartegena" : "cartagena",
    "paulínia" : "paulinia",
    "tân phước tiền giang" : "tan phuoc tien giang",
    "ganaur, sonipat" : "ganore, sonipat",
    "wielkopolskie" : "greater poland",
    "9000 executive prk dr ste c200 healthcare" : "9000 executive park dr ste c200 healthcare",
    "جازن بني مالك" : "jazen bani malik",
    "såo paulo" : "são paulo",
    "c. so  principe eugenio, 3" : "c.i know prince eugenio, 3",
    "arava" : "cart",
    "são paulo  são paulo" : "sao paulo sao paulo",
    "palvoncha telangana" : "palvoncha telankana",
    "ದೋಡಬಳಪುರ" : "dodalpur",
    "pitalito  huila" : "pitalito huila",
    "la reian" : "i ree",
    "sp   taubaté" : "sp taubaté",
    "peruibe" : "peruve",
    "la habana" : "havana",
    "n, la cabecera, san felipe del progreso estado de mexico estado de mexico" : "n, the head, san felipe del progreso state of mexico state of mexico",
    "ventanas" : "windows",
    "bhainsa" : "buffalo",
    "dalkhola" : "dalhala",
    "bengalore" : "bengaloreore",
    "bijapur(kar)" : "bijapur (tax)",
    "سكاكا الجوف" : "sakaka al -jouf",
    "qurum" : "institution",
    "riffa" : "rife",
    "alegrete" : "cheerful",
    "bueos aires caba" : "buenos aires caba",
    "tiruvannamalai" : "thiruvannamalai",
    "dois irmãos" : "two brothers",
    "vllavicencio" : "villavicencio",
    "1940 jefferson hwy" : "1940 jefferson longer",
    "la serena" : "serena",
    "quinta normal" : "normal fifth",
    "niedersachsen" : "lower saxony",
    "dalsingh sarai" : "dalsingh sarah",
    "mersi̇n" : "mersow",
    "naazareno" : "nazarenaw",
    "palani" : "france",
    "10 nguyễn thanh đằng, bà rịa" : "10 nguyen thanh dang, ba ria",
    "sagarejo" : "sagrejo",
    "são josé dos pinhais   paraná" : "são josé dos pinhais paraná",
    "padrauna" : "subtle",
    "nieborów" : "heavenly",
    "várzea grande" : "varzea grande",
    "abugon, sibonga, cebu" : "about, sibonga, cebu",
    "ροσταν 44" : "rostan 44",
    "3330 n 104 ave  234 hotel" : "3330 n 104th ave 234 hotel",
    "malout" : "tend",
    "bhadaur" : "bhadore",
    "são luís do quitunde   alagoas" : "são luís do quitunde alagoas",
    "sulawesi selatan" : "south sulawesi",
    "umm al quwain" : "umm qun",
    "mazowieckie , radom" : "mazowieckie, radom",
    "aldie" : "all",
    "hidrolândia" : "hydroland",
    "江苏" : "jiangsu",
    "sopore" : "slackly",
    "pinhais" : "pinemers",
    "zamb0anga city" : "zamboanga city",
    "vennanthur, namakkal" : "vennandur, namakkal",
    "riyadh, ksa" : "riad, story",
    "dhahran" : "dharan",
    "ggn" : "evidence",
    "santo antônio de pádua" : "santo antônio de padua",
    "ende" : "end",
    "arroio do sal" : "salt",
    "phường 6 , quận gò vấp" : "ward 6, go vap district",
    "iloilo city" : "iloulo city",
    "chittaranjan" : "delicate",
    "perambalur." : "pounding.",
    "almenara" : "beacon",
    "75a, phan đình phùng, phú nhuận" : "75a, phan dinh phung, phu nhuan",
    "nalbari" : "illumon",
    "sitamarhi ,bihar" : "sitamarhi, bihar",
    "32 dương quảng hàm" : "32 duong quang ham",
    "ballari" : "scores",
    "villupuram" : "vilupuram",
    "canaã dos carajás" : "canaan of carajás",
    "jakarta utara" : "north jakarta",
    "jaboatão   pe" : "jaboatão pe",
    "163 trần phú, đà nẵng" : "163 tran phu, da nang",
    "lumut,perak" : "moss, silver",
    "kodungallur" : "kodungallurur",
    "kotli surat malli" : "kotli surat model",
    "av 2g # 44 32" : "of 2g # 44 32",
    "حدائق القبه" : "dome gardens",
    "cn   monasterolo di savigliano" : "cn monasterolo di savigliano",
    "कैलाश कुमार" : "kailash kumar",
    "bulacan bulacan" : "bulcan bulcan",
    "villavcencio" : "villavicencio",
    "10880 wilsire blvd suite 1101 lg pro" : "10880 wilshire blvd suite 1101 lg pro",
    "chowari" : "streets",
    "al sokhna" : "the heat",
    "q. thủ đức" : "q.thu duc",
    "36 hoàng cầu, đống đa, hà nội" : "36 hoang cau, dong da, hanoi",
    "barra mansa" : "mansa bar",
    "103 urb el retiro  medical" : "103 urb el retiro medical",
    "lattarico" : "lactic",
    "69 lakshmi garden, thindal m v k illam" : "69 lakshmi garden, illam in thindal",
    "são gonçalo" : "sao goncalo",
    "darwha" : "tarry",
    "zachoodniopomorskie" : "west pomeranian",
    "wielkopaolska" : "large",
    "1037 des marguerites  it" : "1037 of marguerites it",
    "pedregulho" : "boulder",
    "3351 michelson dr ste 100  medical" : "3351 michelson dr ste 100 medical",
    "tân an, càng long, trà vinh" : "tan an, lau long, tra vinh",
    "lares decora" : "lares decorates",
    "angwin" : "anguish",
    "5300 kell blvd" : "5300 need blvd",
    "diadema" : "diadem",
    "pipartola, balaghat, mp india" : "pipatola, balaghat, pump india",
    "22 đường 5 linh xuân thủ đức" : "22 road 5 linh xuan thu duc",
    "godda" : "new",
    "limoeiro" : "lemon tree",
    "memari" : "keep",
    "קרית גת" : "kiryat gat",
    "hyderabd" : "hyderabad",
    "sacile" : "sail",
    "100 crowley dr  medical" : "100 crowley dr  medical",
    "el shrouk 2" : "el shorouk a",
    "ghughli" : "google",
    "timóteo" : "timothy",
    "anand vihar delhi" : "anand storm delhi",
    "thiruvananamatlai" : "thiruvananthapuram",
    "poza rica" : "picture rica",
    "delhij" : "delichium",
    "lima   peru" : "lima's name",
    "müllheim" : "garbage home",
    "asolo" : "asol",
    "pfalz" : "palatinate",
    "台北市" : "taipei city",
    "islamabad" : "islamabad",
    "azamgarg up" : "azamgarg",
    "cédula de ciudadanía" : "citizenship card",
    "ميسان" : "maysan",
    "andersen afb" : "andersen fig",
    "rionegro" : "black river",
    "vpo rani khera" : "hear",
    "τατοΐου 65" : "tattoo 65",
    "castanhal" : "brown",
    "jazan" : "jizan",
    "ha tiên" : "ha tien",
    "almanya" : "germany",
    "1900 lukken industrial drive west" : "1900 luken industrial drive west",
    "1919 minnesota ct, mississauga, on l5n" : "1919 minnesota ct, mississauga, is l5n",
    "bom jesus" : "good jesus",
    "aadgaon" : "aadga",
    "mour mandi" : "dying",
    "oshogbo" : "embon",
    "mahroni" : "disadvantaged",
    "paschim medinipur (west medinipur)" : "west midnapore (west medinipur)",
    "الجلة الرياض" : "al jala riyadh",
    "สมุทรปราการ" : "samut prakan",
    "abohar" : "warehouse",
    "los gatos" : "the cats",
    "banka" : "knock",
    "dohrighat" : "pink",
    "gautam budha nagar" : "gautam mercury nagar",
    "maceió alagoas" : "maceio alagoas",
    "koszecin" : "cosecin",
    "portão" : "gate",
    "चाकण" : "chickpe",
    "santo domingo este rep. dom." : "saint sunday this rep.sun.",
    "cerejeiras" : "cherry trees",
    "cruzeiro do sul acre" : "southern cruzeiro acre",
    "zachodniopomorskie" : "west pomeranian",
    "fatehnagar" : "fatehanagar",
    "kharabi nagpur" : "morning nagpur",
    "saharsa" : "sarhasa",
    "tp.hồ chí minh" : "ho chi minh city",
    "ciudad de méxico  it" : "mexico city it",
    "lapu lapu city" : "lapa lap city",
    "הר אדר" : "mount adar",
    "faropilhas" : "barefoot",
    "navapur" : "nawapur",
    "sarita chorakiasundeep jain" : "saree",
    "carpina" : "carpine",
    "rio real" : "river royal",
    "pudukkottai" : "spright",
    "cajati" : "gentle",
    "este" : "it is",
    "purna" : "full",
    "mattapoisett" : "matte",
    "cañas" : "white hair",
    "punr" : "rear",
    "harda" : "as well as",
    "3 vpo sunet , brs nagar" : "3 vapo sunat, bars nagar",
    "joão pesssoa" : "joão pessoa",
    "esch sur alzette" : "esch surge of alzette",
    "pooler" : "pools",
    "araras" : "macaws",
    "querètaro" : "wanttaro",
    "daund" : "race",
    "naucalpan , mexico" : "naucalpan, mexico",
    "opolskie" : "opole",
    "ulu tiram, johor bahru" : "ulu oyster, johor bahru",
    "sénégal" : "senegal",
    "dhuliyan" : "dhulian",
    "saudade do iguaçu" : "i miss iguaçu",
    "são luís maranhao" : "sao luis maranhão",
    "marki, mazowieckie" : "brands, masovian voivodeship",
    "minas gerais   belo horizonte" : "minas gerais belo horizonte",
    "25 hoàng sa phường dakao quận 1" : "25 hoang sa ward dakao district 1",
    "barmer rajasthan" : "badmer rajasthan",
    "15an mỹ" : "15an american",
    "newdelhi" : "new delhi",
    "vandavasi" : "trouble",
    "naduvattam" : "inflate",
    "อ เมือง จ ภูเก็ต" : "phuket",
    "pitangui mg" : "pihong ghost gate",
    "mapusa" : "hatch",
    "indiabulls  greens, kon, tal. panvel, pin code 410221" : "indiabulus greens, angle, tal.panvel, pin code 1",
    "churu" : "a thousand",
    "sindgi distik vijaypur" : "life dastic vijaypur",
    "menofia" : "monsterness",
    "ciudad de mexic" : "mexican city",
    "au ricchieri sn" : "a ricciid sn",
    "sriganganagar" : "shri ganga nagar",
    "ไทย" : "thai",
    "ramanagara" : "ragara",
    "hod hasharon" : "competition beef",
    "wa" : "of",
    "at post   satana, tal.satana, district. nashik  hotel" : "at post satana, tal.satana, district.nashik hotel",
    "campo mourão" : "campo mourao"
}

trans_dic['customer_job'] = {
    "denduluru" : "delluder",
    "waukegan" : "waoi cancer",
    "cotonou" : "coton",
    "pundri" : "pundari",
    "bresso" : "brex",
    "armação dos búzios" : "buzios frame",
    "ramgarh cantt" : "ramgarh santat",
    "acharnai" : "relevant",
    "criciuma" : "cricium",
    "junagadh gujarat" : "junagarh gujarat",
    "ấp hòa hữu  xã long đức tptv" : "hoa huu hamlet, long duc commune, tptv",
    "valle de bravo" : "bravo valley",
    "กรุงเทพฯ" : "bangkok",
    "rancho queimado" : "burnt ranch",
    "abha" : "river",
    "villahermosa, tabasco" : "villahermosa tabasco",
    "presidente getulio sc" : "president getulio sc",
    "1 kv2 p.trà nóc" : "1 kv2 p.trà noc",
    "paço do lumiar" : "lumiar paço",
    "aracaju   se" : "aracaju if",
    "morando em são paulo" : "living in são paulo",
    "ayankig" : "youyankig",
    "hatimiyyah street   muwaqqar p. o. box 141351" : "hatimiyyh street muwaqar p.o.box 141351",
    "merut" : "meerut",
    "góra kalwaria" : "mount kalwaria",
    "long hựu tây cần đước long an" : "long huu tay can long an",
    "tuchorza" : "tuchorz",
    "mumvai" : "mumwai",
    "nalagonda" : "disagree",
    "53 e 1st ave" : "೫೩ this ೧ st",
    "maringa pr" : "on mringa",
    "fortaleza ce" : "fortaleza what",
    "2 sengol nagar 3 rd street old vilangudi" : "௨ scepter city ௩ street old",
    "jonadi" : "jondi",
    "châu đôc" : "chau dac",
    "القطيف" : "qatif",
    "quảng ngãi" : "quang ngai",
    "campos rj" : "rj fields",
    "antique philippines" : "ancient philippines",
    "ekangersarai" : "ecadesrai",
    "tp hồ chí minh" : "ho chi minh city",
    "ribeirão das neves" : "stream of snow",
    "raposos" : "fox",
    "ma   timon" : "my drawbar",
    "ottawa ontario" : "i followed ontario",
    "cd de mexico" : "mexico cd",
    "coronel fabriciano mg" : "colonel fabriciano mg",
    "berhampore" : "in berhampur",
    "cossato" : "sock",
    "gaziabadh" : "ghaziabad",
    "dias d'avila ba" : "days d'avila ba",
    "durban , kwa zulu natal" : "durban, to zulu natal",
    "elkhart" : "garden",
    "maumbai" : "mumbai",
    "morbi" : "soccer",
    "via di s. erasmo 12" : "via di s.erasmus 12",
    "novia" : "bride",
    "6252 e grant rd suite 150 tucson, az 85712" : "6252 is a grant rd swing 150 tutzon, i 85712",
    "hồ chí minh" : "ho chi minh",
    "hata" : "even",
    "1602 e yandell dr" : "1602 e yandell dr dr.",
    "arroyo hondo viejo" : "hondo viejo arroyo",
    "juliaca   san roman   puno" : "juliaca san roman puno",
    "pondichéry" : "pondicherry",
    "itapema sc" : "it will spit sc",
    "phường tân phong biên hoà đồng nai" : "tan phong bien hoa ward dong nai",
    "885 haleakala hwy" : "889 videos",
    "kishtwar" : "tendering",
    "las parejas" : "the couples",
    "platón sánchez" : "plato sánchez",
    "palhoça" : "hazel",
    "la misma" : "the same",
    "seria" : "he would be",
    "સુરત" : "surat",
    "nuxis (su)   via dante n. 34" : "nuxis (your) dante n.3. 4",
    "lampa" : "lamp",
    "baquba" : "the bacquuba",
    "bacolod city nergros occidental" : "bacolod city negros occidental",
    "السويس" : "suez",
    "tambopata" : "tamopata",
    "290 thành công, q. tân phú" : "290 success, q.tan phu",
    "vijayapura" : "vijaypura",
    "cordba" : "cordoba",
    "santo andre   sao paulo" : "santo andre sao paulo",
    "buenos aires   azul" : "buenos aires blue",
    "1 penn plz ste 1400 fl 14" : "1 pen plz rose 1400 fl 14",
    "al riyad" : "riyadh",
    "ecatepec , edo mex" : "ecatepec, edo mex",
    "hapur" : "open",
    "guarne" : "cheek",
    "mntpellier" : "montpellier",
    "una" : "a",
    "partappur" : "pratappur",
    "colegiales" : "schoolboys",
    "koribori 8g" : "corbari 1 c",
    "brasilia   águas claras" : "brasilia águas claras",
    "arain rajasthan" : "rajasthan",
    "1850 m. h. del pilar st,  malate" : "1850 m.h.of the pillar st, malate",
    "tapira mg" : "tap mg",
    "santa ines" : "holy ines",
    "palmas   pr" : "palmas pr",
    "hindamata" : "unstated",
    "gyál" : "gate",
    "số 7 đường 3419 phường 7 quận 8" : "no. 7 street 3419 ward 7 district 8",
    "jacareí" : "jacarei",
    "mazowsze" : "mazovia",
    "1605 ave. ponce de león, suite 400 san juan, 00909, puerto rico" : "1605 ave.ponce de león, suite 400 san juan, 00909, puerto rico",
    "purnia" : "purania",
    "pato branco pr" : "ties on the bronze",
    "นนทบุรี" : "nonthaburi",
    "capivari de baixo" : "downstream",
    "auragabad" : "aura",
    "حفر الباطن" : "hafar al -batin",
    "15 phan huy thực, tân kiểng, q7" : "15 phan huy thuc, tan kieng, district 7",
    "cali   valle del cuaca" : "cali valle del cauca",
    "koura" : "gold",
    "الشرقية" : "eastern",
    "gubin" : "burn",
    "huyện chợ gạo  tỉnh tiền giang" : "cho gao district, tien giang province",
    "jatara" : "jatra",
    "sheoganj" : "sheyganj",
    "carrera 66b # 6   26 805006050" : "carrera 66b # 6 26 805006050",
    "almirqab" : "alminqab",
    "ktx khu b dhqg tp.hcm" : "dormitory of building b, ho chi minh city",
    "hydrabad" : "hyderabad",
    "rajkot    gujarat" : "rajkot gujarat",
    "são paulo  sp" : "sao paulo-sp",
    "phước long b" : "phuoc long b",
    "chile santiago" : "santiago chile",
    "quận 2, tp. hcm" : "district 2, tp.hcm",
    "dhrol" : "drol",
    "perintalmanna" : "pond",
    "sumatera selatan" : "south sumatra",
    "bulandshahar" : "bulandshahr",
    "goiania   go" : "goiania go",
    "tân an  ,càng long, trà vinh" : "tan an, lau long, tra vinh",
    "aurngabad" : "aurangabad",
    "sete lagoas   mg" : "sete lagoas mg",
    "cra 29 a # 5 sur 125 valle" : "cra 29 a # 5 of 125 valle",
    "wakas nort" : "end nort",
    "浙江省杭州市" : "hangzhou, zhejiang",
    "mumbay" : "mumbai",
    "נתיבות" : "netivot",
    "al ayat   giza" : "the giza",
    "garhshankar" : "gadhashankar",
    "hoà bình trà ôn vĩnh long" : "hoa binh tra wen vinh long",
    "dhamnod dhar m.p" : "dhamnod dhar mp",
    "yirka" : "the umbri",
    "formello" : "fork",
    "uruli kanchan" : "urali kanchan",
    "台南市" : "tainan city",
    "uran" : "career",
    "14058 mirabella dr  education" : "14058 mirabella dr education",
    "muriae" : "muria",
    "mundra kutch" : "mundra inch",
    "talafar" : "fiction",
    "ciudad del este" : "eastern city",
    "qalubia" : "heartfelt",
    "registro sp" : "sp registration",
    "funza" : "learn",
    "venturosa pe" : "venturous pe",
    "bremerhavcen" : "bremerhaven",
    "são paulo   sp" : "sao paulo-sp",
    "dumaguete city, negrosd oriental" : "dumaguete city, negros oriental",
    "18 nguyễn văn lượng, p6 gò vấp" : "18 nguyen van luong, p6 go vap",
    "kajang" : "brim",
    "sheridan" : "lion",
    "senador canedo" : "senator canedo",
    "kabupaten sanggau, kalimantan barat" : "sanggau regency, west kalimantan",
    "estuar" : "estuary",
    "migo das cruzes" : "mogi das cruzes",
    "thiruvanthapuram" : "thiruvantapuram",
    "chikmagalore" : "the chikamagore",
    "ciudad autonoma de buenos aires" : "autonomous city of buenos aires",
    "porto nacional" : "national port",
    "coonoor tamilnadu" : "coonor tamilnadu",
    "via fontana rutola,22" : "via fontana rutola, 22",
    "al ayat" : "a",
    "ibira" : "pull",
    "avare" : "stingy",
    "canoinhas" : "canoiinhas",
    "الديوانية" : "diwaniyah",
    "pavoorchatram" : "pavoric chatram",
    "nova mutum" : "noa man",
    "manab ray" : "manab re",
    "joelformcheck magnit" : "joelformcheck magnet",
    "lucko" : "lutsk",
    "via spiaggia,328" : "via beach, 328",
    "papanduva" : "papoda",
    "hòa thuận châu thành tỉnh trà vinh" : "harmony of tra vinh province",
    "tasocagi cad. no:64" : "tasocagi cad.no: 64",
    "piazzale s. giuseppe, 10" : "square s.giuseppe, 10",
    "osório" : "osorium",
    "مدينة الجلود بدر" : "the city of leather badr",
    "yahoud  monoson" : "monson jews",
    "thạch bàn long biên hà nội" : "thach ban long bien hanoi",
    "namkhana" : "name",
    "espírito santo do pinhal" : "holy spirit of pinhal",
    "pilkhuwa, distt hapur, west up" : "pilkhuwa, distt hapur, west sub",
    "bhayandar" : "bayander",
    "santo antônio do tauá   pará" : "saint anthony of tauá pará",
    "paranaque  city" : "paranaque city",
    "viale  odino  4" : "viale odin 4",
    "kothara abdasa 370645" : "the statement of the word 1",
    "peixe   tocantins" : "tocantins fish",
    "lan o lakes" : "lan the lakis",
    "la paz" : "peace",
    "navegantes   sc" : "sc navigators",
    "sp são paulo" : "sp sao paulo",
    "jakarta utara, kelapa gading" : "north jakarta, kelapa gading",
    "makka" : "mecca",
    "nova scotia" : "new scotia",
    "amargosa valley" : "bitter",
    "gzira" : "young",
    "partapur" : "paratapur",
    "cascina" : "farmhouse",
    "venda nova do imigrante" : "new sale do immigrant",
    "toronto on" : "toronto is",
    "rio de janeiro e região" : "rio de janeiro and region",
    "terra nova do norte mt" : "northern terr mt",
    "miraj" : "mirage",
    "de leon" : "by leon",
    "brześce" : "brzesce",
    "los andes" : "the andes",
    "saúde ba" : "ba health",
    "3125 avenida topanga" : "to topan 3125 avenue",
    "广东省深圳市" : "shenzhen city, guangdong province",
    "dois vizinhos. pr" : "two neighbors.pr",
    "etah" : "fsl",
    "burla parit" : "burla couples",
    "112 sol building amorsolo st. legaspi village, makati city manila, philippines" : "112 sol building amorsolo st.legaspi village, makati city manila, philippines",
    "carrera 7a # 69 99 d.c." : "career 7a # 69 99 a.c.",
    "la pampa   santa rosa" : "la pampa santa rosa",
    "rio grande" : "big river",
    "niwali" : "preached",
    "al ques" : "at any",
    "guarujá são paulo" : "guarujá sao paulo",
    "siek" : "sick",
    "castello d argile" : "castle of swimming pool",
    "odrano wola" : "wola was rebuke",
    "3300 e.guasti rd  medical" : "3300 e.guasti rd medical",
    "viale della navigazione interna, 48" : "viale della internal navigation, 48",
    "nanaimo" : "seven places",
    "karur" : "some",
    "building 18a 3rd nangang industrial park tangtou shiyantown baoan district  commercial display" : "building 18a 3rd nangang industrial park tangtou shiyan town baoan district commercial display",
    "são paulo, pinheiros" : "sao paulo, pinheiros",
    "matinhos" : "pound",
    "594 howard stsan franciscoca94106" : "594 howard st san francisco ca 94106",
    "bishrampur" : "restor",
    "مكة" : "mecca",
    "la spezia" : "spice",
    "bol" : "he was",
    "maarab" : "arab",
    "kuchina" : "to china",
    "גבעת זאב" : "ze'ev hill",
    "londrina pr" : "at laundrina",
    "terdal" : "consistently",
    "alvorada" : "dawn",
    "baripada" : "range",
    "icó" : "icon",
    "mavelikara" : "mavelikkara",
    "santa catarina" : "st. catarina",
    "sombrio" : "dingy",
    "barueri   são paulo" : "barueri sao paulo",
    "bhubanewar" : "bhubnewar",
    "ghazizbad" : "scope",
    "birghman or london" : "birmingham or london",
    "bergkamen" : "mountain ranges",
    "cao lãnh" : "cao lanh",
    "nishi shinjuku, shinjuku, tokyo" : "nishi -shinjuku, shinjuku, and",
    "bhucho mandi" : "bhucho took a shower",
    "rodeville" : "roseville",
    "1000 primera blvd" : "1000 first blvd",
    "cantton, ohio" : "canton, ohio",
    "153 210, illuppapattu village rajakulam post , kanchipuram" : "೨೧೦ ೨೧೦, illuppapattu village rajakulam post, kanchipuram",
    "otorowo" : "you are aological",
    "pr londrina" : "laundrina on",
    "khorda" : "chord",
    "purifiacacion tolima" : "tolima purification",
    "i̇stanbul (anadolu)" : "istanbul (anatolia)",
    "avenida hostos 410 ave hostos medical" : "hostos avenue 410 ave hostos medical",
    "kasur punjab" : "punjab kasur",
    "الغردقة" : "hurghada",
    "baliguda ,district.. kandhamala, orisa" : "baliguda, district .. kandhamal, orissa",
    "krishnagar" : "black",
    "major izidoro   alagoas" : "major izidoro alagoas",
    "dolnosląskie" : "lower silesian",
    "gevrai" : "gavarai",
    "thủ đức" : "thu duc",
    "vellore,tamil nadu" : "vellore, tamil nadu",
    "amour( baisi)" : "love (baisi)",
    "józefów k. otwocka" : "józefów near otwock",
    "ujazd" : "utille",
    "las pinas city" : "the city pinas",
    "es cariacica" : "cariacica",
    "attiki" : "attica",
    "brasília df" : "brasilia df",
    "9 âu cơ tân bình" : "9 au co tan binh",
    "maceió   alagoas" : "maceio alagoas",
    "القصيم بريدة" : "qassim buraidah",
    "حائل" : "barrier",
    "alipurduar" : "alipurdar",
    "tuxtla gutiérrez" : "tuxtla gutierrez",
    "campos dos goytacazes" : "goytacazes fields",
    "singapoer" : "singapore",
    "calle del trigal 255, quilicura" : "255 trigal street, quilicura",
    "5900 balcones ste 5170" : "5900 ste 5170 balconies",
    "2101 nuuanu ave" : "210170 aflost airline around",
    "senador canedo go" : "senator canedo go",
    "betivoglio" : "bentivoglio",
    "lunavada" : "ransom",
    "gangtok" : "gangts",
    "badaun" : "adults",
    "passos mg" : "mg steps",
    "metropolitana buin" : "metropolitan buin",
    "jakarta pusat" : "central jakarta",
    "lima peru" : "lima's name",
    "ahmedbad" : "ahmed",
    "bijainagar" : "bijayanagar",
    "2414 ถนนเพชรบุรึตัดใหม่" : "2414 new phetchaburi road",
    "dghgfdh" : "dghfh",
    "พระนครศรีอยุธยา" : "ayutthaya",
    "los alamitos" : "the alamitos",
    "são bento do sul" : "sao bento do sul",
    "cra 20 f # 61 b   19 sur" : "cro 20 f # 61 b 19 sur",
    "governador celso ramos" : "governor celso ramos",
    "świętochłowice (woj. śląskie)" : "świętochłowice (śląskie voivodeship)",
    "avcılar" : "hunters",
    "minterrey" : "monterrey",
    "macapá   amapá" : "macapá amapá",
    "sc  itapema" : "sc will shoot",
    "ca' zusto 52" : "ca 'zusto 52",
    "triunfo" : "triumph",
    "airoli" : "aeroli",
    "θέση πέτσα βακαλοπούλου βιοπα παλλήνης  153 51" : "peta peta vakalopoulou biopa pallini 153 51",
    "costa del este" : "east coast",
    "cty tnhh kỹ thuật hts" : "hts technical company limited",
    "santo andre" : "santo other",
    "passa e fica" : "passes and stays",
    "bari, italy" : "barry, italian",
    "ponte nova" : "new bridge",
    "cedula de ciudadania" : "citizenship card",
    "la liberfad" : "freedom",
    "kondotty" : "conduit",
    "mato grosso   sorriso" : "mato grosso smile",
    "ras al khimah" : "ras al khaimah",
    "privato" : "private",
    "prata" : "talk",
    "salto guaira" : "jump guaira",
    "ереван" : "yerevan",
    "nuevo león" : "new lion",
    "jammuand kashmir" : "jammu and kashmir",
    "cali   valle" : "valley",
    "passo fundo   rs" : "passo fundo rs",
    "itaira" : "consuming",
    "21 lạc long quânp8 qtân bình" : "21 lac long quanp8 qtan binh",
    "la grange" : "barn",
    "kannur keray" : "kannur",
    "go luziania" : "him",
    "npida" : "distributing",
    "marinha grande" : "large navy",
    "gramado" : "lawn",
    "campinas   sp" : "campinas, sp",
    "udaipur g 2. ankur complex opp narayan seva sansthan sex.5" : "udaipur c 2. ankur complex o narayan seva sansthan sex. 5",
    "telengana" : "telenganana",
    "930 fairway dr ne" : "930 fairway",
    "panama oeste" : "west panama",
    "suchy las" : "dry forest",
    "bécsi út 3 5" : "vienna road 3 5",
    "277 ave coll y toste" : "277 ave coll and toste",
    "caruaru pe" : "caruaru on",
    "osie" : "axles",
    "orai" : "weather",
    "tupanatinga" : "tupaniatation",
    "kudan kulam" : "kudon pond",
    "cambé   pr" : "camps pr",
    "assandh" : "blessed",
    "cra. 51 #12 sur 75, san fernando, itagüi, medellín, guayabal, medellín, antioquia, colombia" : "cra.51 #12 sur 75, san fernando, itagüi, medellin, guayabal, medellin, antioquia, colombia, colombia, colombia, colombia",
    "gustavo a. madero" : "gustavo a.log",
    "mas grenier" : "more grenier",
    "القنفذة" : "cumulus",
    "chicoloapan de juárez" : "chicololoapan by juarez",
    "melmaruvathur" : "malamaruvathur",
    "khaldeh" : "khalida",
    "850 lincoln ave, suite 11 bohemia ny 11716" : "850 lincoln ave, suite 11 bohemia no. 11716",
    "ain khaled" : "where is khalid",
    "p10 q6 tphcm" : "p10 q6 hcmc",
    "balneário camburiu" : "balneário camboriú",
    "sremska 4a" : "srems' cha",
    "san ramon" : "3 ramon",
    "gusyaquil" : "guayaquil",
    "newton" : "newtonto",
    "ahemdnagar" : "ahmednagar",
    "ajah   lagos" : "ajah lagos",
    "ara" : "we buy",
    "portblair" : "port blair",
    "brusque  sc" : "brusque sc",
    "dedham" : "dadham",
    "leandro n alen 734" : "leandro n alem 734",
    "delhi cant" : "delhi cantt",
    "al nadha" : "the club",
    "rajapalayam" : "rajpalayam",
    "santa ctuz de la sierra" : "santa cruz de la sierra",
    "schaafheim" : "collapheim",
    "são josé dos campos  sp" : "são josé dos campos sp",
    "naxalbari" : "nahalari",
    "charummoodu" : "chaummood",
    "via cav. vitt. veneto" : "via cav.vit.veneto",
    "28 michael adekoya street ilupeju" : "28 michael adekaya street start",
    "gladwyne" : "smooth wines",
    "glenndale" : "glendale",
    "rua rui barbosa n 1095 bairro goiabeiras mato grosso" : "rui barbosa street n 1095 bairro goiabeiras mato grosso",
    "kozhikode" : "leathercode",
    "vadoara" : "vadodara",
    "mimbai" : "branded",
    "dum dum" : "stupid",
    "2520 marsha sharp fwy  apt481" : "2520 marsha sharp more apt481",
    "sorriso   mato grosso" : "smile mato grosso",
    "p1 , tp.cao lãnh , đồng tháp" : "p1, cao lanh city, dong thap",
    "rampurhat   731224" : "rampurhat 1222",
    "กรุงเทพมหานคร" : "bangkok",
    "calle 51#8g 55" : "51#8g 55 street",
    "theni" : "why",
    "nova scctia" : "nova scotia",
    "turmalina" : "tourmaline",
    "nikol,ahmedabad  gujarat" : "nicole, ahmed gogarat",
    "nagercoil" : "nagercoil temple",
    "2124 s michigan" : "2124 s road cancer",
    "sorriso  mt" : "smile mt",
    "la unión" : "the union",
    "klein windhoek klein windhoek" : "small wind corner small wind corner",
    "mandi adampur" : "bathing adix",
    "montréal, qc" : "montreal, qc",
    "via rastrello 22" : "via rastal 22",
    "são joão de meriti" : "st. johns wood",
    "tilhar" : "strain",
    "davao coty" : "gave coty",
    "po tankua ,dist koraput" : "by tankua, dist",
    "688 hưương lộ 2" : "688 huang lo 2",
    "joao pessoa" : "joao person",
    "giza, 6th october" : "giza, october 5th",
    "سلطنة عمان" : "sultanate of oman",
    "simão dias  se" : "simão dias se",
    "bebedouro" : "drinking fountain",
    "las palmas, grand canaria" : "las palmas, gran canaria",
    "al khobar   saudi arabia" : "a saudi arab news",
    "19  giang võ văn kiệt" : "19 giang vo van kiet",
    "assuncion" : "asuncion",
    "ciudad de buenos aires" : "buenos aires city",
    "ave. ricardo alfaro, ph plaza aventura piso 4 ofic 423 panam?" : "bird.ricardo alfaro, ph plaza aventura 4th office 423 panam?",
    "hải dương" : "hai duong",
    "renukoot" : "renucotus",
    "vridhachalam" : "old age",
    "korba" : "to age",
    "león guanajuato" : "guanajuato león",
    "west bandung" : "west ribbon",
    "santa maria   rs" : "santa maria rs",
    "hajipur ,vaishali, bihar" : "hajipur, vaishali, bihar",
    "taigore nagar near veer loric stadium" : "tagore nagar near veer loric stadium",
    "nrw, deutschland" : "nrw, germany",
    "cachoeira paulista" : "paulista waterfall",
    "1000 navaho drive  hotel" : "1000 navaho drive hotel",
    "kout" : "short",
    "490 l'enfant plz, sw" : "490 the child plz, sw",
    "são luís   ma" : "são luís ma",
    "puttur" : "putter",
    "łódzkie" : "łódź",
    "ile ife" : "the love of love",
    "são josé do cedro  sc" : "são josé do cedro sc",
    "bengalooru" : "bangalore",
    "jakarta selatan" : "south jakarta",
    "ابها" : "abha",
    "dheradun" : "dheeradun",
    "channai" : "milk production",
    "rio claro" : "clear river",
    "via g. di vittorio 5 a" : "via g.of vittorio 5 a",
    "1051 calle 3 se ste lc1a 3003 pr, san juan, 00921, puerto rico" : "1051 street 3 sete lc1a 3003 pr, san juan, 00921, puerto rico",
    "bauru   sp" : "bauru sp",
    "celle ligure" : "that ligure",
    "uttarpara kotrung" : "northern",
    "independencia esq heroes de luperon" : "independence esq heroes de luperon",
    "cư xá c . đại học nông lâm tp hcm" : "cu xa c.ho chi minh city university of agriculture and forestry",
    "espeŕanza" : "hope",
    "nh 27 deesa highway badarpura" : "no 27 disa highway badarpur",
    "qracaju" : "grocaju",
    "gen. trias cavite" : "gen.trias cavite",
    "taubaté   são paulo" : "taubaté são paulo",
    "bertioga   sp" : "bertioga sp",
    "hafar al batin" : "drill the clay",
    "barasat" : "rain",
    "nova serrana" : "new serrana",
    "niterói   rj" : "nitero rj",
    "masjid bander" : "bander mosque",
    "pichari" : "backward",
    "chhatapur" : "umbrella",
    "naina tikker" : "naina is ticking",
    "bangaloru" : "bangalore",
    "bezerros" : "calves",
    "juárez chihuahua méxico" : "juárez chihuahua mexico",
    "belo horizinte" : "belo horizonte",
    "dharuhera" : "darhera",
    "1001 camino ct" : "1001 ct path",
    "bucuresti" : "bucharest",
    "софия" : "sofia",
    "nowa wola" : "new wola",
    "rio grande do sul   erechim" : "rio grande do sul erechim",
    "west des moines" : "west monks",
    "terra boa" : "land",
    "viramgam" : "veeranga",
    "presidente juscelino" : "president juscelino",
    "ấp mỹ tân  thị trấn mỹ luông" : "my tan hamlet, my luong town",
    "phagwara" : "fagwara",
    "maloposkie" : "lizposki",
    "niterói   rio de janeiro" : "niterói rio de janeiro",
    "تيماء" : "tayma",
    "san andres islas" : "san andres islands",
    "bodh gaya" : "gone",
    "centro de produção audiovisual   sesc são paulo" : "sesc são paulo audiovisual production center",
    "perumanallur" : "advertise",
    "capacio paestum" : "paestum",
    "شارجه" : "sharjah",
    "karaikudi" : "go from",
    "marau" : "curricular",
    "são josé dos campos   sp" : "são josé dos campos sp",
    "shkoat" : "skoat",
    "debashis chakraborty" : "debashish chakraborty",
    "via mar della cina 254" : "via mar della china 254",
    "stary wiśnicz" : "old wiśnicz",
    "librebille" : "libreville",
    "1855 folsom st ste 601  medical" : "1855 folsom st you 601 medical",
    "paramaribo suriname" : "paramaribo, suriname",
    "mahrajganj" : "maharajganj",
    "alecrim  rs" : "rosemary rs",
    "medan" : "while",
    "artena" : "clay",
    "ertert" : "is",
    "pederneiras" : "bane",
    "würselen" : "sewer",
    "5808 socorro dr" : "5808 dr socorro",
    "ambato" : "hint",
    "tekulasomaram" : "techulsamaram",
    "doctor andrade  63 1er. piso  colonia doctores ciudad de m?xico" : "doctor andrade 63 1st.colonia doctors city of m? xico",
    "centro comercial asturias de ovideo  cra 10 # 1   50   local 10 cundinamarca" : "asturias shopping center of oviedo cra 10 # 1 50 local 10 cundinamarca",
    "atalaya" : "watchtower",
    "jericho" : "jeri",
    "kothrud,pune" : "kothrud, pune",
    "dharmaragar" : "dharmagar",
    "jangaon" : "the jangaoon",
    "belo horizonte   minas gerais" : "belo horizonte minas gerais",
    "3815 faraon st ste e" : "3815 pharaon st you e",
    "baruero" : "barueri",
    "medellín colombia" : "medellin colombia",
    "مسقط" : "muscat",
    "جازان" : "jizan",
    "ciudad satelite" : "ciudad satellite",
    "irara" : "breathy",
    "waukesha" : "you up",
    "pulivendla" : "pulivendula",
    "miagao iloilo" : "misama expections",
    "trà ôn vĩnh long" : "tra wen vinh long",
    "senhora" : "madam",
    "satuba" : "one",
    "ramanagar" : "ramnagar",
    "amk" : "bachelor",
    "fortaleza, ce" : "fortaleza, what",
    "3030 maingate ln" : "3030 mangate lan",
    "barreiros" : "barriers",
    "salmiya" : "peaceful",
    "04, prasadi gali , kotla mubarakpur delhi  delhi" : "04, prasadi gali, mubarakpur delhi delhi boiler",
    "la paz bolivia" : "la paz, bolivia",
    "margao,goa" : "margao, goa",
    "arame   ma" : "aram",
    "latur" : "lazy",
    "alta floresta" : "rainforest",
    "تموز (يوليو)" : "july (july)",
    "paraíso do norte pr" : "north paradise pr",
    "beberibe" : "beeribe",
    "banaso, hazaribagh" : "country, hazaribagh",
    "kochi, kozhikode" : "this, this ← iko",
    "rio branco" : "white river",
    "احد رفيده   الواديين" : "one of the valleys of the valley",
    "عجمان" : "ajman",
    "25c nguyễn bỉnh khiêm, q1, tp.hcm" : "25c nguyen binh khiem, district 1, ho chi minh city",
    "goiás   anápolis" : "goiás anápolis",
    "hessen   meinhard" : "hessen meinhard",
    "negros oriental" : "eastern blacks",
    "manser" : "male",
    "lima perú" : "lima peru",
    "piripiri" : "adhesive",
    "passo fundo" : "deep step",
    "fanna" : "us to",
    "bhongir" : "bhogir",
    "santiago de cali" : "santiago from cali",
    "armação de buzios" : "buzios frame",
    "mombasa,kenya" : "mombasa, kenya",
    "capão da canoa" : "canoa capão",
    "jais" : "them",
    "baki" : "mouth",
    "cambrideshire" : "cambridgeshire",
    "sitamadhi" : "sitamarhi",
    "paraná   francisco beltrão" : "paraná francisco beltrão",
    "budaun" : "budhon",
    "moskova" : "moscow",
    "shendurni" : "stubborn",
    "guwhati" : "gauwh",
    "uberlândia   mg" : "uberlândia mg",
    "małopolska" : "lesser poland",
    "toledo pr" : "on toledo",
    "ixtalhuaca de rayon" : "istalhuacon",
    "amapá   macapá" : "amapá macapá",
    "nova veneza" : "new venice",
    "nandyala" : "nandala",
    "getúlio vargas" : "getulio vargas",
    "barra bonita" : "pretty bar",
    "khober" : "hover",
    "cantagalo" : "sink",
    "bahia   salvador" : "bahia salvador",
    "mérida yucatán" : "merida yucatan",
    "mondai" : "problem",
    "mahesana" : "mehsana",
    "i̇zmir" : "mirzmir",
    "ananthapur" : "anantapur",
    "kepulauan riau" : "riau islands",
    "سوهاج" : "sohag",
    "francisco i. madero mz. 28 lt. 37 m?xico, d.f." : "francisco i.madero mz.28 lt.37 m? xico, d.f.",
    "sahaswan=243638" : "sahawan = 243638",
    "chaguanas" : "selectnas",
    "seleccionar comunapunitaqui" : "select comunapunitaqui",
    "saopaulo" : "baby be drowered",
    "da nang city, viet nam  vi?t nam" : "da nang city, viet nam vi? t nam",
    "foz de iguaçu   pr avenida tancredo neves 6731 jardim itaipu" : "foz de iguaçu pr avenida tancredo neves 6731 jardim itaipu",
    "שריגים (לי און)" : "shss (li on)",
    "i̇stanbul hadımköy" : "istanbul hadımköy",
    "alkhobar" : "the news",
    "reyadh" : "riyadh",
    "neyveli" : "unsheated",
    "medicina" : "medicine",
    "singen" : "to sing",
    "hei longjiang" : "heilongjiang",
    "poznam" : "i know",
    "los ángeles" : "the angels",
    "ahmednagar" : "ahmednaagar",
    "neyba provincia bahoruco" : "neyba bahoruco province",
    "bhawani mandi" : "bhavani people",
    "dĩ an, bình dương" : "di an, binh duong",
    "أنقرة" : "ankara",
    "cancún" : "cancun",
    "metropolitana de santiago" : "metropolitan of santiago",
    "al najaf" : "al najaf",
    "és" : "and",
    "punei" : "place",
    "pará" : "for",
    "campos dos goytqcazes" : "dos goytacazes fields",
    "cobquecura" : "chasing",
    "rj   rio de janeiro" : "rj rio de janeiro",
    "recife  pe" : "recife on",
    "ciudad autonoma de bs. as" : "autonomous city of bs.ace",
    "litleton" : "littleton",
    "sharq" : "east",
    "guidoval" : "he gurid",
    "marechal deodoro" : "marshal deodoro",
    "las piñas   muntinlupa" : "las piñas muntinlupa",
    "paudalho pe" : "padalho pe",
    "chatra" : "student",
    "alriyadh" : "riyadh",
    "76 78 bạch đằng, p. hải châu i" : "76 78 bach dang, p.hai chau i",
    "rs   caxias do sul" : "rs caxias do sul",
    "สงขลา" : "songkhla",
    "capelinha" : "chapel",
    "satata" : "stata",
    "novi beograd" : "new belgrade",
    "kerala" : "cherry",
    "playas de rosarito  baja california" : "rosarito baja california beaches",
    "são paio de oleiros" : "st.",
    "اربد" : "irbid",
    "mg   araxá" : "mg araxá",
    "sete lagoas" : "seven lagoons",
    "gujrat.godhra" : "gujarat. godhra",
    "greatfalls" : "great falls",
    "joinvielle" : "joinville",
    "4 tp thủ đức tphcm" : "4 thu duc city ho chi minh city",
    "siquirres limón" : "limón siquirres",
    "f6" : "timing",
    "lauscha" : "lollipotia",
    "ambalappuzha" : "ambalappuzh",
    "العين" : "eye",
    "أبها" : "abha",
    "komarapalayam" : "comrapalayam",
    "goiania  go" : "goiania go",
    "ktx khu c   đại học nông lâm tp hcm" : "ktx area c, agriculture and forestry university of ho chi minh city",
    "búzios" : "shells",
    "sadhoganj kharawan baragaon vns." : "sadhganj kharwan bargaon ones.",
    "bạc liêu" : "bac lieu",
    "betim   mg" : "betim mg",
    "engativa" : "deceit",
    "turvo" : "cloudy",
    "elkmont" : "quantity",
    "la vega" : "the vega",
    "dique de caxias" : "dike",
    "teresina pi" : "teeasia pi",
    "via oratorio,5" : "via oratory, 5",
    "três barras santa catarina" : "three santa catarina bars",
    "ulu tiram" : "the coast",
    "wiskitki" : "tarpaulin",
    "số 7 đường 3419 phường 7 quận 8." : "no. 7, street 3419 ward 7 district 8.",
    "la palma" : "the palm",
    "5n  ấp đông lân xã bà điểm" : "5n dong lan hamlet, ba diem commune",
    "hydarabaad" : "hysradons",
    "5b tân chánh hiệp 35" : "5b tan chanh hiep 35",
    "m'sila" : "me",
    "28 to ngọc van" : "28 to ngoc van",
    "الدمام" : "dammam",
    "campina verde" : "green campina",
    "rohini" : "expense",
    "são  paulo" : "são paulo",
    "safwa" : "greed",
    "aracaju   sergipe" : "aracaju - sergipe",
    "hauz khas,delhi" : "hauz special, delhi",
    "contagem minas gerais" : "count minas gerais",
    "hadapsar" : "front",
    "pančevo" : "pancevo",
    "davao del sur, davao city" : "gave del sur, gave city",
    "barão de cotegipe" : "baron de cotegipe",
    "5 lorong kemaris 1 bukit bandaraya" : "5 lorong lorong 1 bukit city",
    "salvador  bahia" : "salvador, bahia",
    "klinikum nürnberg (standort nord)" : "clinic nuremberg (north location)",
    "lalmatia" : "lacquer",
    "sumatra barat" : "west sumatra",
    "วังน้อย" : "wang noi",
    "bbsr" : "bubbling",
    "dandeli" : "capable",
    "tarn taran" : "taran taran",
    "13 ryad street shoubra, cairo" : "૧૩ rayal street shoubra, cairo",
    "san isidro davao oriental" : "like an equid davo oriental",
    "dubai invesyments park 1" : "dubai investments park 1",
    "ب" : "b",
    "airwa katra" : "airwa each",
    "balneário  camboriú" : "balneário camboriú",
    "mersin" : "myrtle",
    "araria" : "arriya",
    "la lucila, vicente lopez" : "lucila, vicente lopez",
    "السالمية" : "salmiya",
    "пловдив" : "plovdiv",
    "guanambi" : "guanhabi",
    "puttaparthi" : "puttparthi",
    "1 va ctr," : "1 ctr,",
    "ha noi" : "ha we",
    "antiguo cuscatlan" : "ancient cuscatlan",
    "tabira" : "toy",
    "mousl" : "mosul",
    "via pizzinardi  64" : "via pizzinardi 64",
    "ระยอง" : "rayong",
    "s73w16773 briagate ln  medical monitors" : "s73w16773 briargate ln medical monitors",
    "vinh   nghệ an" : "vinh nghe an",
    "vitória da conquista   ba" : "vitória da conquista ba",
    "joão pessoa:pb" : "joão pessoa: pb",
    "thalavadi" : "thalwadi",
    "160 calle primera" : "160 first street",
    "الجيزه" : "giza",
    "nagpur ,kanhan" : "nagpur, kanhan",
    "103 d tate circle  education" : "103 d tate circle  education",
    "boaco" : "reindeer",
    "calle gala 8" : "street gala 8",
    "test34" : "test 4",
    "mundo novo  ms" : "new world ms",
    "عسير" : "difficult",
    "av 531 no 6 ciudad de mexico" : "av 531 no 6 mexico city",
    "al seeb" : "the sep",
    "52  ต.ป่าตอง อ.กะทู้ จ.ภูเก็ต" : "52 patong subdistrict, kathu district, phuket province",
    "9540 hamburg rd" : "9540 hamburg rd",
    "biñan city" : "biñan cyty",
    "lagos de moreno" : "moreno lakes",
    "hydeeabad" : "hyderabad",
    "san jose del cabo" : "san jose of cabo",
    "minneola" : "memory",
    "ammanabrolu" : "amamanballu",
    "el cove" : "the cove",
    "huixquiliucan" : "huixquilucan",
    "hubali" : "hubli",
    "viçosa" : "lurid",
    "pará canaã dos carajás" : "para canaã dos carajás",
    "موصل" : "insulated",
    "raibareli" : "rai barre",
    "gandu" : "tribute",
    "río tercero   córdoba" : "río tercero córdoba",
    "st julians" : "st. julians",
    "nova lima" : "new lime",
    "la quiaca" : "the quiaca",
    "shikaripura" : "plastic",
    "pelotas" : "pellets",
    "vespasiano   minas gerais" : "vespasian minas gerais",
    "zielona góra" : "green mountain",
    "curitiba e são paulo" : "curitiba and são paulo",
    "podlaskie" : "podlasie",
    "são thome das letras" : "saint thome of letters",
    "bhabra" : "emotional",
    "176 thủ khoa huân phan thiết bt" : "176 valedictorian huan phan thiet bt",
    "12 phạm van bach tân bình" : "12 pham van bach tan binh",
    "434 bình quới, phường 28" : "434 binh quoi, ward 28",
    "bhachau kutch" : "bachau kutch",
    "karwar" : "increase",
    "ภูเก็ต" : "phuket",
    "số7,đường 3419,phạm thế hiển ,q8," : "7, 3419, pham the hien, q8,",
    "maceió  al" : "macea al",
    "λαρισα" : "larissa",
    "guayaquil   ecuador" : "guayaquil, ecuador",
    "ashtapur" : "ashtpur",
    "shakopee" : "shakopi",
    "são joão da boa vista" : "sao joao da boa vista",
    "nabatieh" : "nabaiah",
    "2c trần hưng đạo, p1, q5" : "2c tran hung dao, p1, q5",
    "أبوظبي" : "abu dhabi",
    "cân rhow" : "weigh",
    "hai dương" : "two -positive",
    "santa izabel do oeste" : "santa izabel do west",
    "boituva" : "boituvian",
    "stary żabieniec" : "old żabieniec",
    "hampden" : "hem",
    "新竹縣" : "hsinchu county",
    "strada statale 182 c.da nasari snc" : "state road 182 c.da nasari snc",
    "cachoeiro de itapemirim   es" : "cachoeiro de itapemirim es",
    "7985 kentshire dr  medical" : "7985 i'm",
    "quận 10" : "district 10",
    "1380 fondo de la legua piso 1 â°  commercial display" : "1380 legua fund floor 1 ° commercial display",
    "las condes, santiagom, chile" : "las condes, santiago, chile",
    "bento gonçalves. rs" : "bento gonçalves. lol",
    "hà tiên" : "ha tien",
    "gualeguaychu" : "gualeguare",
    "elmansoura" : "mansoura",
    "sitio de abadia go" : "abbot site",
    "marechal cândido rondon   paraná" : "marshal cândido rondon paraná",
    "vaniyambadi" : "vaniya -enambadi",
    "house no 663 chopal wali gali alipur garhi delhi  110036" : "house no 663 chopal gali alipur garhi delhi 110036",
    "thrithallore" : "trianalore",
    "96 moo 1 klongnueng klongluang" : "96 where 1 klongnueng klongluang",
    "10 zonara st" : "10 zonara st.",
    "roddi" : "give",
    "bilasipara" : "feipy",
    "63 trương định" : "63 truong dinh",
    "a17 12 chung cư samland" : "a17 12 samland apartment",
    "tio hugo rio grande do sul" : "uncle hugo rio grande do sul",
    "plot #35,phase iv,malur  malur taluk  kolar district" : "plot #35, phase iv, malur malur taluk kolar district",
    "durgapur bhiringi" : "durgapur bhirangi",
    "la reina" : "the queen",
    "canningvale" : "canning",
    "governador valadares" : "valadares governor",
    "303 gò dầu phường tân quý" : "303 go dau ward tan quy",
    "seri kembangan" : "kembangan series",
    "sa’ad, riyadh dammam highway,  ksa" : "sa'ad, riyadh dammam highway, ksa",
    "belém e região" : "belém and region",
    "2777 n stemmons fwy  healthcare" : "2777 n stemmons more healthcare",
    "house no. 375, road no. 28, new dohs, mohakhali, dhaka cantonment, dhaka 1206, bangladesh bangladesh" : "house no.1, road no.28, new das, mohakhali, dhaka cantonment, dhaka 1 to 3, bangladesh bangladesh",
    "sp brasil" : "sp brazil",
    "agudos" : "acute",
    "pathanamthitta" : "pathanamtta",
    "via gramsci angolo via de gasperi" : "via gramsci corner via de gasperi",
    "tiruvallur" : "thiruvallur",
    "vadodra" : "vadodara",
    "29222 rancho viejo rd ste 108  medical" : "29222 rancho viejo rd ste 108 medical",
    "mahua" : "mahu",
    "89 hoàng quốc việt,q7" : "89 hoang quoc viet, district 7",
    "الرس" : "alrass",
    "loja" : "store",
    "begunia" : "beguni",
    "nasik (nashik)" : "nasik (we doubt)",
    "sta. cruz" : "stay.cruz",
    "surbiton" : "surbs",
    "lagoa da prata   mg" : "mg silver lagoa",
    "heroica puebla de zaragoza" : "heroic puebla of zaragoza",
    "baga calangute" : "lung calangute",
    "315 alameda blvd ne" : "315 alamed blvd not",
    "palitana" : "glory",
    "anoop shahr" : "anoop city",
    "mianwali colony gurugram" : "miavali colony gurugram",
    "las piñas city" : "the city pineapples",
    "resistencia" : "endurance",
    "capao da canoa" : "canoe",
    "barakar" : "vanity",
    "tukwila" : "us",
    "sanriago" : "after sanria",
    "trà cú trà vinh" : "tra cu tra vinh",
    "tubarao" : "scrapping",
    "dharamapuri" : "dharampuri",
    "barueri" : "barley",
    "sląskie" : "silesian",
    "duque de caxias" : "duke of caxias",
    "chatrapati sambhajnagar" : "chhatrapati",
    "rio grande do sul  it" : "rio grande do sul it",
    "cornélio procópio" : "cornelius procopio",
    "thanh xuân" : "youth",
    "ateli mandi" : "mandy atels",
    "bucaramanda" : "bukramanda",
    "mozzate" : "severed",
    "skawina" : "skaw",
    "wielkopolski" : "wielkopolska",
    "campo bom" : "good camp",
    "naucalpan , estado de mexico" : "naucalpan, state of mexico",
    "nova venécia" : "new venecia",
    "macherla" : "machers",
    "225 baldwin aveneu  healthcare" : "225 baldwin avenue healthcare",
    "oklahoma city" : "ololama city",
    "1759 wehrle dr  it" : "1759 wehrle dr it",
    "kihei" : "bay",
    "saocha" : "laboru",
    "sao paulo   sp" : "sao paulo-sp",
    "cabaceiras do paraguaçu" : "paraguaçu cabaceiras",
    "curitiba pr" : "on karitiba",
    "patos de minas" : "mining ducks",
    "nova iguaçu" : "new iguaçu",
    "brasil" : "brazil",
    "tp.hcn" : "tp.hcm",
    "benagluru" : "bengaluru",
    "ilhabela" : "beautiful island",
    "boa vista   rr" : "boa vista rr",
    "via san martino ,12  fraz. eremo" : "via san martino, 12 fraz.hermitage",
    "hévíz" : "heat water",
    "kupwara" : "to blow",
    "autopista medella. bogotá, colombia" : "medella highway.bogota colombia",
    "duque de caxias   rio de janeiro" : "duque de caxias rio de janeiro",
    "9716 mcfarring drfort worthtx76244" : "9716 mcfarring dr fort worth tx 76244",
    "bielsko biała" : "bielsko-biala",
    "tinsukia" : "chinsquia",
    "wólka kozodawska" : "wólka kozodska",
    "shri madhopur" : "shri madhur",
    "contagem mg" : "count mg",
    "17902 kellas ct" : "17902 clock ct",
    "jatai" : "expressed",
    "menicucci 3" : "menicccci 3",
    "kayseri̇" : "kayseri",
    "pouso alegre" : "happy landing",
    "otro (latinoamérica)" : "another (latin america)",
    "28 ngô quyen" : "28 ngo quyen",
    "sao pauo" : "sao make",
    "5560 katella ave" : "5560 cover ave",
    "30 mân quang 6, sơn trà, đà nẵng" : "30 man quang 6, son tra, da nang",
    "ventanilla   callao" : "callao window",
    "garki" : "forth",
    "al ahsa" : "i count",
    "nandurbar" : "invested",
    "2 2 5 toranomon, minato ku, tokyo" : "2 2 5 toranomon, minato ward, and",
    "25c nguyễn bỉnh khiêm, q1" : "25c nguyen binh khiem, district 1",
    "ينبع" : "stems",
    "nandyal" : "nandala",
    "san martino in rio    re" : "san martino in rio re",
    "deoghar" : "deo -house",
    "cariacica" : "cariacic",
    "pirangi" : "like",
    "nova iguaçu   rio de janeiro" : "nova iguaçu rio de janeiro",
    "avram iancu 20a  it" : "avram iancu 20a it",
    "tulua" : "twinkle",
    "tlalnepantla estado de mexico" : "tlalnepantla state of mexico",
    "hargeisa" : "harge",
    "santo andre são paulo" : "santo andre sao paulo",
    "curitiba   pr" : "on karitiba",
    "são paulo tatui" : "sao paulo tatui",
    "ri9 de.janeiro" : "rurayd d.",
    "impasugong bukidnon" : "impasuto bukidnon",
    "zamboanga city" : "vegeta city",
    "palma decmallorca" : "palma de mallorca",
    "merida yucatán" : "merida yucatan",
    "juárez" : "juare",
    "chandausi" : "check",
    "kennebunkport" : "kenkeybunkport",
    "kareli" : "plaid",
    "tshimaku,tokyo" : "t film, tokyo",
    "ugar" : "fallow",
    "ปัอมปราบฯ" : "pork suppression",
    "kouhoku ku, yokohama" : "kohoku ward, yokohama",
    "פתח תיקווה" : "petah tikva",
    "9647701084310 sulemaniyah" : "9647701084310 sulemanya:",
    "the colonty" : "the colony",
    "dariba" : "remark",
    "świętokrzyskie skarżysko  kamienna" : "świętokrzyskie skarżysko kamienna",
    "caolanh" : "a winter",
    "la mesa" : "table",
    "gran canarias playa del ingles" : "gran canarias playa del english",
    "geebung" : "vomiting",
    "volta redonda   rj" : "volta redonda rj",
    "cam sur" : "cam on",
    "via v. volterra, 9" : "via v.volterra, 9",
    "kraków" : "cracow",
    "17 νοεμβρη 79 πυλαια" : "17 november 79 pylaia",
    "taboão da srra  sp" : "taboão da serra sp",
    "digha" : "dagh",
    "bandung, kota." : "bandung, city.",
    "doveton 3177" : "covton 3177",
    "panabo" : "bailer",
    "avan" : "before",
    "كلباء" : "all",
    "dhi qar" : "ruild",
    "goleta" : "schooner",
    "piedade" : "pity",
    "lachhmangarh sikar" : "lachmangarh sikar",
    "habra" : "there will be",
    "8 ấp 4 xã nhơn đức" : "8 hamlet 4, nhon duc commune",
    "gzb" : "calamity",
    "primavera do leste" : "spring of the east",
    "una, himachal" : "one, himachal",
    "b5 lưu văn liệt, phường 2, tpvl" : "b5 luu van liet, ward 2, tpvl",
    "gola gokaran nath" : "circle gokharan nath",
    "安徽省芜湖市" : "wuhu city, anhui province",
    "caucaia ce" : "caucaia what",
    "barao de cotegipe" : "cotegipe barao",
    "barreiras  ba" : "ba barriers",
    "campos de julio mt" : "julio mt fields",
    "eylül" : "september",
    "margao" : "cursing",
    "itapema   sc" : "it will spit sc",
    "กทม" : "bangkok",
    "norte de santander, cucuta" : "north of santander, cucuta",
    "mooca   sp" : "mooca sp",
    "propriá   sergipe" : "propriá sergipe",
    "alger centre" : "algiers center",
    "constanta" : "constant",
    "prol corregidora norte 1116 of 301 quer?taro" : "north correcting prol 1116 of 301 want? taro",
    "carrera 71 # 50   57 off 201" : "carrera 71 # 50 57 off 201",
    "bagnan" : "garden",
    "pune   kharadi" : "hardy",
    "ชลบุรี" : "chonburi",
    "la florida" : "florida",
    "kasaragod" : "discovery",
    "falakata" : "fatal",
    "formosa do sul   sc" : "formosa do on the sc",
    "otyń" : "obesin",
    "115 trần bá giao f5" : "115 tran ba giao f5",
    "paulista  pernambuco" : "paulista pernambuco",
    "seropédica   rj" : "seropédica rj",
    "chandugarh" : "chandgarh",
    "carrera 11a 94 46 edificio chico 3000 piso 3 bogota" : "carrera 11a 94 46 buy chico 3000 floor 3 bogota",
    "corso canale 32a" : "channel 32a course",
    "ponta grossa   pr" : "ponta groro pr",
    "sâo paulo" : "são paulo",
    "varodrada" : "varodarada",
    "badung, kuta selatan" : "badung, south kuta",
    "via jardins" : "via gardens",
    "chapadinha   maranhão" : "chapadinha maranhão",
    "jakrta" : "jakarta",
    "istanbul   beyoglu" : "istanbul beyoglu",
    "recufe" : "reefs",
    "quatre bornes" : "four terminals",
    "malavalli" : "malwalli",
    "1655 e arlingtonblvd" : "1655 e arlington blvd",
    "buzet sur tarn" : "buzet on tarn",
    "vapi" : "coat of arms",
    "中山區" : "zhongshan area",
    "tiruchengodu" : "we have a cleanliness",
    "małopolks" : "lesser poland",
    "al madinah" : "city",
    "itaberaba   ba" : "plate club",
    "zławieś mała" : "zławieś little",
    "palakonda" : "plakonda",
    "merta city dist nagaur" : "merrata city dist nagaur",
    "venezia mestre" : "venice mestre",
    "botucatu" : "booted",
    "الموصل" : "mosul",
    "tp thủ đức" : "thu duc city",
    "butibori" : "bump",
    "aparecida   sp" : "aparecida sp",
    "16604 arminta st, van nuys, ca 91406" : "16604 arminta sat, van nayas, 91406",
    "fhfu" : "jar",
    "thị xã bình minh" : "binh minh town",
    "buzios" : "bunks",
    "vassouras" : "broom",
    "padra  vadodara" : "bench",
    "baleares" : "whaarse",
    "pernambuco  petrolandia" : "pernambuco petrolandia",
    "van horn" : "horn",
    "8 phan văn hớn" : "8 phan van hon",
    "başakşehi̇r" : "başakşehi",
    "lajeado" : "slab",
    "port bouet rue des caraibes" : "bouet port rue des caraibes",
    "pauri" : "on you",
    "gudalur" : "guddur",
    "cl 20 sur 45 70 ap 904" : "cl 20 of 45 70 ap 904",
    "bình dương" : "binh duong",
    "gangakher" : "gangakhed",
    "barnala" : "childhood",
    "di̇di̇m aydın" : "deep",
    "guadalajara, jalisco" : "guadalajara jalisco",
    "corpus christi" : "body of christ",
    "hunasagi" : "gaid",
    "16 lam sơn, p2, tân bình" : "16 lam son, p2, tan binh",
    "tornesch" : "tornorous",
    "kab. bandung barat" : "regency. west bandung",
    "redenção" : "redemption",
    "50675 yukon rd po box 537" : "50675 oil now rd popular x 537",
    "via provinciale sud, 2" : "provincial south via, 2",
    "espírito santo, es" : "holy espírito, is",
    "obrassano" : "work",
    "carrera 18 n. 48 52" : "race 18 n.48 52",
    "são gabriel" : "saint gabriel",
    "paleo faliro" : "old faliro",
    "sulaimanya" : "solomon",
    "cascavel" : "rattlesnake",
    "śląskie" : "silesian",
    "ciudad madero" : "madero city",
    "tremblay en france" : "tremblay in france",
    "via delle battaglie" : "via delle battles",
    "sorriso" : "smile",
    "45 west flagge street  medical" : "45 west flag street medical",
    "zakopane" : "buried",
    "268 lý thường kiệt quận 10" : "268 ly thuong kiet district 10",
    "مكه" : "mecca",
    "دقهليه" : "dakahleh",
    "samar leyte" : "production of leyte",
    "quatro barras" : "four bars",
    "silvassa" : "silvasa",
    "ulaanbaatar" : "ulbatar",
    "343 s 500 e, apt 118" : "zhz with 500 e, apt 118",
    "muthanna  samawah city" : "muthanna samawah city",
    "puttparthi" : "puttaparthi",
    "praia grande" : "big beach",
    "ciudad de mëxico" : "mexico city",
    "monte verde" : "green mount",
    "kadiri" : "as much as possible",
    "28 tô ngoc vân p thạnh xuan .q12" : "28 to ngoc van p thanh xuan .q12",
    "baturaja" : "king stone",
    "idar" : "id",
    "nouméa" : "noumea",
    "جنوب عبدالله مبارك" : "south abdullah mubarak",
    "sapiranga   rs" : "sapiranga rs",
    "nova ponte" : "new bridge",
    "perote  veracruz" : "perote veracruz",
    "chandigadh" : "chandigarh",
    "baden würtemberg" : "baden württemberg",
    "slask" : "slush",
    "salvador  ba" : "salvador ba",
    "laguna de terminos 221, torre b 604 bis cdmx" : "lagoon of terminnos 221, tower b 604 bis cdmx",
    "台北" : "taipei",
    "marechal candido rondon" : "marshal candido rondon",
    "المحله الكبري  الغربية" : "western major locality",
    "5 av. des hauts fourneaux" : "5 av.high stoves",
    "phường phú thuận, quận 7" : "phu thuan ward, district 7",
    "belo horizonte   mg" : "belo horizonte mg",
    "san mateo rizal" : "san mato rizal",
    "medianeira" : "mediator",
    "alfyom" : "palafume",
    "pest megye" : "pest county",
    "7730 n central dr" : "7730 n central dr.",
    "ktx khu b   đhqg tp.hcm" : "ktx area b vnu hcmc",
    "strovolos" : "stingy",
    "ראש לציון" : "rishon lezion",
    "piauí   teresina" : "piauí teresina",
    "675 đường giải phóng" : "675 liberation road",
    "189 paramus rd. paramus" : "189 paramus rd.paramus",
    "pernambuco   recife" : "pernambuco recife",
    "los olivos" : "the olive trees",
    "ul. łódzka 308" : "ul.łódź 308",
    "nagarnausa" : "carrity",
    "pekanbaru" : "pecanbal",
    "sarangpur" : "rangpur",
    "laxmiganj guna" : "laxmiganj use",
    "sao jose do rio preto   sp" : "sao jose do rio preto sp",
    "karamsad" : "karmasad",
    "santiago de  chile" : "santiago de chile",
    "cao lãnh đồng tháp" : "cao lanh dong thap",
    "lieusaint" : "holy place",
    "mokattam" : "mute",
    "pathanapuram" : "prathanapuram",
    "malapuram" : "malapppuram",
    "campinas shopping móveis" : "campinas shopping mobile",
    "mérida, yucatán, méxico" : "mérida, yucatán, mexico",
    "yucatán" : "yucatan",
    "gharaunda" : "gharonda",
    "sntiago" : "santiago",
    "três rios" : "three rivers",
    "gardabaer" : "keep",
    "são josé do rio preto   sp" : "são josé do rio preto sp",
    "camarines sur" : "camarines on",
    "são caetano" : "sao caetano",
    "chapeco" : "hat",
    "mestre" : "teacher",
    "la follette" : "the madman",
    "via c. battisti, 3b" : "via c.battisti, 3b",
    "lo barnechea" : "barnechea",
    "manhuacu" : "rough comic",
    "foz do iguacu   pr" : "foz do iguacu pr",
    "araçatuba   sp" : "araçatuba sp",
    "कटिहार" : "katihar",
    "paranaíba   ms" : "paranaíba ms",
    "bounsi" : "beyonce",
    ".mpilo and pioneer road maseru" : ".mmp and pioneer road maseru",
    "itajai  sc" : "it will be sc",
    "14 tám danh p4.q8.tphcm" : "14 eight names p4.q8.tphcm",
    "águas formosas" : "famous waters",
    "via de filippis 40" : "way of filippis 40",
    "tangeran" : "tangerang",
    "valleduapar" : "valledupar",
    "san mateo 5 int ala b, boulevares, naucalpan estado de mexico" : "san mateo 5 int ala b, boulevares, naucalpan state of mexico",
    "contagem   minas gerais" : "count minas gerais",
    "bonita ave." : "nice bird.",
    "kiseljak" : "sorrel",
    "cidade" : "city",
    "salfit" : "salt",
    "gretna" : "grena",
    "11100 sepulveda blvd ste 8 pmb 393" : "11100 sepulveda blvd you are 8 pmb zia",
    "barranquilla , colombia" : "barranquilla, colombia",
    "dolnośląskie" : "lower silesian",
    "urb  rio piedras heights 144 wezer" : "urb rio piedras heights 144 wees",
    "cavolo del tomba" : "cabbage of the tomb",
    "nizwa" : "hear",
    "rae bareli" : "raye white",
    "torreón" : "keep",
    "distrito federal" : "federal district",
    "bokakhat" : "bucahat",
    "via circumvallazione esterna 89" : "via external circumvalulation 89",
    "cantabria" : "sneaky",
    "8, lake drive, rajagiriya. na" : "8, lake drive, rajgiria.no",
    "ktx khu a đhqg tp hồ chí minh" : "ktx area a vnu ho chi minh city",
    "juína" : "junior",
    "amapa" : "amapapapapapapapapa about",
    "kanchrapara" : "raw",
    "hải phòng" : "hai phong",
    "calle divino figueroa nº3 bajo c" : "divine street figueroa nº3 under c",
    "ar riyad" : "riyadh",
    "serta" : "as well as",
    "anakapalle" : "unaccounted",
    "tapurah" : "tapur:",
    "al muruj exit5 riyadh" : "the promoter is a feast of riad",
    "località grand chemin 22" : "big chemin location",
    "kuzey kıbrıs" : "north cyprus",
    "ktx khu a đhqg tp hcm" : "ktx area a vnu hcmc",
    "b35, 533" : "akhb, 533",
    "quito ecuador" : "quito, ecuador",
    "sri ganganagar" : "shri ganga nagar",
    "cd.jurez" : "cd.juarez",
    "sas nagar mohali" : "mother -in -law mohali",
    "españa" : "space",
    "via ai ronchi 288, mozzate co." : "away to the ronchi 288, mozzate co.",
    "unanderra" : "anderrera",
    "zduńska wola" : "zdunska wola",
    "bissingen an der teck" : "bissingen on the teck",
    "s.c. de bariloche" : "scby bariloche",
    "wrzeście k" : "september k",
    "belo horizone" : "belo horizonte",
    "śąskie" : "in a good",
    "bauneario" : "road ario",
    "salvado   bahia" : "saved bahia",
    "siddhartha nagar up" : "siddharth nagar sub",
    "hortolandia" : "vegetation",
    "raxaul" : "ragaul",
    "sambhal" : "careful",
    "alto paraiso" : "high paradise",
    "são caetano do sul" : "sao caetano do sul",
    "leh" : "with",
    "2.05 b chung cư mỹ thuận f16 q8" : "2.05 b my thuan apartment f16 q8",
    "смолян" : "smolyan",
    "shrihari kota" : "srihari kota",
    "ciudad de mexico" : "mexico city",
    "asunción" : "assumption",
    "cananda" : "canada",
    "saudi arabia \ al riyad" : "saudi arabia / al riyad",
    "mazowieckie   gabryelin 05 505" : "mazowieckie gabryelin 05 505",
    "ribeirão pires   são paulo" : "ribeirão pires são paulo",
    "bucaramanga   colombia" : "bucaramanga colombia",
    "goianinha" : "gouge",
    "rio grande do sul estação" : "rio grande do sul station",
    "辽宁省大连市" : "dalian, liaoning province",
    "marcham" : "march",
    "kurkheda" : "kurged",
    "haelen" : "hasten",
    "kalimantan selatan" : "south kalimantan",
    "barshi" : "let it go",
    "itabirito   mg" : "it is a mg",
    "quận 11" : "district 11",
    "cần tho" : "can tho",
    "المعادى" : "hostile",
    "רמת השרון" : "ramat hasharon",
    "naugarh siddharth nagar up" : "naugarh siddharth nagar sub",
    "koongal" : "canon",
    "juruti para" : "juruti for",
    "ecuador   loja" : "ecuador loja",
    "edo" : "or",
    "seriate" : "serial",
    "chimalhuacan" : "chimalhuana",
    "huyện tam bình   tĩnh vĩnh long" : "tam calm district vinh long",
    "przysucha" : "priests",
    "kumbakonam" : "kumbakal angle",
    "nellore" : "in",
    "aruppukkottai" : "archae",
    "cao lảnh" : "tall",
    "long xuyên tỉnh an giang" : "long xuyen province an giang",
    "محايل عسير" : "mahayil asir",
    "maadi" : "subject",
    "bhatpar rani" : "bhatpar queen",
    "tekkali" : "indentation",
    "băc giang" : "bác giang",
    "campinas sp" : "campinas, sp",
    "plazoleta la cerámica ste 2 6 ave sanchez vilella esq pr 190 medical" : "pladoleta la ceráma ste 2 6 ave sanchez vilella esq pr 190 medical",
    "sase krstica 43" : "six krstica 43",
    "magé" : "maggish",
    "juiz de fora   mg" : "juiz de fora mg",
    "gautambuddh nagar" : "gautam budh nagar",
    "salvador bahia" : "salvador, bahia",
    "ul. żwirki i wigury 18" : "ul.żwirki i wigury 18",
    "verás" : "you will see",
    "sojat" : "sozat",
    "mubai" : "mumbai",
    "ilocoa norte" : "north ilocos",
    "surat gujarat india" : "indian gujarat letter",
    "itaguaí   rio de janeiro" : "itaguaí rio de janeiro",
    "bulakan" : "bulacan",
    "itiquira" : "itique",
    "มุกดาหาร" : "mukdahan",
    "dokki, giza" : "poisoning, giza",
    "simões filho" : "simão filho",
    "bharuch" : "bharruch",
    "la" : "the",
    "сандански" : "sandanski",
    "rensselaer" : "cleanser",
    "caruaru, pe" : "caruaru, on",
    "macaé   rj" : "macaé rj",
    "sderot" : "deer",
    "imbé   rs" : "imbé rs",
    "godhara" : "godhra",
    "ss129jf" : "ss 129 jaff",
    "beograd" : "white city",
    "al los laureles, el laurel mexico" : "to the laurels, the mexico laurel",
    "douala" : "nation",
    "hạ long" : "ha long",
    "1900 lukken ind dr w" : "1900 luken ind dr w",
    "sesto san giovanni   milano" : "sesto san giovanni milan",
    "valle del cauca" : "cauca's valley",
    "gravataí   rs" : "gravataí rs",
    "tubarão" : "shark",
    "aruppukottai" : "i want to be",
    "santa catarina , sao jose" : "santa catarina, sao jose",
    "peoria" : "peolia",
    "dhbai" : "diani",
    "paragominas   pa" : "paragomin pa",
    "1700 diagonal rd ste 500  commercial display" : "1700 diagonal rd ste 500  commercial display",
    "khaitan" : "khitan",
    "marabá" : "maraba",
    "nova andradina" : "new andradina",
    "av.cra 30 # 17b 25 sur  bogot? d.c" : "av.cra 30 # 17b 25 sur bogot?d.c",
    "kartarpur" : "i'm not sure",
    "via manfredonia km 2,2" : "via manfredonia km 2.2",
    "av. pref. waldemar grubba, 3000 ? vila lalau  sc" : "of.pref.waldemar grubba, 3000?vila lalau sc",
    "norte de santander" : "north of santander",
    "belo jardim" : "beautiful garden",
    "nabadwip" : "ninth",
    "حولي" : "around me",
    "29222 rancho viejo rd ste 108  healthcare" : "29222 rancho vie rd s hand 108 hair lth he",
    "dolnoslaskie" : "donnoslav",
    "arteaga y salazar 108. col. contadero, cuajimalpa cdmx mexico city" : "arteaga y salazar 108. col.contero, cuajimalpa cdmx mexico city",
    "solteiro" : "single",
    "sulaimni" : "sulaimy",
    "senador sá" : "senator sá",
    "aap" : "you",
    "rio das ostras" : "oysters river",
    "shahapur" : "shahpur",
    "sao poaulo" : "sao paulo",
    "padipiranga" : "padipirga",
    "05 520 konstancin,warszawska 21d" : "05 520 konstancin, warszawska 21d",
    "plaghar" : "palathe",
    "châu thành  trà vinh" : "chau thanh tra vinh",
    "barreras" : "barriers",
    "6600 van aalst boulevard  medical" : "6600 van aalst boulevard medical",
    "15 lê đình cẩn, bình tân" : "15 le dinh can, binh tan",
    "pekini ave.41  pekini ave.41" : "pekini ave.41 pekini ave.41",
    "vyara" : "very",
    "liberdastr. 8" : "liberdatr.8",
    "פתח תקווה" : "petah tikva",
    "santa bárbara d’ oeste" : "santa barbara d 'west",
    "chanagancherry, kottayam" : "chanaganceri, kottayam",
    "aracajú sergipe" : "aracaju - sergipe",
    "florencia" : "florence",
    "pimpalgaon baswant" : "baswant pifts",
    "otros" : "others",
    "los alamos" : "poplars",
    "bhabua" : "babua",
    "rizwankhan" : "rizwean",
    "bahia   lauro de freitas" : "bahia lauro de freitas",
    "porto algre" : "porto alegre",
    "gurupi to" : "with gurpi",
    "al obour" : "transit",
    "09 đỗ xuân hợp, bình trưng đông,q2" : "09 do xuan hop, binh trung dong, district 2",
    "mohali , punjab" : "mohali, punjab",
    "sao pedro" : "saint peter",
    "wauwatosa" : "difference from wow wow",
    "kujawsko   pomorskie" : "kujawsko pomorskie",
    "اكتوبر" : "october",
    "13405 stemmons frwy" : "13405 stemmons",
    "rio verde   goiás" : "rio verde goiás",
    "santo andré" : "saint andré",
    "changaramkulam" : "changramakulam",
    "peru   lima" : "the name is lima",
    "monterret" : "monterrey",
    "cai lậy tiền giang" : "cai lay tien giang",
    "sana'a" : "occupation",
    "linh trung, thủ đức, hồ chí minh" : "linh trung, thu duc, ho chi minh",
    "pismo beach" : "beach letter",
    "hyderadbad" : "hyderabad",
    "belagavi" : "belagogi",
    "roca sales" : "sales roca",
    "itu" : "that",
    "união da vitória   pr" : "união da vitória pr",
    "arvada" : "pivot",
    "visconde do rio branco" : "viscount of rio branco",
    "ortigueira" : "ortigue",
    "curitiba   parana" : "curitiba, parana",
    "unite kingdom" : "united kingdom",
    "kaveripakkam" : "frostbite",
    "po box 9897, el paso tx. 79995 1201 kessler" : "po box 9897, el paso tx.79995 1201 kessler",
    "khowai" : "khavai",
    "2a 4a tôn đức thắng, quận 1,  hcm" : "2a 4a ton duc thang, district 1, hcm",
    "shimoga karnataka" : "the bottom is ran",
    "praia grande  sp" : "praia grande sp",
    "saint jean de moirans" : "saint john of moirans",
    "đường số 21 bình chiểu thủ đức" : "road no. 21 binh chieu thu duc",
    "3318 se 6th ave" : "3318 to 6 th",
    "bacood sta mesa manila" : "bacood is mesa manila",
    "khoda colony" : "colonies",
    "65 trương định" : "65 truong dinh",
    "łętownia" : "warehouse",
    "marathovounou 7 latsia" : "marathow 7 latsia",
    "ustronie morskie, pow. kołobrzeski" : "ustronie morskie, areakołobrzeg",
    "nenava" : "random",
    "jastrzębia" : "hawk",
    "abigjan" : "abijan",
    "carrera 29 # 45 45 piso 15 8902130749" : "carrera 29 # 45 45 floor 15 8902130749",
    "newyork parkview apartments 22122" : "new york parkview apartments 22122",
    "kab. lumajang" : "regency.lumajang",
    "payas" : "share",
    "dabra" : "pass",
    "sanand" : "gentleman",
    "turkiye" : "turkey",
    "135 se 6th ave" : "135 to 6 th and",
    "manaus  am" : "manaus am",
    "bangunan jalex, lot 3750, lrg 2d,  kampung melayu subang" : "jalex building, lot 3750, lrg 2d, subang malay village",
    "thaltej" : "taulge",
    "niterói" : "niteroi",
    "modasa" : "outfit",
    "ekangarsarai nalanda bihar" : "ekangarasarai nalanda bihar",
    "新莊區" : "xinzhuang district",
    "ferozepur" : "firozpur",
    "belém do para" : "belém do pará",
    "lộc hòa" : "loc hoa",
    "san josé del cabo" : "san jose del cabo",
    "jid ali" : "good for me",
    "paraiba   patos" : "paraiba patos",
    "geuteng" : "headal",
    "xalle 71d#67 85" : "helle 71d # 67 85",
    "السعوديه" : "saudi arabia",
    "nyiregyhaza" : "corbygougy",
    "biaora" : "baiaora",
    "vize" : "visa",
    "katra" : "every",
    "ive andrica 1, 71250 kiseljak sarajevo" : "this is andrew ೧, ೭೧೨೫೦ kiseljak sare jeevano",
    "cần thơ" : "can tho",
    "joinvile" : "joinville",
    "colombia   tunja" : "colombia tunja",
    "marhaura" : "madhaura",
    "xangri la   rs" : "xangri la rs",
    "pe   serra talhada" : "pe serra talhada",
    "sivasagar" : "sivasagar is the best",
    "dolnyśląsk" : "lower silesia",
    "prbtsar jawla" : "with a tour of a tour",
    "thủ đức, tp hồ chí min" : "thu duc, ho chi minh city",
    "nghệ an" : "nghe an",
    "edo de mex" : "come in edo x",
    "carr 3 oste # 42 45" : "car 3 oste # 42 45",
    "300 las vistas shopping village ave. felisa rincón ste.31" : "300 the views shopping village ave.felisa rincón ste.31",
    "רמת גן" : "ramat gan",
    "gunj basoda" : "ganj basoda",
    "chennau" : "lichen",
    "venda do pinheiro" : "pinheiro sale",
    "lavalle 780" : "on stage 780",
    "fortaleza  ce" : "fortaleza what",
    "ibadan, oyo state" : "ibadan, that state",
    "quận 9" : "district 9",
    "żółwin" : "ywin",
    "บุรีรัมย์" : "buriram",
    "tepeji del rio de ocampo hidalgo" : "tepeji of the river of ocampo hydalgo",
    "abudhabi city" : "abu dhabi city",
    "asdf" : "seashell",
    "lima, lima" : "five, five",
    "bhte minas gerais" : "flowing minas geris",
    "số 04 đường d9, quận tân phú" : "no. 04 d9 street, tan phu district",
    "ashgabat" : "the underum",
    "città della pieve" : "city of the pieve",
    "guadalajara, jal." : "guadlajara, water.",
    "palghat" : "palaghat",
    "tarakeswar" : "tartar",
    "minas gerais   japaraiba" : "minas gerais japaraiba",
    "patos de minas  mg" : "patos de minas mg",
    "khardaha" : "the lines",
    "θηβων 500" : "thebes 500",
    "kanuta" : "kaluuta",
    "dwsa" : "get",
    "mazowieckie warszawa" : "masovian voivodeship warsaw",
    "goias   goiânia" : "goias goiânia",
    "الاقصر" : "the shortest",
    "bangana" : "they are equal",
    "palghar  boisar  mumbai" : "palghar boisar mumbai",
    "gudemaranhalli" : "gudemaranahalli",
    "akola" : "did",
    "594 howard stsan franciscoca94105" : "594 howard st san francisco ca 94105",
    "kashan" : "is the",
    "ấp chợ tập sơn trà cú trà vinh" : "tap son tra cu market hamlet tra vinh",
    "jawa barat" : "west java",
    "edo.mex" : "edo.x",
    "khanouri" : "jerk",
    "lagarto" : "lizard",
    "aquiraz   ceará" : "aquiraz ceará",
    "rosà" : "rose",
    "gridih" : "greedih",
    "montes claros" : "clear montes",
    "al ain" : "where is",
    "khartom" : "khartum",
    "rj   macaé" : "rj macaé",
    "rua j?natas serrano, 182   casa 2   jardim castro alves" : "rua j? natas serrano, 182 house 2 jardim castro alves",
    "929 gessner  healthcare" : "929 gessner healthcare",
    "السنطه" : "eagle",
    "14 đường số 5, đặng văn bi thủ đức" : "14 street no. 5, dang van bi thu duc",
    "plachkovski manastir 7" : "plachkovsky monastery will",
    "santa lucia" : "saint lucia",
    "κ.,κοντου 2, αθηνα, 11255" : "k., kontou 2, athens, 11255",
    "rio de janeiro, rj" : "rio de janeiro - rj",
    "madhopara ,purnia" : "madhopra, puraniya",
    "ciudad apodaca" : "apodaca city",
    "mato grosso do sul   campo grande" : "mato grosso do sul campo grande",
    "kakching" : "i'm not sure",
    "kujawsko pomorskie,nowe" : "kujawsko pomorskie, new",
    "10 nguyễn duy, p12, q8" : "10 nguyen duy, p12, district 8",
    "tirunelveli, tamil nadu 627007" : "tirunelveli, tamil nadu ౬౨౭౦౦౭",
    "rajawali timur no 18 jawa barat" : "rajawali timur no 18 west java",
    "الباحة" : "courtyard",
    "hunsgi" : "hazard",
    "dehradoon" : "dehradun",
    "cd mex" : "cdmx",
    "st. michael im lungau" : "st.michael im lungau",
    "скопје" : "skopje",
    "rasalkhaima" : "ax",
    "mazowieckie" : "masovian",
    "الباجور منوفيه" : "bagor menoufia",
    "山东威海" : "shandong weihai",
    "bolangir" : "collarew",
    "pernambuco   paulista" : "pernambuco paulista",
    "warmińsko mazyrskie" : "warmian maozyrskie",
    "thủ đức, tp.hcm" : "thu duc, ho chi minh city",
    "maceió al" : "macea al",
    "kanhangad" : "canneman",
    "mounofia" : "munophy",
    "ghaziabaad" : "gypsy",
    "monte hermoso" : "monte beautiful",
    "warszawa" : "warsaw",
    "adamantina" : "adamantine",
    "bankura" : "to the bank",
    "santa amrta" : "santa marta",
    "русе" : "ruse",
    "tp thủ đức, tp.hcm" : "thu duc city, ho chi minh city",
    "catalão" : "catalan",
    "premia de mar" : "sea award",
    "dar es salaam tanzania" : "give is salaam tanzania",
    "bhavangar" : "bhavnagar",
    "kakkanad, kochi" : "his excellency d, here",
    "yopal   casanare" : "yopal casanare",
    "bueno aires" : "well airs",
    "ropar" : "call out",
    "herriman" : "faculation",
    "los banos, laguna" : "bans, laguna",
    "khamaria" : "vintage",
    "belait" : "belite",
    "sorriso mt" : "smile mt",
    "srikalahasti" : "sikalahas",
    "bắc giang" : "bac giang",
    "vizianagaram" : "vizayanagaram",
    "cebu,lapu lapu city" : "cebu, lap lap city",
    "curitiba paraná" : "curitiba, parana",
    "jk raod karande mala darewadi krishna house" : "czech rawd kandande mala darevadi krishna house",
    "timarni" : "timar",
    "dolnośłąskie" : "lower silesian",
    "vũng tàu" : "vung tau",
    "la altagracia" : "altagracia",
    "irinjalakuda" : "toolbar",
    "hà nội" : "hanoi",
    "uberaba  mg" : "uberaba mg",
    "ghatal" : "gossip",
    "las palmeras" : "the palm trees",
    "saint paul" : "st.paul",
    "presidente prudente" : "prudent president",
    "sarhochiya" : "sarashia",
    "via a. manzoni" : "away a.manzoni",
    "hạ long quảng ninh" : "ha long quang ninh",
    "sabana perdida" : "lost savanna",
    "москва" : "moscow",
    "libano" : "lebanon",
    "pudukottai" : "i want to be in the prefecture",
    "واسط" : "interface",
    "iguala" : "equal to",
    "región metropolitana de santiago" : "santiago metropolitan region",
    "al khawaneej" : "al khwani",
    "mpumalanga,emalahleni" : "mpumalanga, emalahleni",
    "76 vĩnh hưng" : "76 vinh hung",
    "taubaté   sp" : "taubaté sp",
    "koduvai, tiruppur" : "koduwai, tiruppur",
    "rua comandante costa 3021  cuiab?   mt 3021" : "rua commante costa 3021 cuiab?mt 3021",
    "via 4 novembre n^95" : "via 4 november n^95",
    "manaus  amazonas" : "manaus, amazonas",
    "núcleo bandeirante" : "bandeirante core",
    "القبة" : "dome",
    "தூத்துக்குடி" : "throat",
    "estado de mexico" : "mexico state",
    "nosara" : "and sarah",
    "arlington, ny 12603" : "arlington, 12603",
    "sangotedo" : "three words",
    "kirayan norte miagao ,iloilo" : "kirayan norte miagao ,ilolo",
    "rio verde" : "green river",
    "são luís maranhão" : "sao luis maranhão",
    "ปทุมธานี" : "pathum thani",
    "porto velho, rondônia" : "velho porto, rondônia",
    "364 dương quản hàm" : "364 duong quang ham",
    "tambo grande" : "big tambo",
    "cao lãnh , đồng tháp" : "cao lanh, dong thap",
    "8 phan đình giót, p2, tb" : "8 phan dinh giot, p2, tb",
    "salzburger straße 777" : "salzburger strasse 777",
    "6135 nw 167th st, ste e24 ste e24 cruise" : "6135 nw 167th st, you are e24 you e24 cruise",
    "livramento" : "liberation",
    "malopolskie" : "lesopolskie",
    "şırnak" : "sirnak",
    "taboão da serra   sp" : "taboão da serra sp",
    "台灣台北市" : "taipei city, taiwan",
    "ботевград" : "botevgrad",
    "mehgaon" : "ish",
    "opelousas" : "operator",
    "359 hồ học lãm" : "359 ho hoc lam",
    "sarni" : "sar",
    "guaiba" : "be good",
    "buga" : "print",
    "buenos aires   tigre" : "buenos aires tigre",
    "bangalorr" : "bangalirr",
    "barrio viejo de callosa de segura" : "viejo de callosa de segura",
    "mahemdavad" : "mahmadism",
    "kannad" : "heels",
    "chotomów" : "chotom",
    "salador" : "salter",
    "bagneux" : "prickly",
    "králův dvůr levín" : "the king's court",
    "campo largo pr" : "largo pitch pr",
    "nakhal" : "disgrace",
    "afua" : "originated",
    "purnea" : "poorna",
    "ciudad juarez" : "juarez city",
    "la calorta" : "the carat",
    "ballajura" : "dancing it",
    "calo" : "decline",
    "tirunelveli" : "tirunelevel",
    "rondonópolis   mt" : "rondonópolis mt",
    "612 hangar ln ste. 223, nashville, tn 37217" : "612 hangar ln you are.223, nashville, tn 37217",
    "república dominicana" : "dominican republic",
    "divino" : "divine",
    "poções" : "potions",
    "recife pe" : "recife on",
    "san luis potosí" : "san luis potosi",
    "cuneo" : "wedge",
    "via castelgomberto 96 italia" : "via castelgomberto 96 italy",
    "châu đốc" : "continental director",
    "bhubanerwar" : "bhubnerwar",
    "新北市" : "new taipei city",
    "méxico" : "mexico",
    "via a. manzoni 212" : "away a.manzoni 212",
    "vila velha" : "old village",
    "ayias zonis 60 c limassol" : "of this zone 60 c limassol",
    "thiruppttur" : "tirupur",
    "terrassa barcelona" : "barcelona terrace",
    "offlaga" : "offto",
    "ras al khaima" : "ras al khaimah",
    "guadalajara jalisco méxico" : "guadalajara jalisco mexico",
    "raiadh" : "records",
    "itaberai" : "portion of the board",
    "الرستاق" : "rust",
    "hyde rabad" : "hyderabad",
    "s.d. de rana" : "s.d.of wound",
    "presidente bernardes" : "president bernardes",
    "varca" : "gap",
    "ποσειδωνος 58" : "poseidon 58",
    "aachen   alemanha" : "aachen germany",
    "londrina, pr." : "laundrina, on.",
    "bucaramaga" : "busramga",
    "572, navamin90, klongkum," : "572, navamin 90, kangkum,",
    "gurgaoon" : "gurgaon",
    "michigan" : "road cancer",
    "senador canedo goiás" : "senator canedo goiás",
    "6 ballela road" : "6 read the road",
    "argao cebu city" : "a cream ce is not city",
    "banaglore" : "bangalore",
    "suratgarh" : "photograph",
    "tierra del fuego" : "land of fire",
    "dar es salaam" : "s salam house",
    "zaječar" : "zajecar",
    "bhind" : "bind",
    "deesa" : "goddess",
    "14 đường số 17, p. tân thuận tây" : "14 street no. 17, p.tan thuan tay",
    "imperatriz" : "empress",
    "chikkaballapura" : "chikkallapura",
    "100 first stsan franciscoca94015" : "100 first st san francisco ca 94015",
    "المنوفية" : "menoufia",
    "satana" : "satan",
    "bella vista" : "beautiful view",
    "beit yanai" : "hate love",
    "via lombardia n.8" : "via lombardia ranked # 8",
    "salto sp" : "heel sp",
    "kalaphar" : "black mountain",
    "vijayapur" : "vijaypur",
    "bangalorej" : "bungalore",
    "erith" : "eager",
    "upper pueryo cdo" : "upper puerto cdo",
    "datia (m.p.)" : "data (m.p.)",
    "savadatti" : "conscience",
    "tân đông hiệp, dĩ an, bình dương" : "tan dong hiep, di an, binh duong",
    "chapadinha" : "hat",
    "minas gerais  uberlandia" : "minas gerais uberlandia",
    "negros occidental" : "western blacks",
    "zaragoza, nueva ecija" : "zaragoza, new ecija",
    "eastren area" : "eastern area",
    "merida" : "mary",
    "nawada" : "the rest",
    "338 nguyen van giap ha noi" : "338 nguyen van giap has us",
    "pandikkad" : "bangy wall",
    "رابغ" : "rib",
    "कलकता" : "cadre",
    "dekalb" : "decalb",
    "ηρωων πολυτεχνειου 74" : "heroes people 74",
    "itaú de minas   minas gerais" : "itaú de minas minas gerais",
    "khu phố 1, tân hiệp, biên hòa" : "quarter 1, tan hiep, bien hoa",
    "смедерево" : "smederevo",
    "coxim" : "cushion",
    "sultanpur up" : "sultanpur sub",
    "enna" : "what",
    "novo gama" : "new range",
    "תל אביב" : "tel aviv",
    "kottrakara, kollam" : "kollam, kollam",
    "reibeirão preto sp" : "ribeirão preto sp",
    "bulandhshar" : "bulandshahr",
    "perambalur" : "pounding",
    "asandh" : "blessed",
    "linh trung, tp thủ đức, tp hcm" : "linh trung, thu duc city, ho chi minh city",
    "al khobar" : "the news",
    "jaen, nueva ecija" : "jaen, new ecija",
    "الرحبة" : "splendor",
    "talala gir" : "talala provides",
    "srinsgar" : "srins",
    "dorangla" : "dagang",
    "tihu" : "quiet",
    "elmsford" : "elmford",
    "sidvokodvo" : "sidvoko",
    "ipaba" : "tain",
    "bordon" : "shore",
    "nizgny novgorod" : "nizhny novgorod",
    "la union" : "union",
    "corrientes capital   corrientes" : "corrientes corrientes",
    "kadıköy" : "kadikoy",
    "sangat" : "very",
    "quezon coty" : "quezon city",
    "curitiba, pr" : "karitiba, but",
    "são luís  ma" : "são luís ma",
    "strada provinciale.le sud 101" : "provincial road. south 101",
    "tema" : "her",
    "nairbi" : "nai rujie",
    "hurgada" : "hurghada",
    "dehra du" : "dehra you",
    "igrejinha rs" : "church rs",
    "mettur" : "mector",
    "progreso, yucatán" : "progress, yucatan",
    "santiago, region metropolitana" : "santiago, metropolitan region",
    "hazaribag" : "hazaribagh",
    "shriwardhan" : "scriwarray",
    "pindaré mirim   maranhão" : "pindaré mirim maranhão",
    "via galatina,187   sant'angelo" : "via galatina, 187 sant'angelo",
    "shenzen" : "before shen",
    "גבעתיים" : "givatayim",
    "boituva  sp" : "boituva sp",
    "uberaba   mg" : "uberaba mg",
    "uberlandia   mg" : "uberlandia mg",
    "boirani" : "boyla",
    "canoas" : "canoes",
    "3era seccion lomas verdes" : "3rd lomas verdes section",
    "ak" : "and",
    "buttlar" : "bitch",
    "la cisterna" : "the cistern",
    "sangao" : "three high",
    "mahopac" : "mahoapc",
    "borca" : "a fighter",
    "mazowiecki" : "mazovian",
    "上海" : "shanghai",
    "เมือง" : "city",
    "paschim medinipur" : "west midnapore",
    "via s. anguissola 1" : "way's.anguissola 1",
    "4221 w john carpenter fwy" : "4221 w john carpenter more",
    "mandya" : "cloak",
    "braço do norte" : "northern arm",
    "markapur" : "marakpur",
    "ninavah" : "ninveh",
    "arapongas pr" : "on arpongas",
    "vitória do mearim" : "mearim victory",
    "cancun" : "residual",
    "sheakh zayed" : "the necks drop",
    "الشارقه" : "shark",
    "ilinois" : "illinois",
    "arame" : "wire",
    "kí  túc xá khu b đại học quốc gia" : "dormitory of national university",
    "türki̇ye" : "the turkistan use",
    "ראשון לציון" : "rishon lezion",
    "er" : "is",
    "dharamavaram" : "dharmavaram",
    "6 hẻm số 5 mễ cốc" : "6 alley no. 5 me coc",
    "3 bethesda metro ctr ste 700" : "x better metro str in 700",
    "las piñas" : "the pineapples",
    "bhilwara bapunagar kerali suitings.." : "bhilwara bapunagar kerli suitings ..",
    "praha 9" : "prague 9",
    "quận 3" : "district 3",
    "4880 stevens creek blvdsan joseca95129" : "4880 stevens creek blvd san jose ca 95129",
    "ulwe" : "fight",
    "dos hermanas" : "two sisters",
    "ciudad del carmen" : "carmen city",
    "berhampur" : "delay",
    "shahdol" : "shadol",
    "barauna rn" : "maruna rn",
    "петрич" : "petrich",
    "נתניה" : "netanya",
    "tata hospital actrec  sector 22 kharghar" : "tata hospital actrec seater 22 kharghar",
    "29 nguyễn t minh khai đà nẵng" : "29 nguyen t minh khai da nang",
    "omalur" : "alum",
    "arandu" : "wise",
    "ouagadougou" : "ungaduer",
    "62b kp3 an lạc a bình tân tphcm" : "62b kp3 an lac a binh tan hcmc",
    "extremos" : "extremes",
    "honduras,tegucigalpa" : "honduras, tegucigalpa",
    "marilia  sp" : "marilia sp",
    "los banos" : "the bathrooms",
    "al nakheel, ras al khaimah" : "al nakheel, ras al tent",
    "cheshire" : "chill",
    "hadrmout seyun" : "hadrmout pregnant",
    "itapecerica da serra   sp" : "itapecerica da serra sp",
    "500 chesterfield ctr, ste 355 ste 355" : "500 chesterfield ctr, you are 355 you are 355",
    "pasay city , metro manila" : "pasay city, metro manila",
    "ktx đại học quốc gia khu b" : "national university dormitory area b",
    "العبور" : "transit",
    "saint mitre les remparts" : "saint miter the ramparts",
    "meerut uttar pradesh" : "meerut uttor pradesh",
    "tradate" : "trades",
    "13 kim đồng giáp bát" : "13 kim dong giap bat",
    "aska" : "ash",
    "kanasin" : "cassin",
    "via roma  20" : "via roma 20",
    "corso s. ignazio     88" : "course s.ignazio 88",
    "via degli ippocastani,1" : "via degli ippocastani, 1",
    "678 , chua chu kang cresc #11 622" : "678 , that's the fire cresc #11 622",
    "yverdon les bains" : "yverdon les bains",
    "davao del norte" : "north davao",
    "são sebastião" : "san sebastian",
    "cali  valle" : "valley",
    "शादी" : "marriage",
    "jaru" : "spring",
    "campina grande  pb" : "campina grande pb",
    "מבשרת ציון" : "mevaseret zion",
    "trần đại nghĩa bình tân" : "tran dai nghia binh tan",
    "suite 22, no.4 ajala street (alhaji opeyemi shopping complex), off ikotun idimu road, subol bus stop" : "shuhunte ౨౨, no.౪ alaji street",
    "تبوك" : "tabuk",
    "นครราชสีมา" : "nakhon ratchasima",
    "thiruvalla (tiruvalla)" : "turna (turn)",
    "presidente kennedy" : "president kennedy",
    "jaleswar" : "gap",
    "umm al quain" : "umm al quin",
    "tanajib provence" : "tajiib provence",
    "929 gessner drive  healthcare" : "929 gessner drive healthcare",
    "ariccia" : "habit",
    "goiânia goiás" : "goiania goias",
    "cene" : "prices",
    "prayagraj" : "praisagraj",
    "recifes" : "reef",
    "bollampally veldanda nagarkarnool telangana" : "bollampalay weldand nagarakarnul telangana",
    "shirur" : "shiroor",
    "lac du flambeau" : "flambeau lake",
    "digos city, davao del sur" : "digos city, south davao",
    "chomu" : "why",
    "rewa mp" : "les mp",
    "para abaetetuba" : "to abaetetuba",
    "rio grande do norte" : "large northern river",
    "2777 n stemmons fwy  cloud it" : "2777 n stemmons more cloud it",
    "saihat" : "health",
    "via  tenuta del cavaliere,1" : "via tenuta del cavaliere, 1",
    "metromanila" : "metro manila",
    "tyerl" : "terlle",
    "thoothukudi" : "throat",
    "palams" : "of the palms",
    "chengalpattu" : "excitement",
    "ciudad de panama [panama city]" : "city of panama [paama city]",
    "konopiska" : "hemp",
    "belo orizonte" : "belo horizonte",
    "pargi" : "park",
    "المنيا" : "minya",
    "sta catarina   florianopolis" : "sta catarina florianopolis",
    "são paulo, são bernardo do campo" : "sao paulo, sao bernardo do campo",
    "rinconada del agua 2811 jalisco" : "water rinconada 2811 jalisco",
    "para de minas" : "for mines",
    "via del lavoro artigiano 10" : "via del lavoro craftsman 10",
    "rigadh" : "rigade",
    "gravataí" : "tie",
    "herbertpu, dehradun" : "herbertpur, dehradun",
    "台北市松山區" : "songshan district, taipei city",
    "kapaa" : "softly",
    "buin" : "done",
    "nalgonda" : "phid",
    "ddun" : "tiny",
    "banagalore" : "bangalore",
    "μεσογειων 85" : "mediterranean 85",
    "bogota, colombia." : "bogota colombia.",
    "long vĩnh, gò công tây, tiền giang" : "long vinh, go cong tay, tien giang",
    "rawatbhata" : "his salaries",
    "nueva vizcaya" : "new biscay",
    "tp. hồ chí minh" : "tp.ho chi minh",
    "saudi arabia  riyadh" : "saudi arabia riyadh",
    "bombinhas" : "bombs",
    "puneou" : "puneoula",
    "hueypoxtla" : "caridary",
    "c.da cataniello,12" : "c.da cataniello, 12",
    "viamão   rs" : "viamão rs",
    "koto ku, tokyo" : "ward and residence",
    "calle 52 no. 22   44 bogota" : "52 street no.22 44 bogota",
    "santa fe  rosario" : "santa fe rosario",
    "chakan" : "check",
    "união dos palmares" : "palmares union",
    "8 phan đình giót" : "8 phan dinh giot",
    "ghazipur up" : "ghazipur sub",
    "imbituba sc" : "imbitaba sc",
    "جمصه" : "jamsa",
    "adoni" : "adon",
    "villavicencio, meta" : "villavicencio, goal",
    "pouso redondo" : "round landing",
    "kharepatan" : "weapon",
    "são leopoldo" : "sao leopoldo",
    "visakhapatnm" : "visakhapatnam",
    "cabo frio" : "cabio cabio",
    "vinhedo   são paulo   brasil" : "vineyard são paulo brazil",
    "jagtial" : "hunting",
    "kishinev" : "chisinau",
    "r. catarina eufemia" : "r.catarina eufemia",
    "jabalpu" : "jabalpur",
    "dianopolis" : "intellectual",
    "tiruttani" : "stream",
    "bhubaneswer" : "showerwar",
    "surat kamrej" : "kamrej's letter",
    "karaikkudi" : "karaikudi",
    "edo. de mexico" : "it's from.mexico",
    "são paulo sorocaba" : "sao paulo sorocaba",
    "جده" : "grandmother",
    "demuro, 11" : "room, 11",
    "อุบลราชธานี" : "ubon ratchathani",
    "taiwan , taipei" : "taiwan, taipei",
    "curico" : "curic",
    "woolloomooloo" : "woolboyo",
    "khóm 1 thị trấn càng long" : "pineapple 1 town",
    "kalpetta" : "pale",
    "miraflores   lima" : "miraflores lima",
    "варна" : "varna",
    "jesi" : "you are",
    "ký túc xá khu b" : "dormitory area b",
    "paracatu   mg" : "paracatu mg",
    "jakarta timur, dki jakarta" : "east jakarta, dki jakarta",
    "i̇stanbul (avr)" : "istanbul (avr)",
    "londonamie" : "londonama",
    "tiruvallur district" : "tiruvalur district",
    "nagerkovil" : "nagerkov",
    "miscat" : "moved",
    "гр. софия, област софи" : "sofia, sophie district",
    "jawa timur" : "east java",
    "new delh" : "new delhi",
    "bastos" : "bastly",
    "bjn" : "they make",
    "mazatlán" : "mazatlan",
    "macae" : "macaaye",
    "puñe" : "dagger",
    "costa  mesa" : "costa mesa",
    "أكتوبر" : "october",
    "hòa an" : "hoa an",
    "36, 31 465 kraków" : "36, 31 465 krakow",
    "197 đường 9 phước bình" : "197 street 9 phuoc binh",
    "kharagpur" : "khargpur",
    "cl. 6 sur no. 79 150" : "cl.6 south no.79 150",
    "del mar" : "from sea",
    "puerto colombia" : "colombian port",
    "10,rue 2,lotissement abdelmoumen n 6 immeuble hajjar" : "10, rue 2, abdelmoumen subdivision n 6 hajjar building",
    "الفجيرة" : "fujairah",
    "שוהם" : "onyx",
    "raebareli" : "rae bareli",
    ". mancherial.... telangana" : ".manancherl .... telangana",
    "kopargaon" : "copargaon",
    "tiangua ce" : "hexagram test",
    "bahia   presidente tancredo neves" : "bahia presidente tancredo neves",
    "minhas gerais   araxá" : "my general araxá",
    "la libertad" : "freedom",
    "paraná,  assis chateaubriand" : "paraná, assis chateaubriand",
    "teutônia" : "teutonia",
    "riyadh   central" : "riyadh central",
    "2040 buda?rs, vas?t utca 9." : "2040 buda? rs, vas? t street 9.",
    "kí túc xá khu a" : "dormitory a",
    "বাদসাআনারুললস্কয" : "bang",
    "shahjhanpur" : "shahjahanpur",
    "kuna" : "there are",
    "cancún benito juárez" : "cancun benito juárez",
    "بغداد ." : "baghdad .",
    "65 đường 14" : "65 street 14",
    "anjuna" : "attempt",
    "la verne" : "verne",
    "santa tecla" : "holy button",
    "arni" : "on",
    "đồng xoài bình phước" : "dong xoai binh phuoc",
    "pedra preta" : "black stone",
    "navegantes" : "navigators",
    "haute savoie" : "haute-savoie",
    "shahpura (jaipur dist.)" : "shahpura (jaipur d from)",
    "souhag   maragha city" : "sohag maragha city",
    "1935 silverton rd ne" : "1935 silverton rd no",
    "balanga city" : "they advertise the city",
    "flores da cunha" : "cunha flowers",
    "borgata lavino, 32a" : "borgata developed, 32a",
    "uông bí" : "uong bi",
    "harnatand" : "haranatand",
    "guadalupe nuevo  leon" : "guadalupe nuevo leon",
    "via g. pascoli 39" : "via g.pascoli 39",
    "río de janeiro" : "rio de janeiro",
    "tupã" : "bed",
    "ofibodegas perisur no 8 frente escuela international" : "ofibodegas perisur no 8 front international school",
    "nova crixas" : "new crisles",
    "elsherouk" : "sunrise",
    "via uscioli,114 nocera superiore" : "via usgioli, 114 nocera superiore",
    "p.o. box 411, baysh" : "p.o.box 1 c, age",
    "dzs" : "cbs",
    "kosamba" : "praying",
    "registro" : "record",
    "shimoga" : "below",
    "são joão da boa vista   sp" : "são joão da boa vista sp",
    "francesco laparelli" : "francesco laparchelli",
    "san luis" : "saint louis",
    "santa barbara d’oeste" : "santa barbara d'oeste",
    "vicolo della stufa 14," : "stove 14 alley,",
    "espírito santo" : "holy spirit",
    "مدينه الخبر" : "khobar",
    "barangay alupay rosario batangas" : "barangay alupuds rosario batangas",
    "belém" : "bethlehem",
    "santo ângelo   rs" : "santo ângelo rs",
    "tirupathi" : "tirupati",
    "siddapur" : "sidpur",
    "udarsi" : "generous",
    "tân hiệp, trà cú, trà vinh" : "tan hiep, tra cu, tra vinh",
    "priyanka jhavar" : "priyanka jhawar",
    "52cc khu phố 2 phường tân quy q7" : "52cc quarter 2 tan quy ward district 7",
    "jamkhandi" : "jokhandi",
    "itsitio" : "self -sufficiency",
    "số 2 ngõ 6" : "no. 2 lane 6",
    "kasauli" : "cosmus",
    "pato branco   paraná" : "pato branco paraná",
    "algérie" : "algeria",
    "via dell'informatica 10   37036 san martino buon albergo (veneto), italy" : "via dell'oformatica 10 37036 san martino buon albergo (veneto), italy",
    "brasília" : "brasilia",
    "argyll" : "arch",
    "vitória da conquista." : "victory of the conquest.",
    "ciudad de méxico" : "mexico city",
    "rayachoty" : "rage",
    "ribeião pretp   sp" : "ribeirão preto sp",
    "ras al khaimah, uae" : "the head of the tent, a",
    "mriehel" : "twist",
    "khattar ,navi mumbai" : "khattar, navi mumbai",
    "poá" : "dust",
    "dharmshala (dharamshala)" : "dharamshala (dharamshala)",
    "400 s el camino real  cloud it" : "400 s the royal cloud it",
    "cainta" : "santa",
    "cuernavaca morelos" : "cuernavaca, morelos",
    "can.cun quintanaroo" : "can.cuni quintanaroo",
    "kampli" : "kamplipl",
    "fortaleza   ceará" : "fortaleza, ceara",
    "región" : "region",
    "wielkopolskiej" : "greater poland",
    "sao paulo" : "são paulo",
    "fort belvoir" : "very belvoir",
    "al gudaibiys" : "the jadbis",
    "nahan" : "retain",
    "cancún quintana roo." : "cancun quintana roo.",
    "anápolis   goiás" : "anápolis goiás",
    "chau đốc" : "chau doc",
    "bahia   camaçari" : "bahia camaçari",
    "del" : "of the",
    "omerga" : "to the ommer",
    "raigarh" : "raigad",
    "banediya" : "banadia",
    "pápa" : "pope",
    "yopal  casanare" : "yopal casanare",
    "an de klump 8 47661 issum" : "and the clump 8 47661 issum",
    "doda" : "he adds",
    "cd. lázaro cárdenas" : "cd.cárdenian lazarus",
    "kalinzar" : "calinzar",
    "उदयपुर" : "udaipur",
    "nunamara" : "outer",
    "rusterburg" : "rustenburg",
    "concórdia" : "concord",
    "vereeniging" : "association",
    "deoria" : "rear",
    "katargam" : "abolition",
    "القاهرة‬" : "cairo",
    "general villamil playas" : "general villamil beaches",
    "kendrapara" : "centerpara",
    "alejandro schroeder 6407 uruguay" : "aljandro schroeder 6407 uruguay",
    "المدينه المنورة" : "al madinah al munawwarah",
    "chodecz" : "pedestrian",
    "beed" : "egg",
    "mata roma" : "mata rome",
    "sahibabad, ghaziabad" : "halabad, gas",
    "dolnoślaskie" : "lower silesian",
    "aliso viejo" : "old alder",
    "nova igualu" : "new iguac",
    "tierra blanca" : "white land",
    "518 dungan street" : "518 season street",
    "kavali" : "want to be",
    "nueva ecija" : "nue's ecija",
    "panamá" : "panama",
    "mihijam" : "michiam",
    "sóc trăng" : "soc trang",
    "naucalpan, estado de méxico" : "naucalpan, state of mexico",
    "bình định" : "pacify",
    "fatehnagar distt udaipur" : "fatehanagar distt udipur",
    "amsterdã" : "amsterdam",
    "bolu" : "divided",
    "barranquilla colombia" : "barranquilla, colombia",
    "feijó" : "bee",
    "palwal" : "palval",
    "mexicaltzingo 2352 a y b, col. arcos, guadalajara, jalisco. jalisco" : "mexicantzingo 2352 a and b, col.arcos, guadalajara, jalisco.jalisco",
    "ينبع البحر" : "sea's origin",
    "hẻm 13 . đường sơn thông . k10. p9" : "alley 13.son thong street.k10.p9",
    "jacundá" : "jacket",
    "santa ana 2 mza 24 casa 5" : "santa ana 2 mza 24 house 5",
    "vasai virar" : "slap",
    "são josé do rio preto" : "sao jose do rio preto",
    "siko" : "elbow",
    "são paulo   jundiai" : "sao paulo jundiai",
    "subkowy" : "subdural",
    "islas de la bahia" : "bay islands",
    "caviye" : "poems",
    "קרית מוצקין" : "kiryat motzkin",
    "muntinlupa ciy" : "muntinlupa city",
    "12 a âu cơ , q.tân phú" : "12 a au co, tan phu district",
    "sirhind" : "sirlonging",
    "pasto" : "meal",
    "jaraguá do sul" : "south jaragua",
    "emiliano zapata, ver" : "emiliano zapata, see",
    "caçapava" : "hunt",
    "karawang, west java" : "karang, to go to west",
    "chehalis" : "shalis",
    "warminister" : "warminster",
    "via gregorio ricci curbastro, 29" : "via gregorio ricci curbaster, 29",
    "mundo novo" : "new world",
    "bắc ninh" : "bac ninh",
    "warminsko mazurskie" : "warminsko masurian voivodeship",
    "paraiba" : "paraíba",
    "via garibaldi,9" : "via garibaldi, 9",
    "zagazig" : "titillate",
    "são carlos,sp" : "são carlos, sp",
    "zalec" : "heal",
    "tarwara" : "we are sick",
    "tiruppur" : "twell",
    "lunawada" : "avada",
    "tanda" : "mark",
    "гр. хасково" : "haskovo",
    "jaragua do sul" : "south jaragua",
    "luzern" : "lucerne",
    "fortaleza ceara" : "fortaleza, ceara",
    "k̤o̤l̤k̤a̤t̤a̤" : "k̤oul̤k̤a̤̤̤a",
    "avenida independencia 1297" : "independence avenue 1297",
    "بابل" : "babylon",
    "tân an càng long trà vinh" : "tan an more long tra vinh",
    "udupu" : "stump",
    "madhapur" : "madapur",
    "melgar  tolima" : "melgar tolima",
    "1530 kurunji strt" : "1533",
    "vitoria espírito santo" : "vitoria, espirito santo",
    "espumoso" : "sparkling",
    "plewiska" : "plew",
    "alto taquari" : "high taquari",
    "ceara" : "wax",
    "darsi" : "give up",
    "katihar" : "mytihar",
    "king fahd rd, al olaya,  riyadh 11564, saudi arabia" : "king fahd red, alia, riad 11564, saudi arabia",
    "الحمام" : "the bathroom",
    "alkhobor" : "alhobor",
    "jauru" : "joru",
    "tantipara .barahnagar" : "tantipada.",
    "al wazarat" : "the ministry",
    "puerto deseado   santa cruz" : "puerto deseado santa cruz",
    "lajeado rs" : "laxed rs",
    "ahmed rajabli baku" : "ahmed rajal bey",
    "tambau" : "to sleep",
    "البصره" : "vision",
    "kumbazha" : "digel",
    "1820 chotti bajaria garha" : "1820 chitti bazaria gadha",
    "florianopilis" : "florianopilic",
    "940 redna terrace" : "940 regular terrace",
    "crtagena" : "cartagena",
    "sumidouro" : "sink",
    "dharmapuri" : "dharmapurai",
    "pinheiros" : "pine trees",
    "el nido, palawan" : "the nest, palawan",
    "bengaluru urban" : "bangalore urban",
    "vadodara   390002" : "donarasa 390002",
    "roxana" : "roxane",
    "sao bernardo do campo" : "são bernardo do campo",
    "shivamogga" : "shimoga",
    "san juan. capital" : "san juan.capital",
    "patna , kankarbagh, bihar" : "patna, kankarbagh, bihar",
    "topeka" : "stopy",
    "providencia" : "providence",
    "bhende sk" : "bhende ske",
    "wai" : "water",
    "barranca bermeja" : "barrancabermeja",
    "31 tân trụ, phường 15, q. tân bình" : "31 tan tru, ward 15, district.tan binh",
    "surco" : "groove",
    "não tem" : "it does not have",
    "istanbull" : "istanbul",
    "l'hay les roses" : "l'ho les roses",
    "vĩnh long" : "vinh long",
    "tp thủ  đức , tp hcm" : "thu duc city, ho chi minh city",
    "alvorada do sul" : "southern dawn",
    "pulivendula" : "i'm cleaning up",
    "pimpalner" : "pimpner",
    "hòa bình" : "peace",
    "đồng nai" : "dong nai",
    "warzno" : "brew",
    "rs uruguaiana" : "uruguaiana hospital",
    "siam piwat" : "ciam",
    "bình thuận" : "binh thuan",
    "rio brilhante   ms" : "brilliant river ms",
    "phường tân kiểng, quận 7, tp.hcm" : "tan kieng ward, district 7, ho chi minh city",
    "puerto vallarts" : "puerto vallarta",
    "rio de janeiro rj" : "rio de janeiro - rj",
    "17876 st clair ave.  cloud it" : "17876 st clair ave.  cloud it",
    "khandwa" : "khandva",
    "mindoro" : "mindro",
    "almadinah" : "city",
    "las pinas" : "the pine",
    "الزرقاء" : "blue",
    "lubelskie" : "lublin",
    "ghhv" : "ghahva",
    "กรุงเทพ" : "bangkok",
    "3655 nw 87 ave  cruise" : "3655 nw 87th ave cruise",
    "ponta grossa pr" : "ponta groro pr",
    "الخرج الدلم" : "al -kharj al -dalam",
    "6900 alma dr" : "6900 apples dr",
    "مصر  المنيا  بني مزار" : "egypt minya bani mazar",
    "21 lạc long quânp8tân bình" : "21 lac long quanp8 tan binh",
    "girne (kyrenia)" : "girne (kynie)",
    "friarland" : "freeland",
    "chatrapur" : "chhatarpur",
    "albasrh" : "vision",
    "carrera 69 b no. 98a 31   bogota" : "carrera 69 b no.98a 31 bogota",
    "campo grande   mato grosso do sul" : "campo grande mato grosso do sul",
    "tp quy nhơn, bình định" : "quy nhon city, binh dinh",
    "palari" : "hat",
    "كوم حماده" : "kom hamada",
    "imus, cavite" : "we go, cavite",
    "poznań i okolice" : "poznań and its surroundings",
    "chikhli" : "the subsi",
    "cachoeiras de macacu" : "macacu waterfalls",
    "belçika" : "belgium",
    "são bernardo" : "st bernard",
    "cruzeiro do sul" : "southern cruise",
    "kharghar" : "external",
    "brr alto bosque trv 52 #21d 26" : "brr alto trv 52 #21d 26",
    "sao paull" : "sao paulo",
    "timbauba pe" : "timbauba on",
    "kura place suva" : "kura cries suva",
    "kroi i bardhe no14 prishtine kosove" : "white spring no14 prishtina kosovo",
    "arcoverde" : "green arch",
    "malumichampatty,  coimbatore" : "malumchathatty, codutue",
    "717 general booth blvdvirginia beach, va 23451, usa" : "717 general booth blvd virginia beach, va 23451, usa",
    "sardulgarh" : "sarduhgarh",
    "cuernavaca,  edo. de morelos" : "cuernavaca, edo.of morelos",
    "mỹ phú. mỹ phong mỹ tho tiềng giang" : "my phu.my phong my tho tien giang",
    "51 thống nhất, p10, gò vấp" : "51 thong nhat, p10, go vap",
    "bali , badung" : "bali, badung",
    "taboão da serra" : "taboao da serra",
    "igarassu   pernambuco" : "igarassu pernambuco",
    "jhunjhunun" : "jhunjhunu",
    "7 bình quới" : "7 vase",
    "cogliate" : "they pick up",
    "kfar saba" : "kafr saba",
    "kovvur" : "kovoor",
    "canelones" : "cannelloni",
    "مطروح" : "proportional",
    "narsapur" : "narasapur",
    "via  g  marconi  115" : "via g marconi 115",
    "afyon" : "opium",
    "freiburg im breisgau" : "freiburg in breisgau",
    "pragathi nagar" : "pragati nagar",
    "eusébio" : "eusebius",
    "bangalore , karnataka" : "bangalore, karnataka",
    "бургас" : "burgas",
    "piedras blancas" : "white stones",
    "rillieux la pape" : "rillieux la pope",
    "taoru" : "towel",
    "schriever space force base" : "schlie space force base",
    "minatitlan" : "interest",
    "koregaon" : "the koregaon",
    "5700 lakewright drive suite 300 other" : "5700 lake wright drive suite 300 other",
    "jeddaj" : "jeddah",
    "brovello carpugnino" : "bovello carpugnino",
    "hà nội" : "hanoi",
    "morges" : "morning",
    "bagalkot" : "bang",
    "via dell'industria,3" : "via dell'industria, 3",
    "ciudad de santa fe" : "santa fe city",
    "saint gilles croix de vie" : "saint gilles cross of life",
    "ilha de itamaracá" : "itamaracá island",
    "algeri" : "algiers",
    "mangaratiba rj" : "mangratiba raja",
    "daudnagar." : "daudnagar",
    "مكة المكرمة" : "makkah",
    "từ sơn bắc ninh" : "from son bac ninh",
    "bb@72#biplab" : "bb@72#bbiplab",
    "صنعاء" : "sana'a",
    "września" : "september",
    "marthandam, kanniyakumari" : "marthandam, kanniakumari",
    "repuvlica dominicana" : "dominican republic",
    "diyarbakır" : "diyarbakir",
    "formiga" : "ant",
    "são paulo, são caetano do sul" : "sao paulo, sao caetano do sul",
    "garça" : "heron",
    "lucas do rio verde   mato grosso" : "lucas do rio verde mato grosso",
    "rua turmalina 543" : "tourmaline street 543",
    "olivos" : "olives",
    "15 phan huy thực phường tân kiểng" : "15 phan huy thuc tan kieng ward",
    "15 ấp 6b vĩnh lộc a bình chánh" : "15 hamlet 6b vinh loc a binh chanh",
    "kyrenia" : "souvenir",
    "samdari balotra  badmer" : "sandari balotra barmer",
    "baqdad" : "baghdd",
    "la sarraz" : "unbroke",
    "çameli" : "cameli",
    "kuj.pom." : "also.pom.",
    "chapadão do céu   goias" : "chapadão do céu goias",
    "centro empresarial e industrial de loulé, zona poente, lote 11b" : "loulé business and industrial center, long zone, lot 11b",
    "prueba" : "proof",
    "ras tanura" : "mighted raas",
    "barra do garça mt" : "mt garça bar",
    "ogun state" : "the state war",
    "alwaye" : "he is still",
    "ka gd f" : "the gd f",
    "furrkhabad" : "farrukhabad",
    "cachoeira do piriá" : "piriá waterfall",
    "campo largo" : "long field",
    "recife.    pernambuco" : "recife.pernambuco",
    "san mateo" : "saint matthew",
    "مدينة نصر   القاهرة" : "nasr city, cairo",
    "assis chateaubriand" : "seated chateaubriand",
    "chełm" : "helmet",
    "foz do iguacu   parana" : "foz do iguacu parana",
    "san juan opico" : "san juan monkey",
    "dantiago    recoleta" : "santiago recoleta",
    "3865 e 34th st suite 105, tucson, az 85713" : "3865 is zchtht otse 105, tutzon, i 85713",
    "campinas  sp" : "campinas, sp",
    "جازان محافظة العارضة" : "jazan al -ardah governorate",
    "manzana g casa 2" : "apple g house 2",
    "rua sud?rio maximiano rocha, n 123, bloco 4 ap 202 mg" : "rua maximiano rocha street, n 123, block 4 ap 202 mg",
    "pará." : "for.",
    "thiruvarur" : "tiruvarur",
    "ิกรุงเทพ" : "the gods",
    "nezahualcóyotl  estado de mexico" : "nezahualcóyotl state of mexico",
    "primavera do leste   mt" : "spring of the east mt",
    "ji parana" : "zee parna",
    "ranaghat" : "ranaaghat",
    "birkirkara" : "well",
    "quinto di treviso" : "fifth of treviso",
    "cra32#110 11" : "sharia#110 11",
    "guara   cidade do automovel" : "guara city of automobile",
    "warszaw" : "warsaw",
    "rua santa vitoria 663 minas gerais" : "santa vitoria street 663 minas gerais",
    "القليبوية" : "qalibiya",
    "אזור" : "area",
    "bahía de banderas" : "bay of flags",
    "tphcm" : "hcmc",
    "công ty cdc" : "cdc company",
    "4b mỹ hoà 1,trung chánh, hóc môn" : "4b my hoa 1, trung chanh, hoc mon",
    "h.no 807,lila chowk old abadi lila dharmsaala ,old abadi" : "h.no ౮౦౭, lila chowk old body leela dharamsala, old body",
    "shireer aurangabad" : "shirir aurangabad",
    "itliy" : "lever",
    "bahia blanca" : "white bay",
    "243a  đê la thành" : "243a de la thanh",
    "leverano" : "levers",
    "sor juana ines de la cruz 555 abcd" : "sister juana ines of cruz 555 abcd",
    "baihata chariali" : "behta chariali",
    "البصرة" : "basra",
    "porto seguro" : "safe harbor",
    "benglore" : "bullock",
    "biên hoà" : "bien hoa",
    "tân phú, tp hồ chí minh" : "tan phu, ho chi minh city",
    "niterói rj" : "nitero rj",
    "tuljapur" : "tulajapur",
    "mandideep" : "manitideep",
    "blk iii, cez ii, gen. trias cavite" : "blk iii, via ii, gen.trias cavite",
    "kazhakuttam, trivandrum" : "kazhakutam, trivandrum",
    "teki̇rdağ" : "in",
    "kattappana" : "katappana",
    "se  barra dos coqueiros" : "if barra dos coqueiros",
    "chapra  bihar" : "chhapra bihar",
    "wallaroo" : "pair",
    "bonfinopolis" : "bonfinopol",
    "mazatlan, sin." : "mazatlan, without.",
    "9 landsdowne streetbostonma2215" : "9 lansdowne street boston ma 2215",
    "autoput za novi sad 71" : "novi sad highway 71",
    "sao jose dos campos" : "são josé dos campos",
    "ślaśkie" : "silesian",
    "יד בנימין" : "hand benjamin",
    "santa cruz de sierra  bolivia" : "santa cruz de sierra bolivia",
    "bengalure" : "bangalore",
    "salalah" : "stalemate",
    "ujjain" : "fingers",
    "tanuku" : "tension",
    "capivari de baixo. sc" : "capivari from below.sc",
    "blvr kukulkan zona hotelera km 14.5 jw marriott cancun" : "blvr kukulkan area hotel km 14.5 jw marriott cancun",
    "ceranesi" : "cerane",
    "i̇stanbul" : "istanbul",
    "caoro" : "carro",
    "biharsharif" : "bihar sharif",
    "long biên hà noiij" : "long bien ha noiij",
    "medelin" : "medium",
    "sirkali" : "vermillion",
    "عمان" : "oman",
    "شبين الكوم المنوفيه" : "shebin el -kom menoufia",
    "nabga" : "navaga",
    "mansoura" : "manoura",
    "mazowiecie" : "masovian",
    "ganaur sonipat haryana" : "ganore sonipat haryana",
    "prakasam" : "aura",
    "via g.cesare n.54" : "via g.ceso n.54",
    "jau" : "already",
    "rio brilhante" : "brilliant river",
    "pr cascavel" : "pr rattlesnake",
    "ramat gan" : "ramat gain",
    "presidente getúlio sc" : "president getúlio sc",
    "marica  rio de janeiro" : "marica rio de janeiro",
    "la mirada" : "the look",
    "rytwiany" : "rytwany",
    "pappelweg 4" : "papel road 4",
    "metropolitana" : "metropolitan",
    "via e. de amicis, 23 . 90044 carini (pa)" : "away and.de amicis, 23.90044 carini (pa)",
    "cycleworld, hosa road juncrion" : "sisilebold, jose road junkrin",
    "guasave" : "guasuave",
    "milagro" : "miracle",
    "ديالى" : "diyala",
    "2310 east el segundo blvd" : "2310 east the second blvd",
    "navi mumbai, ghansoli" : "navi mumbai, ghansali",
    "campina grande   pb" : "campina grande pb",
    "sandur" : "sand",
    "navsari" : "nervous",
    "1120 south tyron street, ste 700  medical" : "1120 south tryon street, ste 700 medical",
    "beafort" : "beaufort",
    "6f 1, no.288 5, xinya rd., qianzhen dist., kaohsiung city 80673, taiwan" : "6 rounds 1, no.288 5, new asia tropical. on the third day of the previous array., kaohsiung city 80673, taiwan",
    "浙江省宁波市" : "ningbo city, zhejiang province",
    "h15 tổ 9 kp2 long bình tân biên hòa" : "h15 group 9 kp2 long binh tan bien hoa",
    "kolkatta" : "kolkattat",
    "hubballi" : "giggle",
    "mirjapur" : "mirzapur",
    "jyoraha" : "jyraha",
    "محافظة القاهرة" : "cairo governorate",
    "1305 n holopono st suite 3" : "1305 n juice stuite 3",
    "puttur manalore" : "puttur is ours",
    "narasaraopet" : "narsaraipet",
    "1375 n buena vista dr  commercial display" : "1375 n buena vista dr commercial display",
    "são apulo" : "são paulo",
    "buxar" : "steam",
    "المنصوره" : "asan",
    "estación central" : "central station",
    "صحار" : "sohar",
    "keonjhar" : "odor",
    "orindiuva sp" : "orinduva sp",
    "teresina   pi" : "teeasia pi",
    "chur" : "put",
    "al medhatiya" : "the praise",
    "city of malolos (capital)" : "city of malolos (capatial)",
    "jamtara" : "guess",
    "santa cruz de tenerife" : "santa cruz of tenerife",
    "drabhanga" : "dranska",
    "ikorodu" : "illuminating",
    "karbalaa" : "carquel",
    "kaltim" : "east kalimantan",
    "jacobina" : "jacobin",
    "sonepur" : "sonpur",
    "rialp" : "laugh",
    "குளித்தலை" : "bath head",
    "espirito santo" : "holy spirit",
    "marilandia es" : "marilandia is",
    "lapu lapu city 6015, cebu" : "lapa lap city 6015, cebu",
    "21 calle 7 41 zona 11, mariscal" : "21 street 7 41 zone 11, marshal",
    "attur" : "actor",
    "av. livertador 1285 cochabamba bolivia" : "av.freeder 1285 cochabamba bolivia",
    "auarangabad" : "aurangabad",
    "tapejara  rs" : "tapejara rs",
    "sutton" : "suton",
    "liên chiểu   đà nẵng" : "lien chieu da nang",
    "belem" : "white",
    "puente alto" : "high bridge",
    "itu sp" : "that's sp",
    "222, gurmukh jeevan,11th road, nxt to hinduja hospital,khar (w), maharashtra" : "222, gurmukh jeevan, 11 th road, destroy then hinduja hospital, khar (v), maharashtra",
    "adare manor hotel" : "but the manor hotel",
    "fazenda rio grande (pr)" : "rio grande farm (pr)",
    "tandwa,chatra" : "tandwa, student",
    "aracaju sergipe" : "aracaju - sergipe",
    "bongaon" : "bone",
    "el espinal tolima" : "the tolima espinal",
    "tehuacan  puebla  mexico" : "tehuacan puebla mexico",
    "niterói  rj" : "nitero rj",
    "٦ اكتوبر" : "october 6",
    "florianopois" : "florianop",
    "via deserto 101" : "via desert 101",
    "πατησιων 27" : "patision 27",
    "lima, perú" : "lima peru",
    "sri ganganagarj" : "shri gangnagarz",
    "khu nhà ở phú hồng thịnh, dĩ an, bd" : "phu hong thinh, di an, bd housing area",
    "são miguel do iguaçu   paraná" : "são miguel do iguaçu paraná",
    "murrumbateman" : "murgerumbateman",
    "jakarta barat" : "west jakarta",
    "calle 41 n° 27   79 primer piso" : "41th street no. 27 79 first floor",
    "chaibasa" : "laid",
    "fortaleza ceará" : "fortaleza, ceara",
    "lajas" : "slabs",
    "vinhedo" : "vineyard",
    "la grande" : "the big one",
    "bhadohi, u.p ,india" : "bhadohi, u.p., india",
    "recife   pernambuco" : "recife pernambuco",
    "konstanz" : "consistency",
    "islampur. uttar dinajpur" : "islampur.north dinajpur",
    "via del commercio 9" : "via del commerce 9",
    "diinópolis" : "diinopolis",
    "ratia" : "rat",
    "chennak" : "to chen",
    "hutti" : "incarcerate",
    "la senia" : "no sunnis",
    "uttarpara, hooghly, west bengal" : "north, houghli, west bengal",
    "calle 16a n 55 64 antioquia" : "calle 16a n 55 64 colombia",
    "mossoró" : "mossoro",
    "deogarh" : "dogarah",
    "banten， tangerang selatan" : "banten ， south tangerang",
    "lubeldkie" : "lubeldke",
    "azowieckie" : "masovian",
    "ҭ. ڬܬݬ ѭѭ ҭ.18" : "ҭ.ڬ ܬ ݬ ѭѭ ҭ.18",
    "brotas" : "sprout",
    "são bernardo do campo   sp" : "são bernardo do campo sp",
    "kothaguda, kondapur" : "kottaguda, kondpur",
    "são paulp" : "são paulo",
    "palu" : "hammer",
    "cao lãnh" : "cao lanh",
    "ฉะเชิงเทรา" : "chachoengsao",
    "kasoa" : "kao",
    "tucano bahia" : "bahia toucan",
    "honnudike" : "hount",
    "vitorino freire  ma" : "vitorino freire ma",
    "são vicente" : "saint vincent",
    "santa rosa   rs" : "santa rosa rs",
    "nhị quí cai lậy tiền gianh" : "nhi quy cai lay tien giang",
    "kaohsiung" : "gooohining",
    "144 w 133rd sr apt 1" : "144 w 133rd st apt 1",
    "yamuna nagar" : "yam nagar nagar",
    "contagem" : "score",
    "tarumã   sp" : "tarumã sp",
    "edi̇rne" : "normally",
    "magpur" : "nagpur",
    "bhubaneswar" : "bluewar",
    "vila velha   es" : "vila velha es",
    "picedo" : "piced",
    "washougal" : "washogal",
    "kanpur dehat" : "kanpur countryside",
    "lonavla" : "lonavala",
    "são paulo  ' sp" : "sao paulo-sp",
    "panamá city" : "panama city",
    "yanbu" : "eye",
    "łobodno" : "lobby",
    "sanchez mira" : "sanchez look",
    "dokki" : "poisoning",
    "uthangarai ,krishnagiri" : "utangarai, krishnagiri",
    "nova cruz" : "new cross",
    "viale premuda 12" : "viale pressed 12",
    "עשרת" : "ten",
    "pilibanga" : "the pilot",
    "van nuys" : "van nys",
    "sangli" : "sangli is the best.",
    "são paulo   sp." : "sao paulo-sp.",
    "juarez" : "juerez",
    "ποσειδώνος 4 χαλάνδρι" : "poseidon 4 chalandri",
    "umaria" : "maria",
    "baymont inn 2731 watson bld" : "baymont inn 2731 watson blvd",
    "mundra" : "the man of muman",
    "barreiras" : "barriers",
    "são paulo americana" : "american sao paulo",
    "ras al khaimah gg" : "the head of the tent c",
    "jequié  bahia" : "jequié bahia",
    "cao  lãnh" : "cao lanh",
    "hariharpara" : "hariharpada",
    "damnhour" : "damanhour",
    "sjcampos" : "seacamp",
    "laspinas" : "laspin",
    "arnenia quindío" : "armenia quindio",
    "chattogram" : "chittagong",
    "toluca estado de méxico" : "toluca state of mexico",
    "koraput" : "curtain",
    "جدة" : "grandmother",
    "almodhaibi" : "al -mudhaibi",
    "sunam" : "call",
    "eslam mahdi@hotmail.com" : "islam mahdi",
    "gautang" : "loan",
    "بغداد" : "baghdad",
    "marina del rey" : "navy of the king",
    "kalyoubiyah" : "alkalibi",
    "opa locka" : "opa lock",
    "100 đinh bộ lĩnh p26  q.bình thạnh" : "100 dinh bo linh p26 district binh thanh",
    "merida, yucatan" : "merida, yukatan",
    "shirakawa shi" : "shirakawa",
    "al kharj" : "the outside",
    "la jolla" : "the jolla",
    "varanasí" : "varanza",
    "الجيزة" : "giza",
    "matrouh" : "matrosh",
    "taxila" : "taxilas",
    "moorsville" : "mooresville",
    "via degli allori, 79" : "via degli lapel, 79",
    "shope no 3, mangalwai complex" : "shop no 3, mangalwari complex",
    "zalka" : "zalk",
    "cuerámaro gto méxico" : "gto mexico cuerramaro",
    "bhadrak" : "gentleman",
    "aydın kuşadası" : "aydin kusadasi",
    "chourai" : "shuy",
    "ethachlml" : "somehow",
    "calle larga" : "long street",
    "canela" : "cinnamon",
    "10, ql13, hiệp bình phước" : "10, ql13, hiep binh phuoc",
    "itajaí   santa catarina" : "itajaí santa catarina",
    "rangia" : "rangi",
    "salto" : "heel",
    "serra es" : "serra is",
    "marlloca, espanha" : "mallorca, spain",
    "nala sopara" : "subternair",
    "gadarwara" : "bridge",
    "azamgarh tarwa uttar pradesh 276123" : "azamgarh tarawa uttar pradesh 276123",
    "dheerpura" : "dhirpura",
    "paithan" : "penetration",
    "oaxaca" : "oxaca",
    "puerto vallarta" : "vallarta port",
    "región de coquimbo" : "coquimbo region",
    "centro direzionale milanofiori," : "milanofiori disional center,",
    "beba" : "baby",
    "phulbani" : "fulbani",
    "monclova , coahuila" : "monclova, coahuila",
    "toluca, edo mex" : "toluca, or mex",
    "virgem da lapa" : "virgin da lapa",
    "1175, chiêu liêu bình dương" : "1175, lieu binh duong",
    "al sulay" : "al try",
    "são carlos   sp" : "são carlos sp",
    "des moines" : "monks",
    "pfalzen" : "pfallel",
    "haldibari" : "is l -bali",
    "mossoro" : "mossorus",
    "fonte nuova" : "new source",
    "shahajahanpur" : "shahjahanpur",
    "el obour" : "transit",
    "vitória da conquista" : "victory of the conquest",
    "dozza" : "doness",
    "meteti" : "mete",
    "soro" : "difficult",
    "tiruchirappalli" : "turuchir them",
    "haveri" : "breakdown",
    "cd. júarez" : "cd.júarez",
    "pydibhimavaram" : "paidibhimavaram",
    "mazatlan" : "matlan",
    "isranbul beykoz" : "istanbul beykoz",
    "vengurla" : "vetearla",
    "in 46601, ee. uu." : "in 46601, in.his.",
    "kab.tangerang, gdg serpong" : "kab. tangerang, gading serpong",
    "conselheiro lafaiete" : "counselor lafaiete",
    "akwa ibom" : "immoral",
    "1 travessa francisco pereira, pitua?i bahia" : "1 travessa francisco pereira, pitua? i bahia",
    "shindkheda" : "shirtink",
    "riftvalley" : "rift valley",
    "tabuk" : "taboos",
    "350 ne 24st apt 1105" : "350 not 24th apt 1105",
    "general santos" : "santo",
    "rua keisuke koza, 189 s?o paulo" : "keisuke lua, 189 s?power",
    "sta maria bulacan" : "maria bulacan stays",
    "são josé de ribamar ma" : "sao josé de ribamar ma",
    "468 phan văn trị phường 7 gò vấp" : "468 phan van tri ward 7 go vap",
    "antwerpen" : "antwerp",
    "bengaluru" : "bangalore",
    "campo grande" : "large field",
    "santarem para" : "santarem to",
    "barra de são francisco" : "san francisco barra",
    "villeta" : "villet",
    "maravilha" : "wonder",
    "الكويت" : "kuwait",
    "itarumã" : "istarumã",
    "punjab sailkot" : "punjab salakot",
    "bulchandi.malda" : "bunkanki.al",
    "kesariya" : "saffron",
    "presidente getúlio" : "president getúlio",
    "bharuwa sumerpur" : "bharua sumerpur",
    "mineiros" : "miners",
    "ribeirão pires   sao paulo" : "ribeirão pires sao paulo",
    "mummy ki" : "mummy to",
    "shivmogga" : "to the shivogate",
    "lubelskie lublin" : "lublin lublin",
    "distrito nacional" : "national district",
    "chakan pune" : "check the spectrum",
    "los jagüeyes, namiquipa, chihuahua" : "the jagüeyes, namiquipa, chihuahua",
    "san borja" : "san borje",
    "jubail saudi arabia" : "jubel saudi arabia",
    "keshena" : "cereal",
    "vle roma,4" : "want roma, 4",
    "6 ấp 4, nhơn đức, nhà bè" : "6 hamlet 4, nhon duc, nha be",
    "hanamkonda" : "he hasn't crushed him",
    "murau" : "the region",
    "kenitra" : "chest",
    "manab" : "human",
    "wielbrandowo" : "molbrandowo",
    "radom" : "by working",
    "mardi̇n" : "marḋn",
    "muddebihal" : "mudd is more",
    "campos dos goytacazes,rj" : "campos dos goytacazes, rj",
    "novi ligure" : "new liguers",
    "pontalina" : "pontaline",
    "vitória" : "victory",
    "נהריה" : "nahariya",
    "25 อาคารมูฟอะเมซ ชั้น 4 ห้อง 4b" : "25 movez building, 4th floor 4b",
    "san jose del guaviare" : "san jose of guaviare",
    "الأحساء" : "hasa",
    "cordoba, ver" : "cordoba, see",
    "cd. obregon" : "cd.obregon",
    "puerto montt" : "montt port",
    "marinduqe" : "maruqe",
    "salvador   bahia" : "salvador, bahia",
    "aparecida" : "appeared",
    "san carlos" : "saint charles",
    "vijaywada" : "vijayawada",
    "ontario" : "ontaria",
    "الخبر" : "the news",
    "ciudad de panama" : "city ​​of panama",
    "kahului" : "tickets from kahului",
    "guarulhos   são paulo" : "guarulhos sao paulo",
    "באר שבע" : "beer sheva",
    "jabel" : "jelly",
    "1 skyview drfort worthtx76155" : "1 skyview dr fort worth tx 76155",
    "srungavarapukota, vizianagaram" : "shringwarpukota, vijayanagaram",
    "srinagar j&k" : "srinagar j & k",
    "ابوظيي" : "abuzia",
    "nanded." : "nanded",
    "maricá" : "sissy boy",
    "medchal" : "gambling",
    "quenzon city" : "quezon city",
    "lahar" : "lava",
    "catas altas" : "high tastings",
    "سكاكا" : "sakaka",
    "kudal" : "fences",
    "estado de méxico" : "mexico state",
    "sp franca" : "sp frank",
    "đà nẵng" : "danang",
    "quận 2 sài gòn" : "district 2 saigon",
    "para" : "for",
    "angul" : "corner",
    "via caldaia 17" : "via boiler 17",
    "403 quốc lộ 62  tp tân an long an" : "403 national highway 62 tan an long an city",
    "aydın" : "clear",
    "pueblo libre" : "free town",
    "609 medical center dr. decatur, texas, 76234" : "609 medical center dr. decatur, texas, 76234",
    "ichapuram" : "chick",
    "casa branca" : "white house",
    "ouezon" : "loss of planting",
    "devakottai" : "devakotai",
    "paulinia" : "paulinaia",
    "جيزان" : "jizan",
    "tambogrande" : "tambogging",
    "ourinhos" : "outer",
    "dubay" : "gray",
    "กทม." : "bangkok",
    "اسوان" : "aswan",
    "zakrzew" : "zakhil",
    "carrera 80 # 63b  55 cundinamarca" : "carrera 80 # 63b 55 cundinamarca",
    "viale trento ,20" : "viale trento, 20",
    "vũng liêm" : "vung liem",
    "καποδιστριου 27" : "kapodistriou 27",
    "sete barras" : "seven bars",
    "dharpally dist nizamabad" : "dharpalli dish is nizamabad",
    "sykesville" : "heart rate",
    "via bosco n. 9" : "via bosco n.9",
    "avenida 3c # 45 100" : "3c # 45 100 avenue",
    "bùi xương trạch thanh xuân hà nội" : "bui xuong trach thanh xuan hanoi",
    "kailua" : "halewais",
    "ne" : "it is",
    "partur" : "portion",
    "l'anse" : "the handle",
    "cruzeiro do sul   acre" : "southern cruzeiro acre",
    "piduguralla" : "pigrore",
    "etawah" : "elevator",
    "matamoro" : "math",
    "vadgaon" : "vadga",
    "mpumalanga" : "east",
    "lika" : "equal",
    "são mateus" : "saint matthew",
    "harta" : "assets",
    "katoria" : "android",
    "são paulo, arandu" : "sao paulo, arandu",
    "sobótka" : "bonfire",
    "trà vinh" : "tra vinh",
    "tustin" : "pain",
    "ciudad autónoma de buenos aires" : "autonomous city of buenos aires",
    "sahranpur" : "saharanpur",
    "paraíba joão  pessoa" : "paraíba joão pessoa",
    "aquiraz" : "aquiiraz",
    "mato grosso   cuiaba" : "mato grosso cuiaba",
    "novo cruzeiro   m g" : "new cruise m g",
    "siddipet" : "siddipate",
    "الشرقيه" : "eastern",
    "sc   jaraguá do sul" : "sc jaraguá do sul",
    "台中" : "taichung",
    "sanguem" : "face to face",
    "hawalli" : "around",
    "新骏环路" : "xinjun ring road",
    "101 gosai dr" : "101 later dr",
    "encantado" : "charmed",
    "istanbul   büyükcekmece" : "istanbul büyükcekmece",
    "kogarah" : "kagarah",
    "bijapur   vijayapura" : "bijapur vijaypura",
    "belford roxo" : "purple belford",
    "gurgaoun" : "gurgaon",
    "udumalpet" : "dumalpet",
    "caratinga   mg" : "karatinga mug",
    "vitoria da conquista" : "victory of the conquest",
    "sasthamcotta" : "mammalian",
    "joenvile" : "joinville",
    "99 định công hạ" : "99 dinh cong lower",
    "charqueadas" : "pitty",
    "el paso" : "step",
    "miryalaguda" : "miyalaguda",
    "othon p. blanco" : "othon p.blanch",
    "dadri" : "the",
    "hulhumale" : "humid",
    "ki tuc xa khu a, dhqg ho chi minh" : "ki tuc far from a, dhqg ho chi minh",
    "jr, alemania 2485" : "jr, germany 2485",
    "campo largo   paraná" : "long field paraná",
    "itajobi" : "jobi",
    "maglio" : "mallet",
    "ρουσβελτ 18" : "roussvelt 18",
    "kemaman" : "kember",
    "los lagos" : "the lakes",
    "camarines sur philippines" : "camarines on philippines",
    "dehra dun" : "a good man anton",
    "lagos, surulere" : "lagos, surulate",
    "beirut" : "beir",
    "seoni" : "sony",
    "elazığ" : "elazig",
    "arujá" : "aruja",
    "عدن" : "aden",
    "جيزة" : "giza",
    "ấp vĩnh phú xã vĩnh thạnh" : "vinh phu hamlet, vinh thanh commune",
    "ciudad e méxico" : "mexico city",
    "palompoj" : "palombo",
    "marília" : "marilia",
    "64 nguyễn thị tần p2 q8" : "64 nguyen thi tan p2 q8",
    "mysuru" : "my",
    "macul" : "the brave",
    "eranakulam" : "ernakulam",
    "bagepalli" : "baking palli",
    "barueri   sp" : "barueri sp",
    "são paulo, sp" : "sao paulo-sp",
    "altamont springs" : "altamonte springs",
    "hamilton on" : "hamilton is",
    "dakor" : "significance",
    "united kindgom" : "united kingdom",
    "piotrków trybunalski" : "piotrkow trybunalski",
    "مصر" : "egypt",
    "kuthuparamba" : "to sweat",
    "asturias   gijon" : "asturias gijon",
    "المحلة الكبرى" : "mahalla al -kubra",
    "sindagi" : "inferior",
    "cao lanh" : "kaolin",
    "minamisatsuma" : "southern satsuma",
    "villa maria   cordoba" : "villa maria cordoba",
    "jantral" : "jaralal",
    "دبي" : "dubai",
    "chợ mới" : "new market",
    "uruguaiana" : "uruguayan",
    "khordha" : "khodha",
    "lince   lima" : "lince lima",
    "800 3rd ste nw" : "800 width six nou",
    "balneário camboriú   sc" : "balneário camboriú sc",
    "กม" : "km",
    "goiânia   go" : "goiânia go",
    "via xxv ottobre,  4" : "via xxv october, 4",
    "puerto escondido" : "hidden port",
    "taliparamba" : "respondaramba",
    "phuket" : "fuket",
    "channapatna" : "channapana",
    "ulus mah. manolya cad. no:16" : "ulus mah.magnolia cad.no: 16",
    "bogotá, colombia" : "bogota colombia",
    "nova iguaçu   rj" : "nova iguaçu rj",
    "alangulam" : "alongulam",
    "balangir" : "forbidden",
    "san angelo" : "as an angel",
    "santa catarina em itapema" : "santa catarina in itapema",
    "surar" : "chapter",
    "chikkamagaluru" : "ukuru",
    "cornaredo" : "cornado",
    "الدقهلية" : "dakahlia",
    "concepción" : "conception",
    "jaypur" : "jaipur",
    "piauí   pi" : "piauí pi",
    "candiolo" : "cadlo",
    "الجهراء" : "jahra",
    "méxico city" : "mexico city",
    "draria" : "drarea",
    "สมุทรสาคร" : "samut sakhon",
    "arco (tn)" : "arc (tn)",
    "55 camino aguajito" : "55 aguajito road",
    "banihal" : "bees",
    "jaggayyapeta" : "jaggaiahpet",
    "hosapete" : "assistant pente",
    "calle 167a #5a 4" : "calle 167a #kha 4",
    "al madina" : "city",
    "ganaur" : "ganore",
    "são manuel" : "saint manuel",
    "sonnefeld" : "sunfield",
    "stekene" : "stitch",
    "bangkok (กรุงเทพมหานคร)" : "bangkok (bangkok)",
    "quảng nam" : "quang nam",
    "nguyễn xiển thành phố thủ đức" : "nguyen xien thu duc city",
    "27 phường 17 xô viết nghệ tĩnh" : "27 ward 17 soviet nghe tinh",
    "jangareddygudem" : "jangeddygudem",
    "narkatiya ganj" : "narkatia ganj",
    "undri pune" : "underi puts",
    "havre de grace" : "harbour grace",
    "av. doutor alvaro camargos, 1680  mg" : "av.doctor alvaro camargos, 1680 mg",
    "av. guanabara 95 sa8, jardim s?o vicente, itupeva s?o paulo" : "av.guanabara 95 sa8, jardim s? o vicente, itupeva s? o paulo",
    "تربة" : "soil",
    "slaskie" : "silesian",
    "paruthippara" : "cotton",
    "đồng tháp" : "dong thap",
    "kottakkal malppuram" : "kottakkal malappuram",
    "marigot" : "husband",
    "muheillah" : "timing",
    "الرياض" : "riyadh",
    "chuluota" : "out of him",
    "farropilha" : "farroupilha",
    "ilupeju" : "ilupe",
    "waunakee" : "waukyo",
    "uniao da vitoria" : "victory union",
    "250 e sepulveda blvd #1046" : "250 is sepulveda blvd #1046",
    "hamden" : "hammer",
    "st maarten" : "saint martin",
    "rua nossa senhora de lourdes" : "nossa senhora de lourdes street",
    "חדרה" : "hadera",
    "شربين" : "sherbin",
    "arujá   são paulo" : "arujá sao paulo",
    "strzelno" : "shooting",
    "itaporã ms" : "stone ms",
    "بريدة" : "burial",
    "triprayar (valapad)" : "tripreer (valapad)",
    "laguna sta. cruz" : "santa laguna.cross",
    "via 4 novembre, 2" : "via 4 november, 2",
    "sofija" : "sofia",
    "jumaira" : "jumai",
    "são pedro" : "saint peter",
    "armenia, quindío" : "armenia quindio",
    "binan,laguna" : "binan, laguna",
    "mato grosso   alta floresta" : "mato grosso alta floresta",
    "capela do alto" : "chapel",
    "日本" : "japan",
    "ambelakion 28, 4046 germasogia, limassol, cyprus" : "amblakion 28, 4046 germasogia, limassol, cyprus",
    "الظاهره" : "phenomenon",
    "pedra azul" : "blue stone",
    "ירושלים" : "jerusalem",
    "ahemdabad" : "ahmedabad",
    "mohali" : "local",
    "kosli" : "kosla",
    "como" : "as",
    "governador valadares minas gerais" : "governor valadares minas gerais",
    "sp   pirassununuga" : "sp pirassununuga",
    "río negro" : "black river",
    "lima   san borja" : "five san borja",
    "676 n michigan avenue" : "676 road cancer ave nue",
    "kattumannarkoil" : "overlapping",
    "chamarajanagara" : "chamarajanagar",
    "sta. rosa" : "is.rose",
    "cedry wielkie" : "cedry great",
    "hựu thành, trà ôn, vĩnh long" : "huu thanh, tra on, vinh long",
    "kottur" : "cat",
    "bolivar, magangue" : "boli, maganugu",
    "thiruchangkodu" : "church line",
    "7616 benbrook pwky" : "7616 benbrook parkway",
    "gourav bachan" : "gaurav bachan",
    "233 south beaudry avenue  los angeles ca" : "233 south beaudry avenue los angeles ca",
    "oriximiná   pará" : "oriximiná pará",
    "mednine" : "civilians",
    "strada dei palazzi 5d" : "road of the palazzi 5d",
    "lagoa da prata" : "silver lagoa",
    "rewari" : "leimrit",
    "bhayander" : "bayander",
    "lousada" : "sloe",
    "berwyn" : "pillow",
    "botuporanga" : "bowupounga",
    "hyderbad" : "hyderabad",
    "cuidad de mexico" : "mexico city",
    "addis ababa" : "addis short",
    "pabellon de arteaga" : "pabellon de artaaga",
    "voluntari" : "volunteer",
    "اادوحة" : "idah",
    "manaus am ." : "manaus am.",
    "via sempione 27  italia" : "via sempione 27 italy",
    "brejo santo" : "brejo holy",
    "türkiye cumhuriyeti" : "republic of türkiye",
    "kampala  muyenga" : "kampala muyenga",
    "alhasa" : "whiet",
    "favria" : "favory",
    "الخرج" : "output",
    "davao del sur" : "south davao",
    "secha sadan" : "sikad sika",
    "jatai go" : "expressed",
    "rua 15 de novembro, 1262 rs" : "november 15th street, 1262 rs",
    "abudhabi   western region  ghayathi city" : "abu dhabi and sterene, ghiathi city",
    "mahad" : "thanks",
    "mallapur hyderbad" : "mallapur hyderabad",
    "monterrey, méxico" : "monterrey mexico",
    "florianópolis   sc" : "florianópolis sc",
    "recife   pe" : "recife on",
    "são paulo sp" : "sao paulo-sp",
    "calle 89 no 67 136 811014571" : "street 89 no 67 136 811014571",
    "cd mx" : "cdmx",
    "soufriere" : "sufriere",
    "são paulo   são paulo" : "sao paulo sao paulo",
    "indoren" : "indo",
    "chrompet" : "cromarty",
    "εθνικης αντιστασεως" : "national resistance",
    "imus cavite" : "we go to cavite",
    "colgong" : "colong",
    "rio dr janeiro" : "rio de janeiro",
    "curitiba parana" : "curitiba, parana",
    "alamuru" : "opportunity",
    "el salvador" : "the savior",
    "czeladź" : "czeladz",
    "thủ đức, tp hcm" : "thu duc, ho chi minh city",
    "khorasan razavi" : "khorasan razvi",
    "المدينه المنوره" : "al madinah al munawwarah",
    "neily" : "nile",
    "calle 64 b # 1  71" : "calle 64 b # 1 71",
    "eustis" : "welcome",
    "كربلاء المقدسة" : "holy karbala",
    "via vittorio emanuele,17" : "via vittorio emanuele, 17",
    "jawabarat" : "answer",
    "madinat abqaiq" : "madinat abqiqi",
    "nearambedkar hospet" : "nearambakkar hospet",
    "mohania" : "mohinia",
    "θηβων 500 ιλιον" : "thebes 500 ilion",
    "açailândia   ma" : "açailândia ma",
    "boa esperança" : "good hope",
    "po box 701362  san antonio" : "well box 701362 san antonio",
    "quảng ninh" : "quang ninh",
    "5949 via del cielo  cloud it" : "5949 via del cielo cloud it",
    "blvd. miguel aleman n?m. 1101, san mateo atenco edo. de m?xico estado de m?xico" : "blvdmiguel aleman n? m.1101, san mateo atenco edo.of the same state of music",
    "oshawa" : "shawawa",
    "campo novo do parecis" : "parecis novo campo",
    "buenos aires , florida" : "buenos aires, florida",
    "dourados" : "golden",
    "sindekela" : "sindekelala",
    "kaij" : "hallter",
    "duliajan" : "dulia",
    "2500 menaul blvd ne" : "2500 menal blvd not",
    "اسكندرية" : "alexandria",
    "ตาก" : "dry",
    "anuppur" : "transmitter",
    "gurugam" : "gurgam",
    "santos domingo" : "saints sunday",
    "sancristobal" : "sanctobal",
    "ji paraná" : "her paraded",
    "santa cruz   rio gallegos" : "santa cruz rio gallegos",
    "florida 1 piso: 12 ofic: 48   entre: esquina rivadavia   capital federal (1005) buenos aires" : "florida 1 floor: 12 off: 48 between: corner rivadavia capital federal (1005) buenos aires",
    "kaimur bhabuan" : "kaimur bhabhua",
    "playa del carmen" : "carmen beach",
    "401st afsb" : "401st aff",
    "villa alemana" : "german village",
    "bangaluru" : "bangluru",
    "mancherial telangana" : "manancherl telangana",
    "purwkhn" : "prize",
    "atílio vivacqua" : "atilio vivacqua",
    "99 định công hạ  hoàng mai" : "99 dinh cong ha hoang mai",
    "caieiras   são paulo" : "caieiras são paulo",
    "tekirdağ" : "tekirdag",
    "rosário oeste mt" : "rosario west mt",
    "getulina" : "we have it",
    "no.08, sir chittampalam a gardiner mw," : "no.08, sir chittampam gardin mw,",
    "piro" : "score",
    "tandwa" : "struggle",
    "chinsurah" : "chinasur:",
    "vitória da conquista   bahia" : "vitória da conquista bahia",
    "rio de janeiro   rj" : "rio de janeiro - rj",
    "jognda" : "jogna",
    "kodagu" : "kernel",
    "gurgoan" : "gurgaon",
    "6october" : "6 october",
    "lavras" : "mulch",
    "blairsville" : "blairville",
    "begusaria" : "begusariia",
    "gaurulhos" : "guarulhos",
    "kathumar" : "jelum",
    "juan de acosta atlantico" : "juan de acosta atlantic",
    "motihari  chowk" : "motihari chowk",
    "dasuya" : "put out",
    "balneário piçarras" : "balneario piçarras",
    "القاهرة" : "cairo",
    "100 vestavia parkwaybirminghamal35216" : "100 vestavia parkway birmingham al 35216",
    "atlantico" : "atlantic",
    "pompeia, sao paulo" : "pompeii, sao paulo",
    "ponda goa" : "crush goa",
    "la plata" : "at payment",
    "espinar" : "spine",
    "tocantins   palmas" : "tocantins palmas",
    "volta redonda" : "round",
    "agbara" : "power",
    "olinda pe" : "olinda the",
    "virar east" : "east",
    "mato grosso,  sinop" : "mato grosso, sinop",
    "banglore" : "bengaluru",
    "ابوظبي" : "abu dhabi",
    "kalwa" : "calf",
    "perú lima" : "lima peru",
    "nova resende" : "new resende",
    "mumbai" : "moon",
    "cote d'ivoire" : "ivory coast",
    "διγενή 25" : "digeni 25",
    "na vyhlídce 268 268" : "on the prospect of 268 268",
    "carpina   pernambuco" : "carpina pernambuco",
    "słupska" : "słupsk",
    "bünde" : "fründe",
    "el cerrito" : "the hill",
    "vitória   espirito santo" : "vitoria, espirito santo",
    "khagaria" : "khagariia",
    "النوبارية" : "nubaria",
    "joelformcheck magnit joelformcheck magnit" : "joelformcheck mets joelformcheck magnet",
    "pan india" : "indian bread",
    "colombia c2: solo información" : "colombia c2: only information",
    "via fontana dell'oste 52" : "via fontana dell'osta 52",
    "barranquilla, atlantico" : "barranquilla, atlantic",
    "calle goya, 90" : "goya street, 90",
    "roha" : "steam",
    "aquiraz   ce" : "aquiraz ce",
    "المدينة المنورة" : "al madinah al munawwarah",
    "três corações" : "three hearts",
    "ponda" : "crush",
    "via barone s. ferla" : "via barone s.ferla",
    "sc   maravilha" : "sc wonder",
    "boca raton" : "bottle of raton",
    "raigad" : "rig",
    "mahnar" : "mahanar",
    "ribeirão preto   sp" : "ribeirão preto sp",
    "arsikere" : "arsic",
    "abudhabi" : "abu dhabi",
    "viluppuram" : "developuram",
    "rua piau" : "piau street",
    "carazinho" : "cad",
    "odenton" : "denton",
    "rio das ostras   rj" : "rio das ostras rj",
    "región metropolitana" : "metropolitan region",
    "nagpur india" : "india",
    "台灣" : "taiwan",
    "vadodara" : "let them",
    "correntes" : "chains",
    "gangavathi" : "gangavati",
    "balneário camboriú  sc" : "balneário camboriú sc",
    "barka" : "helloon",
    "jalna" : "burn",
    "kota bandung" : "bandung",
    "cordilheira alta   sc" : "high cordilher sc",
    "portharcourt" : "porthcourt",
    "i̇zmir urla" : "ilkir urla",
    "avenida de europa 4 planta baja, local b" : "avenida de europa 4 ground floor, local b",
    "rua leonor maria barbosa condom?nio village ba" : "rua leonor maria barbosa condom?nicil village i",
    "teste" : "test",
    "frontera hidalgo" : "hidalgo border",
    "avc systemy audiowizualne jakub pacholewicz, ul. widokowa 3" : "avc audiovisual systems jakub pacholewicz, ul.viewing 3",
    "kosagumuda" : "kosa no waste",
    "waraseoni, dist balaghat" : "varasai, district balaghat",
    "kerala  kannur" : "kerala kannur",
    "breaux bridge" : "breaux bridge",
    "piler" : "arrows",
    "chikodi" : "health",
    "stevensville" : "stevensville",
    "amparo" : "protection",
    "mau ranipur" : "it's ranipur",
    "parana , medianeira" : "parana, medianeira",
    "peje" : "pe said",
    "parbhani" : "perbhani",
    "edappal" : "adapal",
    "san jose del monte" : "san jose of monte",
    "lastra a signa" : "slab in signa",
    "bazas" : "bazae",
    "116đường cầu kinh" : "116 cau kinh road",
    "rajam" : "stoning",
    "خميس مشيط" : "khamis mushait",
    "noorsarai" : "joun",
    "sveta nedelja" : "holy sunday",
    "basavanbagewadi" : "basebagewadi",
    "sayoun" : "sion",
    "shahabad markanda distt kurukshetra" : "shahabad markanda distra kurukshetra",
    "benito juárez" : "benito juarez",
    "proszowice, kraków, małopolskie" : "proszowice, kraków, lesser poland",
    "rajashthan" : "rampage",
    "sialkot" : "sigot",
    "la rioja" : "the rioja",
    "piedras negras" : "black stones",
    "amethi" : "to",
    "40 s k bole marg" : "3 s.",
    "tada" : "then",
    "bazpur uttrakhand" : "bajpur uttarakhand",
    "الاسكندرية" : "alexandria",
    "via d.f. rosso" : "via d.f.red",
    "nazaré da mata  pernambuco" : "nazaré da mata pernambuco",
    "la chorrera" : "the chorrera",
    "sao paulo vargen grande do sul" : "sao paulo vargem granding do sul",
    "moshi,pimpri  chinchawad" : "moshi, pimpri chinchwad",
    "umri" : "age",
    "duabi" : "dubai",
    "fort pierre" : "strong stone",
    "manavalanagar" : "manwalnagar",
    "akole" : "we can not",
    "boa vista" : "good view",
    "san carlos," : "saint charles,",
    "partur,jalna" : "part, jalna",
    "dubi" : "look",
    "kodambakkam" : "kodubakkam",
    "league coty" : "league city",
    "barra velha" : "old bar",
    "pinhalzinho" : "pineglass",
    "الرياض السعودية" : "al riyadh, saudi arabia",
    "fraiburgo" : "fraiburg",
    "maharashatra" : "maharashtra",
    "andana" : "story",
    "b0g0tá" : "b0g0g0",
    "dubia" : "doubts",
    "gaziantep i̇slahiye" : "gazi̇antep i̇slahi̇ye",
    "sumatera utara" : "north sumatra",
    "shagamu" : "shop",
    "cachoeiro" : "waterfall",
    "18 r. aquino st., hagdang bato libis, mandaluyong city metro manila" : "18 r.aquino st., hagdang bato libis, mandaluyong city metro manila",
    "jantral vijapur mehsana gujarat india" : "bharanal vijapur mehsana gujarat india",
    "dreieich" : "triad",
    "via tosco romagnola  1422" : "via tosco romagnola 1422",
    "montería" : "hunting",
    "信義區" : "xinyi district",
    "resende" : "reset",
    "النبطية" : "nabatieh",
    "schladming" : "sladming",
    "quitó" : "he removed",
    "san josé" : "saint joseph",
    "são gabriel do oeste   ms" : "são gabriel do oeste ms",
    "ny 11358" : "the 11358",
    "castelo" : "castle",
    "betul" : "proper",
    "rua celso ramos, 145   vila andrade   s?o paulo   sp   cep: 05734 080 sp" : "rua celso ramos, 145 vila andrade são paulo sp zip code: 05734 080 sp",
    "malur" : "anoint",
    "paita" : "shirt",
    "marib" : "maribe",
    "7001 e fish lake rd #200  cloud it" : "7001 e fish lake rd #200  cloud it",
    "cuiabá   mt" : "cuiabá mt",
    "vasai road, mumbai" : "wasi road, mumbai",
    "thủ dầu một, tỉnh bình dương" : "thu dau mot, binh duong province",
    "reasi" : "reaction",
    "yanbu al sinaiyah" : "eye aris resistance marks",
    "chile, santiago" : "santiago chile",
    "contai" : "anyway",
    "paranaque, metro manila" : "paranaque, metro manila",
    "mestre venezia" : "mestre venice",
    "madikeri" : "madatti",
    "haag in oberbayern" : "haag in upper bavaria",
    "700 patroon creek blvdalbanyny12206" : "700 pattern creek blvdalbanyny12206",
    "passos" : "steps",
    "الشارقة" : "sharjah",
    "malerkotla" : "painter kotla",
    "itanhaém  sp" : "itanhaém sp",
    "aracaju se" : "aracaju if",
    "el dorado hills" : "the dorado hills",
    "57 neumann road    capalaba  qld  4157  us qld" : "57 neumann road capalaba qld 4157 us qld",
    "maceió   al" : "macea al",
    "atizapan, estado de méxico" : "atizapan, state of mexico",
    "provinsi banten" : "banten province",
    "sta. rosa city" : "is.rosa city",
    "645 hulet dr ste 200" : "645 hollowed dr ste 200",
    "franca são paulo" : "franca sao paulo",
    "rajghar" : "royal",
    "sqlvador" : "salvador",
    "sjo" : "sea",
    "al qusais" : "al quasais",
    "πάροδος θηβαΐδος 21" : "thebaidis 21",
    "pozzonovo" : "refuse",
    "chakia" : "hugs",
    "neópolis" : "neopolis",
    "القصيم بريده" : "al -qassim is his mail",
    "kí túc xá khu a đhqg tp. hcm" : "dormitory a vnu tp.hcm",
    "ny" : "the",
    "pilani jhunjunu" : "pilani jhhununu",
    "sai paulo" : "you know paulo",
    "tibu" : "chick",
    "brgy 13 naglicuan, pasuquin, in" : "brgy 13 naglikuan, pasuquin, in",
    "wynnum" : "whiten",
    "tadiprru" : "earlierpru",
    "torreblanca" : "white tower",
    "ramnagar" : "ramnagar ramnagar",
    "nizmabad" : "nizamabad",
    "bhestan, surat." : "bhestan, letter.",
    "biloxi" : "comparison",
    "balıkesir" : "balikesir",
    "al jubail" : "to the jubail",
    "kosova" : "chuang",
    "santarém  pará" : "santarém pará",
    "linh  trung,  thủ đức" : "linh trung, thu duc",
    "ghatsila" : "ghatila",
    "joão pessoal" : "joão pessoa",
    "kuj pomorskie" : "kuj pomeranian",
    "新竹" : "hsinchu",
    "taloja" : "houses",
    "joão pessoa   pb" : "joão pessoa pb",
    "videira" : "vine",
    "warmińsko  mazurskie" : "warmian masurian voivodeship",
    "johannesburh" : "johannesburg",
    "tamluk" : "ownership",
    "chennaj" : "chennai",
    "via d. alighieri 13" : "via d.alighieri 13",
    "chengalpattu district" : "district",
    "malkapur" : "malakpur",
    "thatta" : "mockery",
    "wolfschlugen" : "wolf",
    "recife  pernambuco" : "recife pernambuco",
    "san lorenzo" : "saint lawrence",
    "samana" : "the same",
    "tangerang selatan" : "south tangerang",
    "nossa senhora do socorro" : "our lady of socorro",
    "via venezia n. 69" : "via venezia n.69",
    "tezpur" : "tamur",
    "balikesi̇r edremi̇t" : "fish edrememi",
    "175 e. schroon river road" : "175 e.river road schron",
    "boa vista  rr" : "boa vista rr",
    "jharsuguda" : "guard",
    "bình thạnh" : "binh thanh",
    "akluj" : "accuse",
    "santo andré   sp" : "santo andré sp",
    "dolny śląsk" : "lower silesia",
    "merkez" : "centre",
    "são paulo (sp)" : "sao paulo-sp)",
    "aurangabad mh" : "aurangabad maha",
    "pomorskie" : "pomeranian",
    "zahle" : "pay",
    "margoa" : "painting",
    "rahata city." : "live city",
    "doda jammu" : "adds to the jammu",
    "bergdietikon" : "bergdietic",
    "nord rhein" : "north rhine",
    "londres" : "london",
    "accettura" : "accepting",
    "bình chánh, tp hồ chí minh" : "binh chanh, ho chi minh city",
    "las palmas de gran canaria" : "the gran canarian palms",
    "coronel fabriciano" : "colonel fabriciano",
    "bhusawal" : "bhusaval",
    "173 xuân thiy" : "173 xuan thuy",
    "navi mumbai (new mumbai)" : "navi mumbai (new moon)",
    "novo hamburgo" : "new hamburg",
    "camarenis sur" : "south camarines",
    "s.s. 115 avola siracusa km 388,8" : "s.s.115 avola siracusa km 388.8",
    "dhanbad" : "dhanbad dhanbad",
    "sutrapada" : "tomorrow",
    "criciúma" : "criciuma",
    "casado" : "married",
    "hosadurga" : "hosadadugha",
    "niš" : "nis",
    "chittaurgarh" : "chittorgarh",
    "neemrana" : "neemountain",
    "kothagudem" : "newgudem",
    "الاغوار الجنوبيه" : "southern jordan valley",
    "via l. da vinci 12" : "via l.da vinci 12",
    "santiago chile" : "santiago, chile",
    "alterosa" : "alterious",
    "berlin   neukölln" : "berlin neukölln",
    "catigua" : "catiguua",
    "caçador" : "hunter",
    "hathars" : "hathras",
    "kruszyn" : "crumble",
    "rangapara" : "raduate",
    "campi salentina" : "salento campi",
    "76 đường 34, an phú, q.2" : "76 street 34, an phu, district 2",
    "villahermosa" : "beautiful villa",
    "19 picadlly rd hampstead" : "19 piccadilly rd hampstead",
    "estrada das carrascas" : "carrascas road",
    "minas gerais visconde do rio branco" : "minas gerais viscount of rio branco",
    "pusad" : "sweaters",
    "via alba 20   roddi" : "via alba 20 roddi",
    "sao paulo sp" : "sao paulo-sp",
    "rio grande do sul   porto alegre" : "rio grande do sul porto alegre",
    "8795 runamuck pl, unit b  medical" : "8795 runamuck pl, unit b medical",
    "20370 passagio drive  medical" : "20370 passage drive medical",
    "sordio" : "sordy",
    "oswego" : "oestrus",
    "pattukkottai" : "tukkottai",
    "virum, denmark" : "before, dermark",
    "gia tân 2, thống nhất, đồng nai" : "gia tan 2, thong nhat, dong nai",
    "cabo sa lucas" : "cabo in lucas",
    "umhlanga rocks" : "reed rocks",
    "guarulhos são paulo" : "guarulhos sao paulo",
    "marília  sp" : "marília sp",
    "via campo marinaro, 8 italia" : "via campo marinaro, 8 italy",
    "mahdia" : "can",
    "paratwada" : "paste",
    "solteiro(a)" : "single",
    "cabreúva" : "cabreuva",
    "pontotoc" : "pontoc",
    "jebel ali" : "mount on",
    "via dante n. 34   nuxis (su)" : "travel n.34 nuxis (su)",
    "phú quí cai lậy tiền giang" : "phu qui cai lay tien giang",
    "hristo smirnenski blv., 30" : "hristo smirnenski bl., 30",
    "cruzeiro" : "cruise",
    "pr   foz do iguaçu" : "pr foz do iguaçu",
    "curitiba.   paraná" : "curitiba.paraná",
    "113 หมู่ 6 ลาดพร้าว 124" : "113 moo 6 lat phrao 124",
    "bambavde, kolhapur" : "bambavade, kolhapur",
    "riaydh" : "riaeyd",
    "pampangga" : "pampanga",
    "pomona" : "pomon",
    "prishtina" : "pristina",
    "fdzs" : "fed",
    "limeira" : "lime",
    "davao city" : "gave city",
    "av. boyacá n 67 96 813013550" : "av.boyacá n 67 96 813013550",
    "mahi" : "assignment",
    "مكه المكرمه" : "makkah",
    "rura,kanpur dehat" : "rura, kanpur dehat",
    "majri" : "stream",
    "al qatif ash sharqiyah" : "al qatif ash sharqi",
    "ecuador manabí  manta" : "ecuador manabí manta",
    "pelotas rs" : "rs balls",
    "minh khai  hai bà trưng hà nội" : "minh khai hai ba trung hanoi",
    "bairagania" : "bargania",
    "putinga" : "to putin",
    "burien" : "yellowish garden",
    "conselheiro pena" : "pena",
    "becharaji" : "disgraceful",
    "1100 lee ave, lafayette, la 70501," : "1100 lee ave, lafayette, 70501,",
    "rio do sul" : "south river",
    "vitotoria" : "mtotoria",
    "gò công đông tiền giang" : "go cong dong tien giang",
    "18, đường số 12, hiệp bình chánh" : "18, road no. 12, hiep binh chanh",
    "bhanga" : "bank",
    "śląsk" : "silesia",
    "san isidro" : "yese isiddro",
    "56a dương thị mười" : "56a duong thi muoi",
    "pangasinan,pozorrubio" : "pangasinan, pozorrubio",
    "garhmuktesar" : "gadhmuktear",
    "nagpur yatmaval" : "nagpur yatmawal",
    "los angeles" : "the angels",
    "beni mellal" : "bani boredom",
    "barra mansa rj" : "mansa rj bar",
    "hajipur,  phubihar" : "hajipur, fubihar",
    "naigoan" : "went",
    "via dei fiumi" : "via dei riversi",
    "buengkum" : "builder",
    "sindgi" : "are",
    "sanyo antônio do monte" : "santo antônio do monte",
    "ambajogai" : "to the ambition",
    "bhrompur" : "bharompur",
    "te a, 2222 w. grand river ave, okemos, mi, 48864" : "a, 2222 w.grand river ave, ok, mi, 48864",
    "kolkata, india" : "kolka, india",
    "centro de distribucion amelia  it" : "amelia it distribution center",
    "2826 nw 72 av" : "2826 nw 72 of",
    "davanagere" : "davangere",
    "dibai" : "glance",
    "taguatinga sul" : "southern taguatinga",
    "tanta" : "so much",
    "puerto wilches" : "wilches port",
    "colina" : "hill",
    "9110 forest crossingthe woodlandstx77381" : "9110 forest crossing the woodlands tx 77381",
    "mandigobindgarh(amandeepanand786@gmail.com)" : "mandigobindagarha (amandipananda",
    "são paulino" : "sao paulino",
    "cabo frio   rj" : "cabo frio rj",
    "74 molave st. z 1 central signal village, taguig city philippines" : "74 molave st.z 1 central signal village, taguig city philippines",
    "warmińsko mazurskie" : "warmian masurian voivodeship",
    "mankapur gonda" : "manikpur gonda",
    "itajaí   sc" : "itajaí sc",
    "dumka" : "opinion",
    "raichur" : "raycur",
    "governador valadares mg" : "governor valadares mg",
    "hyderabab" : "hyderabad",
    "serra   espírito santo" : "serra espírito santo",
    "vico 2° ginnasio 8" : "vico 2nd gymnasium 8",
    "missoula" : "touched",
    "hailakandi" : "hallakandi",
    "ras alkhaimah" : "ras al khaimah",
    "yeddumailaram" : "bull",
    "butler" : "thighs",
    "rio de janiero" : "rio de janeiro",
    "batala" : "butter",
    "atlanta, ga" : "ah, but",
    "brasília   df" : "brasilia df",
    "อยุธยา" : "ayutthaya",
    "الاحساء" : "hasa",
    "al fujairah" : "the dawn",
    "crisciuma" : "i'm going to",
    "الانبار" : "anbar",
    "ciudad de mexi o" : "mexico city",
    "assis são paulo" : "assis sao paulo",
    "thanh hoá" : "thanh hoa",
    "viale alpi orientali nord, 6 int. 3" : "viale alps orientali nord, 6 int.3",
    "kodikonda check post" : "choose check post",
    "gidderbaha" : "budgebaha",
    "bauneario camburiu" : "balneário camboriú",
    "narre warren" : "fool warren",
    "รพ.สมิติเวช" : "samitivej hospital",
    "mukherjinagar" : "mukherjnagar",
    "terlingua" : "elder",
    "ernakulum" : "ernamum",
    "6300 harry hines blvdste. 1400dallastx75235" : "6300 harry hines blvd ste. 1400 dallas tx 75235",
    "ciudad arce" : "arce city",
    "sta. maria bulacan" : "stay.maria bulacan",
    "colíder" : "collide",
    "new canaan" : "new year",
    "piedras negraa" : "black stones",
    "campo verde mt" : "green field mt",
    "102c đường 79, phước long b," : "102c street 79, phuoc long b,",
    "padre hurtado  central 673las condes santiago" : "father hurtado central 673las condes santiago",
    "salvador   ba" : "salvador ba",
    "milão" : "milan",
    "nürnberg" : "nuremberg",
    "galati" : "galata",
    "biała" : "white",
    "bhndara" : "bhandara",
    "chatrapati sambhajinagar" : "chhatrapati sambhajinagar",
    "la trinidad" : "the trinity",
    "mandvi" : "mandavi",
    "lima lima" : "five five",
    "leicesterhsire" : "leicestershire",
    "den haag" : "the hague",
    "sojatcity" : "sojat city",
    "santa bárbara d'oeste" : "santa barbara d'oeste",
    "104 caldo" : "104 hot",
    "القاهره" : "cairo",
    "611 w. park street fait medical" : "611 w.park street make medical",
    "tiruchirapalli" : "truuchrapalli",
    "luna la union" : "luna laon",
    "odisha baripada" : "odisha barada",
    "parli vaijnath" : "married vijayanath",
    "trà ôn, vĩnh long" : "tra on, vinh long",
    "kovilpatti" : "kovilpat",
    "दिल्ली" : "delhi",
    "pangkal pinang" : "town",
    "जैतारण आनंदपुर कालू" : "jaitaran anandpur kalu",
    "newnan" : "they were new",
    "jawa tengah" : "central java",
    "ahad rofaida ksa" : "sunday rofaida ksa",
    "barraquilla" : "barranquilla",
    "jardim do seridó rn" : "seridó rn garden",
    "مبارك الكبير" : "mubarak al -kabeer",
    "gov. valadares" : "gov.valadares",
    "macae rj" : "macae raj",
    "karaikal" : "stains",
    "tusayan" : "show",
    "hoogbocht 1" : "beam 1",
    "810 n kingston dr peoria, il 61604 2145" : "810 n kingston dr peoria, il",
    "jabel ali" : "mount on",
    "pandhurna" : "bad",
    "thành phố hồ chí minh" : "ho chi minh city",
    "gaouteng" : "karcarian hurts",
    "tân bình" : "tan binh",
    "kapadwanj" : "kapadwanjj",
    "ouro branco" : "white cholocate bonbon",
    "itaboraí   rj" : "itaboraí rj",
    "κ. βαρναλη 130 132" : "k. varnali 130 132",
    "xalapa e ver" : "xalapa and see",
    "imbituva pr" : "pilva pr",
    "rosario   santa fe" : "rosario santa fe",
    "jakarta timur" : "east jakarta",
    "arkport" : "carpet",
    "ichalkaranji" : "akinkaranan",
    "via col vento" : "gone with the wind",
    "nordrhein westfalen   herford" : "north rhine westphalia herford",
    "amarillo" : "yellow",
    "uruara" : "unafare",
    "mancherial" : "mancheril",
    "russas" : "russian",
    "500 valley rd.,  suite 202  it" : "500 valley rd.,  suite 202  it",
    "campo novo" : "new field",
    "tp ho chi minh" : "ho chi minh city",
    "nowe skalmierzyce" : "new skalmierzyce",
    "san nicolas de los garza" : "san nicolas de los heros",
    "9700 maumelle blvd" : "9700 multidio blvd",
    "changanacherry, kottayam" : "changanacheri, kottayam",
    "keshav rai patan" : "keshav rai collapse",
    "ongole" : "credit",
    "bogazkoy" : "bogatskoy",
    "mumbai ( mumbai)" : "mumbai (mumbai)",
    "القليوبيه" : "qaliubiya",
    "เพชรบูรณ์" : "phetchabun",
    "cachoeira do piriá pará" : "piriá pará waterfall",
    "30 ung văn khiêm bình thạnh" : "30 ung van khiem binh thanh",
    "nova andradina   mato grosso do sul" : "nova andradina mato grosso do sul",
    "małopolskie, 34 436 maniowy" : "lesser poland, 34 436 maniowy",
    "baradwar" : "paradise",
    "dhuri" : "the wall",
    "el segundo" : "the second",
    "anjugramam" : "anjugaramam",
    "2 alexfisher drive    burliegh heads  qld  4220  us qld" : "2 alex fisher drive burleigh heads qld 4220 us qld",
    "hai phong" : "two phong",
    "pedro leopoldo" : "pedro leopold",
    "thành phố hồ chí minhp" : "ho chi minh city",
    "arambagh" : "arambagi",
    "11 penns trail bldg. 100" : "11 pennies trail bldg.100th",
    "centro" : "center",
    "salete" : "silet",
    "shivapark" : "shivakarak",
    "9x chu văn an" : "9x is văn",
    "rua doutor álvaro albuquerque" : "rua doctor álvaro albuquerque",
    "184 lê đình can" : "184 le dinh can",
    "małopolskie" : "lesser poland",
    "hưng yên" : "hung yen",
    "jaora" : "loser",
    "franca   sp" : "franca sp",
    "14334 e layton pl  medical" : "14334 e layton pl medical",
    "gaziabad" : "attractive",
    "bakı" : "baku",
    "birsa, balaghat, mp india" : "birsa, balaghat, pump india",
    "kalimantan barat" : "west kalimantan",
    "nashirabad" : "nasirabad",
    "cancún, q. roo" : "cancun, q.roo",
    "distrito capital de bogotá" : "capital district of bogotá",
    "ofibodegas fraijanes bodega 23 km. 18 carretera a san jose  commercial display" : "ofibodegas fraijanes bodega 23 km.18 road to san jose commercial display",
    "الطائف" : "taif",
    "vespasiano" : "vespasian",
    "via irlanda, 12" : "via ireland, 12",
    "vitoria es" : "vitoria is",
    "gulbarga" : "to a floron",
    "gorari" : "lalli",
    "botad" : "cured",
    "φιλολαου 28 ηλιουπολη" : "filololaou 28 ilioupoli",
    "ipatinga  mg" : "ipatinga mg",
    "santa cruz do capibaribe   pe" : "santa cruz do capibaribe pe",
    "se" : "himself",
    "karachi gulshane shamim fb area" : "karachi gulzhane shamim fb area",
    "corso m. d'azeglio 30" : "course m.d'azeglio 30",
    "kodinar" : "codeinar",
    "malappuram, manjeri" : "malappuram, manjari",
    "cartegena" : "cartagena",
    "paulínia" : "paulinia",
    "tân phước tiền giang" : "tan phuoc tien giang",
    "ganaur, sonipat" : "ganore, sonipat",
    "wielkopolskie" : "greater poland",
    "9000 executive prk dr ste c200 healthcare" : "9000 executive park dr ste c200 healthcare",
    "جازن بني مالك" : "jazen bani malik",
    "såo paulo" : "são paulo",
    "c. so  principe eugenio, 3" : "c.i know prince eugenio, 3",
    "arava" : "cart",
    "são paulo  são paulo" : "sao paulo sao paulo",
    "palvoncha telangana" : "palvoncha telankana",
    "ದೋಡಬಳಪುರ" : "dodalpur",
    "pitalito  huila" : "pitalito huila",
    "la reian" : "i ree",
    "sp   taubaté" : "sp taubaté",
    "peruibe" : "peruve",
    "la habana" : "havana",
    "n, la cabecera, san felipe del progreso estado de mexico estado de mexico" : "n, the head, san felipe del progreso state of mexico state of mexico",
    "ventanas" : "windows",
    "bhainsa" : "buffalo",
    "dalkhola" : "dalhala",
    "bengalore" : "bengaloreore",
    "bijapur(kar)" : "bijapur (tax)",
    "سكاكا الجوف" : "sakaka al -jouf",
    "qurum" : "institution",
    "riffa" : "rife",
    "alegrete" : "cheerful",
    "bueos aires caba" : "buenos aires caba",
    "tiruvannamalai" : "thiruvannamalai",
    "dois irmãos" : "two brothers",
    "vllavicencio" : "villavicencio",
    "1940 jefferson hwy" : "1940 jefferson longer",
    "la serena" : "serena",
    "quinta normal" : "normal fifth",
    "niedersachsen" : "lower saxony",
    "dalsingh sarai" : "dalsingh sarah",
    "mersi̇n" : "mersow",
    "naazareno" : "nazarenaw",
    "palani" : "france",
    "10 nguyễn thanh đằng, bà rịa" : "10 nguyen thanh dang, ba ria",
    "sagarejo" : "sagrejo",
    "são josé dos pinhais   paraná" : "são josé dos pinhais paraná",
    "padrauna" : "subtle",
    "nieborów" : "heavenly",
    "várzea grande" : "varzea grande",
    "abugon, sibonga, cebu" : "about, sibonga, cebu",
    "ροσταν 44" : "rostan 44",
    "3330 n 104 ave  234 hotel" : "3330 n 104th ave 234 hotel",
    "malout" : "tend",
    "bhadaur" : "bhadore",
    "são luís do quitunde   alagoas" : "são luís do quitunde alagoas",
    "sulawesi selatan" : "south sulawesi",
    "umm al quwain" : "umm qun",
    "mazowieckie , radom" : "mazowieckie, radom",
    "aldie" : "all",
    "hidrolândia" : "hydroland",
    "江苏" : "jiangsu",
    "sopore" : "slackly",
    "pinhais" : "pinemers",
    "zamb0anga city" : "zamboanga city",
    "vennanthur, namakkal" : "vennandur, namakkal",
    "riyadh, ksa" : "riad, story",
    "dhahran" : "dharan",
    "ggn" : "evidence",
    "santo antônio de pádua" : "santo antônio de padua",
    "ende" : "end",
    "arroio do sal" : "salt",
    "phường 6 , quận gò vấp" : "ward 6, go vap district",
    "iloilo city" : "iloulo city",
    "chittaranjan" : "delicate",
    "perambalur." : "pounding.",
    "almenara" : "beacon",
    "75a, phan đình phùng, phú nhuận" : "75a, phan dinh phung, phu nhuan",
    "nalbari" : "illumon",
    "sitamarhi ,bihar" : "sitamarhi, bihar",
    "32 dương quảng hàm" : "32 duong quang ham",
    "ballari" : "scores",
    "villupuram" : "vilupuram",
    "canaã dos carajás" : "canaan of carajás",
    "jakarta utara" : "north jakarta",
    "jaboatão   pe" : "jaboatão pe",
    "163 trần phú, đà nẵng" : "163 tran phu, da nang",
    "lumut,perak" : "moss, silver",
    "kodungallur" : "kodungallurur",
    "kotli surat malli" : "kotli surat model",
    "av 2g # 44 32" : "of 2g # 44 32",
    "حدائق القبه" : "dome gardens",
    "cn   monasterolo di savigliano" : "cn monasterolo di savigliano",
    "कैलाश कुमार" : "kailash kumar",
    "bulacan bulacan" : "bulcan bulcan",
    "villavcencio" : "villavicencio",
    "10880 wilsire blvd suite 1101 lg pro" : "10880 wilshire blvd suite 1101 lg pro",
    "chowari" : "streets",
    "al sokhna" : "the heat",
    "q. thủ đức" : "q.thu duc",
    "36 hoàng cầu, đống đa, hà nội" : "36 hoang cau, dong da, hanoi",
    "barra mansa" : "mansa bar",
    "103 urb el retiro  medical" : "103 urb el retiro medical",
    "lattarico" : "lactic",
    "69 lakshmi garden, thindal m v k illam" : "69 lakshmi garden, illam in thindal",
    "são gonçalo" : "sao goncalo",
    "darwha" : "tarry",
    "zachoodniopomorskie" : "west pomeranian",
    "wielkopaolska" : "large",
    "1037 des marguerites  it" : "1037 of marguerites it",
    "pedregulho" : "boulder",
    "3351 michelson dr ste 100  medical" : "3351 michelson dr ste 100 medical",
    "tân an, càng long, trà vinh" : "tan an, lau long, tra vinh",
    "lares decora" : "lares decorates",
    "angwin" : "anguish",
    "5300 kell blvd" : "5300 need blvd",
    "diadema" : "diadem",
    "pipartola, balaghat, mp india" : "pipatola, balaghat, pump india",
    "22 đường 5 linh xuân thủ đức" : "22 road 5 linh xuan thu duc",
    "godda" : "new",
    "limoeiro" : "lemon tree",
    "memari" : "keep",
    "קרית גת" : "kiryat gat",
    "hyderabd" : "hyderabad",
    "sacile" : "sail",
    "100 crowley dr  medical" : "100 crowley dr  medical",
    "el shrouk 2" : "el shorouk a",
    "ghughli" : "google",
    "timóteo" : "timothy",
    "anand vihar delhi" : "anand storm delhi",
    "thiruvananamatlai" : "thiruvananthapuram",
    "poza rica" : "picture rica",
    "delhij" : "delichium",
    "lima   peru" : "lima's name",
    "müllheim" : "garbage home",
    "asolo" : "asol",
    "pfalz" : "palatinate",
    "台北市" : "taipei city",
    "islamabad" : "islamabad",
    "azamgarg up" : "azamgarg",
    "cédula de ciudadanía" : "citizenship card",
    "ميسان" : "maysan",
    "andersen afb" : "andersen fig",
    "rionegro" : "black river",
    "vpo rani khera" : "hear",
    "τατοΐου 65" : "tattoo 65",
    "castanhal" : "brown",
    "jazan" : "jizan",
    "ha tiên" : "ha tien",
    "almanya" : "germany",
    "1900 lukken industrial drive west" : "1900 luken industrial drive west",
    "1919 minnesota ct, mississauga, on l5n" : "1919 minnesota ct, mississauga, is l5n",
    "bom jesus" : "good jesus",
    "aadgaon" : "aadga",
    "mour mandi" : "dying",
    "oshogbo" : "embon",
    "mahroni" : "disadvantaged",
    "paschim medinipur (west medinipur)" : "west midnapore (west medinipur)",
    "الجلة الرياض" : "al jala riyadh",
    "สมุทรปราการ" : "samut prakan",
    "abohar" : "warehouse",
    "los gatos" : "the cats",
    "banka" : "knock",
    "dohrighat" : "pink",
    "gautam budha nagar" : "gautam mercury nagar",
    "maceió alagoas" : "maceio alagoas",
    "koszecin" : "cosecin",
    "portão" : "gate",
    "चाकण" : "chickpe",
    "santo domingo este rep. dom." : "saint sunday this rep.sun.",
    "cerejeiras" : "cherry trees",
    "cruzeiro do sul acre" : "southern cruzeiro acre",
    "zachodniopomorskie" : "west pomeranian",
    "fatehnagar" : "fatehanagar",
    "kharabi nagpur" : "morning nagpur",
    "saharsa" : "sarhasa",
    "tp.hồ chí minh" : "ho chi minh city",
    "ciudad de méxico  it" : "mexico city it",
    "lapu lapu city" : "lapa lap city",
    "הר אדר" : "mount adar",
    "faropilhas" : "barefoot",
    "navapur" : "nawapur",
    "sarita chorakiasundeep jain" : "saree",
    "carpina" : "carpine",
    "rio real" : "river royal",
    "pudukkottai" : "spright",
    "cajati" : "gentle",
    "este" : "it is",
    "purna" : "full",
    "mattapoisett" : "matte",
    "cañas" : "white hair",
    "punr" : "rear",
    "harda" : "as well as",
    "3 vpo sunet , brs nagar" : "3 vapo sunat, bars nagar",
    "joão pesssoa" : "joão pessoa",
    "esch sur alzette" : "esch surge of alzette",
    "pooler" : "pools",
    "araras" : "macaws",
    "querètaro" : "wanttaro",
    "daund" : "race",
    "naucalpan , mexico" : "naucalpan, mexico",
    "opolskie" : "opole",
    "ulu tiram, johor bahru" : "ulu oyster, johor bahru",
    "sénégal" : "senegal",
    "dhuliyan" : "dhulian",
    "saudade do iguaçu" : "i miss iguaçu",
    "são luís maranhao" : "sao luis maranhão",
    "marki, mazowieckie" : "brands, masovian voivodeship",
    "minas gerais   belo horizonte" : "minas gerais belo horizonte",
    "25 hoàng sa phường dakao quận 1" : "25 hoang sa ward dakao district 1",
    "barmer rajasthan" : "badmer rajasthan",
    "15an mỹ" : "15an american",
    "newdelhi" : "new delhi",
    "vandavasi" : "trouble",
    "naduvattam" : "inflate",
    "อ เมือง จ ภูเก็ต" : "phuket",
    "pitangui mg" : "pihong ghost gate",
    "mapusa" : "hatch",
    "indiabulls  greens, kon, tal. panvel, pin code 410221" : "indiabulus greens, angle, tal.panvel, pin code 1",
    "churu" : "a thousand",
    "sindgi distik vijaypur" : "life dastic vijaypur",
    "menofia" : "monsterness",
    "ciudad de mexic" : "mexican city",
    "au ricchieri sn" : "a ricciid sn",
    "sriganganagar" : "shri ganga nagar",
    "ไทย" : "thai",
    "ramanagara" : "ragara",
    "hod hasharon" : "competition beef",
    "wa" : "of",
    "at post   satana, tal.satana, district. nashik  hotel" : "at post satana, tal.satana, district.nashik hotel",
    "campo mourão" : "campo mourao"
}

trans_dic['customer_country'] = {
    "denduluru" : "delluder",
    "waukegan" : "waoi cancer",
    "cotonou" : "coton",
    "pundri" : "pundari",
    "bresso" : "brex",
    "armação dos búzios" : "buzios frame",
    "ramgarh cantt" : "ramgarh santat",
    "acharnai" : "relevant",
    "criciuma" : "cricium",
    "junagadh gujarat" : "junagarh gujarat",
    "ấp hòa hữu  xã long đức tptv" : "hoa huu hamlet, long duc commune, tptv",
    "valle de bravo" : "bravo valley",
    "กรุงเทพฯ" : "bangkok",
    "rancho queimado" : "burnt ranch",
    "abha" : "river",
    "villahermosa, tabasco" : "villahermosa tabasco",
    "presidente getulio sc" : "president getulio sc",
    "1 kv2 p.trà nóc" : "1 kv2 p.trà noc",
    "paço do lumiar" : "lumiar paço",
    "aracaju   se" : "aracaju if",
    "morando em são paulo" : "living in são paulo",
    "ayankig" : "youyankig",
    "hatimiyyah street   muwaqqar p. o. box 141351" : "hatimiyyh street muwaqar p.o.box 141351",
    "merut" : "meerut",
    "góra kalwaria" : "mount kalwaria",
    "long hựu tây cần đước long an" : "long huu tay can long an",
    "tuchorza" : "tuchorz",
    "mumvai" : "mumwai",
    "nalagonda" : "disagree",
    "53 e 1st ave" : "೫೩ this ೧ st",
    "maringa pr" : "on mringa",
    "fortaleza ce" : "fortaleza what",
    "2 sengol nagar 3 rd street old vilangudi" : "௨ scepter city ௩ street old",
    "jonadi" : "jondi",
    "châu đôc" : "chau dac",
    "القطيف" : "qatif",
    "quảng ngãi" : "quang ngai",
    "campos rj" : "rj fields",
    "antique philippines" : "ancient philippines",
    "ekangersarai" : "ecadesrai",
    "tp hồ chí minh" : "ho chi minh city",
    "ribeirão das neves" : "stream of snow",
    "raposos" : "fox",
    "ma   timon" : "my drawbar",
    "ottawa ontario" : "i followed ontario",
    "cd de mexico" : "mexico cd",
    "coronel fabriciano mg" : "colonel fabriciano mg",
    "berhampore" : "in berhampur",
    "cossato" : "sock",
    "gaziabadh" : "ghaziabad",
    "dias d'avila ba" : "days d'avila ba",
    "durban , kwa zulu natal" : "durban, to zulu natal",
    "elkhart" : "garden",
    "maumbai" : "mumbai",
    "morbi" : "soccer",
    "via di s. erasmo 12" : "via di s.erasmus 12",
    "novia" : "bride",
    "6252 e grant rd suite 150 tucson, az 85712" : "6252 is a grant rd swing 150 tutzon, i 85712",
    "hồ chí minh" : "ho chi minh",
    "hata" : "even",
    "1602 e yandell dr" : "1602 e yandell dr dr.",
    "arroyo hondo viejo" : "hondo viejo arroyo",
    "juliaca   san roman   puno" : "juliaca san roman puno",
    "pondichéry" : "pondicherry",
    "itapema sc" : "it will spit sc",
    "phường tân phong biên hoà đồng nai" : "tan phong bien hoa ward dong nai",
    "885 haleakala hwy" : "889 videos",
    "kishtwar" : "tendering",
    "las parejas" : "the couples",
    "platón sánchez" : "plato sánchez",
    "palhoça" : "hazel",
    "la misma" : "the same",
    "seria" : "he would be",
    "સુરત" : "surat",
    "nuxis (su)   via dante n. 34" : "nuxis (your) dante n.3. 4",
    "lampa" : "lamp",
    "baquba" : "the bacquuba",
    "bacolod city nergros occidental" : "bacolod city negros occidental",
    "السويس" : "suez",
    "tambopata" : "tamopata",
    "290 thành công, q. tân phú" : "290 success, q.tan phu",
    "vijayapura" : "vijaypura",
    "cordba" : "cordoba",
    "santo andre   sao paulo" : "santo andre sao paulo",
    "buenos aires   azul" : "buenos aires blue",
    "1 penn plz ste 1400 fl 14" : "1 pen plz rose 1400 fl 14",
    "al riyad" : "riyadh",
    "ecatepec , edo mex" : "ecatepec, edo mex",
    "hapur" : "open",
    "guarne" : "cheek",
    "mntpellier" : "montpellier",
    "una" : "a",
    "partappur" : "pratappur",
    "colegiales" : "schoolboys",
    "koribori 8g" : "corbari 1 c",
    "brasilia   águas claras" : "brasilia águas claras",
    "arain rajasthan" : "rajasthan",
    "1850 m. h. del pilar st,  malate" : "1850 m.h.of the pillar st, malate",
    "tapira mg" : "tap mg",
    "santa ines" : "holy ines",
    "palmas   pr" : "palmas pr",
    "hindamata" : "unstated",
    "gyál" : "gate",
    "số 7 đường 3419 phường 7 quận 8" : "no. 7 street 3419 ward 7 district 8",
    "jacareí" : "jacarei",
    "mazowsze" : "mazovia",
    "1605 ave. ponce de león, suite 400 san juan, 00909, puerto rico" : "1605 ave.ponce de león, suite 400 san juan, 00909, puerto rico",
    "purnia" : "purania",
    "pato branco pr" : "ties on the bronze",
    "นนทบุรี" : "nonthaburi",
    "capivari de baixo" : "downstream",
    "auragabad" : "aura",
    "حفر الباطن" : "hafar al -batin",
    "15 phan huy thực, tân kiểng, q7" : "15 phan huy thuc, tan kieng, district 7",
    "cali   valle del cuaca" : "cali valle del cauca",
    "koura" : "gold",
    "الشرقية" : "eastern",
    "gubin" : "burn",
    "huyện chợ gạo  tỉnh tiền giang" : "cho gao district, tien giang province",
    "jatara" : "jatra",
    "sheoganj" : "sheyganj",
    "carrera 66b # 6   26 805006050" : "carrera 66b # 6 26 805006050",
    "almirqab" : "alminqab",
    "ktx khu b dhqg tp.hcm" : "dormitory of building b, ho chi minh city",
    "hydrabad" : "hyderabad",
    "rajkot    gujarat" : "rajkot gujarat",
    "são paulo  sp" : "sao paulo-sp",
    "phước long b" : "phuoc long b",
    "chile santiago" : "santiago chile",
    "quận 2, tp. hcm" : "district 2, tp.hcm",
    "dhrol" : "drol",
    "perintalmanna" : "pond",
    "sumatera selatan" : "south sumatra",
    "bulandshahar" : "bulandshahr",
    "goiania   go" : "goiania go",
    "tân an  ,càng long, trà vinh" : "tan an, lau long, tra vinh",
    "aurngabad" : "aurangabad",
    "sete lagoas   mg" : "sete lagoas mg",
    "cra 29 a # 5 sur 125 valle" : "cra 29 a # 5 of 125 valle",
    "wakas nort" : "end nort",
    "浙江省杭州市" : "hangzhou, zhejiang",
    "mumbay" : "mumbai",
    "נתיבות" : "netivot",
    "al ayat   giza" : "the giza",
    "garhshankar" : "gadhashankar",
    "hoà bình trà ôn vĩnh long" : "hoa binh tra wen vinh long",
    "dhamnod dhar m.p" : "dhamnod dhar mp",
    "yirka" : "the umbri",
    "formello" : "fork",
    "uruli kanchan" : "urali kanchan",
    "台南市" : "tainan city",
    "uran" : "career",
    "14058 mirabella dr  education" : "14058 mirabella dr education",
    "muriae" : "muria",
    "mundra kutch" : "mundra inch",
    "talafar" : "fiction",
    "ciudad del este" : "eastern city",
    "qalubia" : "heartfelt",
    "registro sp" : "sp registration",
    "funza" : "learn",
    "venturosa pe" : "venturous pe",
    "bremerhavcen" : "bremerhaven",
    "são paulo   sp" : "sao paulo-sp",
    "dumaguete city, negrosd oriental" : "dumaguete city, negros oriental",
    "18 nguyễn văn lượng, p6 gò vấp" : "18 nguyen van luong, p6 go vap",
    "kajang" : "brim",
    "sheridan" : "lion",
    "senador canedo" : "senator canedo",
    "kabupaten sanggau, kalimantan barat" : "sanggau regency, west kalimantan",
    "estuar" : "estuary",
    "migo das cruzes" : "mogi das cruzes",
    "thiruvanthapuram" : "thiruvantapuram",
    "chikmagalore" : "the chikamagore",
    "ciudad autonoma de buenos aires" : "autonomous city of buenos aires",
    "porto nacional" : "national port",
    "coonoor tamilnadu" : "coonor tamilnadu",
    "via fontana rutola,22" : "via fontana rutola, 22",
    "al ayat" : "a",
    "ibira" : "pull",
    "avare" : "stingy",
    "canoinhas" : "canoiinhas",
    "الديوانية" : "diwaniyah",
    "pavoorchatram" : "pavoric chatram",
    "nova mutum" : "noa man",
    "manab ray" : "manab re",
    "joelformcheck magnit" : "joelformcheck magnet",
    "lucko" : "lutsk",
    "via spiaggia,328" : "via beach, 328",
    "papanduva" : "papoda",
    "hòa thuận châu thành tỉnh trà vinh" : "harmony of tra vinh province",
    "tasocagi cad. no:64" : "tasocagi cad.no: 64",
    "piazzale s. giuseppe, 10" : "square s.giuseppe, 10",
    "osório" : "osorium",
    "مدينة الجلود بدر" : "the city of leather badr",
    "yahoud  monoson" : "monson jews",
    "thạch bàn long biên hà nội" : "thach ban long bien hanoi",
    "namkhana" : "name",
    "espírito santo do pinhal" : "holy spirit of pinhal",
    "pilkhuwa, distt hapur, west up" : "pilkhuwa, distt hapur, west sub",
    "bhayandar" : "bayander",
    "santo antônio do tauá   pará" : "saint anthony of tauá pará",
    "paranaque  city" : "paranaque city",
    "viale  odino  4" : "viale odin 4",
    "kothara abdasa 370645" : "the statement of the word 1",
    "peixe   tocantins" : "tocantins fish",
    "lan o lakes" : "lan the lakis",
    "la paz" : "peace",
    "navegantes   sc" : "sc navigators",
    "sp são paulo" : "sp sao paulo",
    "jakarta utara, kelapa gading" : "north jakarta, kelapa gading",
    "makka" : "mecca",
    "nova scotia" : "new scotia",
    "amargosa valley" : "bitter",
    "gzira" : "young",
    "partapur" : "paratapur",
    "cascina" : "farmhouse",
    "venda nova do imigrante" : "new sale do immigrant",
    "toronto on" : "toronto is",
    "rio de janeiro e região" : "rio de janeiro and region",
    "terra nova do norte mt" : "northern terr mt",
    "miraj" : "mirage",
    "de leon" : "by leon",
    "brześce" : "brzesce",
    "los andes" : "the andes",
    "saúde ba" : "ba health",
    "3125 avenida topanga" : "to topan 3125 avenue",
    "广东省深圳市" : "shenzhen city, guangdong province",
    "dois vizinhos. pr" : "two neighbors.pr",
    "etah" : "fsl",
    "burla parit" : "burla couples",
    "112 sol building amorsolo st. legaspi village, makati city manila, philippines" : "112 sol building amorsolo st.legaspi village, makati city manila, philippines",
    "carrera 7a # 69 99 d.c." : "career 7a # 69 99 a.c.",
    "la pampa   santa rosa" : "la pampa santa rosa",
    "rio grande" : "big river",
    "niwali" : "preached",
    "al ques" : "at any",
    "guarujá são paulo" : "guarujá sao paulo",
    "siek" : "sick",
    "castello d argile" : "castle of swimming pool",
    "odrano wola" : "wola was rebuke",
    "3300 e.guasti rd  medical" : "3300 e.guasti rd medical",
    "viale della navigazione interna, 48" : "viale della internal navigation, 48",
    "nanaimo" : "seven places",
    "karur" : "some",
    "building 18a 3rd nangang industrial park tangtou shiyantown baoan district  commercial display" : "building 18a 3rd nangang industrial park tangtou shiyan town baoan district commercial display",
    "são paulo, pinheiros" : "sao paulo, pinheiros",
    "matinhos" : "pound",
    "594 howard stsan franciscoca94106" : "594 howard st san francisco ca 94106",
    "bishrampur" : "restor",
    "مكة" : "mecca",
    "la spezia" : "spice",
    "bol" : "he was",
    "maarab" : "arab",
    "kuchina" : "to china",
    "גבעת זאב" : "ze'ev hill",
    "londrina pr" : "at laundrina",
    "terdal" : "consistently",
    "alvorada" : "dawn",
    "baripada" : "range",
    "icó" : "icon",
    "mavelikara" : "mavelikkara",
    "santa catarina" : "st. catarina",
    "sombrio" : "dingy",
    "barueri   são paulo" : "barueri sao paulo",
    "bhubanewar" : "bhubnewar",
    "ghazizbad" : "scope",
    "birghman or london" : "birmingham or london",
    "bergkamen" : "mountain ranges",
    "cao lãnh" : "cao lanh",
    "nishi shinjuku, shinjuku, tokyo" : "nishi -shinjuku, shinjuku, and",
    "bhucho mandi" : "bhucho took a shower",
    "rodeville" : "roseville",
    "1000 primera blvd" : "1000 first blvd",
    "cantton, ohio" : "canton, ohio",
    "153 210, illuppapattu village rajakulam post , kanchipuram" : "೨೧೦ ೨೧೦, illuppapattu village rajakulam post, kanchipuram",
    "otorowo" : "you are aological",
    "pr londrina" : "laundrina on",
    "khorda" : "chord",
    "purifiacacion tolima" : "tolima purification",
    "i̇stanbul (anadolu)" : "istanbul (anatolia)",
    "avenida hostos 410 ave hostos medical" : "hostos avenue 410 ave hostos medical",
    "kasur punjab" : "punjab kasur",
    "الغردقة" : "hurghada",
    "baliguda ,district.. kandhamala, orisa" : "baliguda, district .. kandhamal, orissa",
    "krishnagar" : "black",
    "major izidoro   alagoas" : "major izidoro alagoas",
    "dolnosląskie" : "lower silesian",
    "gevrai" : "gavarai",
    "thủ đức" : "thu duc",
    "vellore,tamil nadu" : "vellore, tamil nadu",
    "amour( baisi)" : "love (baisi)",
    "józefów k. otwocka" : "józefów near otwock",
    "ujazd" : "utille",
    "las pinas city" : "the city pinas",
    "es cariacica" : "cariacica",
    "attiki" : "attica",
    "brasília df" : "brasilia df",
    "9 âu cơ tân bình" : "9 au co tan binh",
    "maceió   alagoas" : "maceio alagoas",
    "القصيم بريدة" : "qassim buraidah",
    "حائل" : "barrier",
    "alipurduar" : "alipurdar",
    "tuxtla gutiérrez" : "tuxtla gutierrez",
    "campos dos goytacazes" : "goytacazes fields",
    "singapoer" : "singapore",
    "calle del trigal 255, quilicura" : "255 trigal street, quilicura",
    "5900 balcones ste 5170" : "5900 ste 5170 balconies",
    "2101 nuuanu ave" : "210170 aflost airline around",
    "senador canedo go" : "senator canedo go",
    "betivoglio" : "bentivoglio",
    "lunavada" : "ransom",
    "gangtok" : "gangts",
    "badaun" : "adults",
    "passos mg" : "mg steps",
    "metropolitana buin" : "metropolitan buin",
    "jakarta pusat" : "central jakarta",
    "lima peru" : "lima's name",
    "ahmedbad" : "ahmed",
    "bijainagar" : "bijayanagar",
    "2414 ถนนเพชรบุรึตัดใหม่" : "2414 new phetchaburi road",
    "dghgfdh" : "dghfh",
    "พระนครศรีอยุธยา" : "ayutthaya",
    "los alamitos" : "the alamitos",
    "são bento do sul" : "sao bento do sul",
    "cra 20 f # 61 b   19 sur" : "cro 20 f # 61 b 19 sur",
    "governador celso ramos" : "governor celso ramos",
    "świętochłowice (woj. śląskie)" : "świętochłowice (śląskie voivodeship)",
    "avcılar" : "hunters",
    "minterrey" : "monterrey",
    "macapá   amapá" : "macapá amapá",
    "sc  itapema" : "sc will shoot",
    "ca' zusto 52" : "ca 'zusto 52",
    "triunfo" : "triumph",
    "airoli" : "aeroli",
    "θέση πέτσα βακαλοπούλου βιοπα παλλήνης  153 51" : "peta peta vakalopoulou biopa pallini 153 51",
    "costa del este" : "east coast",
    "cty tnhh kỹ thuật hts" : "hts technical company limited",
    "santo andre" : "santo other",
    "passa e fica" : "passes and stays",
    "bari, italy" : "barry, italian",
    "ponte nova" : "new bridge",
    "cedula de ciudadania" : "citizenship card",
    "la liberfad" : "freedom",
    "kondotty" : "conduit",
    "mato grosso   sorriso" : "mato grosso smile",
    "ras al khimah" : "ras al khaimah",
    "privato" : "private",
    "prata" : "talk",
    "salto guaira" : "jump guaira",
    "ереван" : "yerevan",
    "nuevo león" : "new lion",
    "jammuand kashmir" : "jammu and kashmir",
    "cali   valle" : "valley",
    "passo fundo   rs" : "passo fundo rs",
    "itaira" : "consuming",
    "21 lạc long quânp8 qtân bình" : "21 lac long quanp8 qtan binh",
    "la grange" : "barn",
    "kannur keray" : "kannur",
    "go luziania" : "him",
    "npida" : "distributing",
    "marinha grande" : "large navy",
    "gramado" : "lawn",
    "campinas   sp" : "campinas, sp",
    "udaipur g 2. ankur complex opp narayan seva sansthan sex.5" : "udaipur c 2. ankur complex o narayan seva sansthan sex. 5",
    "telengana" : "telenganana",
    "930 fairway dr ne" : "930 fairway",
    "panama oeste" : "west panama",
    "suchy las" : "dry forest",
    "bécsi út 3 5" : "vienna road 3 5",
    "277 ave coll y toste" : "277 ave coll and toste",
    "caruaru pe" : "caruaru on",
    "osie" : "axles",
    "orai" : "weather",
    "tupanatinga" : "tupaniatation",
    "kudan kulam" : "kudon pond",
    "cambé   pr" : "camps pr",
    "assandh" : "blessed",
    "cra. 51 #12 sur 75, san fernando, itagüi, medellín, guayabal, medellín, antioquia, colombia" : "cra.51 #12 sur 75, san fernando, itagüi, medellin, guayabal, medellin, antioquia, colombia, colombia, colombia, colombia",
    "gustavo a. madero" : "gustavo a.log",
    "mas grenier" : "more grenier",
    "القنفذة" : "cumulus",
    "chicoloapan de juárez" : "chicololoapan by juarez",
    "melmaruvathur" : "malamaruvathur",
    "khaldeh" : "khalida",
    "850 lincoln ave, suite 11 bohemia ny 11716" : "850 lincoln ave, suite 11 bohemia no. 11716",
    "ain khaled" : "where is khalid",
    "p10 q6 tphcm" : "p10 q6 hcmc",
    "balneário camburiu" : "balneário camboriú",
    "sremska 4a" : "srems' cha",
    "san ramon" : "3 ramon",
    "gusyaquil" : "guayaquil",
    "newton" : "newtonto",
    "ahemdnagar" : "ahmednagar",
    "ajah   lagos" : "ajah lagos",
    "ara" : "we buy",
    "portblair" : "port blair",
    "brusque  sc" : "brusque sc",
    "dedham" : "dadham",
    "leandro n alen 734" : "leandro n alem 734",
    "delhi cant" : "delhi cantt",
    "al nadha" : "the club",
    "rajapalayam" : "rajpalayam",
    "santa ctuz de la sierra" : "santa cruz de la sierra",
    "schaafheim" : "collapheim",
    "são josé dos campos  sp" : "são josé dos campos sp",
    "naxalbari" : "nahalari",
    "charummoodu" : "chaummood",
    "via cav. vitt. veneto" : "via cav.vit.veneto",
    "28 michael adekoya street ilupeju" : "28 michael adekaya street start",
    "gladwyne" : "smooth wines",
    "glenndale" : "glendale",
    "rua rui barbosa n 1095 bairro goiabeiras mato grosso" : "rui barbosa street n 1095 bairro goiabeiras mato grosso",
    "kozhikode" : "leathercode",
    "vadoara" : "vadodara",
    "mimbai" : "branded",
    "dum dum" : "stupid",
    "2520 marsha sharp fwy  apt481" : "2520 marsha sharp more apt481",
    "sorriso   mato grosso" : "smile mato grosso",
    "p1 , tp.cao lãnh , đồng tháp" : "p1, cao lanh city, dong thap",
    "rampurhat   731224" : "rampurhat 1222",
    "กรุงเทพมหานคร" : "bangkok",
    "calle 51#8g 55" : "51#8g 55 street",
    "theni" : "why",
    "nova scctia" : "nova scotia",
    "turmalina" : "tourmaline",
    "nikol,ahmedabad  gujarat" : "nicole, ahmed gogarat",
    "nagercoil" : "nagercoil temple",
    "2124 s michigan" : "2124 s road cancer",
    "sorriso  mt" : "smile mt",
    "la unión" : "the union",
    "klein windhoek klein windhoek" : "small wind corner small wind corner",
    "mandi adampur" : "bathing adix",
    "montréal, qc" : "montreal, qc",
    "via rastrello 22" : "via rastal 22",
    "são joão de meriti" : "st. johns wood",
    "tilhar" : "strain",
    "davao coty" : "gave coty",
    "po tankua ,dist koraput" : "by tankua, dist",
    "688 hưương lộ 2" : "688 huang lo 2",
    "joao pessoa" : "joao person",
    "giza, 6th october" : "giza, october 5th",
    "سلطنة عمان" : "sultanate of oman",
    "simão dias  se" : "simão dias se",
    "bebedouro" : "drinking fountain",
    "las palmas, grand canaria" : "las palmas, gran canaria",
    "al khobar   saudi arabia" : "a saudi arab news",
    "19  giang võ văn kiệt" : "19 giang vo van kiet",
    "assuncion" : "asuncion",
    "ciudad de buenos aires" : "buenos aires city",
    "ave. ricardo alfaro, ph plaza aventura piso 4 ofic 423 panam?" : "bird.ricardo alfaro, ph plaza aventura 4th office 423 panam?",
    "hải dương" : "hai duong",
    "renukoot" : "renucotus",
    "vridhachalam" : "old age",
    "korba" : "to age",
    "león guanajuato" : "guanajuato león",
    "west bandung" : "west ribbon",
    "santa maria   rs" : "santa maria rs",
    "hajipur ,vaishali, bihar" : "hajipur, vaishali, bihar",
    "taigore nagar near veer loric stadium" : "tagore nagar near veer loric stadium",
    "nrw, deutschland" : "nrw, germany",
    "cachoeira paulista" : "paulista waterfall",
    "1000 navaho drive  hotel" : "1000 navaho drive hotel",
    "kout" : "short",
    "490 l'enfant plz, sw" : "490 the child plz, sw",
    "são luís   ma" : "são luís ma",
    "puttur" : "putter",
    "łódzkie" : "łódź",
    "ile ife" : "the love of love",
    "são josé do cedro  sc" : "são josé do cedro sc",
    "bengalooru" : "bangalore",
    "jakarta selatan" : "south jakarta",
    "ابها" : "abha",
    "dheradun" : "dheeradun",
    "channai" : "milk production",
    "rio claro" : "clear river",
    "via g. di vittorio 5 a" : "via g.of vittorio 5 a",
    "1051 calle 3 se ste lc1a 3003 pr, san juan, 00921, puerto rico" : "1051 street 3 sete lc1a 3003 pr, san juan, 00921, puerto rico",
    "bauru   sp" : "bauru sp",
    "celle ligure" : "that ligure",
    "uttarpara kotrung" : "northern",
    "independencia esq heroes de luperon" : "independence esq heroes de luperon",
    "cư xá c . đại học nông lâm tp hcm" : "cu xa c.ho chi minh city university of agriculture and forestry",
    "espeŕanza" : "hope",
    "nh 27 deesa highway badarpura" : "no 27 disa highway badarpur",
    "qracaju" : "grocaju",
    "gen. trias cavite" : "gen.trias cavite",
    "taubaté   são paulo" : "taubaté são paulo",
    "bertioga   sp" : "bertioga sp",
    "hafar al batin" : "drill the clay",
    "barasat" : "rain",
    "nova serrana" : "new serrana",
    "niterói   rj" : "nitero rj",
    "masjid bander" : "bander mosque",
    "pichari" : "backward",
    "chhatapur" : "umbrella",
    "naina tikker" : "naina is ticking",
    "bangaloru" : "bangalore",
    "bezerros" : "calves",
    "juárez chihuahua méxico" : "juárez chihuahua mexico",
    "belo horizinte" : "belo horizonte",
    "dharuhera" : "darhera",
    "1001 camino ct" : "1001 ct path",
    "bucuresti" : "bucharest",
    "софия" : "sofia",
    "nowa wola" : "new wola",
    "rio grande do sul   erechim" : "rio grande do sul erechim",
    "west des moines" : "west monks",
    "terra boa" : "land",
    "viramgam" : "veeranga",
    "presidente juscelino" : "president juscelino",
    "ấp mỹ tân  thị trấn mỹ luông" : "my tan hamlet, my luong town",
    "phagwara" : "fagwara",
    "maloposkie" : "lizposki",
    "niterói   rio de janeiro" : "niterói rio de janeiro",
    "تيماء" : "tayma",
    "san andres islas" : "san andres islands",
    "bodh gaya" : "gone",
    "centro de produção audiovisual   sesc são paulo" : "sesc são paulo audiovisual production center",
    "perumanallur" : "advertise",
    "capacio paestum" : "paestum",
    "شارجه" : "sharjah",
    "karaikudi" : "go from",
    "marau" : "curricular",
    "são josé dos campos   sp" : "são josé dos campos sp",
    "shkoat" : "skoat",
    "debashis chakraborty" : "debashish chakraborty",
    "via mar della cina 254" : "via mar della china 254",
    "stary wiśnicz" : "old wiśnicz",
    "librebille" : "libreville",
    "1855 folsom st ste 601  medical" : "1855 folsom st you 601 medical",
    "paramaribo suriname" : "paramaribo, suriname",
    "mahrajganj" : "maharajganj",
    "alecrim  rs" : "rosemary rs",
    "medan" : "while",
    "artena" : "clay",
    "ertert" : "is",
    "pederneiras" : "bane",
    "würselen" : "sewer",
    "5808 socorro dr" : "5808 dr socorro",
    "ambato" : "hint",
    "tekulasomaram" : "techulsamaram",
    "doctor andrade  63 1er. piso  colonia doctores ciudad de m?xico" : "doctor andrade 63 1st.colonia doctors city of m? xico",
    "centro comercial asturias de ovideo  cra 10 # 1   50   local 10 cundinamarca" : "asturias shopping center of oviedo cra 10 # 1 50 local 10 cundinamarca",
    "atalaya" : "watchtower",
    "jericho" : "jeri",
    "kothrud,pune" : "kothrud, pune",
    "dharmaragar" : "dharmagar",
    "jangaon" : "the jangaoon",
    "belo horizonte   minas gerais" : "belo horizonte minas gerais",
    "3815 faraon st ste e" : "3815 pharaon st you e",
    "baruero" : "barueri",
    "medellín colombia" : "medellin colombia",
    "مسقط" : "muscat",
    "جازان" : "jizan",
    "ciudad satelite" : "ciudad satellite",
    "irara" : "breathy",
    "waukesha" : "you up",
    "pulivendla" : "pulivendula",
    "miagao iloilo" : "misama expections",
    "trà ôn vĩnh long" : "tra wen vinh long",
    "senhora" : "madam",
    "satuba" : "one",
    "ramanagar" : "ramnagar",
    "amk" : "bachelor",
    "fortaleza, ce" : "fortaleza, what",
    "3030 maingate ln" : "3030 mangate lan",
    "barreiros" : "barriers",
    "salmiya" : "peaceful",
    "04, prasadi gali , kotla mubarakpur delhi  delhi" : "04, prasadi gali, mubarakpur delhi delhi boiler",
    "la paz bolivia" : "la paz, bolivia",
    "margao,goa" : "margao, goa",
    "arame   ma" : "aram",
    "latur" : "lazy",
    "alta floresta" : "rainforest",
    "تموز (يوليو)" : "july (july)",
    "paraíso do norte pr" : "north paradise pr",
    "beberibe" : "beeribe",
    "banaso, hazaribagh" : "country, hazaribagh",
    "kochi, kozhikode" : "this, this ← iko",
    "rio branco" : "white river",
    "احد رفيده   الواديين" : "one of the valleys of the valley",
    "عجمان" : "ajman",
    "25c nguyễn bỉnh khiêm, q1, tp.hcm" : "25c nguyen binh khiem, district 1, ho chi minh city",
    "goiás   anápolis" : "goiás anápolis",
    "hessen   meinhard" : "hessen meinhard",
    "negros oriental" : "eastern blacks",
    "manser" : "male",
    "lima perú" : "lima peru",
    "piripiri" : "adhesive",
    "passo fundo" : "deep step",
    "fanna" : "us to",
    "bhongir" : "bhogir",
    "santiago de cali" : "santiago from cali",
    "armação de buzios" : "buzios frame",
    "mombasa,kenya" : "mombasa, kenya",
    "capão da canoa" : "canoa capão",
    "jais" : "them",
    "baki" : "mouth",
    "cambrideshire" : "cambridgeshire",
    "sitamadhi" : "sitamarhi",
    "paraná   francisco beltrão" : "paraná francisco beltrão",
    "budaun" : "budhon",
    "moskova" : "moscow",
    "shendurni" : "stubborn",
    "guwhati" : "gauwh",
    "uberlândia   mg" : "uberlândia mg",
    "małopolska" : "lesser poland",
    "toledo pr" : "on toledo",
    "ixtalhuaca de rayon" : "istalhuacon",
    "amapá   macapá" : "amapá macapá",
    "nova veneza" : "new venice",
    "nandyala" : "nandala",
    "getúlio vargas" : "getulio vargas",
    "barra bonita" : "pretty bar",
    "khober" : "hover",
    "cantagalo" : "sink",
    "bahia   salvador" : "bahia salvador",
    "mérida yucatán" : "merida yucatan",
    "mondai" : "problem",
    "mahesana" : "mehsana",
    "i̇zmir" : "mirzmir",
    "ananthapur" : "anantapur",
    "kepulauan riau" : "riau islands",
    "سوهاج" : "sohag",
    "francisco i. madero mz. 28 lt. 37 m?xico, d.f." : "francisco i.madero mz.28 lt.37 m? xico, d.f.",
    "sahaswan=243638" : "sahawan = 243638",
    "chaguanas" : "selectnas",
    "seleccionar comunapunitaqui" : "select comunapunitaqui",
    "saopaulo" : "baby be drowered",
    "da nang city, viet nam  vi?t nam" : "da nang city, viet nam vi? t nam",
    "foz de iguaçu   pr avenida tancredo neves 6731 jardim itaipu" : "foz de iguaçu pr avenida tancredo neves 6731 jardim itaipu",
    "שריגים (לי און)" : "shss (li on)",
    "i̇stanbul hadımköy" : "istanbul hadımköy",
    "alkhobar" : "the news",
    "reyadh" : "riyadh",
    "neyveli" : "unsheated",
    "medicina" : "medicine",
    "singen" : "to sing",
    "hei longjiang" : "heilongjiang",
    "poznam" : "i know",
    "los ángeles" : "the angels",
    "ahmednagar" : "ahmednaagar",
    "neyba provincia bahoruco" : "neyba bahoruco province",
    "bhawani mandi" : "bhavani people",
    "dĩ an, bình dương" : "di an, binh duong",
    "أنقرة" : "ankara",
    "cancún" : "cancun",
    "metropolitana de santiago" : "metropolitan of santiago",
    "al najaf" : "al najaf",
    "és" : "and",
    "punei" : "place",
    "pará" : "for",
    "campos dos goytqcazes" : "dos goytacazes fields",
    "cobquecura" : "chasing",
    "rj   rio de janeiro" : "rj rio de janeiro",
    "recife  pe" : "recife on",
    "ciudad autonoma de bs. as" : "autonomous city of bs.ace",
    "litleton" : "littleton",
    "sharq" : "east",
    "guidoval" : "he gurid",
    "marechal deodoro" : "marshal deodoro",
    "las piñas   muntinlupa" : "las piñas muntinlupa",
    "paudalho pe" : "padalho pe",
    "chatra" : "student",
    "alriyadh" : "riyadh",
    "76 78 bạch đằng, p. hải châu i" : "76 78 bach dang, p.hai chau i",
    "rs   caxias do sul" : "rs caxias do sul",
    "สงขลา" : "songkhla",
    "capelinha" : "chapel",
    "satata" : "stata",
    "novi beograd" : "new belgrade",
    "kerala" : "cherry",
    "playas de rosarito  baja california" : "rosarito baja california beaches",
    "são paio de oleiros" : "st.",
    "اربد" : "irbid",
    "mg   araxá" : "mg araxá",
    "sete lagoas" : "seven lagoons",
    "gujrat.godhra" : "gujarat. godhra",
    "greatfalls" : "great falls",
    "joinvielle" : "joinville",
    "4 tp thủ đức tphcm" : "4 thu duc city ho chi minh city",
    "siquirres limón" : "limón siquirres",
    "f6" : "timing",
    "lauscha" : "lollipotia",
    "ambalappuzha" : "ambalappuzh",
    "العين" : "eye",
    "أبها" : "abha",
    "komarapalayam" : "comrapalayam",
    "goiania  go" : "goiania go",
    "ktx khu c   đại học nông lâm tp hcm" : "ktx area c, agriculture and forestry university of ho chi minh city",
    "búzios" : "shells",
    "sadhoganj kharawan baragaon vns." : "sadhganj kharwan bargaon ones.",
    "bạc liêu" : "bac lieu",
    "betim   mg" : "betim mg",
    "engativa" : "deceit",
    "turvo" : "cloudy",
    "elkmont" : "quantity",
    "la vega" : "the vega",
    "dique de caxias" : "dike",
    "teresina pi" : "teeasia pi",
    "via oratorio,5" : "via oratory, 5",
    "três barras santa catarina" : "three santa catarina bars",
    "ulu tiram" : "the coast",
    "wiskitki" : "tarpaulin",
    "số 7 đường 3419 phường 7 quận 8." : "no. 7, street 3419 ward 7 district 8.",
    "la palma" : "the palm",
    "5n  ấp đông lân xã bà điểm" : "5n dong lan hamlet, ba diem commune",
    "hydarabaad" : "hysradons",
    "5b tân chánh hiệp 35" : "5b tan chanh hiep 35",
    "m'sila" : "me",
    "28 to ngọc van" : "28 to ngoc van",
    "الدمام" : "dammam",
    "campina verde" : "green campina",
    "rohini" : "expense",
    "são  paulo" : "são paulo",
    "safwa" : "greed",
    "aracaju   sergipe" : "aracaju - sergipe",
    "hauz khas,delhi" : "hauz special, delhi",
    "contagem minas gerais" : "count minas gerais",
    "hadapsar" : "front",
    "pančevo" : "pancevo",
    "davao del sur, davao city" : "gave del sur, gave city",
    "barão de cotegipe" : "baron de cotegipe",
    "5 lorong kemaris 1 bukit bandaraya" : "5 lorong lorong 1 bukit city",
    "salvador  bahia" : "salvador, bahia",
    "klinikum nürnberg (standort nord)" : "clinic nuremberg (north location)",
    "lalmatia" : "lacquer",
    "sumatra barat" : "west sumatra",
    "วังน้อย" : "wang noi",
    "bbsr" : "bubbling",
    "dandeli" : "capable",
    "tarn taran" : "taran taran",
    "13 ryad street shoubra, cairo" : "૧૩ rayal street shoubra, cairo",
    "san isidro davao oriental" : "like an equid davo oriental",
    "dubai invesyments park 1" : "dubai investments park 1",
    "ب" : "b",
    "airwa katra" : "airwa each",
    "balneário  camboriú" : "balneário camboriú",
    "mersin" : "myrtle",
    "araria" : "arriya",
    "la lucila, vicente lopez" : "lucila, vicente lopez",
    "السالمية" : "salmiya",
    "пловдив" : "plovdiv",
    "guanambi" : "guanhabi",
    "puttaparthi" : "puttparthi",
    "1 va ctr," : "1 ctr,",
    "ha noi" : "ha we",
    "antiguo cuscatlan" : "ancient cuscatlan",
    "tabira" : "toy",
    "mousl" : "mosul",
    "via pizzinardi  64" : "via pizzinardi 64",
    "ระยอง" : "rayong",
    "s73w16773 briagate ln  medical monitors" : "s73w16773 briargate ln medical monitors",
    "vinh   nghệ an" : "vinh nghe an",
    "vitória da conquista   ba" : "vitória da conquista ba",
    "joão pessoa:pb" : "joão pessoa: pb",
    "thalavadi" : "thalwadi",
    "160 calle primera" : "160 first street",
    "الجيزه" : "giza",
    "nagpur ,kanhan" : "nagpur, kanhan",
    "103 d tate circle  education" : "103 d tate circle  education",
    "boaco" : "reindeer",
    "calle gala 8" : "street gala 8",
    "test34" : "test 4",
    "mundo novo  ms" : "new world ms",
    "عسير" : "difficult",
    "av 531 no 6 ciudad de mexico" : "av 531 no 6 mexico city",
    "al seeb" : "the sep",
    "52  ต.ป่าตอง อ.กะทู้ จ.ภูเก็ต" : "52 patong subdistrict, kathu district, phuket province",
    "9540 hamburg rd" : "9540 hamburg rd",
    "biñan city" : "biñan cyty",
    "lagos de moreno" : "moreno lakes",
    "hydeeabad" : "hyderabad",
    "san jose del cabo" : "san jose of cabo",
    "minneola" : "memory",
    "ammanabrolu" : "amamanballu",
    "el cove" : "the cove",
    "huixquiliucan" : "huixquilucan",
    "hubali" : "hubli",
    "viçosa" : "lurid",
    "pará canaã dos carajás" : "para canaã dos carajás",
    "موصل" : "insulated",
    "raibareli" : "rai barre",
    "gandu" : "tribute",
    "río tercero   córdoba" : "río tercero córdoba",
    "st julians" : "st. julians",
    "nova lima" : "new lime",
    "la quiaca" : "the quiaca",
    "shikaripura" : "plastic",
    "pelotas" : "pellets",
    "vespasiano   minas gerais" : "vespasian minas gerais",
    "zielona góra" : "green mountain",
    "curitiba e são paulo" : "curitiba and são paulo",
    "podlaskie" : "podlasie",
    "são thome das letras" : "saint thome of letters",
    "bhabra" : "emotional",
    "176 thủ khoa huân phan thiết bt" : "176 valedictorian huan phan thiet bt",
    "12 phạm van bach tân bình" : "12 pham van bach tan binh",
    "434 bình quới, phường 28" : "434 binh quoi, ward 28",
    "bhachau kutch" : "bachau kutch",
    "karwar" : "increase",
    "ภูเก็ต" : "phuket",
    "số7,đường 3419,phạm thế hiển ,q8," : "7, 3419, pham the hien, q8,",
    "maceió  al" : "macea al",
    "λαρισα" : "larissa",
    "guayaquil   ecuador" : "guayaquil, ecuador",
    "ashtapur" : "ashtpur",
    "shakopee" : "shakopi",
    "são joão da boa vista" : "sao joao da boa vista",
    "nabatieh" : "nabaiah",
    "2c trần hưng đạo, p1, q5" : "2c tran hung dao, p1, q5",
    "أبوظبي" : "abu dhabi",
    "cân rhow" : "weigh",
    "hai dương" : "two -positive",
    "santa izabel do oeste" : "santa izabel do west",
    "boituva" : "boituvian",
    "stary żabieniec" : "old żabieniec",
    "hampden" : "hem",
    "新竹縣" : "hsinchu county",
    "strada statale 182 c.da nasari snc" : "state road 182 c.da nasari snc",
    "cachoeiro de itapemirim   es" : "cachoeiro de itapemirim es",
    "7985 kentshire dr  medical" : "7985 i'm",
    "quận 10" : "district 10",
    "1380 fondo de la legua piso 1 â°  commercial display" : "1380 legua fund floor 1 ° commercial display",
    "las condes, santiagom, chile" : "las condes, santiago, chile",
    "bento gonçalves. rs" : "bento gonçalves. lol",
    "hà tiên" : "ha tien",
    "gualeguaychu" : "gualeguare",
    "elmansoura" : "mansoura",
    "sitio de abadia go" : "abbot site",
    "marechal cândido rondon   paraná" : "marshal cândido rondon paraná",
    "vaniyambadi" : "vaniya -enambadi",
    "house no 663 chopal wali gali alipur garhi delhi  110036" : "house no 663 chopal gali alipur garhi delhi 110036",
    "thrithallore" : "trianalore",
    "96 moo 1 klongnueng klongluang" : "96 where 1 klongnueng klongluang",
    "10 zonara st" : "10 zonara st.",
    "roddi" : "give",
    "bilasipara" : "feipy",
    "63 trương định" : "63 truong dinh",
    "a17 12 chung cư samland" : "a17 12 samland apartment",
    "tio hugo rio grande do sul" : "uncle hugo rio grande do sul",
    "plot #35,phase iv,malur  malur taluk  kolar district" : "plot #35, phase iv, malur malur taluk kolar district",
    "durgapur bhiringi" : "durgapur bhirangi",
    "la reina" : "the queen",
    "canningvale" : "canning",
    "governador valadares" : "valadares governor",
    "303 gò dầu phường tân quý" : "303 go dau ward tan quy",
    "seri kembangan" : "kembangan series",
    "sa’ad, riyadh dammam highway,  ksa" : "sa'ad, riyadh dammam highway, ksa",
    "belém e região" : "belém and region",
    "2777 n stemmons fwy  healthcare" : "2777 n stemmons more healthcare",
    "house no. 375, road no. 28, new dohs, mohakhali, dhaka cantonment, dhaka 1206, bangladesh bangladesh" : "house no.1, road no.28, new das, mohakhali, dhaka cantonment, dhaka 1 to 3, bangladesh bangladesh",
    "sp brasil" : "sp brazil",
    "agudos" : "acute",
    "pathanamthitta" : "pathanamtta",
    "via gramsci angolo via de gasperi" : "via gramsci corner via de gasperi",
    "tiruvallur" : "thiruvallur",
    "vadodra" : "vadodara",
    "29222 rancho viejo rd ste 108  medical" : "29222 rancho viejo rd ste 108 medical",
    "mahua" : "mahu",
    "89 hoàng quốc việt,q7" : "89 hoang quoc viet, district 7",
    "الرس" : "alrass",
    "loja" : "store",
    "begunia" : "beguni",
    "nasik (nashik)" : "nasik (we doubt)",
    "sta. cruz" : "stay.cruz",
    "surbiton" : "surbs",
    "lagoa da prata   mg" : "mg silver lagoa",
    "heroica puebla de zaragoza" : "heroic puebla of zaragoza",
    "baga calangute" : "lung calangute",
    "315 alameda blvd ne" : "315 alamed blvd not",
    "palitana" : "glory",
    "anoop shahr" : "anoop city",
    "mianwali colony gurugram" : "miavali colony gurugram",
    "las piñas city" : "the city pineapples",
    "resistencia" : "endurance",
    "capao da canoa" : "canoe",
    "barakar" : "vanity",
    "tukwila" : "us",
    "sanriago" : "after sanria",
    "trà cú trà vinh" : "tra cu tra vinh",
    "tubarao" : "scrapping",
    "dharamapuri" : "dharampuri",
    "barueri" : "barley",
    "sląskie" : "silesian",
    "duque de caxias" : "duke of caxias",
    "chatrapati sambhajnagar" : "chhatrapati",
    "rio grande do sul  it" : "rio grande do sul it",
    "cornélio procópio" : "cornelius procopio",
    "thanh xuân" : "youth",
    "ateli mandi" : "mandy atels",
    "bucaramanda" : "bukramanda",
    "mozzate" : "severed",
    "skawina" : "skaw",
    "wielkopolski" : "wielkopolska",
    "campo bom" : "good camp",
    "naucalpan , estado de mexico" : "naucalpan, state of mexico",
    "nova venécia" : "new venecia",
    "macherla" : "machers",
    "225 baldwin aveneu  healthcare" : "225 baldwin avenue healthcare",
    "oklahoma city" : "ololama city",
    "1759 wehrle dr  it" : "1759 wehrle dr it",
    "kihei" : "bay",
    "saocha" : "laboru",
    "sao paulo   sp" : "sao paulo-sp",
    "cabaceiras do paraguaçu" : "paraguaçu cabaceiras",
    "curitiba pr" : "on karitiba",
    "patos de minas" : "mining ducks",
    "nova iguaçu" : "new iguaçu",
    "brasil" : "brazil",
    "tp.hcn" : "tp.hcm",
    "benagluru" : "bengaluru",
    "ilhabela" : "beautiful island",
    "boa vista   rr" : "boa vista rr",
    "via san martino ,12  fraz. eremo" : "via san martino, 12 fraz.hermitage",
    "hévíz" : "heat water",
    "kupwara" : "to blow",
    "autopista medella. bogotá, colombia" : "medella highway.bogota colombia",
    "duque de caxias   rio de janeiro" : "duque de caxias rio de janeiro",
    "9716 mcfarring drfort worthtx76244" : "9716 mcfarring dr fort worth tx 76244",
    "bielsko biała" : "bielsko-biala",
    "tinsukia" : "chinsquia",
    "wólka kozodawska" : "wólka kozodska",
    "shri madhopur" : "shri madhur",
    "contagem mg" : "count mg",
    "17902 kellas ct" : "17902 clock ct",
    "jatai" : "expressed",
    "menicucci 3" : "menicccci 3",
    "kayseri̇" : "kayseri",
    "pouso alegre" : "happy landing",
    "otro (latinoamérica)" : "another (latin america)",
    "28 ngô quyen" : "28 ngo quyen",
    "sao pauo" : "sao make",
    "5560 katella ave" : "5560 cover ave",
    "30 mân quang 6, sơn trà, đà nẵng" : "30 man quang 6, son tra, da nang",
    "ventanilla   callao" : "callao window",
    "garki" : "forth",
    "al ahsa" : "i count",
    "nandurbar" : "invested",
    "2 2 5 toranomon, minato ku, tokyo" : "2 2 5 toranomon, minato ward, and",
    "25c nguyễn bỉnh khiêm, q1" : "25c nguyen binh khiem, district 1",
    "ينبع" : "stems",
    "nandyal" : "nandala",
    "san martino in rio    re" : "san martino in rio re",
    "deoghar" : "deo -house",
    "cariacica" : "cariacic",
    "pirangi" : "like",
    "nova iguaçu   rio de janeiro" : "nova iguaçu rio de janeiro",
    "avram iancu 20a  it" : "avram iancu 20a it",
    "tulua" : "twinkle",
    "tlalnepantla estado de mexico" : "tlalnepantla state of mexico",
    "hargeisa" : "harge",
    "santo andre são paulo" : "santo andre sao paulo",
    "curitiba   pr" : "on karitiba",
    "são paulo tatui" : "sao paulo tatui",
    "ri9 de.janeiro" : "rurayd d.",
    "impasugong bukidnon" : "impasuto bukidnon",
    "zamboanga city" : "vegeta city",
    "palma decmallorca" : "palma de mallorca",
    "merida yucatán" : "merida yucatan",
    "juárez" : "juare",
    "chandausi" : "check",
    "kennebunkport" : "kenkeybunkport",
    "kareli" : "plaid",
    "tshimaku,tokyo" : "t film, tokyo",
    "ugar" : "fallow",
    "ปัอมปราบฯ" : "pork suppression",
    "kouhoku ku, yokohama" : "kohoku ward, yokohama",
    "פתח תיקווה" : "petah tikva",
    "9647701084310 sulemaniyah" : "9647701084310 sulemanya:",
    "the colonty" : "the colony",
    "dariba" : "remark",
    "świętokrzyskie skarżysko  kamienna" : "świętokrzyskie skarżysko kamienna",
    "caolanh" : "a winter",
    "la mesa" : "table",
    "gran canarias playa del ingles" : "gran canarias playa del english",
    "geebung" : "vomiting",
    "volta redonda   rj" : "volta redonda rj",
    "cam sur" : "cam on",
    "via v. volterra, 9" : "via v.volterra, 9",
    "kraków" : "cracow",
    "17 νοεμβρη 79 πυλαια" : "17 november 79 pylaia",
    "taboão da srra  sp" : "taboão da serra sp",
    "digha" : "dagh",
    "bandung, kota." : "bandung, city.",
    "doveton 3177" : "covton 3177",
    "panabo" : "bailer",
    "avan" : "before",
    "كلباء" : "all",
    "dhi qar" : "ruild",
    "goleta" : "schooner",
    "piedade" : "pity",
    "lachhmangarh sikar" : "lachmangarh sikar",
    "habra" : "there will be",
    "8 ấp 4 xã nhơn đức" : "8 hamlet 4, nhon duc commune",
    "gzb" : "calamity",
    "primavera do leste" : "spring of the east",
    "una, himachal" : "one, himachal",
    "b5 lưu văn liệt, phường 2, tpvl" : "b5 luu van liet, ward 2, tpvl",
    "gola gokaran nath" : "circle gokharan nath",
    "安徽省芜湖市" : "wuhu city, anhui province",
    "caucaia ce" : "caucaia what",
    "barao de cotegipe" : "cotegipe barao",
    "barreiras  ba" : "ba barriers",
    "campos de julio mt" : "julio mt fields",
    "eylül" : "september",
    "margao" : "cursing",
    "itapema   sc" : "it will spit sc",
    "กทม" : "bangkok",
    "norte de santander, cucuta" : "north of santander, cucuta",
    "mooca   sp" : "mooca sp",
    "propriá   sergipe" : "propriá sergipe",
    "alger centre" : "algiers center",
    "constanta" : "constant",
    "prol corregidora norte 1116 of 301 quer?taro" : "north correcting prol 1116 of 301 want? taro",
    "carrera 71 # 50   57 off 201" : "carrera 71 # 50 57 off 201",
    "bagnan" : "garden",
    "pune   kharadi" : "hardy",
    "ชลบุรี" : "chonburi",
    "la florida" : "florida",
    "kasaragod" : "discovery",
    "falakata" : "fatal",
    "formosa do sul   sc" : "formosa do on the sc",
    "otyń" : "obesin",
    "115 trần bá giao f5" : "115 tran ba giao f5",
    "paulista  pernambuco" : "paulista pernambuco",
    "seropédica   rj" : "seropédica rj",
    "chandugarh" : "chandgarh",
    "carrera 11a 94 46 edificio chico 3000 piso 3 bogota" : "carrera 11a 94 46 buy chico 3000 floor 3 bogota",
    "corso canale 32a" : "channel 32a course",
    "ponta grossa   pr" : "ponta groro pr",
    "sâo paulo" : "são paulo",
    "varodrada" : "varodarada",
    "badung, kuta selatan" : "badung, south kuta",
    "via jardins" : "via gardens",
    "chapadinha   maranhão" : "chapadinha maranhão",
    "jakrta" : "jakarta",
    "istanbul   beyoglu" : "istanbul beyoglu",
    "recufe" : "reefs",
    "quatre bornes" : "four terminals",
    "malavalli" : "malwalli",
    "1655 e arlingtonblvd" : "1655 e arlington blvd",
    "buzet sur tarn" : "buzet on tarn",
    "vapi" : "coat of arms",
    "中山區" : "zhongshan area",
    "tiruchengodu" : "we have a cleanliness",
    "małopolks" : "lesser poland",
    "al madinah" : "city",
    "itaberaba   ba" : "plate club",
    "zławieś mała" : "zławieś little",
    "palakonda" : "plakonda",
    "merta city dist nagaur" : "merrata city dist nagaur",
    "venezia mestre" : "venice mestre",
    "botucatu" : "booted",
    "الموصل" : "mosul",
    "tp thủ đức" : "thu duc city",
    "butibori" : "bump",
    "aparecida   sp" : "aparecida sp",
    "16604 arminta st, van nuys, ca 91406" : "16604 arminta sat, van nayas, 91406",
    "fhfu" : "jar",
    "thị xã bình minh" : "binh minh town",
    "buzios" : "bunks",
    "vassouras" : "broom",
    "padra  vadodara" : "bench",
    "baleares" : "whaarse",
    "pernambuco  petrolandia" : "pernambuco petrolandia",
    "van horn" : "horn",
    "8 phan văn hớn" : "8 phan van hon",
    "başakşehi̇r" : "başakşehi",
    "lajeado" : "slab",
    "port bouet rue des caraibes" : "bouet port rue des caraibes",
    "pauri" : "on you",
    "gudalur" : "guddur",
    "cl 20 sur 45 70 ap 904" : "cl 20 of 45 70 ap 904",
    "bình dương" : "binh duong",
    "gangakher" : "gangakhed",
    "barnala" : "childhood",
    "di̇di̇m aydın" : "deep",
    "guadalajara, jalisco" : "guadalajara jalisco",
    "corpus christi" : "body of christ",
    "hunasagi" : "gaid",
    "16 lam sơn, p2, tân bình" : "16 lam son, p2, tan binh",
    "tornesch" : "tornorous",
    "kab. bandung barat" : "regency. west bandung",
    "redenção" : "redemption",
    "50675 yukon rd po box 537" : "50675 oil now rd popular x 537",
    "via provinciale sud, 2" : "provincial south via, 2",
    "espírito santo, es" : "holy espírito, is",
    "obrassano" : "work",
    "carrera 18 n. 48 52" : "race 18 n.48 52",
    "são gabriel" : "saint gabriel",
    "paleo faliro" : "old faliro",
    "sulaimanya" : "solomon",
    "cascavel" : "rattlesnake",
    "śląskie" : "silesian",
    "ciudad madero" : "madero city",
    "tremblay en france" : "tremblay in france",
    "via delle battaglie" : "via delle battles",
    "sorriso" : "smile",
    "45 west flagge street  medical" : "45 west flag street medical",
    "zakopane" : "buried",
    "268 lý thường kiệt quận 10" : "268 ly thuong kiet district 10",
    "مكه" : "mecca",
    "دقهليه" : "dakahleh",
    "samar leyte" : "production of leyte",
    "quatro barras" : "four bars",
    "silvassa" : "silvasa",
    "ulaanbaatar" : "ulbatar",
    "343 s 500 e, apt 118" : "zhz with 500 e, apt 118",
    "muthanna  samawah city" : "muthanna samawah city",
    "puttparthi" : "puttaparthi",
    "praia grande" : "big beach",
    "ciudad de mëxico" : "mexico city",
    "monte verde" : "green mount",
    "kadiri" : "as much as possible",
    "28 tô ngoc vân p thạnh xuan .q12" : "28 to ngoc van p thanh xuan .q12",
    "baturaja" : "king stone",
    "idar" : "id",
    "nouméa" : "noumea",
    "جنوب عبدالله مبارك" : "south abdullah mubarak",
    "sapiranga   rs" : "sapiranga rs",
    "nova ponte" : "new bridge",
    "perote  veracruz" : "perote veracruz",
    "chandigadh" : "chandigarh",
    "baden würtemberg" : "baden württemberg",
    "slask" : "slush",
    "salvador  ba" : "salvador ba",
    "laguna de terminos 221, torre b 604 bis cdmx" : "lagoon of terminnos 221, tower b 604 bis cdmx",
    "台北" : "taipei",
    "marechal candido rondon" : "marshal candido rondon",
    "المحله الكبري  الغربية" : "western major locality",
    "5 av. des hauts fourneaux" : "5 av.high stoves",
    "phường phú thuận, quận 7" : "phu thuan ward, district 7",
    "belo horizonte   mg" : "belo horizonte mg",
    "san mateo rizal" : "san mato rizal",
    "medianeira" : "mediator",
    "alfyom" : "palafume",
    "pest megye" : "pest county",
    "7730 n central dr" : "7730 n central dr.",
    "ktx khu b   đhqg tp.hcm" : "ktx area b vnu hcmc",
    "strovolos" : "stingy",
    "ראש לציון" : "rishon lezion",
    "piauí   teresina" : "piauí teresina",
    "675 đường giải phóng" : "675 liberation road",
    "189 paramus rd. paramus" : "189 paramus rd.paramus",
    "pernambuco   recife" : "pernambuco recife",
    "los olivos" : "the olive trees",
    "ul. łódzka 308" : "ul.łódź 308",
    "nagarnausa" : "carrity",
    "pekanbaru" : "pecanbal",
    "sarangpur" : "rangpur",
    "laxmiganj guna" : "laxmiganj use",
    "sao jose do rio preto   sp" : "sao jose do rio preto sp",
    "karamsad" : "karmasad",
    "santiago de  chile" : "santiago de chile",
    "cao lãnh đồng tháp" : "cao lanh dong thap",
    "lieusaint" : "holy place",
    "mokattam" : "mute",
    "pathanapuram" : "prathanapuram",
    "malapuram" : "malapppuram",
    "campinas shopping móveis" : "campinas shopping mobile",
    "mérida, yucatán, méxico" : "mérida, yucatán, mexico",
    "yucatán" : "yucatan",
    "gharaunda" : "gharonda",
    "sntiago" : "santiago",
    "três rios" : "three rivers",
    "gardabaer" : "keep",
    "são josé do rio preto   sp" : "são josé do rio preto sp",
    "camarines sur" : "camarines on",
    "são caetano" : "sao caetano",
    "chapeco" : "hat",
    "mestre" : "teacher",
    "la follette" : "the madman",
    "via c. battisti, 3b" : "via c.battisti, 3b",
    "lo barnechea" : "barnechea",
    "manhuacu" : "rough comic",
    "foz do iguacu   pr" : "foz do iguacu pr",
    "araçatuba   sp" : "araçatuba sp",
    "कटिहार" : "katihar",
    "paranaíba   ms" : "paranaíba ms",
    "bounsi" : "beyonce",
    ".mpilo and pioneer road maseru" : ".mmp and pioneer road maseru",
    "itajai  sc" : "it will be sc",
    "14 tám danh p4.q8.tphcm" : "14 eight names p4.q8.tphcm",
    "águas formosas" : "famous waters",
    "via de filippis 40" : "way of filippis 40",
    "tangeran" : "tangerang",
    "valleduapar" : "valledupar",
    "san mateo 5 int ala b, boulevares, naucalpan estado de mexico" : "san mateo 5 int ala b, boulevares, naucalpan state of mexico",
    "contagem   minas gerais" : "count minas gerais",
    "bonita ave." : "nice bird.",
    "kiseljak" : "sorrel",
    "cidade" : "city",
    "salfit" : "salt",
    "gretna" : "grena",
    "11100 sepulveda blvd ste 8 pmb 393" : "11100 sepulveda blvd you are 8 pmb zia",
    "barranquilla , colombia" : "barranquilla, colombia",
    "dolnośląskie" : "lower silesian",
    "urb  rio piedras heights 144 wezer" : "urb rio piedras heights 144 wees",
    "cavolo del tomba" : "cabbage of the tomb",
    "nizwa" : "hear",
    "rae bareli" : "raye white",
    "torreón" : "keep",
    "distrito federal" : "federal district",
    "bokakhat" : "bucahat",
    "via circumvallazione esterna 89" : "via external circumvalulation 89",
    "cantabria" : "sneaky",
    "8, lake drive, rajagiriya. na" : "8, lake drive, rajgiria.no",
    "ktx khu a đhqg tp hồ chí minh" : "ktx area a vnu ho chi minh city",
    "juína" : "junior",
    "amapa" : "amapapapapapapapapa about",
    "kanchrapara" : "raw",
    "hải phòng" : "hai phong",
    "calle divino figueroa nº3 bajo c" : "divine street figueroa nº3 under c",
    "ar riyad" : "riyadh",
    "serta" : "as well as",
    "anakapalle" : "unaccounted",
    "tapurah" : "tapur:",
    "al muruj exit5 riyadh" : "the promoter is a feast of riad",
    "località grand chemin 22" : "big chemin location",
    "kuzey kıbrıs" : "north cyprus",
    "ktx khu a đhqg tp hcm" : "ktx area a vnu hcmc",
    "b35, 533" : "akhb, 533",
    "quito ecuador" : "quito, ecuador",
    "sri ganganagar" : "shri ganga nagar",
    "cd.jurez" : "cd.juarez",
    "sas nagar mohali" : "mother -in -law mohali",
    "españa" : "space",
    "via ai ronchi 288, mozzate co." : "away to the ronchi 288, mozzate co.",
    "unanderra" : "anderrera",
    "zduńska wola" : "zdunska wola",
    "bissingen an der teck" : "bissingen on the teck",
    "s.c. de bariloche" : "scby bariloche",
    "wrzeście k" : "september k",
    "belo horizone" : "belo horizonte",
    "śąskie" : "in a good",
    "bauneario" : "road ario",
    "salvado   bahia" : "saved bahia",
    "siddhartha nagar up" : "siddharth nagar sub",
    "hortolandia" : "vegetation",
    "raxaul" : "ragaul",
    "sambhal" : "careful",
    "alto paraiso" : "high paradise",
    "são caetano do sul" : "sao caetano do sul",
    "leh" : "with",
    "2.05 b chung cư mỹ thuận f16 q8" : "2.05 b my thuan apartment f16 q8",
    "смолян" : "smolyan",
    "shrihari kota" : "srihari kota",
    "ciudad de mexico" : "mexico city",
    "asunción" : "assumption",
    "cananda" : "canada",
    "saudi arabia \ al riyad" : "saudi arabia / al riyad",
    "mazowieckie   gabryelin 05 505" : "mazowieckie gabryelin 05 505",
    "ribeirão pires   são paulo" : "ribeirão pires são paulo",
    "bucaramanga   colombia" : "bucaramanga colombia",
    "goianinha" : "gouge",
    "rio grande do sul estação" : "rio grande do sul station",
    "辽宁省大连市" : "dalian, liaoning province",
    "marcham" : "march",
    "kurkheda" : "kurged",
    "haelen" : "hasten",
    "kalimantan selatan" : "south kalimantan",
    "barshi" : "let it go",
    "itabirito   mg" : "it is a mg",
    "quận 11" : "district 11",
    "cần tho" : "can tho",
    "المعادى" : "hostile",
    "רמת השרון" : "ramat hasharon",
    "naugarh siddharth nagar up" : "naugarh siddharth nagar sub",
    "koongal" : "canon",
    "juruti para" : "juruti for",
    "ecuador   loja" : "ecuador loja",
    "edo" : "or",
    "seriate" : "serial",
    "chimalhuacan" : "chimalhuana",
    "huyện tam bình   tĩnh vĩnh long" : "tam calm district vinh long",
    "przysucha" : "priests",
    "kumbakonam" : "kumbakal angle",
    "nellore" : "in",
    "aruppukkottai" : "archae",
    "cao lảnh" : "tall",
    "long xuyên tỉnh an giang" : "long xuyen province an giang",
    "محايل عسير" : "mahayil asir",
    "maadi" : "subject",
    "bhatpar rani" : "bhatpar queen",
    "tekkali" : "indentation",
    "băc giang" : "bác giang",
    "campinas sp" : "campinas, sp",
    "plazoleta la cerámica ste 2 6 ave sanchez vilella esq pr 190 medical" : "pladoleta la ceráma ste 2 6 ave sanchez vilella esq pr 190 medical",
    "sase krstica 43" : "six krstica 43",
    "magé" : "maggish",
    "juiz de fora   mg" : "juiz de fora mg",
    "gautambuddh nagar" : "gautam budh nagar",
    "salvador bahia" : "salvador, bahia",
    "ul. żwirki i wigury 18" : "ul.żwirki i wigury 18",
    "verás" : "you will see",
    "sojat" : "sozat",
    "mubai" : "mumbai",
    "ilocoa norte" : "north ilocos",
    "surat gujarat india" : "indian gujarat letter",
    "itaguaí   rio de janeiro" : "itaguaí rio de janeiro",
    "bulakan" : "bulacan",
    "itiquira" : "itique",
    "มุกดาหาร" : "mukdahan",
    "dokki, giza" : "poisoning, giza",
    "simões filho" : "simão filho",
    "bharuch" : "bharruch",
    "la" : "the",
    "сандански" : "sandanski",
    "rensselaer" : "cleanser",
    "caruaru, pe" : "caruaru, on",
    "macaé   rj" : "macaé rj",
    "sderot" : "deer",
    "imbé   rs" : "imbé rs",
    "godhara" : "godhra",
    "ss129jf" : "ss 129 jaff",
    "beograd" : "white city",
    "al los laureles, el laurel mexico" : "to the laurels, the mexico laurel",
    "douala" : "nation",
    "hạ long" : "ha long",
    "1900 lukken ind dr w" : "1900 luken ind dr w",
    "sesto san giovanni   milano" : "sesto san giovanni milan",
    "valle del cauca" : "cauca's valley",
    "gravataí   rs" : "gravataí rs",
    "tubarão" : "shark",
    "aruppukottai" : "i want to be",
    "santa catarina , sao jose" : "santa catarina, sao jose",
    "peoria" : "peolia",
    "dhbai" : "diani",
    "paragominas   pa" : "paragomin pa",
    "1700 diagonal rd ste 500  commercial display" : "1700 diagonal rd ste 500  commercial display",
    "khaitan" : "khitan",
    "marabá" : "maraba",
    "nova andradina" : "new andradina",
    "av.cra 30 # 17b 25 sur  bogot? d.c" : "av.cra 30 # 17b 25 sur bogot?d.c",
    "kartarpur" : "i'm not sure",
    "via manfredonia km 2,2" : "via manfredonia km 2.2",
    "av. pref. waldemar grubba, 3000 ? vila lalau  sc" : "of.pref.waldemar grubba, 3000?vila lalau sc",
    "norte de santander" : "north of santander",
    "belo jardim" : "beautiful garden",
    "nabadwip" : "ninth",
    "حولي" : "around me",
    "29222 rancho viejo rd ste 108  healthcare" : "29222 rancho vie rd s hand 108 hair lth he",
    "dolnoslaskie" : "donnoslav",
    "arteaga y salazar 108. col. contadero, cuajimalpa cdmx mexico city" : "arteaga y salazar 108. col.contero, cuajimalpa cdmx mexico city",
    "solteiro" : "single",
    "sulaimni" : "sulaimy",
    "senador sá" : "senator sá",
    "aap" : "you",
    "rio das ostras" : "oysters river",
    "shahapur" : "shahpur",
    "sao poaulo" : "sao paulo",
    "padipiranga" : "padipirga",
    "05 520 konstancin,warszawska 21d" : "05 520 konstancin, warszawska 21d",
    "plaghar" : "palathe",
    "châu thành  trà vinh" : "chau thanh tra vinh",
    "barreras" : "barriers",
    "6600 van aalst boulevard  medical" : "6600 van aalst boulevard medical",
    "15 lê đình cẩn, bình tân" : "15 le dinh can, binh tan",
    "pekini ave.41  pekini ave.41" : "pekini ave.41 pekini ave.41",
    "vyara" : "very",
    "liberdastr. 8" : "liberdatr.8",
    "פתח תקווה" : "petah tikva",
    "santa bárbara d’ oeste" : "santa barbara d 'west",
    "chanagancherry, kottayam" : "chanaganceri, kottayam",
    "aracajú sergipe" : "aracaju - sergipe",
    "florencia" : "florence",
    "pimpalgaon baswant" : "baswant pifts",
    "otros" : "others",
    "los alamos" : "poplars",
    "bhabua" : "babua",
    "rizwankhan" : "rizwean",
    "bahia   lauro de freitas" : "bahia lauro de freitas",
    "porto algre" : "porto alegre",
    "gurupi to" : "with gurpi",
    "al obour" : "transit",
    "09 đỗ xuân hợp, bình trưng đông,q2" : "09 do xuan hop, binh trung dong, district 2",
    "mohali , punjab" : "mohali, punjab",
    "sao pedro" : "saint peter",
    "wauwatosa" : "difference from wow wow",
    "kujawsko   pomorskie" : "kujawsko pomorskie",
    "اكتوبر" : "october",
    "13405 stemmons frwy" : "13405 stemmons",
    "rio verde   goiás" : "rio verde goiás",
    "santo andré" : "saint andré",
    "changaramkulam" : "changramakulam",
    "peru   lima" : "the name is lima",
    "monterret" : "monterrey",
    "cai lậy tiền giang" : "cai lay tien giang",
    "sana'a" : "occupation",
    "linh trung, thủ đức, hồ chí minh" : "linh trung, thu duc, ho chi minh",
    "pismo beach" : "beach letter",
    "hyderadbad" : "hyderabad",
    "belagavi" : "belagogi",
    "roca sales" : "sales roca",
    "itu" : "that",
    "união da vitória   pr" : "união da vitória pr",
    "arvada" : "pivot",
    "visconde do rio branco" : "viscount of rio branco",
    "ortigueira" : "ortigue",
    "curitiba   parana" : "curitiba, parana",
    "unite kingdom" : "united kingdom",
    "kaveripakkam" : "frostbite",
    "po box 9897, el paso tx. 79995 1201 kessler" : "po box 9897, el paso tx.79995 1201 kessler",
    "khowai" : "khavai",
    "2a 4a tôn đức thắng, quận 1,  hcm" : "2a 4a ton duc thang, district 1, hcm",
    "shimoga karnataka" : "the bottom is ran",
    "praia grande  sp" : "praia grande sp",
    "saint jean de moirans" : "saint john of moirans",
    "đường số 21 bình chiểu thủ đức" : "road no. 21 binh chieu thu duc",
    "3318 se 6th ave" : "3318 to 6 th",
    "bacood sta mesa manila" : "bacood is mesa manila",
    "khoda colony" : "colonies",
    "65 trương định" : "65 truong dinh",
    "łętownia" : "warehouse",
    "marathovounou 7 latsia" : "marathow 7 latsia",
    "ustronie morskie, pow. kołobrzeski" : "ustronie morskie, areakołobrzeg",
    "nenava" : "random",
    "jastrzębia" : "hawk",
    "abigjan" : "abijan",
    "carrera 29 # 45 45 piso 15 8902130749" : "carrera 29 # 45 45 floor 15 8902130749",
    "newyork parkview apartments 22122" : "new york parkview apartments 22122",
    "kab. lumajang" : "regency.lumajang",
    "payas" : "share",
    "dabra" : "pass",
    "sanand" : "gentleman",
    "turkiye" : "turkey",
    "135 se 6th ave" : "135 to 6 th and",
    "manaus  am" : "manaus am",
    "bangunan jalex, lot 3750, lrg 2d,  kampung melayu subang" : "jalex building, lot 3750, lrg 2d, subang malay village",
    "thaltej" : "taulge",
    "niterói" : "niteroi",
    "modasa" : "outfit",
    "ekangarsarai nalanda bihar" : "ekangarasarai nalanda bihar",
    "新莊區" : "xinzhuang district",
    "ferozepur" : "firozpur",
    "belém do para" : "belém do pará",
    "lộc hòa" : "loc hoa",
    "san josé del cabo" : "san jose del cabo",
    "jid ali" : "good for me",
    "paraiba   patos" : "paraiba patos",
    "geuteng" : "headal",
    "xalle 71d#67 85" : "helle 71d # 67 85",
    "السعوديه" : "saudi arabia",
    "nyiregyhaza" : "corbygougy",
    "biaora" : "baiaora",
    "vize" : "visa",
    "katra" : "every",
    "ive andrica 1, 71250 kiseljak sarajevo" : "this is andrew ೧, ೭೧೨೫೦ kiseljak sare jeevano",
    "cần thơ" : "can tho",
    "joinvile" : "joinville",
    "colombia   tunja" : "colombia tunja",
    "marhaura" : "madhaura",
    "xangri la   rs" : "xangri la rs",
    "pe   serra talhada" : "pe serra talhada",
    "sivasagar" : "sivasagar is the best",
    "dolnyśląsk" : "lower silesia",
    "prbtsar jawla" : "with a tour of a tour",
    "thủ đức, tp hồ chí min" : "thu duc, ho chi minh city",
    "nghệ an" : "nghe an",
    "edo de mex" : "come in edo x",
    "carr 3 oste # 42 45" : "car 3 oste # 42 45",
    "300 las vistas shopping village ave. felisa rincón ste.31" : "300 the views shopping village ave.felisa rincón ste.31",
    "רמת גן" : "ramat gan",
    "gunj basoda" : "ganj basoda",
    "chennau" : "lichen",
    "venda do pinheiro" : "pinheiro sale",
    "lavalle 780" : "on stage 780",
    "fortaleza  ce" : "fortaleza what",
    "ibadan, oyo state" : "ibadan, that state",
    "quận 9" : "district 9",
    "żółwin" : "ywin",
    "บุรีรัมย์" : "buriram",
    "tepeji del rio de ocampo hidalgo" : "tepeji of the river of ocampo hydalgo",
    "abudhabi city" : "abu dhabi city",
    "asdf" : "seashell",
    "lima, lima" : "five, five",
    "bhte minas gerais" : "flowing minas geris",
    "số 04 đường d9, quận tân phú" : "no. 04 d9 street, tan phu district",
    "ashgabat" : "the underum",
    "città della pieve" : "city of the pieve",
    "guadalajara, jal." : "guadlajara, water.",
    "palghat" : "palaghat",
    "tarakeswar" : "tartar",
    "minas gerais   japaraiba" : "minas gerais japaraiba",
    "patos de minas  mg" : "patos de minas mg",
    "khardaha" : "the lines",
    "θηβων 500" : "thebes 500",
    "kanuta" : "kaluuta",
    "dwsa" : "get",
    "mazowieckie warszawa" : "masovian voivodeship warsaw",
    "goias   goiânia" : "goias goiânia",
    "الاقصر" : "the shortest",
    "bangana" : "they are equal",
    "palghar  boisar  mumbai" : "palghar boisar mumbai",
    "gudemaranhalli" : "gudemaranahalli",
    "akola" : "did",
    "594 howard stsan franciscoca94105" : "594 howard st san francisco ca 94105",
    "kashan" : "is the",
    "ấp chợ tập sơn trà cú trà vinh" : "tap son tra cu market hamlet tra vinh",
    "jawa barat" : "west java",
    "edo.mex" : "edo.x",
    "khanouri" : "jerk",
    "lagarto" : "lizard",
    "aquiraz   ceará" : "aquiraz ceará",
    "rosà" : "rose",
    "gridih" : "greedih",
    "montes claros" : "clear montes",
    "al ain" : "where is",
    "khartom" : "khartum",
    "rj   macaé" : "rj macaé",
    "rua j?natas serrano, 182   casa 2   jardim castro alves" : "rua j? natas serrano, 182 house 2 jardim castro alves",
    "929 gessner  healthcare" : "929 gessner healthcare",
    "السنطه" : "eagle",
    "14 đường số 5, đặng văn bi thủ đức" : "14 street no. 5, dang van bi thu duc",
    "plachkovski manastir 7" : "plachkovsky monastery will",
    "santa lucia" : "saint lucia",
    "κ.,κοντου 2, αθηνα, 11255" : "k., kontou 2, athens, 11255",
    "rio de janeiro, rj" : "rio de janeiro - rj",
    "madhopara ,purnia" : "madhopra, puraniya",
    "ciudad apodaca" : "apodaca city",
    "mato grosso do sul   campo grande" : "mato grosso do sul campo grande",
    "kakching" : "i'm not sure",
    "kujawsko pomorskie,nowe" : "kujawsko pomorskie, new",
    "10 nguyễn duy, p12, q8" : "10 nguyen duy, p12, district 8",
    "tirunelveli, tamil nadu 627007" : "tirunelveli, tamil nadu ౬౨౭౦౦౭",
    "rajawali timur no 18 jawa barat" : "rajawali timur no 18 west java",
    "الباحة" : "courtyard",
    "hunsgi" : "hazard",
    "dehradoon" : "dehradun",
    "cd mex" : "cdmx",
    "st. michael im lungau" : "st.michael im lungau",
    "скопје" : "skopje",
    "rasalkhaima" : "ax",
    "mazowieckie" : "masovian",
    "الباجور منوفيه" : "bagor menoufia",
    "山东威海" : "shandong weihai",
    "bolangir" : "collarew",
    "pernambuco   paulista" : "pernambuco paulista",
    "warmińsko mazyrskie" : "warmian maozyrskie",
    "thủ đức, tp.hcm" : "thu duc, ho chi minh city",
    "maceió al" : "macea al",
    "kanhangad" : "canneman",
    "mounofia" : "munophy",
    "ghaziabaad" : "gypsy",
    "monte hermoso" : "monte beautiful",
    "warszawa" : "warsaw",
    "adamantina" : "adamantine",
    "bankura" : "to the bank",
    "santa amrta" : "santa marta",
    "русе" : "ruse",
    "tp thủ đức, tp.hcm" : "thu duc city, ho chi minh city",
    "catalão" : "catalan",
    "premia de mar" : "sea award",
    "dar es salaam tanzania" : "give is salaam tanzania",
    "bhavangar" : "bhavnagar",
    "kakkanad, kochi" : "his excellency d, here",
    "yopal   casanare" : "yopal casanare",
    "bueno aires" : "well airs",
    "ropar" : "call out",
    "herriman" : "faculation",
    "los banos, laguna" : "bans, laguna",
    "khamaria" : "vintage",
    "belait" : "belite",
    "sorriso mt" : "smile mt",
    "srikalahasti" : "sikalahas",
    "bắc giang" : "bac giang",
    "vizianagaram" : "vizayanagaram",
    "cebu,lapu lapu city" : "cebu, lap lap city",
    "curitiba paraná" : "curitiba, parana",
    "jk raod karande mala darewadi krishna house" : "czech rawd kandande mala darevadi krishna house",
    "timarni" : "timar",
    "dolnośłąskie" : "lower silesian",
    "vũng tàu" : "vung tau",
    "la altagracia" : "altagracia",
    "irinjalakuda" : "toolbar",
    "hà nội" : "hanoi",
    "uberaba  mg" : "uberaba mg",
    "ghatal" : "gossip",
    "las palmeras" : "the palm trees",
    "saint paul" : "st.paul",
    "presidente prudente" : "prudent president",
    "sarhochiya" : "sarashia",
    "via a. manzoni" : "away a.manzoni",
    "hạ long quảng ninh" : "ha long quang ninh",
    "sabana perdida" : "lost savanna",
    "москва" : "moscow",
    "libano" : "lebanon",
    "pudukottai" : "i want to be in the prefecture",
    "واسط" : "interface",
    "iguala" : "equal to",
    "región metropolitana de santiago" : "santiago metropolitan region",
    "al khawaneej" : "al khwani",
    "mpumalanga,emalahleni" : "mpumalanga, emalahleni",
    "76 vĩnh hưng" : "76 vinh hung",
    "taubaté   sp" : "taubaté sp",
    "koduvai, tiruppur" : "koduwai, tiruppur",
    "rua comandante costa 3021  cuiab?   mt 3021" : "rua commante costa 3021 cuiab?mt 3021",
    "via 4 novembre n^95" : "via 4 november n^95",
    "manaus  amazonas" : "manaus, amazonas",
    "núcleo bandeirante" : "bandeirante core",
    "القبة" : "dome",
    "தூத்துக்குடி" : "throat",
    "estado de mexico" : "mexico state",
    "nosara" : "and sarah",
    "arlington, ny 12603" : "arlington, 12603",
    "sangotedo" : "three words",
    "kirayan norte miagao ,iloilo" : "kirayan norte miagao ,ilolo",
    "rio verde" : "green river",
    "são luís maranhão" : "sao luis maranhão",
    "ปทุมธานี" : "pathum thani",
    "porto velho, rondônia" : "velho porto, rondônia",
    "364 dương quản hàm" : "364 duong quang ham",
    "tambo grande" : "big tambo",
    "cao lãnh , đồng tháp" : "cao lanh, dong thap",
    "8 phan đình giót, p2, tb" : "8 phan dinh giot, p2, tb",
    "salzburger straße 777" : "salzburger strasse 777",
    "6135 nw 167th st, ste e24 ste e24 cruise" : "6135 nw 167th st, you are e24 you e24 cruise",
    "livramento" : "liberation",
    "malopolskie" : "lesopolskie",
    "şırnak" : "sirnak",
    "taboão da serra   sp" : "taboão da serra sp",
    "台灣台北市" : "taipei city, taiwan",
    "ботевград" : "botevgrad",
    "mehgaon" : "ish",
    "opelousas" : "operator",
    "359 hồ học lãm" : "359 ho hoc lam",
    "sarni" : "sar",
    "guaiba" : "be good",
    "buga" : "print",
    "buenos aires   tigre" : "buenos aires tigre",
    "bangalorr" : "bangalirr",
    "barrio viejo de callosa de segura" : "viejo de callosa de segura",
    "mahemdavad" : "mahmadism",
    "kannad" : "heels",
    "chotomów" : "chotom",
    "salador" : "salter",
    "bagneux" : "prickly",
    "králův dvůr levín" : "the king's court",
    "campo largo pr" : "largo pitch pr",
    "nakhal" : "disgrace",
    "afua" : "originated",
    "purnea" : "poorna",
    "ciudad juarez" : "juarez city",
    "la calorta" : "the carat",
    "ballajura" : "dancing it",
    "calo" : "decline",
    "tirunelveli" : "tirunelevel",
    "rondonópolis   mt" : "rondonópolis mt",
    "612 hangar ln ste. 223, nashville, tn 37217" : "612 hangar ln you are.223, nashville, tn 37217",
    "república dominicana" : "dominican republic",
    "divino" : "divine",
    "poções" : "potions",
    "recife pe" : "recife on",
    "san luis potosí" : "san luis potosi",
    "cuneo" : "wedge",
    "via castelgomberto 96 italia" : "via castelgomberto 96 italy",
    "châu đốc" : "continental director",
    "bhubanerwar" : "bhubnerwar",
    "新北市" : "new taipei city",
    "méxico" : "mexico",
    "via a. manzoni 212" : "away a.manzoni 212",
    "vila velha" : "old village",
    "ayias zonis 60 c limassol" : "of this zone 60 c limassol",
    "thiruppttur" : "tirupur",
    "terrassa barcelona" : "barcelona terrace",
    "offlaga" : "offto",
    "ras al khaima" : "ras al khaimah",
    "guadalajara jalisco méxico" : "guadalajara jalisco mexico",
    "raiadh" : "records",
    "itaberai" : "portion of the board",
    "الرستاق" : "rust",
    "hyde rabad" : "hyderabad",
    "s.d. de rana" : "s.d.of wound",
    "presidente bernardes" : "president bernardes",
    "varca" : "gap",
    "ποσειδωνος 58" : "poseidon 58",
    "aachen   alemanha" : "aachen germany",
    "londrina, pr." : "laundrina, on.",
    "bucaramaga" : "busramga",
    "572, navamin90, klongkum," : "572, navamin 90, kangkum,",
    "gurgaoon" : "gurgaon",
    "michigan" : "road cancer",
    "senador canedo goiás" : "senator canedo goiás",
    "6 ballela road" : "6 read the road",
    "argao cebu city" : "a cream ce is not city",
    "banaglore" : "bangalore",
    "suratgarh" : "photograph",
    "tierra del fuego" : "land of fire",
    "dar es salaam" : "s salam house",
    "zaječar" : "zajecar",
    "bhind" : "bind",
    "deesa" : "goddess",
    "14 đường số 17, p. tân thuận tây" : "14 street no. 17, p.tan thuan tay",
    "imperatriz" : "empress",
    "chikkaballapura" : "chikkallapura",
    "100 first stsan franciscoca94015" : "100 first st san francisco ca 94015",
    "المنوفية" : "menoufia",
    "satana" : "satan",
    "bella vista" : "beautiful view",
    "beit yanai" : "hate love",
    "via lombardia n.8" : "via lombardia ranked # 8",
    "salto sp" : "heel sp",
    "kalaphar" : "black mountain",
    "vijayapur" : "vijaypur",
    "bangalorej" : "bungalore",
    "erith" : "eager",
    "upper pueryo cdo" : "upper puerto cdo",
    "datia (m.p.)" : "data (m.p.)",
    "savadatti" : "conscience",
    "tân đông hiệp, dĩ an, bình dương" : "tan dong hiep, di an, binh duong",
    "chapadinha" : "hat",
    "minas gerais  uberlandia" : "minas gerais uberlandia",
    "negros occidental" : "western blacks",
    "zaragoza, nueva ecija" : "zaragoza, new ecija",
    "eastren area" : "eastern area",
    "merida" : "mary",
    "nawada" : "the rest",
    "338 nguyen van giap ha noi" : "338 nguyen van giap has us",
    "pandikkad" : "bangy wall",
    "رابغ" : "rib",
    "कलकता" : "cadre",
    "dekalb" : "decalb",
    "ηρωων πολυτεχνειου 74" : "heroes people 74",
    "itaú de minas   minas gerais" : "itaú de minas minas gerais",
    "khu phố 1, tân hiệp, biên hòa" : "quarter 1, tan hiep, bien hoa",
    "смедерево" : "smederevo",
    "coxim" : "cushion",
    "sultanpur up" : "sultanpur sub",
    "enna" : "what",
    "novo gama" : "new range",
    "תל אביב" : "tel aviv",
    "kottrakara, kollam" : "kollam, kollam",
    "reibeirão preto sp" : "ribeirão preto sp",
    "bulandhshar" : "bulandshahr",
    "perambalur" : "pounding",
    "asandh" : "blessed",
    "linh trung, tp thủ đức, tp hcm" : "linh trung, thu duc city, ho chi minh city",
    "al khobar" : "the news",
    "jaen, nueva ecija" : "jaen, new ecija",
    "الرحبة" : "splendor",
    "talala gir" : "talala provides",
    "srinsgar" : "srins",
    "dorangla" : "dagang",
    "tihu" : "quiet",
    "elmsford" : "elmford",
    "sidvokodvo" : "sidvoko",
    "ipaba" : "tain",
    "bordon" : "shore",
    "nizgny novgorod" : "nizhny novgorod",
    "la union" : "union",
    "corrientes capital   corrientes" : "corrientes corrientes",
    "kadıköy" : "kadikoy",
    "sangat" : "very",
    "quezon coty" : "quezon city",
    "curitiba, pr" : "karitiba, but",
    "são luís  ma" : "são luís ma",
    "strada provinciale.le sud 101" : "provincial road. south 101",
    "tema" : "her",
    "nairbi" : "nai rujie",
    "hurgada" : "hurghada",
    "dehra du" : "dehra you",
    "igrejinha rs" : "church rs",
    "mettur" : "mector",
    "progreso, yucatán" : "progress, yucatan",
    "santiago, region metropolitana" : "santiago, metropolitan region",
    "hazaribag" : "hazaribagh",
    "shriwardhan" : "scriwarray",
    "pindaré mirim   maranhão" : "pindaré mirim maranhão",
    "via galatina,187   sant'angelo" : "via galatina, 187 sant'angelo",
    "shenzen" : "before shen",
    "גבעתיים" : "givatayim",
    "boituva  sp" : "boituva sp",
    "uberaba   mg" : "uberaba mg",
    "uberlandia   mg" : "uberlandia mg",
    "boirani" : "boyla",
    "canoas" : "canoes",
    "3era seccion lomas verdes" : "3rd lomas verdes section",
    "ak" : "and",
    "buttlar" : "bitch",
    "la cisterna" : "the cistern",
    "sangao" : "three high",
    "mahopac" : "mahoapc",
    "borca" : "a fighter",
    "mazowiecki" : "mazovian",
    "上海" : "shanghai",
    "เมือง" : "city",
    "paschim medinipur" : "west midnapore",
    "via s. anguissola 1" : "way's.anguissola 1",
    "4221 w john carpenter fwy" : "4221 w john carpenter more",
    "mandya" : "cloak",
    "braço do norte" : "northern arm",
    "markapur" : "marakpur",
    "ninavah" : "ninveh",
    "arapongas pr" : "on arpongas",
    "vitória do mearim" : "mearim victory",
    "cancun" : "residual",
    "sheakh zayed" : "the necks drop",
    "الشارقه" : "shark",
    "ilinois" : "illinois",
    "arame" : "wire",
    "kí  túc xá khu b đại học quốc gia" : "dormitory of national university",
    "türki̇ye" : "the turkistan use",
    "ראשון לציון" : "rishon lezion",
    "er" : "is",
    "dharamavaram" : "dharmavaram",
    "6 hẻm số 5 mễ cốc" : "6 alley no. 5 me coc",
    "3 bethesda metro ctr ste 700" : "x better metro str in 700",
    "las piñas" : "the pineapples",
    "bhilwara bapunagar kerali suitings.." : "bhilwara bapunagar kerli suitings ..",
    "praha 9" : "prague 9",
    "quận 3" : "district 3",
    "4880 stevens creek blvdsan joseca95129" : "4880 stevens creek blvd san jose ca 95129",
    "ulwe" : "fight",
    "dos hermanas" : "two sisters",
    "ciudad del carmen" : "carmen city",
    "berhampur" : "delay",
    "shahdol" : "shadol",
    "barauna rn" : "maruna rn",
    "петрич" : "petrich",
    "נתניה" : "netanya",
    "tata hospital actrec  sector 22 kharghar" : "tata hospital actrec seater 22 kharghar",
    "29 nguyễn t minh khai đà nẵng" : "29 nguyen t minh khai da nang",
    "omalur" : "alum",
    "arandu" : "wise",
    "ouagadougou" : "ungaduer",
    "62b kp3 an lạc a bình tân tphcm" : "62b kp3 an lac a binh tan hcmc",
    "extremos" : "extremes",
    "honduras,tegucigalpa" : "honduras, tegucigalpa",
    "marilia  sp" : "marilia sp",
    "los banos" : "the bathrooms",
    "al nakheel, ras al khaimah" : "al nakheel, ras al tent",
    "cheshire" : "chill",
    "hadrmout seyun" : "hadrmout pregnant",
    "itapecerica da serra   sp" : "itapecerica da serra sp",
    "500 chesterfield ctr, ste 355 ste 355" : "500 chesterfield ctr, you are 355 you are 355",
    "pasay city , metro manila" : "pasay city, metro manila",
    "ktx đại học quốc gia khu b" : "national university dormitory area b",
    "العبور" : "transit",
    "saint mitre les remparts" : "saint miter the ramparts",
    "meerut uttar pradesh" : "meerut uttor pradesh",
    "tradate" : "trades",
    "13 kim đồng giáp bát" : "13 kim dong giap bat",
    "aska" : "ash",
    "kanasin" : "cassin",
    "via roma  20" : "via roma 20",
    "corso s. ignazio     88" : "course s.ignazio 88",
    "via degli ippocastani,1" : "via degli ippocastani, 1",
    "678 , chua chu kang cresc #11 622" : "678 , that's the fire cresc #11 622",
    "yverdon les bains" : "yverdon les bains",
    "davao del norte" : "north davao",
    "são sebastião" : "san sebastian",
    "cali  valle" : "valley",
    "शादी" : "marriage",
    "jaru" : "spring",
    "campina grande  pb" : "campina grande pb",
    "מבשרת ציון" : "mevaseret zion",
    "trần đại nghĩa bình tân" : "tran dai nghia binh tan",
    "suite 22, no.4 ajala street (alhaji opeyemi shopping complex), off ikotun idimu road, subol bus stop" : "shuhunte ౨౨, no.౪ alaji street",
    "تبوك" : "tabuk",
    "นครราชสีมา" : "nakhon ratchasima",
    "thiruvalla (tiruvalla)" : "turna (turn)",
    "presidente kennedy" : "president kennedy",
    "jaleswar" : "gap",
    "umm al quain" : "umm al quin",
    "tanajib provence" : "tajiib provence",
    "929 gessner drive  healthcare" : "929 gessner drive healthcare",
    "ariccia" : "habit",
    "goiânia goiás" : "goiania goias",
    "cene" : "prices",
    "prayagraj" : "praisagraj",
    "recifes" : "reef",
    "bollampally veldanda nagarkarnool telangana" : "bollampalay weldand nagarakarnul telangana",
    "shirur" : "shiroor",
    "lac du flambeau" : "flambeau lake",
    "digos city, davao del sur" : "digos city, south davao",
    "chomu" : "why",
    "rewa mp" : "les mp",
    "para abaetetuba" : "to abaetetuba",
    "rio grande do norte" : "large northern river",
    "2777 n stemmons fwy  cloud it" : "2777 n stemmons more cloud it",
    "saihat" : "health",
    "via  tenuta del cavaliere,1" : "via tenuta del cavaliere, 1",
    "metromanila" : "metro manila",
    "tyerl" : "terlle",
    "thoothukudi" : "throat",
    "palams" : "of the palms",
    "chengalpattu" : "excitement",
    "ciudad de panama [panama city]" : "city of panama [paama city]",
    "konopiska" : "hemp",
    "belo orizonte" : "belo horizonte",
    "pargi" : "park",
    "المنيا" : "minya",
    "sta catarina   florianopolis" : "sta catarina florianopolis",
    "são paulo, são bernardo do campo" : "sao paulo, sao bernardo do campo",
    "rinconada del agua 2811 jalisco" : "water rinconada 2811 jalisco",
    "para de minas" : "for mines",
    "via del lavoro artigiano 10" : "via del lavoro craftsman 10",
    "rigadh" : "rigade",
    "gravataí" : "tie",
    "herbertpu, dehradun" : "herbertpur, dehradun",
    "台北市松山區" : "songshan district, taipei city",
    "kapaa" : "softly",
    "buin" : "done",
    "nalgonda" : "phid",
    "ddun" : "tiny",
    "banagalore" : "bangalore",
    "μεσογειων 85" : "mediterranean 85",
    "bogota, colombia." : "bogota colombia.",
    "long vĩnh, gò công tây, tiền giang" : "long vinh, go cong tay, tien giang",
    "rawatbhata" : "his salaries",
    "nueva vizcaya" : "new biscay",
    "tp. hồ chí minh" : "tp.ho chi minh",
    "saudi arabia  riyadh" : "saudi arabia riyadh",
    "bombinhas" : "bombs",
    "puneou" : "puneoula",
    "hueypoxtla" : "caridary",
    "c.da cataniello,12" : "c.da cataniello, 12",
    "viamão   rs" : "viamão rs",
    "koto ku, tokyo" : "ward and residence",
    "calle 52 no. 22   44 bogota" : "52 street no.22 44 bogota",
    "santa fe  rosario" : "santa fe rosario",
    "chakan" : "check",
    "união dos palmares" : "palmares union",
    "8 phan đình giót" : "8 phan dinh giot",
    "ghazipur up" : "ghazipur sub",
    "imbituba sc" : "imbitaba sc",
    "جمصه" : "jamsa",
    "adoni" : "adon",
    "villavicencio, meta" : "villavicencio, goal",
    "pouso redondo" : "round landing",
    "kharepatan" : "weapon",
    "são leopoldo" : "sao leopoldo",
    "visakhapatnm" : "visakhapatnam",
    "cabo frio" : "cabio cabio",
    "vinhedo   são paulo   brasil" : "vineyard são paulo brazil",
    "jagtial" : "hunting",
    "kishinev" : "chisinau",
    "r. catarina eufemia" : "r.catarina eufemia",
    "jabalpu" : "jabalpur",
    "dianopolis" : "intellectual",
    "tiruttani" : "stream",
    "bhubaneswer" : "showerwar",
    "surat kamrej" : "kamrej's letter",
    "karaikkudi" : "karaikudi",
    "edo. de mexico" : "it's from.mexico",
    "são paulo sorocaba" : "sao paulo sorocaba",
    "جده" : "grandmother",
    "demuro, 11" : "room, 11",
    "อุบลราชธานี" : "ubon ratchathani",
    "taiwan , taipei" : "taiwan, taipei",
    "curico" : "curic",
    "woolloomooloo" : "woolboyo",
    "khóm 1 thị trấn càng long" : "pineapple 1 town",
    "kalpetta" : "pale",
    "miraflores   lima" : "miraflores lima",
    "варна" : "varna",
    "jesi" : "you are",
    "ký túc xá khu b" : "dormitory area b",
    "paracatu   mg" : "paracatu mg",
    "jakarta timur, dki jakarta" : "east jakarta, dki jakarta",
    "i̇stanbul (avr)" : "istanbul (avr)",
    "londonamie" : "londonama",
    "tiruvallur district" : "tiruvalur district",
    "nagerkovil" : "nagerkov",
    "miscat" : "moved",
    "гр. софия, област софи" : "sofia, sophie district",
    "jawa timur" : "east java",
    "new delh" : "new delhi",
    "bastos" : "bastly",
    "bjn" : "they make",
    "mazatlán" : "mazatlan",
    "macae" : "macaaye",
    "puñe" : "dagger",
    "costa  mesa" : "costa mesa",
    "أكتوبر" : "october",
    "hòa an" : "hoa an",
    "36, 31 465 kraków" : "36, 31 465 krakow",
    "197 đường 9 phước bình" : "197 street 9 phuoc binh",
    "kharagpur" : "khargpur",
    "cl. 6 sur no. 79 150" : "cl.6 south no.79 150",
    "del mar" : "from sea",
    "puerto colombia" : "colombian port",
    "10,rue 2,lotissement abdelmoumen n 6 immeuble hajjar" : "10, rue 2, abdelmoumen subdivision n 6 hajjar building",
    "الفجيرة" : "fujairah",
    "שוהם" : "onyx",
    "raebareli" : "rae bareli",
    ". mancherial.... telangana" : ".manancherl .... telangana",
    "kopargaon" : "copargaon",
    "tiangua ce" : "hexagram test",
    "bahia   presidente tancredo neves" : "bahia presidente tancredo neves",
    "minhas gerais   araxá" : "my general araxá",
    "la libertad" : "freedom",
    "paraná,  assis chateaubriand" : "paraná, assis chateaubriand",
    "teutônia" : "teutonia",
    "riyadh   central" : "riyadh central",
    "2040 buda?rs, vas?t utca 9." : "2040 buda? rs, vas? t street 9.",
    "kí túc xá khu a" : "dormitory a",
    "বাদসাআনারুললস্কয" : "bang",
    "shahjhanpur" : "shahjahanpur",
    "kuna" : "there are",
    "cancún benito juárez" : "cancun benito juárez",
    "بغداد ." : "baghdad .",
    "65 đường 14" : "65 street 14",
    "anjuna" : "attempt",
    "la verne" : "verne",
    "santa tecla" : "holy button",
    "arni" : "on",
    "đồng xoài bình phước" : "dong xoai binh phuoc",
    "pedra preta" : "black stone",
    "navegantes" : "navigators",
    "haute savoie" : "haute-savoie",
    "shahpura (jaipur dist.)" : "shahpura (jaipur d from)",
    "souhag   maragha city" : "sohag maragha city",
    "1935 silverton rd ne" : "1935 silverton rd no",
    "balanga city" : "they advertise the city",
    "flores da cunha" : "cunha flowers",
    "borgata lavino, 32a" : "borgata developed, 32a",
    "uông bí" : "uong bi",
    "harnatand" : "haranatand",
    "guadalupe nuevo  leon" : "guadalupe nuevo leon",
    "via g. pascoli 39" : "via g.pascoli 39",
    "río de janeiro" : "rio de janeiro",
    "tupã" : "bed",
    "ofibodegas perisur no 8 frente escuela international" : "ofibodegas perisur no 8 front international school",
    "nova crixas" : "new crisles",
    "elsherouk" : "sunrise",
    "via uscioli,114 nocera superiore" : "via usgioli, 114 nocera superiore",
    "p.o. box 411, baysh" : "p.o.box 1 c, age",
    "dzs" : "cbs",
    "kosamba" : "praying",
    "registro" : "record",
    "shimoga" : "below",
    "são joão da boa vista   sp" : "são joão da boa vista sp",
    "francesco laparelli" : "francesco laparchelli",
    "san luis" : "saint louis",
    "santa barbara d’oeste" : "santa barbara d'oeste",
    "vicolo della stufa 14," : "stove 14 alley,",
    "espírito santo" : "holy spirit",
    "مدينه الخبر" : "khobar",
    "barangay alupay rosario batangas" : "barangay alupuds rosario batangas",
    "belém" : "bethlehem",
    "santo ângelo   rs" : "santo ângelo rs",
    "tirupathi" : "tirupati",
    "siddapur" : "sidpur",
    "udarsi" : "generous",
    "tân hiệp, trà cú, trà vinh" : "tan hiep, tra cu, tra vinh",
    "priyanka jhavar" : "priyanka jhawar",
    "52cc khu phố 2 phường tân quy q7" : "52cc quarter 2 tan quy ward district 7",
    "jamkhandi" : "jokhandi",
    "itsitio" : "self -sufficiency",
    "số 2 ngõ 6" : "no. 2 lane 6",
    "kasauli" : "cosmus",
    "pato branco   paraná" : "pato branco paraná",
    "algérie" : "algeria",
    "via dell'informatica 10   37036 san martino buon albergo (veneto), italy" : "via dell'oformatica 10 37036 san martino buon albergo (veneto), italy",
    "brasília" : "brasilia",
    "argyll" : "arch",
    "vitória da conquista." : "victory of the conquest.",
    "ciudad de méxico" : "mexico city",
    "rayachoty" : "rage",
    "ribeião pretp   sp" : "ribeirão preto sp",
    "ras al khaimah, uae" : "the head of the tent, a",
    "mriehel" : "twist",
    "khattar ,navi mumbai" : "khattar, navi mumbai",
    "poá" : "dust",
    "dharmshala (dharamshala)" : "dharamshala (dharamshala)",
    "400 s el camino real  cloud it" : "400 s the royal cloud it",
    "cainta" : "santa",
    "cuernavaca morelos" : "cuernavaca, morelos",
    "can.cun quintanaroo" : "can.cuni quintanaroo",
    "kampli" : "kamplipl",
    "fortaleza   ceará" : "fortaleza, ceara",
    "región" : "region",
    "wielkopolskiej" : "greater poland",
    "sao paulo" : "são paulo",
    "fort belvoir" : "very belvoir",
    "al gudaibiys" : "the jadbis",
    "nahan" : "retain",
    "cancún quintana roo." : "cancun quintana roo.",
    "anápolis   goiás" : "anápolis goiás",
    "chau đốc" : "chau doc",
    "bahia   camaçari" : "bahia camaçari",
    "del" : "of the",
    "omerga" : "to the ommer",
    "raigarh" : "raigad",
    "banediya" : "banadia",
    "pápa" : "pope",
    "yopal  casanare" : "yopal casanare",
    "an de klump 8 47661 issum" : "and the clump 8 47661 issum",
    "doda" : "he adds",
    "cd. lázaro cárdenas" : "cd.cárdenian lazarus",
    "kalinzar" : "calinzar",
    "उदयपुर" : "udaipur",
    "nunamara" : "outer",
    "rusterburg" : "rustenburg",
    "concórdia" : "concord",
    "vereeniging" : "association",
    "deoria" : "rear",
    "katargam" : "abolition",
    "القاهرة‬" : "cairo",
    "general villamil playas" : "general villamil beaches",
    "kendrapara" : "centerpara",
    "alejandro schroeder 6407 uruguay" : "aljandro schroeder 6407 uruguay",
    "المدينه المنورة" : "al madinah al munawwarah",
    "chodecz" : "pedestrian",
    "beed" : "egg",
    "mata roma" : "mata rome",
    "sahibabad, ghaziabad" : "halabad, gas",
    "dolnoślaskie" : "lower silesian",
    "aliso viejo" : "old alder",
    "nova igualu" : "new iguac",
    "tierra blanca" : "white land",
    "518 dungan street" : "518 season street",
    "kavali" : "want to be",
    "nueva ecija" : "nue's ecija",
    "panamá" : "panama",
    "mihijam" : "michiam",
    "sóc trăng" : "soc trang",
    "naucalpan, estado de méxico" : "naucalpan, state of mexico",
    "bình định" : "pacify",
    "fatehnagar distt udaipur" : "fatehanagar distt udipur",
    "amsterdã" : "amsterdam",
    "bolu" : "divided",
    "barranquilla colombia" : "barranquilla, colombia",
    "feijó" : "bee",
    "palwal" : "palval",
    "mexicaltzingo 2352 a y b, col. arcos, guadalajara, jalisco. jalisco" : "mexicantzingo 2352 a and b, col.arcos, guadalajara, jalisco.jalisco",
    "ينبع البحر" : "sea's origin",
    "hẻm 13 . đường sơn thông . k10. p9" : "alley 13.son thong street.k10.p9",
    "jacundá" : "jacket",
    "santa ana 2 mza 24 casa 5" : "santa ana 2 mza 24 house 5",
    "vasai virar" : "slap",
    "são josé do rio preto" : "sao jose do rio preto",
    "siko" : "elbow",
    "são paulo   jundiai" : "sao paulo jundiai",
    "subkowy" : "subdural",
    "islas de la bahia" : "bay islands",
    "caviye" : "poems",
    "קרית מוצקין" : "kiryat motzkin",
    "muntinlupa ciy" : "muntinlupa city",
    "12 a âu cơ , q.tân phú" : "12 a au co, tan phu district",
    "sirhind" : "sirlonging",
    "pasto" : "meal",
    "jaraguá do sul" : "south jaragua",
    "emiliano zapata, ver" : "emiliano zapata, see",
    "caçapava" : "hunt",
    "karawang, west java" : "karang, to go to west",
    "chehalis" : "shalis",
    "warminister" : "warminster",
    "via gregorio ricci curbastro, 29" : "via gregorio ricci curbaster, 29",
    "mundo novo" : "new world",
    "bắc ninh" : "bac ninh",
    "warminsko mazurskie" : "warminsko masurian voivodeship",
    "paraiba" : "paraíba",
    "via garibaldi,9" : "via garibaldi, 9",
    "zagazig" : "titillate",
    "são carlos,sp" : "são carlos, sp",
    "zalec" : "heal",
    "tarwara" : "we are sick",
    "tiruppur" : "twell",
    "lunawada" : "avada",
    "tanda" : "mark",
    "гр. хасково" : "haskovo",
    "jaragua do sul" : "south jaragua",
    "luzern" : "lucerne",
    "fortaleza ceara" : "fortaleza, ceara",
    "k̤o̤l̤k̤a̤t̤a̤" : "k̤oul̤k̤a̤̤̤a",
    "avenida independencia 1297" : "independence avenue 1297",
    "بابل" : "babylon",
    "tân an càng long trà vinh" : "tan an more long tra vinh",
    "udupu" : "stump",
    "madhapur" : "madapur",
    "melgar  tolima" : "melgar tolima",
    "1530 kurunji strt" : "1533",
    "vitoria espírito santo" : "vitoria, espirito santo",
    "espumoso" : "sparkling",
    "plewiska" : "plew",
    "alto taquari" : "high taquari",
    "ceara" : "wax",
    "darsi" : "give up",
    "katihar" : "mytihar",
    "king fahd rd, al olaya,  riyadh 11564, saudi arabia" : "king fahd red, alia, riad 11564, saudi arabia",
    "الحمام" : "the bathroom",
    "alkhobor" : "alhobor",
    "jauru" : "joru",
    "tantipara .barahnagar" : "tantipada.",
    "al wazarat" : "the ministry",
    "puerto deseado   santa cruz" : "puerto deseado santa cruz",
    "lajeado rs" : "laxed rs",
    "ahmed rajabli baku" : "ahmed rajal bey",
    "tambau" : "to sleep",
    "البصره" : "vision",
    "kumbazha" : "digel",
    "1820 chotti bajaria garha" : "1820 chitti bazaria gadha",
    "florianopilis" : "florianopilic",
    "940 redna terrace" : "940 regular terrace",
    "crtagena" : "cartagena",
    "sumidouro" : "sink",
    "dharmapuri" : "dharmapurai",
    "pinheiros" : "pine trees",
    "el nido, palawan" : "the nest, palawan",
    "bengaluru urban" : "bangalore urban",
    "vadodara   390002" : "donarasa 390002",
    "roxana" : "roxane",
    "sao bernardo do campo" : "são bernardo do campo",
    "shivamogga" : "shimoga",
    "san juan. capital" : "san juan.capital",
    "patna , kankarbagh, bihar" : "patna, kankarbagh, bihar",
    "topeka" : "stopy",
    "providencia" : "providence",
    "bhende sk" : "bhende ske",
    "wai" : "water",
    "barranca bermeja" : "barrancabermeja",
    "31 tân trụ, phường 15, q. tân bình" : "31 tan tru, ward 15, district.tan binh",
    "surco" : "groove",
    "não tem" : "it does not have",
    "istanbull" : "istanbul",
    "l'hay les roses" : "l'ho les roses",
    "vĩnh long" : "vinh long",
    "tp thủ  đức , tp hcm" : "thu duc city, ho chi minh city",
    "alvorada do sul" : "southern dawn",
    "pulivendula" : "i'm cleaning up",
    "pimpalner" : "pimpner",
    "hòa bình" : "peace",
    "đồng nai" : "dong nai",
    "warzno" : "brew",
    "rs uruguaiana" : "uruguaiana hospital",
    "siam piwat" : "ciam",
    "bình thuận" : "binh thuan",
    "rio brilhante   ms" : "brilliant river ms",
    "phường tân kiểng, quận 7, tp.hcm" : "tan kieng ward, district 7, ho chi minh city",
    "puerto vallarts" : "puerto vallarta",
    "rio de janeiro rj" : "rio de janeiro - rj",
    "17876 st clair ave.  cloud it" : "17876 st clair ave.  cloud it",
    "khandwa" : "khandva",
    "mindoro" : "mindro",
    "almadinah" : "city",
    "las pinas" : "the pine",
    "الزرقاء" : "blue",
    "lubelskie" : "lublin",
    "ghhv" : "ghahva",
    "กรุงเทพ" : "bangkok",
    "3655 nw 87 ave  cruise" : "3655 nw 87th ave cruise",
    "ponta grossa pr" : "ponta groro pr",
    "الخرج الدلم" : "al -kharj al -dalam",
    "6900 alma dr" : "6900 apples dr",
    "مصر  المنيا  بني مزار" : "egypt minya bani mazar",
    "21 lạc long quânp8tân bình" : "21 lac long quanp8 tan binh",
    "girne (kyrenia)" : "girne (kynie)",
    "friarland" : "freeland",
    "chatrapur" : "chhatarpur",
    "albasrh" : "vision",
    "carrera 69 b no. 98a 31   bogota" : "carrera 69 b no.98a 31 bogota",
    "campo grande   mato grosso do sul" : "campo grande mato grosso do sul",
    "tp quy nhơn, bình định" : "quy nhon city, binh dinh",
    "palari" : "hat",
    "كوم حماده" : "kom hamada",
    "imus, cavite" : "we go, cavite",
    "poznań i okolice" : "poznań and its surroundings",
    "chikhli" : "the subsi",
    "cachoeiras de macacu" : "macacu waterfalls",
    "belçika" : "belgium",
    "são bernardo" : "st bernard",
    "cruzeiro do sul" : "southern cruise",
    "kharghar" : "external",
    "brr alto bosque trv 52 #21d 26" : "brr alto trv 52 #21d 26",
    "sao paull" : "sao paulo",
    "timbauba pe" : "timbauba on",
    "kura place suva" : "kura cries suva",
    "kroi i bardhe no14 prishtine kosove" : "white spring no14 prishtina kosovo",
    "arcoverde" : "green arch",
    "malumichampatty,  coimbatore" : "malumchathatty, codutue",
    "717 general booth blvdvirginia beach, va 23451, usa" : "717 general booth blvd virginia beach, va 23451, usa",
    "sardulgarh" : "sarduhgarh",
    "cuernavaca,  edo. de morelos" : "cuernavaca, edo.of morelos",
    "mỹ phú. mỹ phong mỹ tho tiềng giang" : "my phu.my phong my tho tien giang",
    "51 thống nhất, p10, gò vấp" : "51 thong nhat, p10, go vap",
    "bali , badung" : "bali, badung",
    "taboão da serra" : "taboao da serra",
    "igarassu   pernambuco" : "igarassu pernambuco",
    "jhunjhunun" : "jhunjhunu",
    "7 bình quới" : "7 vase",
    "cogliate" : "they pick up",
    "kfar saba" : "kafr saba",
    "kovvur" : "kovoor",
    "canelones" : "cannelloni",
    "مطروح" : "proportional",
    "narsapur" : "narasapur",
    "via  g  marconi  115" : "via g marconi 115",
    "afyon" : "opium",
    "freiburg im breisgau" : "freiburg in breisgau",
    "pragathi nagar" : "pragati nagar",
    "eusébio" : "eusebius",
    "bangalore , karnataka" : "bangalore, karnataka",
    "бургас" : "burgas",
    "piedras blancas" : "white stones",
    "rillieux la pape" : "rillieux la pope",
    "taoru" : "towel",
    "schriever space force base" : "schlie space force base",
    "minatitlan" : "interest",
    "koregaon" : "the koregaon",
    "5700 lakewright drive suite 300 other" : "5700 lake wright drive suite 300 other",
    "jeddaj" : "jeddah",
    "brovello carpugnino" : "bovello carpugnino",
    "hà nội" : "hanoi",
    "morges" : "morning",
    "bagalkot" : "bang",
    "via dell'industria,3" : "via dell'industria, 3",
    "ciudad de santa fe" : "santa fe city",
    "saint gilles croix de vie" : "saint gilles cross of life",
    "ilha de itamaracá" : "itamaracá island",
    "algeri" : "algiers",
    "mangaratiba rj" : "mangratiba raja",
    "daudnagar." : "daudnagar",
    "مكة المكرمة" : "makkah",
    "từ sơn bắc ninh" : "from son bac ninh",
    "bb@72#biplab" : "bb@72#bbiplab",
    "صنعاء" : "sana'a",
    "września" : "september",
    "marthandam, kanniyakumari" : "marthandam, kanniakumari",
    "repuvlica dominicana" : "dominican republic",
    "diyarbakır" : "diyarbakir",
    "formiga" : "ant",
    "são paulo, são caetano do sul" : "sao paulo, sao caetano do sul",
    "garça" : "heron",
    "lucas do rio verde   mato grosso" : "lucas do rio verde mato grosso",
    "rua turmalina 543" : "tourmaline street 543",
    "olivos" : "olives",
    "15 phan huy thực phường tân kiểng" : "15 phan huy thuc tan kieng ward",
    "15 ấp 6b vĩnh lộc a bình chánh" : "15 hamlet 6b vinh loc a binh chanh",
    "kyrenia" : "souvenir",
    "samdari balotra  badmer" : "sandari balotra barmer",
    "baqdad" : "baghdd",
    "la sarraz" : "unbroke",
    "çameli" : "cameli",
    "kuj.pom." : "also.pom.",
    "chapadão do céu   goias" : "chapadão do céu goias",
    "centro empresarial e industrial de loulé, zona poente, lote 11b" : "loulé business and industrial center, long zone, lot 11b",
    "prueba" : "proof",
    "ras tanura" : "mighted raas",
    "barra do garça mt" : "mt garça bar",
    "ogun state" : "the state war",
    "alwaye" : "he is still",
    "ka gd f" : "the gd f",
    "furrkhabad" : "farrukhabad",
    "cachoeira do piriá" : "piriá waterfall",
    "campo largo" : "long field",
    "recife.    pernambuco" : "recife.pernambuco",
    "san mateo" : "saint matthew",
    "مدينة نصر   القاهرة" : "nasr city, cairo",
    "assis chateaubriand" : "seated chateaubriand",
    "chełm" : "helmet",
    "foz do iguacu   parana" : "foz do iguacu parana",
    "san juan opico" : "san juan monkey",
    "dantiago    recoleta" : "santiago recoleta",
    "3865 e 34th st suite 105, tucson, az 85713" : "3865 is zchtht otse 105, tutzon, i 85713",
    "campinas  sp" : "campinas, sp",
    "جازان محافظة العارضة" : "jazan al -ardah governorate",
    "manzana g casa 2" : "apple g house 2",
    "rua sud?rio maximiano rocha, n 123, bloco 4 ap 202 mg" : "rua maximiano rocha street, n 123, block 4 ap 202 mg",
    "pará." : "for.",
    "thiruvarur" : "tiruvarur",
    "ิกรุงเทพ" : "the gods",
    "nezahualcóyotl  estado de mexico" : "nezahualcóyotl state of mexico",
    "primavera do leste   mt" : "spring of the east mt",
    "ji parana" : "zee parna",
    "ranaghat" : "ranaaghat",
    "birkirkara" : "well",
    "quinto di treviso" : "fifth of treviso",
    "cra32#110 11" : "sharia#110 11",
    "guara   cidade do automovel" : "guara city of automobile",
    "warszaw" : "warsaw",
    "rua santa vitoria 663 minas gerais" : "santa vitoria street 663 minas gerais",
    "القليبوية" : "qalibiya",
    "אזור" : "area",
    "bahía de banderas" : "bay of flags",
    "tphcm" : "hcmc",
    "công ty cdc" : "cdc company",
    "4b mỹ hoà 1,trung chánh, hóc môn" : "4b my hoa 1, trung chanh, hoc mon",
    "h.no 807,lila chowk old abadi lila dharmsaala ,old abadi" : "h.no ౮౦౭, lila chowk old body leela dharamsala, old body",
    "shireer aurangabad" : "shirir aurangabad",
    "itliy" : "lever",
    "bahia blanca" : "white bay",
    "243a  đê la thành" : "243a de la thanh",
    "leverano" : "levers",
    "sor juana ines de la cruz 555 abcd" : "sister juana ines of cruz 555 abcd",
    "baihata chariali" : "behta chariali",
    "البصرة" : "basra",
    "porto seguro" : "safe harbor",
    "benglore" : "bullock",
    "biên hoà" : "bien hoa",
    "tân phú, tp hồ chí minh" : "tan phu, ho chi minh city",
    "niterói rj" : "nitero rj",
    "tuljapur" : "tulajapur",
    "mandideep" : "manitideep",
    "blk iii, cez ii, gen. trias cavite" : "blk iii, via ii, gen.trias cavite",
    "kazhakuttam, trivandrum" : "kazhakutam, trivandrum",
    "teki̇rdağ" : "in",
    "kattappana" : "katappana",
    "se  barra dos coqueiros" : "if barra dos coqueiros",
    "chapra  bihar" : "chhapra bihar",
    "wallaroo" : "pair",
    "bonfinopolis" : "bonfinopol",
    "mazatlan, sin." : "mazatlan, without.",
    "9 landsdowne streetbostonma2215" : "9 lansdowne street boston ma 2215",
    "autoput za novi sad 71" : "novi sad highway 71",
    "sao jose dos campos" : "são josé dos campos",
    "ślaśkie" : "silesian",
    "יד בנימין" : "hand benjamin",
    "santa cruz de sierra  bolivia" : "santa cruz de sierra bolivia",
    "bengalure" : "bangalore",
    "salalah" : "stalemate",
    "ujjain" : "fingers",
    "tanuku" : "tension",
    "capivari de baixo. sc" : "capivari from below.sc",
    "blvr kukulkan zona hotelera km 14.5 jw marriott cancun" : "blvr kukulkan area hotel km 14.5 jw marriott cancun",
    "ceranesi" : "cerane",
    "i̇stanbul" : "istanbul",
    "caoro" : "carro",
    "biharsharif" : "bihar sharif",
    "long biên hà noiij" : "long bien ha noiij",
    "medelin" : "medium",
    "sirkali" : "vermillion",
    "عمان" : "oman",
    "شبين الكوم المنوفيه" : "shebin el -kom menoufia",
    "nabga" : "navaga",
    "mansoura" : "manoura",
    "mazowiecie" : "masovian",
    "ganaur sonipat haryana" : "ganore sonipat haryana",
    "prakasam" : "aura",
    "via g.cesare n.54" : "via g.ceso n.54",
    "jau" : "already",
    "rio brilhante" : "brilliant river",
    "pr cascavel" : "pr rattlesnake",
    "ramat gan" : "ramat gain",
    "presidente getúlio sc" : "president getúlio sc",
    "marica  rio de janeiro" : "marica rio de janeiro",
    "la mirada" : "the look",
    "rytwiany" : "rytwany",
    "pappelweg 4" : "papel road 4",
    "metropolitana" : "metropolitan",
    "via e. de amicis, 23 . 90044 carini (pa)" : "away and.de amicis, 23.90044 carini (pa)",
    "cycleworld, hosa road juncrion" : "sisilebold, jose road junkrin",
    "guasave" : "guasuave",
    "milagro" : "miracle",
    "ديالى" : "diyala",
    "2310 east el segundo blvd" : "2310 east the second blvd",
    "navi mumbai, ghansoli" : "navi mumbai, ghansali",
    "campina grande   pb" : "campina grande pb",
    "sandur" : "sand",
    "navsari" : "nervous",
    "1120 south tyron street, ste 700  medical" : "1120 south tryon street, ste 700 medical",
    "beafort" : "beaufort",
    "6f 1, no.288 5, xinya rd., qianzhen dist., kaohsiung city 80673, taiwan" : "6 rounds 1, no.288 5, new asia tropical. on the third day of the previous array., kaohsiung city 80673, taiwan",
    "浙江省宁波市" : "ningbo city, zhejiang province",
    "h15 tổ 9 kp2 long bình tân biên hòa" : "h15 group 9 kp2 long binh tan bien hoa",
    "kolkatta" : "kolkattat",
    "hubballi" : "giggle",
    "mirjapur" : "mirzapur",
    "jyoraha" : "jyraha",
    "محافظة القاهرة" : "cairo governorate",
    "1305 n holopono st suite 3" : "1305 n juice stuite 3",
    "puttur manalore" : "puttur is ours",
    "narasaraopet" : "narsaraipet",
    "1375 n buena vista dr  commercial display" : "1375 n buena vista dr commercial display",
    "são apulo" : "são paulo",
    "buxar" : "steam",
    "المنصوره" : "asan",
    "estación central" : "central station",
    "صحار" : "sohar",
    "keonjhar" : "odor",
    "orindiuva sp" : "orinduva sp",
    "teresina   pi" : "teeasia pi",
    "chur" : "put",
    "al medhatiya" : "the praise",
    "city of malolos (capital)" : "city of malolos (capatial)",
    "jamtara" : "guess",
    "santa cruz de tenerife" : "santa cruz of tenerife",
    "drabhanga" : "dranska",
    "ikorodu" : "illuminating",
    "karbalaa" : "carquel",
    "kaltim" : "east kalimantan",
    "jacobina" : "jacobin",
    "sonepur" : "sonpur",
    "rialp" : "laugh",
    "குளித்தலை" : "bath head",
    "espirito santo" : "holy spirit",
    "marilandia es" : "marilandia is",
    "lapu lapu city 6015, cebu" : "lapa lap city 6015, cebu",
    "21 calle 7 41 zona 11, mariscal" : "21 street 7 41 zone 11, marshal",
    "attur" : "actor",
    "av. livertador 1285 cochabamba bolivia" : "av.freeder 1285 cochabamba bolivia",
    "auarangabad" : "aurangabad",
    "tapejara  rs" : "tapejara rs",
    "sutton" : "suton",
    "liên chiểu   đà nẵng" : "lien chieu da nang",
    "belem" : "white",
    "puente alto" : "high bridge",
    "itu sp" : "that's sp",
    "222, gurmukh jeevan,11th road, nxt to hinduja hospital,khar (w), maharashtra" : "222, gurmukh jeevan, 11 th road, destroy then hinduja hospital, khar (v), maharashtra",
    "adare manor hotel" : "but the manor hotel",
    "fazenda rio grande (pr)" : "rio grande farm (pr)",
    "tandwa,chatra" : "tandwa, student",
    "aracaju sergipe" : "aracaju - sergipe",
    "bongaon" : "bone",
    "el espinal tolima" : "the tolima espinal",
    "tehuacan  puebla  mexico" : "tehuacan puebla mexico",
    "niterói  rj" : "nitero rj",
    "٦ اكتوبر" : "october 6",
    "florianopois" : "florianop",
    "via deserto 101" : "via desert 101",
    "πατησιων 27" : "patision 27",
    "lima, perú" : "lima peru",
    "sri ganganagarj" : "shri gangnagarz",
    "khu nhà ở phú hồng thịnh, dĩ an, bd" : "phu hong thinh, di an, bd housing area",
    "são miguel do iguaçu   paraná" : "são miguel do iguaçu paraná",
    "murrumbateman" : "murgerumbateman",
    "jakarta barat" : "west jakarta",
    "calle 41 n° 27   79 primer piso" : "41th street no. 27 79 first floor",
    "chaibasa" : "laid",
    "fortaleza ceará" : "fortaleza, ceara",
    "lajas" : "slabs",
    "vinhedo" : "vineyard",
    "la grande" : "the big one",
    "bhadohi, u.p ,india" : "bhadohi, u.p., india",
    "recife   pernambuco" : "recife pernambuco",
    "konstanz" : "consistency",
    "islampur. uttar dinajpur" : "islampur.north dinajpur",
    "via del commercio 9" : "via del commerce 9",
    "diinópolis" : "diinopolis",
    "ratia" : "rat",
    "chennak" : "to chen",
    "hutti" : "incarcerate",
    "la senia" : "no sunnis",
    "uttarpara, hooghly, west bengal" : "north, houghli, west bengal",
    "calle 16a n 55 64 antioquia" : "calle 16a n 55 64 colombia",
    "mossoró" : "mossoro",
    "deogarh" : "dogarah",
    "banten， tangerang selatan" : "banten ， south tangerang",
    "lubeldkie" : "lubeldke",
    "azowieckie" : "masovian",
    "ҭ. ڬܬݬ ѭѭ ҭ.18" : "ҭ.ڬ ܬ ݬ ѭѭ ҭ.18",
    "brotas" : "sprout",
    "são bernardo do campo   sp" : "são bernardo do campo sp",
    "kothaguda, kondapur" : "kottaguda, kondpur",
    "são paulp" : "são paulo",
    "palu" : "hammer",
    "cao lãnh" : "cao lanh",
    "ฉะเชิงเทรา" : "chachoengsao",
    "kasoa" : "kao",
    "tucano bahia" : "bahia toucan",
    "honnudike" : "hount",
    "vitorino freire  ma" : "vitorino freire ma",
    "são vicente" : "saint vincent",
    "santa rosa   rs" : "santa rosa rs",
    "nhị quí cai lậy tiền gianh" : "nhi quy cai lay tien giang",
    "kaohsiung" : "gooohining",
    "144 w 133rd sr apt 1" : "144 w 133rd st apt 1",
    "yamuna nagar" : "yam nagar nagar",
    "contagem" : "score",
    "tarumã   sp" : "tarumã sp",
    "edi̇rne" : "normally",
    "magpur" : "nagpur",
    "bhubaneswar" : "bluewar",
    "vila velha   es" : "vila velha es",
    "picedo" : "piced",
    "washougal" : "washogal",
    "kanpur dehat" : "kanpur countryside",
    "lonavla" : "lonavala",
    "são paulo  ' sp" : "sao paulo-sp",
    "panamá city" : "panama city",
    "yanbu" : "eye",
    "łobodno" : "lobby",
    "sanchez mira" : "sanchez look",
    "dokki" : "poisoning",
    "uthangarai ,krishnagiri" : "utangarai, krishnagiri",
    "nova cruz" : "new cross",
    "viale premuda 12" : "viale pressed 12",
    "עשרת" : "ten",
    "pilibanga" : "the pilot",
    "van nuys" : "van nys",
    "sangli" : "sangli is the best.",
    "são paulo   sp." : "sao paulo-sp.",
    "juarez" : "juerez",
    "ποσειδώνος 4 χαλάνδρι" : "poseidon 4 chalandri",
    "umaria" : "maria",
    "baymont inn 2731 watson bld" : "baymont inn 2731 watson blvd",
    "mundra" : "the man of muman",
    "barreiras" : "barriers",
    "são paulo americana" : "american sao paulo",
    "ras al khaimah gg" : "the head of the tent c",
    "jequié  bahia" : "jequié bahia",
    "cao  lãnh" : "cao lanh",
    "hariharpara" : "hariharpada",
    "damnhour" : "damanhour",
    "sjcampos" : "seacamp",
    "laspinas" : "laspin",
    "arnenia quindío" : "armenia quindio",
    "chattogram" : "chittagong",
    "toluca estado de méxico" : "toluca state of mexico",
    "koraput" : "curtain",
    "جدة" : "grandmother",
    "almodhaibi" : "al -mudhaibi",
    "sunam" : "call",
    "eslam mahdi@hotmail.com" : "islam mahdi",
    "gautang" : "loan",
    "بغداد" : "baghdad",
    "marina del rey" : "navy of the king",
    "kalyoubiyah" : "alkalibi",
    "opa locka" : "opa lock",
    "100 đinh bộ lĩnh p26  q.bình thạnh" : "100 dinh bo linh p26 district binh thanh",
    "merida, yucatan" : "merida, yukatan",
    "shirakawa shi" : "shirakawa",
    "al kharj" : "the outside",
    "la jolla" : "the jolla",
    "varanasí" : "varanza",
    "الجيزة" : "giza",
    "matrouh" : "matrosh",
    "taxila" : "taxilas",
    "moorsville" : "mooresville",
    "via degli allori, 79" : "via degli lapel, 79",
    "shope no 3, mangalwai complex" : "shop no 3, mangalwari complex",
    "zalka" : "zalk",
    "cuerámaro gto méxico" : "gto mexico cuerramaro",
    "bhadrak" : "gentleman",
    "aydın kuşadası" : "aydin kusadasi",
    "chourai" : "shuy",
    "ethachlml" : "somehow",
    "calle larga" : "long street",
    "canela" : "cinnamon",
    "10, ql13, hiệp bình phước" : "10, ql13, hiep binh phuoc",
    "itajaí   santa catarina" : "itajaí santa catarina",
    "rangia" : "rangi",
    "salto" : "heel",
    "serra es" : "serra is",
    "marlloca, espanha" : "mallorca, spain",
    "nala sopara" : "subternair",
    "gadarwara" : "bridge",
    "azamgarh tarwa uttar pradesh 276123" : "azamgarh tarawa uttar pradesh 276123",
    "dheerpura" : "dhirpura",
    "paithan" : "penetration",
    "oaxaca" : "oxaca",
    "puerto vallarta" : "vallarta port",
    "región de coquimbo" : "coquimbo region",
    "centro direzionale milanofiori," : "milanofiori disional center,",
    "beba" : "baby",
    "phulbani" : "fulbani",
    "monclova , coahuila" : "monclova, coahuila",
    "toluca, edo mex" : "toluca, or mex",
    "virgem da lapa" : "virgin da lapa",
    "1175, chiêu liêu bình dương" : "1175, lieu binh duong",
    "al sulay" : "al try",
    "são carlos   sp" : "são carlos sp",
    "des moines" : "monks",
    "pfalzen" : "pfallel",
    "haldibari" : "is l -bali",
    "mossoro" : "mossorus",
    "fonte nuova" : "new source",
    "shahajahanpur" : "shahjahanpur",
    "el obour" : "transit",
    "vitória da conquista" : "victory of the conquest",
    "dozza" : "doness",
    "meteti" : "mete",
    "soro" : "difficult",
    "tiruchirappalli" : "turuchir them",
    "haveri" : "breakdown",
    "cd. júarez" : "cd.júarez",
    "pydibhimavaram" : "paidibhimavaram",
    "mazatlan" : "matlan",
    "isranbul beykoz" : "istanbul beykoz",
    "vengurla" : "vetearla",
    "in 46601, ee. uu." : "in 46601, in.his.",
    "kab.tangerang, gdg serpong" : "kab. tangerang, gading serpong",
    "conselheiro lafaiete" : "counselor lafaiete",
    "akwa ibom" : "immoral",
    "1 travessa francisco pereira, pitua?i bahia" : "1 travessa francisco pereira, pitua? i bahia",
    "shindkheda" : "shirtink",
    "riftvalley" : "rift valley",
    "tabuk" : "taboos",
    "350 ne 24st apt 1105" : "350 not 24th apt 1105",
    "general santos" : "santo",
    "rua keisuke koza, 189 s?o paulo" : "keisuke lua, 189 s?power",
    "sta maria bulacan" : "maria bulacan stays",
    "são josé de ribamar ma" : "sao josé de ribamar ma",
    "468 phan văn trị phường 7 gò vấp" : "468 phan van tri ward 7 go vap",
    "antwerpen" : "antwerp",
    "bengaluru" : "bangalore",
    "campo grande" : "large field",
    "santarem para" : "santarem to",
    "barra de são francisco" : "san francisco barra",
    "villeta" : "villet",
    "maravilha" : "wonder",
    "الكويت" : "kuwait",
    "itarumã" : "istarumã",
    "punjab sailkot" : "punjab salakot",
    "bulchandi.malda" : "bunkanki.al",
    "kesariya" : "saffron",
    "presidente getúlio" : "president getúlio",
    "bharuwa sumerpur" : "bharua sumerpur",
    "mineiros" : "miners",
    "ribeirão pires   sao paulo" : "ribeirão pires sao paulo",
    "mummy ki" : "mummy to",
    "shivmogga" : "to the shivogate",
    "lubelskie lublin" : "lublin lublin",
    "distrito nacional" : "national district",
    "chakan pune" : "check the spectrum",
    "los jagüeyes, namiquipa, chihuahua" : "the jagüeyes, namiquipa, chihuahua",
    "san borja" : "san borje",
    "jubail saudi arabia" : "jubel saudi arabia",
    "keshena" : "cereal",
    "vle roma,4" : "want roma, 4",
    "6 ấp 4, nhơn đức, nhà bè" : "6 hamlet 4, nhon duc, nha be",
    "hanamkonda" : "he hasn't crushed him",
    "murau" : "the region",
    "kenitra" : "chest",
    "manab" : "human",
    "wielbrandowo" : "molbrandowo",
    "radom" : "by working",
    "mardi̇n" : "marḋn",
    "muddebihal" : "mudd is more",
    "campos dos goytacazes,rj" : "campos dos goytacazes, rj",
    "novi ligure" : "new liguers",
    "pontalina" : "pontaline",
    "vitória" : "victory",
    "נהריה" : "nahariya",
    "25 อาคารมูฟอะเมซ ชั้น 4 ห้อง 4b" : "25 movez building, 4th floor 4b",
    "san jose del guaviare" : "san jose of guaviare",
    "الأحساء" : "hasa",
    "cordoba, ver" : "cordoba, see",
    "cd. obregon" : "cd.obregon",
    "puerto montt" : "montt port",
    "marinduqe" : "maruqe",
    "salvador   bahia" : "salvador, bahia",
    "aparecida" : "appeared",
    "san carlos" : "saint charles",
    "vijaywada" : "vijayawada",
    "ontario" : "ontaria",
    "الخبر" : "the news",
    "ciudad de panama" : "city ​​of panama",
    "kahului" : "tickets from kahului",
    "guarulhos   são paulo" : "guarulhos sao paulo",
    "באר שבע" : "beer sheva",
    "jabel" : "jelly",
    "1 skyview drfort worthtx76155" : "1 skyview dr fort worth tx 76155",
    "srungavarapukota, vizianagaram" : "shringwarpukota, vijayanagaram",
    "srinagar j&k" : "srinagar j & k",
    "ابوظيي" : "abuzia",
    "nanded." : "nanded",
    "maricá" : "sissy boy",
    "medchal" : "gambling",
    "quenzon city" : "quezon city",
    "lahar" : "lava",
    "catas altas" : "high tastings",
    "سكاكا" : "sakaka",
    "kudal" : "fences",
    "estado de méxico" : "mexico state",
    "sp franca" : "sp frank",
    "đà nẵng" : "danang",
    "quận 2 sài gòn" : "district 2 saigon",
    "para" : "for",
    "angul" : "corner",
    "via caldaia 17" : "via boiler 17",
    "403 quốc lộ 62  tp tân an long an" : "403 national highway 62 tan an long an city",
    "aydın" : "clear",
    "pueblo libre" : "free town",
    "609 medical center dr. decatur, texas, 76234" : "609 medical center dr. decatur, texas, 76234",
    "ichapuram" : "chick",
    "casa branca" : "white house",
    "ouezon" : "loss of planting",
    "devakottai" : "devakotai",
    "paulinia" : "paulinaia",
    "جيزان" : "jizan",
    "tambogrande" : "tambogging",
    "ourinhos" : "outer",
    "dubay" : "gray",
    "กทม." : "bangkok",
    "اسوان" : "aswan",
    "zakrzew" : "zakhil",
    "carrera 80 # 63b  55 cundinamarca" : "carrera 80 # 63b 55 cundinamarca",
    "viale trento ,20" : "viale trento, 20",
    "vũng liêm" : "vung liem",
    "καποδιστριου 27" : "kapodistriou 27",
    "sete barras" : "seven bars",
    "dharpally dist nizamabad" : "dharpalli dish is nizamabad",
    "sykesville" : "heart rate",
    "via bosco n. 9" : "via bosco n.9",
    "avenida 3c # 45 100" : "3c # 45 100 avenue",
    "bùi xương trạch thanh xuân hà nội" : "bui xuong trach thanh xuan hanoi",
    "kailua" : "halewais",
    "ne" : "it is",
    "partur" : "portion",
    "l'anse" : "the handle",
    "cruzeiro do sul   acre" : "southern cruzeiro acre",
    "piduguralla" : "pigrore",
    "etawah" : "elevator",
    "matamoro" : "math",
    "vadgaon" : "vadga",
    "mpumalanga" : "east",
    "lika" : "equal",
    "são mateus" : "saint matthew",
    "harta" : "assets",
    "katoria" : "android",
    "são paulo, arandu" : "sao paulo, arandu",
    "sobótka" : "bonfire",
    "trà vinh" : "tra vinh",
    "tustin" : "pain",
    "ciudad autónoma de buenos aires" : "autonomous city of buenos aires",
    "sahranpur" : "saharanpur",
    "paraíba joão  pessoa" : "paraíba joão pessoa",
    "aquiraz" : "aquiiraz",
    "mato grosso   cuiaba" : "mato grosso cuiaba",
    "novo cruzeiro   m g" : "new cruise m g",
    "siddipet" : "siddipate",
    "الشرقيه" : "eastern",
    "sc   jaraguá do sul" : "sc jaraguá do sul",
    "台中" : "taichung",
    "sanguem" : "face to face",
    "hawalli" : "around",
    "新骏环路" : "xinjun ring road",
    "101 gosai dr" : "101 later dr",
    "encantado" : "charmed",
    "istanbul   büyükcekmece" : "istanbul büyükcekmece",
    "kogarah" : "kagarah",
    "bijapur   vijayapura" : "bijapur vijaypura",
    "belford roxo" : "purple belford",
    "gurgaoun" : "gurgaon",
    "udumalpet" : "dumalpet",
    "caratinga   mg" : "karatinga mug",
    "vitoria da conquista" : "victory of the conquest",
    "sasthamcotta" : "mammalian",
    "joenvile" : "joinville",
    "99 định công hạ" : "99 dinh cong lower",
    "charqueadas" : "pitty",
    "el paso" : "step",
    "miryalaguda" : "miyalaguda",
    "othon p. blanco" : "othon p.blanch",
    "dadri" : "the",
    "hulhumale" : "humid",
    "ki tuc xa khu a, dhqg ho chi minh" : "ki tuc far from a, dhqg ho chi minh",
    "jr, alemania 2485" : "jr, germany 2485",
    "campo largo   paraná" : "long field paraná",
    "itajobi" : "jobi",
    "maglio" : "mallet",
    "ρουσβελτ 18" : "roussvelt 18",
    "kemaman" : "kember",
    "los lagos" : "the lakes",
    "camarines sur philippines" : "camarines on philippines",
    "dehra dun" : "a good man anton",
    "lagos, surulere" : "lagos, surulate",
    "beirut" : "beir",
    "seoni" : "sony",
    "elazığ" : "elazig",
    "arujá" : "aruja",
    "عدن" : "aden",
    "جيزة" : "giza",
    "ấp vĩnh phú xã vĩnh thạnh" : "vinh phu hamlet, vinh thanh commune",
    "ciudad e méxico" : "mexico city",
    "palompoj" : "palombo",
    "marília" : "marilia",
    "64 nguyễn thị tần p2 q8" : "64 nguyen thi tan p2 q8",
    "mysuru" : "my",
    "macul" : "the brave",
    "eranakulam" : "ernakulam",
    "bagepalli" : "baking palli",
    "barueri   sp" : "barueri sp",
    "são paulo, sp" : "sao paulo-sp",
    "altamont springs" : "altamonte springs",
    "hamilton on" : "hamilton is",
    "dakor" : "significance",
    "united kindgom" : "united kingdom",
    "piotrków trybunalski" : "piotrkow trybunalski",
    "مصر" : "egypt",
    "kuthuparamba" : "to sweat",
    "asturias   gijon" : "asturias gijon",
    "المحلة الكبرى" : "mahalla al -kubra",
    "sindagi" : "inferior",
    "cao lanh" : "kaolin",
    "minamisatsuma" : "southern satsuma",
    "villa maria   cordoba" : "villa maria cordoba",
    "jantral" : "jaralal",
    "دبي" : "dubai",
    "chợ mới" : "new market",
    "uruguaiana" : "uruguayan",
    "khordha" : "khodha",
    "lince   lima" : "lince lima",
    "800 3rd ste nw" : "800 width six nou",
    "balneário camboriú   sc" : "balneário camboriú sc",
    "กม" : "km",
    "goiânia   go" : "goiânia go",
    "via xxv ottobre,  4" : "via xxv october, 4",
    "puerto escondido" : "hidden port",
    "taliparamba" : "respondaramba",
    "phuket" : "fuket",
    "channapatna" : "channapana",
    "ulus mah. manolya cad. no:16" : "ulus mah.magnolia cad.no: 16",
    "bogotá, colombia" : "bogota colombia",
    "nova iguaçu   rj" : "nova iguaçu rj",
    "alangulam" : "alongulam",
    "balangir" : "forbidden",
    "san angelo" : "as an angel",
    "santa catarina em itapema" : "santa catarina in itapema",
    "surar" : "chapter",
    "chikkamagaluru" : "ukuru",
    "cornaredo" : "cornado",
    "الدقهلية" : "dakahlia",
    "concepción" : "conception",
    "jaypur" : "jaipur",
    "piauí   pi" : "piauí pi",
    "candiolo" : "cadlo",
    "الجهراء" : "jahra",
    "méxico city" : "mexico city",
    "draria" : "drarea",
    "สมุทรสาคร" : "samut sakhon",
    "arco (tn)" : "arc (tn)",
    "55 camino aguajito" : "55 aguajito road",
    "banihal" : "bees",
    "jaggayyapeta" : "jaggaiahpet",
    "hosapete" : "assistant pente",
    "calle 167a #5a 4" : "calle 167a #kha 4",
    "al madina" : "city",
    "ganaur" : "ganore",
    "são manuel" : "saint manuel",
    "sonnefeld" : "sunfield",
    "stekene" : "stitch",
    "bangkok (กรุงเทพมหานคร)" : "bangkok (bangkok)",
    "quảng nam" : "quang nam",
    "nguyễn xiển thành phố thủ đức" : "nguyen xien thu duc city",
    "27 phường 17 xô viết nghệ tĩnh" : "27 ward 17 soviet nghe tinh",
    "jangareddygudem" : "jangeddygudem",
    "narkatiya ganj" : "narkatia ganj",
    "undri pune" : "underi puts",
    "havre de grace" : "harbour grace",
    "av. doutor alvaro camargos, 1680  mg" : "av.doctor alvaro camargos, 1680 mg",
    "av. guanabara 95 sa8, jardim s?o vicente, itupeva s?o paulo" : "av.guanabara 95 sa8, jardim s? o vicente, itupeva s? o paulo",
    "تربة" : "soil",
    "slaskie" : "silesian",
    "paruthippara" : "cotton",
    "đồng tháp" : "dong thap",
    "kottakkal malppuram" : "kottakkal malappuram",
    "marigot" : "husband",
    "muheillah" : "timing",
    "الرياض" : "riyadh",
    "chuluota" : "out of him",
    "farropilha" : "farroupilha",
    "ilupeju" : "ilupe",
    "waunakee" : "waukyo",
    "uniao da vitoria" : "victory union",
    "250 e sepulveda blvd #1046" : "250 is sepulveda blvd #1046",
    "hamden" : "hammer",
    "st maarten" : "saint martin",
    "rua nossa senhora de lourdes" : "nossa senhora de lourdes street",
    "חדרה" : "hadera",
    "شربين" : "sherbin",
    "arujá   são paulo" : "arujá sao paulo",
    "strzelno" : "shooting",
    "itaporã ms" : "stone ms",
    "بريدة" : "burial",
    "triprayar (valapad)" : "tripreer (valapad)",
    "laguna sta. cruz" : "santa laguna.cross",
    "via 4 novembre, 2" : "via 4 november, 2",
    "sofija" : "sofia",
    "jumaira" : "jumai",
    "são pedro" : "saint peter",
    "armenia, quindío" : "armenia quindio",
    "binan,laguna" : "binan, laguna",
    "mato grosso   alta floresta" : "mato grosso alta floresta",
    "capela do alto" : "chapel",
    "日本" : "japan",
    "ambelakion 28, 4046 germasogia, limassol, cyprus" : "amblakion 28, 4046 germasogia, limassol, cyprus",
    "الظاهره" : "phenomenon",
    "pedra azul" : "blue stone",
    "ירושלים" : "jerusalem",
    "ahemdabad" : "ahmedabad",
    "mohali" : "local",
    "kosli" : "kosla",
    "como" : "as",
    "governador valadares minas gerais" : "governor valadares minas gerais",
    "sp   pirassununuga" : "sp pirassununuga",
    "río negro" : "black river",
    "lima   san borja" : "five san borja",
    "676 n michigan avenue" : "676 road cancer ave nue",
    "kattumannarkoil" : "overlapping",
    "chamarajanagara" : "chamarajanagar",
    "sta. rosa" : "is.rose",
    "cedry wielkie" : "cedry great",
    "hựu thành, trà ôn, vĩnh long" : "huu thanh, tra on, vinh long",
    "kottur" : "cat",
    "bolivar, magangue" : "boli, maganugu",
    "thiruchangkodu" : "church line",
    "7616 benbrook pwky" : "7616 benbrook parkway",
    "gourav bachan" : "gaurav bachan",
    "233 south beaudry avenue  los angeles ca" : "233 south beaudry avenue los angeles ca",
    "oriximiná   pará" : "oriximiná pará",
    "mednine" : "civilians",
    "strada dei palazzi 5d" : "road of the palazzi 5d",
    "lagoa da prata" : "silver lagoa",
    "rewari" : "leimrit",
    "bhayander" : "bayander",
    "lousada" : "sloe",
    "berwyn" : "pillow",
    "botuporanga" : "bowupounga",
    "hyderbad" : "hyderabad",
    "cuidad de mexico" : "mexico city",
    "addis ababa" : "addis short",
    "pabellon de arteaga" : "pabellon de artaaga",
    "voluntari" : "volunteer",
    "اادوحة" : "idah",
    "manaus am ." : "manaus am.",
    "via sempione 27  italia" : "via sempione 27 italy",
    "brejo santo" : "brejo holy",
    "türkiye cumhuriyeti" : "republic of türkiye",
    "kampala  muyenga" : "kampala muyenga",
    "alhasa" : "whiet",
    "favria" : "favory",
    "الخرج" : "output",
    "davao del sur" : "south davao",
    "secha sadan" : "sikad sika",
    "jatai go" : "expressed",
    "rua 15 de novembro, 1262 rs" : "november 15th street, 1262 rs",
    "abudhabi   western region  ghayathi city" : "abu dhabi and sterene, ghiathi city",
    "mahad" : "thanks",
    "mallapur hyderbad" : "mallapur hyderabad",
    "monterrey, méxico" : "monterrey mexico",
    "florianópolis   sc" : "florianópolis sc",
    "recife   pe" : "recife on",
    "são paulo sp" : "sao paulo-sp",
    "calle 89 no 67 136 811014571" : "street 89 no 67 136 811014571",
    "cd mx" : "cdmx",
    "soufriere" : "sufriere",
    "são paulo   são paulo" : "sao paulo sao paulo",
    "indoren" : "indo",
    "chrompet" : "cromarty",
    "εθνικης αντιστασεως" : "national resistance",
    "imus cavite" : "we go to cavite",
    "colgong" : "colong",
    "rio dr janeiro" : "rio de janeiro",
    "curitiba parana" : "curitiba, parana",
    "alamuru" : "opportunity",
    "el salvador" : "the savior",
    "czeladź" : "czeladz",
    "thủ đức, tp hcm" : "thu duc, ho chi minh city",
    "khorasan razavi" : "khorasan razvi",
    "المدينه المنوره" : "al madinah al munawwarah",
    "neily" : "nile",
    "calle 64 b # 1  71" : "calle 64 b # 1 71",
    "eustis" : "welcome",
    "كربلاء المقدسة" : "holy karbala",
    "via vittorio emanuele,17" : "via vittorio emanuele, 17",
    "jawabarat" : "answer",
    "madinat abqaiq" : "madinat abqiqi",
    "nearambedkar hospet" : "nearambakkar hospet",
    "mohania" : "mohinia",
    "θηβων 500 ιλιον" : "thebes 500 ilion",
    "açailândia   ma" : "açailândia ma",
    "boa esperança" : "good hope",
    "po box 701362  san antonio" : "well box 701362 san antonio",
    "quảng ninh" : "quang ninh",
    "5949 via del cielo  cloud it" : "5949 via del cielo cloud it",
    "blvd. miguel aleman n?m. 1101, san mateo atenco edo. de m?xico estado de m?xico" : "blvdmiguel aleman n? m.1101, san mateo atenco edo.of the same state of music",
    "oshawa" : "shawawa",
    "campo novo do parecis" : "parecis novo campo",
    "buenos aires , florida" : "buenos aires, florida",
    "dourados" : "golden",
    "sindekela" : "sindekelala",
    "kaij" : "hallter",
    "duliajan" : "dulia",
    "2500 menaul blvd ne" : "2500 menal blvd not",
    "اسكندرية" : "alexandria",
    "ตาก" : "dry",
    "anuppur" : "transmitter",
    "gurugam" : "gurgam",
    "santos domingo" : "saints sunday",
    "sancristobal" : "sanctobal",
    "ji paraná" : "her paraded",
    "santa cruz   rio gallegos" : "santa cruz rio gallegos",
    "florida 1 piso: 12 ofic: 48   entre: esquina rivadavia   capital federal (1005) buenos aires" : "florida 1 floor: 12 off: 48 between: corner rivadavia capital federal (1005) buenos aires",
    "kaimur bhabuan" : "kaimur bhabhua",
    "playa del carmen" : "carmen beach",
    "401st afsb" : "401st aff",
    "villa alemana" : "german village",
    "bangaluru" : "bangluru",
    "mancherial telangana" : "manancherl telangana",
    "purwkhn" : "prize",
    "atílio vivacqua" : "atilio vivacqua",
    "99 định công hạ  hoàng mai" : "99 dinh cong ha hoang mai",
    "caieiras   são paulo" : "caieiras são paulo",
    "tekirdağ" : "tekirdag",
    "rosário oeste mt" : "rosario west mt",
    "getulina" : "we have it",
    "no.08, sir chittampalam a gardiner mw," : "no.08, sir chittampam gardin mw,",
    "piro" : "score",
    "tandwa" : "struggle",
    "chinsurah" : "chinasur:",
    "vitória da conquista   bahia" : "vitória da conquista bahia",
    "rio de janeiro   rj" : "rio de janeiro - rj",
    "jognda" : "jogna",
    "kodagu" : "kernel",
    "gurgoan" : "gurgaon",
    "6october" : "6 october",
    "lavras" : "mulch",
    "blairsville" : "blairville",
    "begusaria" : "begusariia",
    "gaurulhos" : "guarulhos",
    "kathumar" : "jelum",
    "juan de acosta atlantico" : "juan de acosta atlantic",
    "motihari  chowk" : "motihari chowk",
    "dasuya" : "put out",
    "balneário piçarras" : "balneario piçarras",
    "القاهرة" : "cairo",
    "100 vestavia parkwaybirminghamal35216" : "100 vestavia parkway birmingham al 35216",
    "atlantico" : "atlantic",
    "pompeia, sao paulo" : "pompeii, sao paulo",
    "ponda goa" : "crush goa",
    "la plata" : "at payment",
    "espinar" : "spine",
    "tocantins   palmas" : "tocantins palmas",
    "volta redonda" : "round",
    "agbara" : "power",
    "olinda pe" : "olinda the",
    "virar east" : "east",
    "mato grosso,  sinop" : "mato grosso, sinop",
    "banglore" : "bengaluru",
    "ابوظبي" : "abu dhabi",
    "kalwa" : "calf",
    "perú lima" : "lima peru",
    "nova resende" : "new resende",
    "mumbai" : "moon",
    "cote d'ivoire" : "ivory coast",
    "διγενή 25" : "digeni 25",
    "na vyhlídce 268 268" : "on the prospect of 268 268",
    "carpina   pernambuco" : "carpina pernambuco",
    "słupska" : "słupsk",
    "bünde" : "fründe",
    "el cerrito" : "the hill",
    "vitória   espirito santo" : "vitoria, espirito santo",
    "khagaria" : "khagariia",
    "النوبارية" : "nubaria",
    "joelformcheck magnit joelformcheck magnit" : "joelformcheck mets joelformcheck magnet",
    "pan india" : "indian bread",
    "colombia c2: solo información" : "colombia c2: only information",
    "via fontana dell'oste 52" : "via fontana dell'osta 52",
    "barranquilla, atlantico" : "barranquilla, atlantic",
    "calle goya, 90" : "goya street, 90",
    "roha" : "steam",
    "aquiraz   ce" : "aquiraz ce",
    "المدينة المنورة" : "al madinah al munawwarah",
    "três corações" : "three hearts",
    "ponda" : "crush",
    "via barone s. ferla" : "via barone s.ferla",
    "sc   maravilha" : "sc wonder",
    "boca raton" : "bottle of raton",
    "raigad" : "rig",
    "mahnar" : "mahanar",
    "ribeirão preto   sp" : "ribeirão preto sp",
    "arsikere" : "arsic",
    "abudhabi" : "abu dhabi",
    "viluppuram" : "developuram",
    "rua piau" : "piau street",
    "carazinho" : "cad",
    "odenton" : "denton",
    "rio das ostras   rj" : "rio das ostras rj",
    "región metropolitana" : "metropolitan region",
    "nagpur india" : "india",
    "台灣" : "taiwan",
    "vadodara" : "let them",
    "correntes" : "chains",
    "gangavathi" : "gangavati",
    "balneário camboriú  sc" : "balneário camboriú sc",
    "barka" : "helloon",
    "jalna" : "burn",
    "kota bandung" : "bandung",
    "cordilheira alta   sc" : "high cordilher sc",
    "portharcourt" : "porthcourt",
    "i̇zmir urla" : "ilkir urla",
    "avenida de europa 4 planta baja, local b" : "avenida de europa 4 ground floor, local b",
    "rua leonor maria barbosa condom?nio village ba" : "rua leonor maria barbosa condom?nicil village i",
    "teste" : "test",
    "frontera hidalgo" : "hidalgo border",
    "avc systemy audiowizualne jakub pacholewicz, ul. widokowa 3" : "avc audiovisual systems jakub pacholewicz, ul.viewing 3",
    "kosagumuda" : "kosa no waste",
    "waraseoni, dist balaghat" : "varasai, district balaghat",
    "kerala  kannur" : "kerala kannur",
    "breaux bridge" : "breaux bridge",
    "piler" : "arrows",
    "chikodi" : "health",
    "stevensville" : "stevensville",
    "amparo" : "protection",
    "mau ranipur" : "it's ranipur",
    "parana , medianeira" : "parana, medianeira",
    "peje" : "pe said",
    "parbhani" : "perbhani",
    "edappal" : "adapal",
    "san jose del monte" : "san jose of monte",
    "lastra a signa" : "slab in signa",
    "bazas" : "bazae",
    "116đường cầu kinh" : "116 cau kinh road",
    "rajam" : "stoning",
    "خميس مشيط" : "khamis mushait",
    "noorsarai" : "joun",
    "sveta nedelja" : "holy sunday",
    "basavanbagewadi" : "basebagewadi",
    "sayoun" : "sion",
    "shahabad markanda distt kurukshetra" : "shahabad markanda distra kurukshetra",
    "benito juárez" : "benito juarez",
    "proszowice, kraków, małopolskie" : "proszowice, kraków, lesser poland",
    "rajashthan" : "rampage",
    "sialkot" : "sigot",
    "la rioja" : "the rioja",
    "piedras negras" : "black stones",
    "amethi" : "to",
    "40 s k bole marg" : "3 s.",
    "tada" : "then",
    "bazpur uttrakhand" : "bajpur uttarakhand",
    "الاسكندرية" : "alexandria",
    "via d.f. rosso" : "via d.f.red",
    "nazaré da mata  pernambuco" : "nazaré da mata pernambuco",
    "la chorrera" : "the chorrera",
    "sao paulo vargen grande do sul" : "sao paulo vargem granding do sul",
    "moshi,pimpri  chinchawad" : "moshi, pimpri chinchwad",
    "umri" : "age",
    "duabi" : "dubai",
    "fort pierre" : "strong stone",
    "manavalanagar" : "manwalnagar",
    "akole" : "we can not",
    "boa vista" : "good view",
    "san carlos," : "saint charles,",
    "partur,jalna" : "part, jalna",
    "dubi" : "look",
    "kodambakkam" : "kodubakkam",
    "league coty" : "league city",
    "barra velha" : "old bar",
    "pinhalzinho" : "pineglass",
    "الرياض السعودية" : "al riyadh, saudi arabia",
    "fraiburgo" : "fraiburg",
    "maharashatra" : "maharashtra",
    "andana" : "story",
    "b0g0tá" : "b0g0g0",
    "dubia" : "doubts",
    "gaziantep i̇slahiye" : "gazi̇antep i̇slahi̇ye",
    "sumatera utara" : "north sumatra",
    "shagamu" : "shop",
    "cachoeiro" : "waterfall",
    "18 r. aquino st., hagdang bato libis, mandaluyong city metro manila" : "18 r.aquino st., hagdang bato libis, mandaluyong city metro manila",
    "jantral vijapur mehsana gujarat india" : "bharanal vijapur mehsana gujarat india",
    "dreieich" : "triad",
    "via tosco romagnola  1422" : "via tosco romagnola 1422",
    "montería" : "hunting",
    "信義區" : "xinyi district",
    "resende" : "reset",
    "النبطية" : "nabatieh",
    "schladming" : "sladming",
    "quitó" : "he removed",
    "san josé" : "saint joseph",
    "são gabriel do oeste   ms" : "são gabriel do oeste ms",
    "ny 11358" : "the 11358",
    "castelo" : "castle",
    "betul" : "proper",
    "rua celso ramos, 145   vila andrade   s?o paulo   sp   cep: 05734 080 sp" : "rua celso ramos, 145 vila andrade são paulo sp zip code: 05734 080 sp",
    "malur" : "anoint",
    "paita" : "shirt",
    "marib" : "maribe",
    "7001 e fish lake rd #200  cloud it" : "7001 e fish lake rd #200  cloud it",
    "cuiabá   mt" : "cuiabá mt",
    "vasai road, mumbai" : "wasi road, mumbai",
    "thủ dầu một, tỉnh bình dương" : "thu dau mot, binh duong province",
    "reasi" : "reaction",
    "yanbu al sinaiyah" : "eye aris resistance marks",
    "chile, santiago" : "santiago chile",
    "contai" : "anyway",
    "paranaque, metro manila" : "paranaque, metro manila",
    "mestre venezia" : "mestre venice",
    "madikeri" : "madatti",
    "haag in oberbayern" : "haag in upper bavaria",
    "700 patroon creek blvdalbanyny12206" : "700 pattern creek blvdalbanyny12206",
    "passos" : "steps",
    "الشارقة" : "sharjah",
    "malerkotla" : "painter kotla",
    "itanhaém  sp" : "itanhaém sp",
    "aracaju se" : "aracaju if",
    "el dorado hills" : "the dorado hills",
    "57 neumann road    capalaba  qld  4157  us qld" : "57 neumann road capalaba qld 4157 us qld",
    "maceió   al" : "macea al",
    "atizapan, estado de méxico" : "atizapan, state of mexico",
    "provinsi banten" : "banten province",
    "sta. rosa city" : "is.rosa city",
    "645 hulet dr ste 200" : "645 hollowed dr ste 200",
    "franca são paulo" : "franca sao paulo",
    "rajghar" : "royal",
    "sqlvador" : "salvador",
    "sjo" : "sea",
    "al qusais" : "al quasais",
    "πάροδος θηβαΐδος 21" : "thebaidis 21",
    "pozzonovo" : "refuse",
    "chakia" : "hugs",
    "neópolis" : "neopolis",
    "القصيم بريده" : "al -qassim is his mail",
    "kí túc xá khu a đhqg tp. hcm" : "dormitory a vnu tp.hcm",
    "ny" : "the",
    "pilani jhunjunu" : "pilani jhhununu",
    "sai paulo" : "you know paulo",
    "tibu" : "chick",
    "brgy 13 naglicuan, pasuquin, in" : "brgy 13 naglikuan, pasuquin, in",
    "wynnum" : "whiten",
    "tadiprru" : "earlierpru",
    "torreblanca" : "white tower",
    "ramnagar" : "ramnagar ramnagar",
    "nizmabad" : "nizamabad",
    "bhestan, surat." : "bhestan, letter.",
    "biloxi" : "comparison",
    "balıkesir" : "balikesir",
    "al jubail" : "to the jubail",
    "kosova" : "chuang",
    "santarém  pará" : "santarém pará",
    "linh  trung,  thủ đức" : "linh trung, thu duc",
    "ghatsila" : "ghatila",
    "joão pessoal" : "joão pessoa",
    "kuj pomorskie" : "kuj pomeranian",
    "新竹" : "hsinchu",
    "taloja" : "houses",
    "joão pessoa   pb" : "joão pessoa pb",
    "videira" : "vine",
    "warmińsko  mazurskie" : "warmian masurian voivodeship",
    "johannesburh" : "johannesburg",
    "tamluk" : "ownership",
    "chennaj" : "chennai",
    "via d. alighieri 13" : "via d.alighieri 13",
    "chengalpattu district" : "district",
    "malkapur" : "malakpur",
    "thatta" : "mockery",
    "wolfschlugen" : "wolf",
    "recife  pernambuco" : "recife pernambuco",
    "san lorenzo" : "saint lawrence",
    "samana" : "the same",
    "tangerang selatan" : "south tangerang",
    "nossa senhora do socorro" : "our lady of socorro",
    "via venezia n. 69" : "via venezia n.69",
    "tezpur" : "tamur",
    "balikesi̇r edremi̇t" : "fish edrememi",
    "175 e. schroon river road" : "175 e.river road schron",
    "boa vista  rr" : "boa vista rr",
    "jharsuguda" : "guard",
    "bình thạnh" : "binh thanh",
    "akluj" : "accuse",
    "santo andré   sp" : "santo andré sp",
    "dolny śląsk" : "lower silesia",
    "merkez" : "centre",
    "são paulo (sp)" : "sao paulo-sp)",
    "aurangabad mh" : "aurangabad maha",
    "pomorskie" : "pomeranian",
    "zahle" : "pay",
    "margoa" : "painting",
    "rahata city." : "live city",
    "doda jammu" : "adds to the jammu",
    "bergdietikon" : "bergdietic",
    "nord rhein" : "north rhine",
    "londres" : "london",
    "accettura" : "accepting",
    "bình chánh, tp hồ chí minh" : "binh chanh, ho chi minh city",
    "las palmas de gran canaria" : "the gran canarian palms",
    "coronel fabriciano" : "colonel fabriciano",
    "bhusawal" : "bhusaval",
    "173 xuân thiy" : "173 xuan thuy",
    "navi mumbai (new mumbai)" : "navi mumbai (new moon)",
    "novo hamburgo" : "new hamburg",
    "camarenis sur" : "south camarines",
    "s.s. 115 avola siracusa km 388,8" : "s.s.115 avola siracusa km 388.8",
    "dhanbad" : "dhanbad dhanbad",
    "sutrapada" : "tomorrow",
    "criciúma" : "criciuma",
    "casado" : "married",
    "hosadurga" : "hosadadugha",
    "niš" : "nis",
    "chittaurgarh" : "chittorgarh",
    "neemrana" : "neemountain",
    "kothagudem" : "newgudem",
    "الاغوار الجنوبيه" : "southern jordan valley",
    "via l. da vinci 12" : "via l.da vinci 12",
    "santiago chile" : "santiago, chile",
    "alterosa" : "alterious",
    "berlin   neukölln" : "berlin neukölln",
    "catigua" : "catiguua",
    "caçador" : "hunter",
    "hathars" : "hathras",
    "kruszyn" : "crumble",
    "rangapara" : "raduate",
    "campi salentina" : "salento campi",
    "76 đường 34, an phú, q.2" : "76 street 34, an phu, district 2",
    "villahermosa" : "beautiful villa",
    "19 picadlly rd hampstead" : "19 piccadilly rd hampstead",
    "estrada das carrascas" : "carrascas road",
    "minas gerais visconde do rio branco" : "minas gerais viscount of rio branco",
    "pusad" : "sweaters",
    "via alba 20   roddi" : "via alba 20 roddi",
    "sao paulo sp" : "sao paulo-sp",
    "rio grande do sul   porto alegre" : "rio grande do sul porto alegre",
    "8795 runamuck pl, unit b  medical" : "8795 runamuck pl, unit b medical",
    "20370 passagio drive  medical" : "20370 passage drive medical",
    "sordio" : "sordy",
    "oswego" : "oestrus",
    "pattukkottai" : "tukkottai",
    "virum, denmark" : "before, dermark",
    "gia tân 2, thống nhất, đồng nai" : "gia tan 2, thong nhat, dong nai",
    "cabo sa lucas" : "cabo in lucas",
    "umhlanga rocks" : "reed rocks",
    "guarulhos são paulo" : "guarulhos sao paulo",
    "marília  sp" : "marília sp",
    "via campo marinaro, 8 italia" : "via campo marinaro, 8 italy",
    "mahdia" : "can",
    "paratwada" : "paste",
    "solteiro(a)" : "single",
    "cabreúva" : "cabreuva",
    "pontotoc" : "pontoc",
    "jebel ali" : "mount on",
    "via dante n. 34   nuxis (su)" : "travel n.34 nuxis (su)",
    "phú quí cai lậy tiền giang" : "phu qui cai lay tien giang",
    "hristo smirnenski blv., 30" : "hristo smirnenski bl., 30",
    "cruzeiro" : "cruise",
    "pr   foz do iguaçu" : "pr foz do iguaçu",
    "curitiba.   paraná" : "curitiba.paraná",
    "113 หมู่ 6 ลาดพร้าว 124" : "113 moo 6 lat phrao 124",
    "bambavde, kolhapur" : "bambavade, kolhapur",
    "riaydh" : "riaeyd",
    "pampangga" : "pampanga",
    "pomona" : "pomon",
    "prishtina" : "pristina",
    "fdzs" : "fed",
    "limeira" : "lime",
    "davao city" : "gave city",
    "av. boyacá n 67 96 813013550" : "av.boyacá n 67 96 813013550",
    "mahi" : "assignment",
    "مكه المكرمه" : "makkah",
    "rura,kanpur dehat" : "rura, kanpur dehat",
    "majri" : "stream",
    "al qatif ash sharqiyah" : "al qatif ash sharqi",
    "ecuador manabí  manta" : "ecuador manabí manta",
    "pelotas rs" : "rs balls",
    "minh khai  hai bà trưng hà nội" : "minh khai hai ba trung hanoi",
    "bairagania" : "bargania",
    "putinga" : "to putin",
    "burien" : "yellowish garden",
    "conselheiro pena" : "pena",
    "becharaji" : "disgraceful",
    "1100 lee ave, lafayette, la 70501," : "1100 lee ave, lafayette, 70501,",
    "rio do sul" : "south river",
    "vitotoria" : "mtotoria",
    "gò công đông tiền giang" : "go cong dong tien giang",
    "18, đường số 12, hiệp bình chánh" : "18, road no. 12, hiep binh chanh",
    "bhanga" : "bank",
    "śląsk" : "silesia",
    "san isidro" : "yese isiddro",
    "56a dương thị mười" : "56a duong thi muoi",
    "pangasinan,pozorrubio" : "pangasinan, pozorrubio",
    "garhmuktesar" : "gadhmuktear",
    "nagpur yatmaval" : "nagpur yatmawal",
    "los angeles" : "the angels",
    "beni mellal" : "bani boredom",
    "barra mansa rj" : "mansa rj bar",
    "hajipur,  phubihar" : "hajipur, fubihar",
    "naigoan" : "went",
    "via dei fiumi" : "via dei riversi",
    "buengkum" : "builder",
    "sindgi" : "are",
    "sanyo antônio do monte" : "santo antônio do monte",
    "ambajogai" : "to the ambition",
    "bhrompur" : "bharompur",
    "te a, 2222 w. grand river ave, okemos, mi, 48864" : "a, 2222 w.grand river ave, ok, mi, 48864",
    "kolkata, india" : "kolka, india",
    "centro de distribucion amelia  it" : "amelia it distribution center",
    "2826 nw 72 av" : "2826 nw 72 of",
    "davanagere" : "davangere",
    "dibai" : "glance",
    "taguatinga sul" : "southern taguatinga",
    "tanta" : "so much",
    "puerto wilches" : "wilches port",
    "colina" : "hill",
    "9110 forest crossingthe woodlandstx77381" : "9110 forest crossing the woodlands tx 77381",
    "mandigobindgarh(amandeepanand786@gmail.com)" : "mandigobindagarha (amandipananda",
    "são paulino" : "sao paulino",
    "cabo frio   rj" : "cabo frio rj",
    "74 molave st. z 1 central signal village, taguig city philippines" : "74 molave st.z 1 central signal village, taguig city philippines",
    "warmińsko mazurskie" : "warmian masurian voivodeship",
    "mankapur gonda" : "manikpur gonda",
    "itajaí   sc" : "itajaí sc",
    "dumka" : "opinion",
    "raichur" : "raycur",
    "governador valadares mg" : "governor valadares mg",
    "hyderabab" : "hyderabad",
    "serra   espírito santo" : "serra espírito santo",
    "vico 2° ginnasio 8" : "vico 2nd gymnasium 8",
    "missoula" : "touched",
    "hailakandi" : "hallakandi",
    "ras alkhaimah" : "ras al khaimah",
    "yeddumailaram" : "bull",
    "butler" : "thighs",
    "rio de janiero" : "rio de janeiro",
    "batala" : "butter",
    "atlanta, ga" : "ah, but",
    "brasília   df" : "brasilia df",
    "อยุธยา" : "ayutthaya",
    "الاحساء" : "hasa",
    "al fujairah" : "the dawn",
    "crisciuma" : "i'm going to",
    "الانبار" : "anbar",
    "ciudad de mexi o" : "mexico city",
    "assis são paulo" : "assis sao paulo",
    "thanh hoá" : "thanh hoa",
    "viale alpi orientali nord, 6 int. 3" : "viale alps orientali nord, 6 int.3",
    "kodikonda check post" : "choose check post",
    "gidderbaha" : "budgebaha",
    "bauneario camburiu" : "balneário camboriú",
    "narre warren" : "fool warren",
    "รพ.สมิติเวช" : "samitivej hospital",
    "mukherjinagar" : "mukherjnagar",
    "terlingua" : "elder",
    "ernakulum" : "ernamum",
    "6300 harry hines blvdste. 1400dallastx75235" : "6300 harry hines blvd ste. 1400 dallas tx 75235",
    "ciudad arce" : "arce city",
    "sta. maria bulacan" : "stay.maria bulacan",
    "colíder" : "collide",
    "new canaan" : "new year",
    "piedras negraa" : "black stones",
    "campo verde mt" : "green field mt",
    "102c đường 79, phước long b," : "102c street 79, phuoc long b,",
    "padre hurtado  central 673las condes santiago" : "father hurtado central 673las condes santiago",
    "salvador   ba" : "salvador ba",
    "milão" : "milan",
    "nürnberg" : "nuremberg",
    "galati" : "galata",
    "biała" : "white",
    "bhndara" : "bhandara",
    "chatrapati sambhajinagar" : "chhatrapati sambhajinagar",
    "la trinidad" : "the trinity",
    "mandvi" : "mandavi",
    "lima lima" : "five five",
    "leicesterhsire" : "leicestershire",
    "den haag" : "the hague",
    "sojatcity" : "sojat city",
    "santa bárbara d'oeste" : "santa barbara d'oeste",
    "104 caldo" : "104 hot",
    "القاهره" : "cairo",
    "611 w. park street fait medical" : "611 w.park street make medical",
    "tiruchirapalli" : "truuchrapalli",
    "luna la union" : "luna laon",
    "odisha baripada" : "odisha barada",
    "parli vaijnath" : "married vijayanath",
    "trà ôn, vĩnh long" : "tra on, vinh long",
    "kovilpatti" : "kovilpat",
    "दिल्ली" : "delhi",
    "pangkal pinang" : "town",
    "जैतारण आनंदपुर कालू" : "jaitaran anandpur kalu",
    "newnan" : "they were new",
    "jawa tengah" : "central java",
    "ahad rofaida ksa" : "sunday rofaida ksa",
    "barraquilla" : "barranquilla",
    "jardim do seridó rn" : "seridó rn garden",
    "مبارك الكبير" : "mubarak al -kabeer",
    "gov. valadares" : "gov.valadares",
    "macae rj" : "macae raj",
    "karaikal" : "stains",
    "tusayan" : "show",
    "hoogbocht 1" : "beam 1",
    "810 n kingston dr peoria, il 61604 2145" : "810 n kingston dr peoria, il",
    "jabel ali" : "mount on",
    "pandhurna" : "bad",
    "thành phố hồ chí minh" : "ho chi minh city",
    "gaouteng" : "karcarian hurts",
    "tân bình" : "tan binh",
    "kapadwanj" : "kapadwanjj",
    "ouro branco" : "white cholocate bonbon",
    "itaboraí   rj" : "itaboraí rj",
    "κ. βαρναλη 130 132" : "k. varnali 130 132",
    "xalapa e ver" : "xalapa and see",
    "imbituva pr" : "pilva pr",
    "rosario   santa fe" : "rosario santa fe",
    "jakarta timur" : "east jakarta",
    "arkport" : "carpet",
    "ichalkaranji" : "akinkaranan",
    "via col vento" : "gone with the wind",
    "nordrhein westfalen   herford" : "north rhine westphalia herford",
    "amarillo" : "yellow",
    "uruara" : "unafare",
    "mancherial" : "mancheril",
    "russas" : "russian",
    "500 valley rd.,  suite 202  it" : "500 valley rd.,  suite 202  it",
    "campo novo" : "new field",
    "tp ho chi minh" : "ho chi minh city",
    "nowe skalmierzyce" : "new skalmierzyce",
    "san nicolas de los garza" : "san nicolas de los heros",
    "9700 maumelle blvd" : "9700 multidio blvd",
    "changanacherry, kottayam" : "changanacheri, kottayam",
    "keshav rai patan" : "keshav rai collapse",
    "ongole" : "credit",
    "bogazkoy" : "bogatskoy",
    "mumbai ( mumbai)" : "mumbai (mumbai)",
    "القليوبيه" : "qaliubiya",
    "เพชรบูรณ์" : "phetchabun",
    "cachoeira do piriá pará" : "piriá pará waterfall",
    "30 ung văn khiêm bình thạnh" : "30 ung van khiem binh thanh",
    "nova andradina   mato grosso do sul" : "nova andradina mato grosso do sul",
    "małopolskie, 34 436 maniowy" : "lesser poland, 34 436 maniowy",
    "baradwar" : "paradise",
    "dhuri" : "the wall",
    "el segundo" : "the second",
    "anjugramam" : "anjugaramam",
    "2 alexfisher drive    burliegh heads  qld  4220  us qld" : "2 alex fisher drive burleigh heads qld 4220 us qld",
    "hai phong" : "two phong",
    "pedro leopoldo" : "pedro leopold",
    "thành phố hồ chí minhp" : "ho chi minh city",
    "arambagh" : "arambagi",
    "11 penns trail bldg. 100" : "11 pennies trail bldg.100th",
    "centro" : "center",
    "salete" : "silet",
    "shivapark" : "shivakarak",
    "9x chu văn an" : "9x is văn",
    "rua doutor álvaro albuquerque" : "rua doctor álvaro albuquerque",
    "184 lê đình can" : "184 le dinh can",
    "małopolskie" : "lesser poland",
    "hưng yên" : "hung yen",
    "jaora" : "loser",
    "franca   sp" : "franca sp",
    "14334 e layton pl  medical" : "14334 e layton pl medical",
    "gaziabad" : "attractive",
    "bakı" : "baku",
    "birsa, balaghat, mp india" : "birsa, balaghat, pump india",
    "kalimantan barat" : "west kalimantan",
    "nashirabad" : "nasirabad",
    "cancún, q. roo" : "cancun, q.roo",
    "distrito capital de bogotá" : "capital district of bogotá",
    "ofibodegas fraijanes bodega 23 km. 18 carretera a san jose  commercial display" : "ofibodegas fraijanes bodega 23 km.18 road to san jose commercial display",
    "الطائف" : "taif",
    "vespasiano" : "vespasian",
    "via irlanda, 12" : "via ireland, 12",
    "vitoria es" : "vitoria is",
    "gulbarga" : "to a floron",
    "gorari" : "lalli",
    "botad" : "cured",
    "φιλολαου 28 ηλιουπολη" : "filololaou 28 ilioupoli",
    "ipatinga  mg" : "ipatinga mg",
    "santa cruz do capibaribe   pe" : "santa cruz do capibaribe pe",
    "se" : "himself",
    "karachi gulshane shamim fb area" : "karachi gulzhane shamim fb area",
    "corso m. d'azeglio 30" : "course m.d'azeglio 30",
    "kodinar" : "codeinar",
    "malappuram, manjeri" : "malappuram, manjari",
    "cartegena" : "cartagena",
    "paulínia" : "paulinia",
    "tân phước tiền giang" : "tan phuoc tien giang",
    "ganaur, sonipat" : "ganore, sonipat",
    "wielkopolskie" : "greater poland",
    "9000 executive prk dr ste c200 healthcare" : "9000 executive park dr ste c200 healthcare",
    "جازن بني مالك" : "jazen bani malik",
    "såo paulo" : "são paulo",
    "c. so  principe eugenio, 3" : "c.i know prince eugenio, 3",
    "arava" : "cart",
    "são paulo  são paulo" : "sao paulo sao paulo",
    "palvoncha telangana" : "palvoncha telankana",
    "ದೋಡಬಳಪುರ" : "dodalpur",
    "pitalito  huila" : "pitalito huila",
    "la reian" : "i ree",
    "sp   taubaté" : "sp taubaté",
    "peruibe" : "peruve",
    "la habana" : "havana",
    "n, la cabecera, san felipe del progreso estado de mexico estado de mexico" : "n, the head, san felipe del progreso state of mexico state of mexico",
    "ventanas" : "windows",
    "bhainsa" : "buffalo",
    "dalkhola" : "dalhala",
    "bengalore" : "bengaloreore",
    "bijapur(kar)" : "bijapur (tax)",
    "سكاكا الجوف" : "sakaka al -jouf",
    "qurum" : "institution",
    "riffa" : "rife",
    "alegrete" : "cheerful",
    "bueos aires caba" : "buenos aires caba",
    "tiruvannamalai" : "thiruvannamalai",
    "dois irmãos" : "two brothers",
    "vllavicencio" : "villavicencio",
    "1940 jefferson hwy" : "1940 jefferson longer",
    "la serena" : "serena",
    "quinta normal" : "normal fifth",
    "niedersachsen" : "lower saxony",
    "dalsingh sarai" : "dalsingh sarah",
    "mersi̇n" : "mersow",
    "naazareno" : "nazarenaw",
    "palani" : "france",
    "10 nguyễn thanh đằng, bà rịa" : "10 nguyen thanh dang, ba ria",
    "sagarejo" : "sagrejo",
    "são josé dos pinhais   paraná" : "são josé dos pinhais paraná",
    "padrauna" : "subtle",
    "nieborów" : "heavenly",
    "várzea grande" : "varzea grande",
    "abugon, sibonga, cebu" : "about, sibonga, cebu",
    "ροσταν 44" : "rostan 44",
    "3330 n 104 ave  234 hotel" : "3330 n 104th ave 234 hotel",
    "malout" : "tend",
    "bhadaur" : "bhadore",
    "são luís do quitunde   alagoas" : "são luís do quitunde alagoas",
    "sulawesi selatan" : "south sulawesi",
    "umm al quwain" : "umm qun",
    "mazowieckie , radom" : "mazowieckie, radom",
    "aldie" : "all",
    "hidrolândia" : "hydroland",
    "江苏" : "jiangsu",
    "sopore" : "slackly",
    "pinhais" : "pinemers",
    "zamb0anga city" : "zamboanga city",
    "vennanthur, namakkal" : "vennandur, namakkal",
    "riyadh, ksa" : "riad, story",
    "dhahran" : "dharan",
    "ggn" : "evidence",
    "santo antônio de pádua" : "santo antônio de padua",
    "ende" : "end",
    "arroio do sal" : "salt",
    "phường 6 , quận gò vấp" : "ward 6, go vap district",
    "iloilo city" : "iloulo city",
    "chittaranjan" : "delicate",
    "perambalur." : "pounding.",
    "almenara" : "beacon",
    "75a, phan đình phùng, phú nhuận" : "75a, phan dinh phung, phu nhuan",
    "nalbari" : "illumon",
    "sitamarhi ,bihar" : "sitamarhi, bihar",
    "32 dương quảng hàm" : "32 duong quang ham",
    "ballari" : "scores",
    "villupuram" : "vilupuram",
    "canaã dos carajás" : "canaan of carajás",
    "jakarta utara" : "north jakarta",
    "jaboatão   pe" : "jaboatão pe",
    "163 trần phú, đà nẵng" : "163 tran phu, da nang",
    "lumut,perak" : "moss, silver",
    "kodungallur" : "kodungallurur",
    "kotli surat malli" : "kotli surat model",
    "av 2g # 44 32" : "of 2g # 44 32",
    "حدائق القبه" : "dome gardens",
    "cn   monasterolo di savigliano" : "cn monasterolo di savigliano",
    "कैलाश कुमार" : "kailash kumar",
    "bulacan bulacan" : "bulcan bulcan",
    "villavcencio" : "villavicencio",
    "10880 wilsire blvd suite 1101 lg pro" : "10880 wilshire blvd suite 1101 lg pro",
    "chowari" : "streets",
    "al sokhna" : "the heat",
    "q. thủ đức" : "q.thu duc",
    "36 hoàng cầu, đống đa, hà nội" : "36 hoang cau, dong da, hanoi",
    "barra mansa" : "mansa bar",
    "103 urb el retiro  medical" : "103 urb el retiro medical",
    "lattarico" : "lactic",
    "69 lakshmi garden, thindal m v k illam" : "69 lakshmi garden, illam in thindal",
    "são gonçalo" : "sao goncalo",
    "darwha" : "tarry",
    "zachoodniopomorskie" : "west pomeranian",
    "wielkopaolska" : "large",
    "1037 des marguerites  it" : "1037 of marguerites it",
    "pedregulho" : "boulder",
    "3351 michelson dr ste 100  medical" : "3351 michelson dr ste 100 medical",
    "tân an, càng long, trà vinh" : "tan an, lau long, tra vinh",
    "lares decora" : "lares decorates",
    "angwin" : "anguish",
    "5300 kell blvd" : "5300 need blvd",
    "diadema" : "diadem",
    "pipartola, balaghat, mp india" : "pipatola, balaghat, pump india",
    "22 đường 5 linh xuân thủ đức" : "22 road 5 linh xuan thu duc",
    "godda" : "new",
    "limoeiro" : "lemon tree",
    "memari" : "keep",
    "קרית גת" : "kiryat gat",
    "hyderabd" : "hyderabad",
    "sacile" : "sail",
    "100 crowley dr  medical" : "100 crowley dr  medical",
    "el shrouk 2" : "el shorouk a",
    "ghughli" : "google",
    "timóteo" : "timothy",
    "anand vihar delhi" : "anand storm delhi",
    "thiruvananamatlai" : "thiruvananthapuram",
    "poza rica" : "picture rica",
    "delhij" : "delichium",
    "lima   peru" : "lima's name",
    "müllheim" : "garbage home",
    "asolo" : "asol",
    "pfalz" : "palatinate",
    "台北市" : "taipei city",
    "islamabad" : "islamabad",
    "azamgarg up" : "azamgarg",
    "cédula de ciudadanía" : "citizenship card",
    "ميسان" : "maysan",
    "andersen afb" : "andersen fig",
    "rionegro" : "black river",
    "vpo rani khera" : "hear",
    "τατοΐου 65" : "tattoo 65",
    "castanhal" : "brown",
    "jazan" : "jizan",
    "ha tiên" : "ha tien",
    "almanya" : "germany",
    "1900 lukken industrial drive west" : "1900 luken industrial drive west",
    "1919 minnesota ct, mississauga, on l5n" : "1919 minnesota ct, mississauga, is l5n",
    "bom jesus" : "good jesus",
    "aadgaon" : "aadga",
    "mour mandi" : "dying",
    "oshogbo" : "embon",
    "mahroni" : "disadvantaged",
    "paschim medinipur (west medinipur)" : "west midnapore (west medinipur)",
    "الجلة الرياض" : "al jala riyadh",
    "สมุทรปราการ" : "samut prakan",
    "abohar" : "warehouse",
    "los gatos" : "the cats",
    "banka" : "knock",
    "dohrighat" : "pink",
    "gautam budha nagar" : "gautam mercury nagar",
    "maceió alagoas" : "maceio alagoas",
    "koszecin" : "cosecin",
    "portão" : "gate",
    "चाकण" : "chickpe",
    "santo domingo este rep. dom." : "saint sunday this rep.sun.",
    "cerejeiras" : "cherry trees",
    "cruzeiro do sul acre" : "southern cruzeiro acre",
    "zachodniopomorskie" : "west pomeranian",
    "fatehnagar" : "fatehanagar",
    "kharabi nagpur" : "morning nagpur",
    "saharsa" : "sarhasa",
    "tp.hồ chí minh" : "ho chi minh city",
    "ciudad de méxico  it" : "mexico city it",
    "lapu lapu city" : "lapa lap city",
    "הר אדר" : "mount adar",
    "faropilhas" : "barefoot",
    "navapur" : "nawapur",
    "sarita chorakiasundeep jain" : "saree",
    "carpina" : "carpine",
    "rio real" : "river royal",
    "pudukkottai" : "spright",
    "cajati" : "gentle",
    "este" : "it is",
    "purna" : "full",
    "mattapoisett" : "matte",
    "cañas" : "white hair",
    "punr" : "rear",
    "harda" : "as well as",
    "3 vpo sunet , brs nagar" : "3 vapo sunat, bars nagar",
    "joão pesssoa" : "joão pessoa",
    "esch sur alzette" : "esch surge of alzette",
    "pooler" : "pools",
    "araras" : "macaws",
    "querètaro" : "wanttaro",
    "daund" : "race",
    "naucalpan , mexico" : "naucalpan, mexico",
    "opolskie" : "opole",
    "ulu tiram, johor bahru" : "ulu oyster, johor bahru",
    "sénégal" : "senegal",
    "dhuliyan" : "dhulian",
    "saudade do iguaçu" : "i miss iguaçu",
    "são luís maranhao" : "sao luis maranhão",
    "marki, mazowieckie" : "brands, masovian voivodeship",
    "minas gerais   belo horizonte" : "minas gerais belo horizonte",
    "25 hoàng sa phường dakao quận 1" : "25 hoang sa ward dakao district 1",
    "barmer rajasthan" : "badmer rajasthan",
    "15an mỹ" : "15an american",
    "newdelhi" : "new delhi",
    "vandavasi" : "trouble",
    "naduvattam" : "inflate",
    "อ เมือง จ ภูเก็ต" : "phuket",
    "pitangui mg" : "pihong ghost gate",
    "mapusa" : "hatch",
    "indiabulls  greens, kon, tal. panvel, pin code 410221" : "indiabulus greens, angle, tal.panvel, pin code 1",
    "churu" : "a thousand",
    "sindgi distik vijaypur" : "life dastic vijaypur",
    "menofia" : "monsterness",
    "ciudad de mexic" : "mexican city",
    "au ricchieri sn" : "a ricciid sn",
    "sriganganagar" : "shri ganga nagar",
    "ไทย" : "thai",
    "ramanagara" : "ragara",
    "hod hasharon" : "competition beef",
    "wa" : "of",
    "at post   satana, tal.satana, district. nashik  hotel" : "at post satana, tal.satana, district.nashik hotel",
    "campo mourão" : "campo mourao"
}

In [21]:
#####################
### customer_type ###
#####################

# 같은 의미의 값들을 변환
df_all["customer_type"] = df_all["customer_type"].str.lower().replace({"end customer": "end-customer", "end-user": "end-customer", 'specifier/ influencer': "specifier / influencer","software/solution provider": "software / solution provider","installer": "installer/contractor","distributor": "dealer/distributor","homeowner" : "home owner" ,"others": "other", "etc.":"other", "technical assistant" : "technician", "commercial end-user" : "end-customer"})

# 빈도수 높은 상위 10개의 값만 남겨두고, 다른 값은 'ohter'로 변환
top_type = [x for x, y in df_all["customer_type"].value_counts().items() if y >= 10] 
df_all["customer_type"] = df_all["customer_type"].map(lambda x : x if x in top_type else 'other')

# 라벨링 진행 및 데이터 넣기
le = LabelEncoder()
df_all['customer_type'] = le.fit_transform(df_all['customer_type'])
df_train_prepare['customer_type'] = df_all['customer_type'].values[ : idx_train]        # 학습용 데이터
df_test_prepare['customer_type'] = df_all['customer_type'].values[idx_train :]          # 테스트 데이터


####################
### customer_job ###
####################

# 영어가 아닌 문자를 번역한 결과로 변환
df_all["customer_job"] = df_all["customer_job"].str.lower().map(lambda x : trans_dic['customer_job'][x] if x in trans_dic['customer_job'].keys() else x)

# 텍스트 불용어 처리
li_split = []
[[li_split.append(val) for val in x.replace("_", " ").replace("-", " ").replace("/", " ").replace("\t", " ").replace("(", " ").replace(")", " ").split(" ") if len(val) > 2] for x in df_train["customer_job"].dropna()]

# 3번 이상 등장하는 단어들의 집합 구성
li_tem = np.array(sorted([k for k, v in Counter(li_split).items() if v >= 3]))

# 파생 단어를 원형 단어로 포함시키기 1 : 원형 단어가 남아있는 단어를 변형시키기
str_tem, idx_tem, idx_li = li_tem[0], 0, []
for i in range(1, len(li_tem) - 1) :
    if not li_tem[i].startswith(li_tem[idx_tem]) :  
        idx_li.append(i)
        idx_tem = i

# 파생 단어를 원형 단어로 포함시키기 2 : 원형 단어가 남아있지 않는 단어를 찾아 변형시키기
chg_dic = {
    'amministrativo' : 'admin', 'arquitecto' : 'architect', 'consulting' : 'consultant', 'consultor' : 'consultant', 'educator' : 'education', 
    'finanzen' : 'finance', 'head' : 'ceo', 'integrador' : 'integrator', 'kommunikation' : 'communication', 'kommunikáció' : 'communication', 
    'lead' : 'head', 'manager' : 'managing', 'medien' : 'media', 'média' : 'media', 'procurement' : 'producer', 'projektmanagement' : 'project', 
    'projektmenedzsment' : 'project', 'serving' : 'service', 'művészet' : 'art', 'pénzügy' : 'finance'
}

del_li = ['and', 'egyéb', 'end', 'for', 'other', 'otro', 'sonstiges', 'user', 'the', 'und'] + list(chg_dic.keys())      # 불용어 지정
labels_li = [x for x in li_tem[idx_li] if x not in del_li] + ['others']
label_dic = {k : v for v, k in enumerate(labels_li) if k not in del_li}     # 직업별 라벨 딕셔너리 생성

for k, v in chg_dic.items() : label_dic[k] = label_dic[v]       # 파생 단어를 원형 단어의 라벨값으로 변경

# 직업 딕셔너리 라벨 매핑
label_li = []
labels = labels_li + list(chg_dic.keys())
for d in df_all["customer_job"] :
    li_tem = []
    if type(d) == str :
        for l in labels :
            if l in d : li_tem.append(label_dic[l])

        if not li_tem : li_tem = [label_dic['others']]
        
    label_li.append(sum([(1000 ** idx) * num for idx, num in enumerate(sorted(li_tem))]))       # 1000 자리수를 만들어 중복 값의 경우에는 새로운 라벨 생성

# 데이터 넣기
df_train_prepare['customer_job'] = label_li[ : idx_train]       # 학습용 데이터
df_test_prepare['customer_job'] = label_li[idx_train :]         # 테스트 데이터


####################
### inquiry_type ###
####################

# 텍스트 불용어 처리
df_all['inquiry_type'] = df_all["inquiry_type"].str.lower().map(lambda x : x.replace("-", " ").replace("_", " ").replace("/", " ").strip().lower() 
                                                                                                                            if type(x) == str else x)

# 영어가 아닌 문자를 번역한 결과로 변환
df_all['inquiry_type'] = df_all['inquiry_type'].str.lower().map(lambda x : trans_dic['inquiry_type'][x] if x in trans_dic['inquiry_type'].keys() else x)

# 빈도수 높은 상위 50개의 값만 남겨두고, 다른 값은 'ohter'로 변환
label_li = [val for val, cnt in df_all['inquiry_type'].value_counts().items() if cnt >= 50]
df_all['inquiry_type'] = df_all['inquiry_type'].map(lambda x : x if type(x) == str and x in label_li else 'others')

# 라벨링 진행 및 데이터 넣기
le = LabelEncoder()
df_all['inquiry_type'] = le.fit_transform(df_all['inquiry_type'])
df_train_prepare['inquiry_type'] = df_all['inquiry_type'].values[ : idx_train]          # 학습용 데이터
df_test_prepare['inquiry_type'] = df_all['inquiry_type'].values[idx_train :]            # 테스트 데이터


########################
### customer_country ###
########################

# 문자를 "/"를 기준으로 자른 후, 제일 마지막 값만 저장하기
df_all['customer_country'] = df_all['customer_country'].map(lambda x : x.split("/")[-1].strip() if type(x) == str else x).str.lower()

# 영어가 아닌 문자를 번역한 결과로 변환
df_all['customer_country'] = df_all['customer_country'].map(lambda x : trans_dic['customer_country'][x] if x in trans_dic['customer_country'].keys() else x)

# 동의어 찾아서 변환하기
chg_dic = {'usa' : 'united states', "u.s" : "united states", "us" : "united states", "uae dubai" : 'u.a.e', 'türkiye' : 'turkey'}
df_all['customer_country'] = df_all['customer_country'].replace(chg_dic)

# 빈도수 4개 이상의 값만 남겨두고, 다른 값은 'ohters'로 변환
lebal_li = [k[0] for k, v in df_all[['customer_country']].value_counts().items() if k[0] and v >= 4 and len(k[0]) > 2]
df_all['customer_country'] = df_all['customer_country'].map(lambda x : x if x in lebal_li else 'others')

# 라벨링 진행 및 데이터 넣기
le = LabelEncoder()
df_all['customer_country'] = le.fit_transform(df_all['customer_country'])
df_train_prepare['customer_country'] = df_all['customer_country'].values[ : idx_train]      # 학습용 데이터
df_test_prepare['customer_country'] = df_all['customer_country'].values[idx_train :]        # 테스트 데이터


########################
### business_subarea ###
########################

# 문자를 소문자로 변환하기
df_all['business_subarea'] = df_all["business_subarea"].str.lower()

# 분류를 위한 딕셔너리 생성
cat2val = {
    'agr' : ['agriculture', 'botanical garden / green house'],
    'air' : ['airline', 'airport / airfield / helipad', 'aerospace'],
    'fin' : ['banking', 'developer/property', 'finance', 'holdings', 'insurance', 'investment', 'real-estate agency'],
    'str' : ['bookstore', 'cvs (convenience store)', 'duty free shop', 'hyper market & grocery', 'luxury(watch/jewelry shop)', 'other stores', 'shopping mall'],
    'clu' : ['broadcasting & media', 'cinema /theater', 'entertainment', 'exhibition / convention center', 'museum / gallery', 'sports', 'sports entertainment', 'travel agency', 'advertising', 'outdoor advertisement', 'fashion'],
    'tra' : ['bus terminal', 'car dealership', 'cruise', 'railway & metro station', 'sea passenger terminal / port'],
    'acc' : ['apartment', 'casino resort', 'dormitory', 'hotel', 'officetel', 'resort', 'townhouse', 'villa / single-family home'],
    'pla' : ['chemical factory / plant', 'distribution center', 'logistics', 'manufacturing', 'manufacturing factory / plant', 'pharmaceutical factory', 'construction', 'engineering', 'mining'],
    'hea' : ['clinic', 'cosmetics', 'fitness', 'general hospital', 'healthcare', 'hospital', 'ltc(long-term care)', 'pharmaceutical', 'pharmacy'],
    'off' : ['consulting', 'law firm'],
    'tec' : ['electronics & telco', 'it/software', 'network/cabling', 'telecom base station / data, call', 'telecommunication'],
    'env' : ['energy', 'energy storage & saving', 'gas station', 'power plant', 'renewable energy'],
    'foo' : ['f&b(food and beverage)', 'qsr(quick service restaurant)', 'restaurant'],
    'stu' : ['higher education (college & university)', 'institute & academy', 'k12 kindergarten & schools'],
    'pub' : ['police / fire station', 'general government office', 'welfare facilities', 'military', 'religious facility'],
    'mixed-use (multi complex)' : ['mixed-use (multi complex)'],
    'others' : ['others', 'other']
}
val2cat = {}
for cat, li in cat2val.items() : 
    for s in li : val2cat[s] = cat     # {'카테고리' : '값'} 에서 {'값' : '카테고리'} 딕셔너리 생성

# 딕셔너리에 존재하는 값이면 catgory 별로 변환, 없는 경우는 'ohters'로 변환
label_li = cat2val.keys()
df_all['business_subarea'] = df_all['business_subarea'].map(lambda x : x if type(x) == str and x in label_li else 'others')
# label_li = val2cat.keys()
# df_all['business_subarea'] = df_all['business_subarea'].map(lambda x : val2cat[x] if type(x) == str and x in label_li else 'others')

# 라벨링 진행 및 데이터 넣기
le = LabelEncoder()
df_all['business_subarea'] = le.fit_transform(df_all['business_subarea'])
df_train_prepare['business_subarea'] = df_all['business_subarea'].values[ : idx_train]      # 학습용 데이터
df_test_prepare['business_subarea'] = df_all['business_subarea'].values[idx_train :]        # 테스트 데이터


#########################
### customer_position ###
#########################

# 텍스트 불용어 처리
df_all['customer_position'] = df_all["customer_position"].str.lower().map(lambda x : x.replace("-", " ").replace("_", " ").replace("/", " ").strip() if type(x) == str else x)

# 분류를 위한 딕셔너리 생성
cat2val = { 
    'buss' : ['business partner', 'entrepreneurship', 'leadership executive office owner'],
    'part' : ['partner'],
    'arch' : ['architect consultant', 'architecture consult', 'architect consultant'],
    'devl' : ['business development', 'business development sales'],
    'cons' : ['career coach', 'commercial consultant', 'consultant', 'consulting'],
    'prof' : ['assistant professor', 'assistant professor of enlish', 'associate professor', 'associate professor in electronics engg', 'asst prof.', 'software  solution provider', 'guest faculty', 'pathologist', 'prof.', 'professional trainer', 'professor', 'professor of mathematics', 'quantitative aptitude faculty', 'radiology professional', 'senior lecturer', 'technical', 'técnico', 'academic specialist', 'director cum faculty at gaining apex coaching centre', 'principal at oxford integrated pu science college', 'principal & director', 'physics faculty'],
    'rese' : ['market intelligence research', 'research'],
    'anal' : ['associate analyst', 'associate analyst'],
    'mana' : ['administrative', 'operations', 'av management', 'business unit director', 'director', 'genel müdür', 'lider de desarrollo', 'manager', 'product management', 'gerente'],
    'syst' : ['system integrator'],
    'teac' : ['education', 'academic coordinator post graduate teacher (accountancy, business studies)  tgt (ict)', 'education professional', 'educator', 'education professional', 'english trainer for ielts,toefl,pte,gre,sat exams.', 'hon dean', 'physics teacher', 'science teacher', 'teacher', 'teacher middle school coordinator', 'math and physics teacher', 'maths lecturer', 'neet  olympiad expert faculty', 'other   please specify   cedia association', 'pgt chemistry', 'chemistry teacher', 'pgt physics', 'physics and mathematics teacher', 'academic coordinator  post graduate teacher (accountancy, business studies)  tgt (ict)'],
    'comm' : ['commercial end user', 'end user'],
    'carg' : ['cargo', 'distributor'],
    'cust' : ['customer'],
    'empl' : ['employee', 'employee' , 'intern', 'no influence', 'not applicable', 'trainee', 'entry level'],
    'engg' : ['engineering'],
    'entr' : ['entrylevel'],
    'exhi' : ['exhibition', 'exhibitiontv', 'installer'],
    'manu' : ['manufacturer'],
    'this' : ['this is a consume display requirement for home purpose.'],
    'head' : ['c level executive', 'c levelexecutive', 'ceo founder', 'ceo fundador', 'chairman', 'chief executive officer', 'co founder', 'the big boss', 'president'],
    'deci' : ['decision influencer', 'decision maker', 'decision influencer', 'decision maker'],
    'owne' : ['homeowner', 'proprietário(a)'],
    'foun' : ['founde', 'founder'],
    'medi' : ['főorvos', 'hospital', 'medical device manufacturer', 'medical imaging specialist', 'surgery professional', 'tierarzt'],
    'vice' : ['vice president', 'vicepresident', 'vp'],
    'sale' : ['sales', 'subsidiary sales (ise)'],
    'gove' : ['government', 'bulgaria'],
    'unpd' : ['unpaid'],
    'others' : ['others', 'other', 'mindenes'],
    'none' : ['none']
}
val2cat = {}
for cat, li in cat2val.items() : 
    for s in li : val2cat[s] = cat       # {'카테고리' : '값'} 에서 {'값' : '카테고리'} 딕셔너리 생성

# 딕셔너리에 존재하는 값이면 catgory 별로 변환, 없는 경우는 'ohters'로 변환
label_li = val2cat.keys()
df_all["customer_position"] = df_all["customer_position"].map(lambda x : x if type(x) == str and x in label_li else 'others')
# df_all["customer_position"] = df_all["customer_position"].map(lambda x : val2cat[x] if type(x) == str and x in label_li else 'others')

# 라벨링 진행 및 데이터 넣기
le = LabelEncoder()
df_all['customer_position'] = le.fit_transform(df_all['customer_position'])
df_train_prepare['customer_position'] = df_all["customer_position"].values[ : idx_train]        # 학습용 데이터
df_test_prepare['customer_position'] = df_all["customer_position"].values[idx_train :]          # 테스트 데이터


#########################
### expected_timeline ###
#########################

def func(x, most_ex_time) :      # 문자 전처리 하여 카테고리 생성하는 함수
    try:
        if x not in most_ex_time:
            if "not" in x or "n't" in x or 'no ' in x or 'drop' in x: return "neg"
            elif re.search(r"\b\d+(?:th)\b", x) is not None: return "detailed"
            
            for i in ["month", "day", "week", "detail", "jan", 'feb','mar', 'apil', 'may', 'june','july', 'aug','sept','oct', 'nov', 'dec' ]:
                if i in x: return "detailed"
            return x
        else: return x
        
    except: 
        return "none"

# 불용어 처리 및 동의어 처리
df_all["expected_timeline"] = df_all["expected_timeline"].map(lambda x : x.replace("_"," ").replace(".", "").replace("then", "than") if type(x) == str else x)

# 시간 구간인 문자열 가져오기
most_ex_time = ["less than 3 months", "3 months ~ 6 months", "more than a year", "9 months ~ 1 year", "6 months ~ 9 months", "less than 6 months", "being followed up"]

# 함수 적용하여 전처리 진행
df_all["expected_timeline"] = df_all["expected_timeline"].map(lambda x : func(x, most_ex_time))

# 빈도수 상위 11개의 값만 남기고, 이외의 값을 'etc'로 변환
top_ex_time = df_all["expected_timeline"].value_counts()[:11]
df_all["expected_timeline"] = df_all["expected_timeline"].apply(lambda x : "etc" if x not in top_ex_time else x)

# 라벨링 진행 및 데이터 넣기
le = LabelEncoder()
df_all['expected_timeline'] = le.fit_transform(df_all['expected_timeline'])
df_train_prepare['expected_timeline'] = df_all["expected_timeline"].values[ : idx_train]        # 학습용 데이터
df_test_prepare['expected_timeline'] = df_all["expected_timeline"].values[idx_train :]          # 테스트 데이터


########################
### product_category ###
########################

def func(x, top_category):      # 문자 전처리 하여 카테고리 생성하는 함수
    try :
        if x not in top_category:
            if "," in x:
                return "items"
            for i in ["signage", "tv", "vrf", "split", "rac", "chiller","monitor","display",  "inverter" ]:
                if i in x: return i
                elif "one" in x: return "one : quick"
                else : return x
        else: return x
        
    except:
        return "none"

def func2(x, top_30_category) :      # 카테고리 리스트에 없는 값을 'etc.'로 변환하는 함수
    if x not in top_30_category : return "etc."
    else: return x

# 빈도수 상위 11개의 값 가져오고 함수 적용하기
top_10_category = df_all["product_category"].value_counts()[:11]
df_all["product_category"] = df_all["product_category"].map(lambda x : func(x, top_10_category))

# 빈도수 상위 30개의 값 가져오고 함수 적용하기
top_30_category = df_all["product_category"].value_counts()[:30]
df_all["product_category"] = df_all["product_category"].map(lambda x : func2(x, top_30_category))

# 라벨링 진행 및 데이터 넣기
le = LabelEncoder()
df_all['product_category'] = le.fit_transform(df_all['product_category'])
df_train_prepare['product_category'] = df_all['product_category'].values[ : idx_train]      # 학습용 데이터
df_test_prepare['product_category'] = df_all['product_category'].values[idx_train :]        # 테스트 데이터


In [22]:
cols = ['bant_submit', 'lead_desc_length', 'id_strategic_ver',
       'it_strategic_ver', 'idit_strategic_ver', 'ver_win_rate_x',
       'ver_win_ratio_per_bu', 'com_reg_ver_win_rate',
       'historical_existing_cnt', 'business_unit', 'response_corporate',
       'enterprise', 'ver_cus', 'ver_pro', 'customer_idx', 'customer_type',
       'product_modelname', 'customer_job', 'inquiry_type', 'customer_country',
       'business_subarea', 'customer_position', 'expected_timeline',
       'product_category', 'lead_owner']

df_train_prepare = df_train_prepare[cols]
df_test_prepare = df_test_prepare[cols]

## 3. 모델링

### 3.1. 모델 학습을 위한 데이터 구축

3.1.1. 학습 모델 분리

In [23]:
# 데이터를 trian, validatoin 세트로 분리
x_train, x_val, y_train, y_val = train_test_split( df_train_prepare, df_train["is_converted"].map(lambda x : 1 if x else 0), test_size=0.2,
                                                stratify = df_train["is_converted"], shuffle=True, random_state=1015)

x_train.shape, y_train.shape, x_val.shape, y_val.shape

((44624, 25), (44624,), (11156, 25), (11156,))

In [24]:
# target 데이터 분포 확인
y_train.value_counts()      # 불균형 존재하여 추가로 다운샘플링 진행 결정

0    40928
1     3696
Name: is_converted, dtype: int64

3.1.2. 다운샘플링 진행

In [25]:
# 각 데이터 라벨을 적은 라벨의 데이터 양만큼 가져오기
x_train_1 = x_train[y_train == 1]
x_train_0 = x_train[y_train == 0].sample(len(x_train_1), random_state = 1015)

# 데이터 합치기 및 적용
x_train = pd.concat([x_train_0, x_train_1], axis = 0)
y_train = y_train[x_train.index]

# target 데이터 분포 확인
y_train.value_counts()      # 다운샘플링 진행하여 동일 데이터 수 선택 완료

0    3696
1    3696
Name: is_converted, dtype: int64

3.1.3. 원핫 인코딩 데이터 구축

In [26]:
# 원핫 인코딩을 위한 특성별 열 지정 ( categorical or numeric )
cat_features = ['bant_submit', 'customer_country', 'customer_type', 'customer_job', 'product_modelname', 'expected_timeline', 'inquiry_type', 
                'product_category', 'customer_position', 'business_subarea', 'business_unit', 'customer_idx', "ver_cus", "ver_pro", 'enterprise', 
                'response_corporate', 'lead_owner']
num_features = [x for x in x_train.columns if x not in cat_features]

# 원핫 인코딩을 위해 데이터 병합
df_concat = pd.concat([x_train[cat_features], x_val[cat_features], df_test_prepare[cat_features]], axis = 0).reset_index(drop = True)

# 병합된 데이터 프레임를 원핫인코딩 데이터프레임으로 변환
df_concat_dum = pd.get_dummies(df_concat.astype(str)).astype(int)

# 데이터 분리
x_train_ohe = pd.concat([x_train[num_features].reset_index(drop = True), df_concat_dum.iloc[:len(x_train)].reset_index(drop = True)], axis = 1)
x_val_ohe = pd.concat([x_val[num_features].reset_index(drop = True), df_concat_dum.iloc[len(x_train) : len(x_train) + len(x_val)].reset_index(drop = True)], axis = 1)
df_test_prepare_ohe = pd.concat([df_test_prepare[num_features].reset_index(drop = True), df_concat_dum.iloc[len(x_train) + len(x_val) : ].reset_index(drop = True)], axis = 1)

# 모델 학습에 필요한 데이터 인덱스 재지정
x_train_ohe = x_train_ohe.set_index(y_train.index)
x_val_ohe = x_val_ohe.set_index(y_val.index)

### 3.2 모델별 최적 파라미터 찾기

3.2.1. CatBoost 모델

In [28]:
# # 모델 학습 및 평가지표 생성 함수
# def objective_cat(trial, x_train, y_train, val_x, val_y):

#     # 모델 선언 및 파라미터 지정
#     model = CatBoostClassifier(
#         verbose=False, cat_features=cat_features, random_seed=1015, loss_function='Logloss',
#         learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
#         depth=trial.suggest_int("depth", 3, 10),
#         l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True)
#     )

#     # 모델 학습 후 예측
#     model.fit(x_train, y_train)
#     pred_y = model.predict(val_x)

#     # F1_score 점수 반환
#     return f1_score(val_y, pred_y)

# # 최적 파라미터 검색 시작
# study_cat = optuna.create_study(study_name="catboost", direction="maximize", sampler = TPESampler(seed=1015))
# study_cat.optimize(lambda trial: objective_cat(trial, x_train, y_train, x_val, y_val), n_trials=30)

# # 최종 F1_score 및 파라미터 출력
# print(study_cat.best_trial.value)
# print(study_cat.best_trial.params)

# # 모델 저장
# model_cat = CatBoostClassifier(**study_cat.best_trial.params, verbose=False, loss_function='Logloss', cat_features = cat_features, random_state=1015)
# model_cat.fit(x_train, y_train)
# joblib.dump(model_cat, 'model_cat.pkl')

# # << 최종 결과 >>
# # F1_score : 0.7175944737911417
# # Parmas : {'learning_rate': 0.012993331162100623, 'depth': 9, 'l2_leaf_reg': 0.00026910135175641804}

[I 2024-03-01 23:54:56,285] A new study created in memory with name: catboost
[I 2024-03-01 23:55:01,629] Trial 0 finished with value: 0.709729511505854 and parameters: {'learning_rate': 0.017989659220170835, 'depth': 4, 'l2_leaf_reg': 0.015020498677267236}. Best is trial 0 with value: 0.709729511505854.
[I 2024-03-01 23:55:17,600] Trial 1 finished with value: 0.7105580088317944 and parameters: {'learning_rate': 0.019801570091934378, 'depth': 9, 'l2_leaf_reg': 4.617566267645222}. Best is trial 1 with value: 0.7105580088317944.
[I 2024-03-01 23:55:21,752] Trial 2 finished with value: 0.7065916398713826 and parameters: {'learning_rate': 0.038055929652979915, 'depth': 3, 'l2_leaf_reg': 3.211569622408195}. Best is trial 1 with value: 0.7105580088317944.
[I 2024-03-01 23:55:38,555] Trial 3 finished with value: 0.7091055600322319 and parameters: {'learning_rate': 0.009339574235634876, 'depth': 10, 'l2_leaf_reg': 4.562273697030293e-05}. Best is trial 1 with value: 0.7105580088317944.
[I 2024-

0.7175944737911417
{'learning_rate': 0.012993331162100623, 'depth': 9, 'l2_leaf_reg': 0.00026910135175641804}


In [29]:
# 위 코드에서 저장한 모델 가져오기
!wget https://github.com/BusyCod/4th_SBS/raw/main/model/model_cat
model_cat = joblib.load("model_cat")

--2024-03-02 01:18:39--  https://github.com/BusyCod/4th_SBS/raw/main/model/model_cat
github.com (github.com) 해석 중... 20.200.245.247
다음으로 연결 중: github.com (github.com)|20.200.245.247|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 302 Found
위치: https://raw.githubusercontent.com/BusyCod/4th_SBS/main/model/model_cat [따라감]
--2024-03-02 01:18:39--  https://raw.githubusercontent.com/BusyCod/4th_SBS/main/model/model_cat
raw.githubusercontent.com (raw.githubusercontent.com) 해석 중... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
다음으로 연결 중: raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 17545004 (17M) [application/octet-stream]
저장 위치: `model_cat.2'

model_cat.2         100%[===================>]  16.73M  5.82MB/s    /  2.9s    

2024-03-02 01:18:43 (5.82 MB/s) - `model_cat.2' 저장함 [17545004/17545004]



3.2.2. RandomForest 모델

In [33]:
# # 모델 학습 및 평가지표 생성 함수
# def objective_RF(trial, train_x, train_y, val_x, val_y):

#     # 파라미터 지정
#     param = {
#         'n_estimators': trial.suggest_int('n_estimators', 10, 2000),
#         'criterion': trial.suggest_categorical("criterion", ['entropy', 'gini']),
#         'max_depth' : trial.suggest_int('max_depth', 3, 40),
#         'max_features': trial.suggest_categorical("max_features", ['auto', 'sqrt'])
#     }

#     # 모델 선언
#     model = RandomForestClassifier(**param, random_state= 1015)

#     # 모델 학습 후 예측
#     model.fit(train_x, train_y)
#     pred_y = model.predict(val_x)

#     # F1_score 점수 반환
#     return f1_score(val_y, pred_y)

# # 최적 파라미터 검색 시작
# study_rf = optuna.create_study(study_name='RF', direction='maximize', sampler=TPESampler(seed=1015))
# study_rf.optimize(lambda trial: objective_RF(trial, x_train, y_train, x_val, y_val), n_trials=30)

# # 최종 F1_score 및 파라미터 출력
# print(study_rf.best_trial.value)
# print(study_rf.best_trial.params)

# # 모델 저장
# model_rfc = RandomForestClassifier(**study_rf.best_trial.params, random_state=1015)
# model_rfc.fit(x_train, y_train)
# joblib.dump(model_rfc, 'model_rfc.pkl')

# # << 최종 결과 >>
# # F1_score : 0.6873261819626539
# # Parmas : {'n_estimators': 440, 'criterion': 'gini', 'max_depth': 34, 'max_features': 'auto'}

[I 2024-03-02 00:02:24,907] A new study created in memory with name: RF
[I 2024-03-02 00:02:31,479] Trial 0 finished with value: 0.6800630417651695 and parameters: {'n_estimators': 1259, 'criterion': 'gini', 'max_depth': 27, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.6800630417651695.
/Users/my_macbook_pro/anaconda3/envs/my_project/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
[I 2024-03-02 00:02:39,718] Trial 1 finished with value: 0.6795123869445537 and parameters: {'n_estimators': 1583, 'criterion': 'gini', 'max_depth': 21, 'max_features': 'auto'}. Best is trial 0 with value: 0.6800630417651695.
/Users/my_macbook_pro/anaconda3/envs/my_project/lib/python3.10/site-packages/sklearn/ense

0.6873261819626539
{'n_estimators': 440, 'criterion': 'gini', 'max_depth': 34, 'max_features': 'auto'}


In [30]:
# 위 코드에서 저장한 모델 가져오기
!wget https://github.com/BusyCod/4th_SBS/raw/main/model/model_rfc.pkl
model_rfc = joblib.load("model_rfc.pkl")

--2024-03-02 01:18:46--  https://github.com/BusyCod/4th_SBS/raw/main/model/model_rfc.pkl
github.com (github.com) 해석 중... 20.200.245.247
다음으로 연결 중: github.com (github.com)|20.200.245.247|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 302 Found
위치: https://raw.githubusercontent.com/BusyCod/4th_SBS/main/model/model_rfc.pkl [따라감]
--2024-03-02 01:18:46--  https://raw.githubusercontent.com/BusyCod/4th_SBS/main/model/model_rfc.pkl
raw.githubusercontent.com (raw.githubusercontent.com) 해석 중... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
다음으로 연결 중: raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 47679815 (45M) [application/octet-stream]
저장 위치: `model_rfc.pkl.3'

model_rfc.pkl.3     100%[===================>]  45.47M  6.60MB/s    /  7.4s    

2024-03-02 01:18:56 (6.16 MB/s) - `model_rfc.pkl.3' 저장함 [47679815/47679815]



3.2.3. XGboost 모델

In [38]:
# # 모델 학습 및 평가지표 생성 함수
# def objective_xgb(trial, train_x, train_y, val_x, val_y):

#     # 파라미터 지정
#     param = {
#         'n_estimators': trial.suggest_int('n_estimators', 10, 3000),
#         'objective': 'binary:logistic',
#         'max_depth' : trial.suggest_int('max_depth', 3, 40),
#         'booster' : "gbtree",
#         "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
#         "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
#         "gamma" : trial.suggest_float("gamma", 1e-8, 1.0, log=True),
#         "min_child_weight" : trial.suggest_int("min_child_weight", 2, 10),
#         "eta" : trial.suggest_float("eta", 1e-8, 1.0, log=True),
#     }

#     # 모델 선언
#     model = XGBClassifier(**param, random_state = 1015)

#     # 모델 학습 후 예측
#     model.fit(train_x, train_y)
#     pred_y = model.predict(val_x)

#     # F1_score 점수 반환
#     return f1_score(val_y, pred_y)

# # 최적 파라미터 검색 시작
# study_xgb = optuna.create_study(study_name='XGB', direction='maximize', sampler=TPESampler(seed=1015))
# study_xgb.optimize(lambda trial: objective_xgb(trial, x_train, y_train, x_val, y_val), n_trials=30)

# # 최종 F1_score 및 파라미터 출력
# print(study_xgb.best_trial.value)
# print(study_xgb.best_trial.params)

# # 모델 저장
# model_xgb = XGBClassifier(**study_xgb.best_trial.params, random_state=1015)
# model_xgb.fit(x_train, y_train)
# joblib.dump(model_xgb, 'model_xgb.pkl')

# # << 최종 결과 >>
# # F1_score : 0.6940711462450593
# # Parmas : {'n_estimators': 2853, 'max_depth': 15, 'lambda': 0.0038739276904590555, 'alpha': 3.293522702408573e-07, 'gamma': 1.5947302895482284e-05, 'min_child_weight': 7, 'eta': 0.042740299222558564}

[I 2024-03-02 00:06:02,898] A new study created in memory with name: XGB
[I 2024-03-02 00:06:08,435] Trial 0 finished with value: 0.6910377358490565 and parameters: {'n_estimators': 1886, 'max_depth': 8, 'lambda': 0.0008736701164924911, 'alpha': 0.0015374411807530426, 'gamma': 0.036376395678523486, 'min_child_weight': 9, 'eta': 0.02097439016360574}. Best is trial 0 with value: 0.6910377358490565.
[I 2024-03-02 00:06:10,203] Trial 1 finished with value: 0.5769501054111034 and parameters: {'n_estimators': 361, 'max_depth': 35, 'lambda': 7.608662384223036e-05, 'alpha': 0.21497025772900671, 'gamma': 8.459507280273826e-06, 'min_child_weight': 9, 'eta': 2.6018067062142906e-08}. Best is trial 0 with value: 0.6910377358490565.
[I 2024-03-02 00:06:20,039] Trial 2 finished with value: 0.6940711462450593 and parameters: {'n_estimators': 2853, 'max_depth': 15, 'lambda': 0.0038739276904590555, 'alpha': 3.293522702408573e-07, 'gamma': 1.5947302895482284e-05, 'min_child_weight': 7, 'eta': 0.042740299

0.6940711462450593
{'n_estimators': 2853, 'max_depth': 15, 'lambda': 0.0038739276904590555, 'alpha': 3.293522702408573e-07, 'gamma': 1.5947302895482284e-05, 'min_child_weight': 7, 'eta': 0.042740299222558564}


In [31]:
# 위 코드에서 저장한 모델 가져오기
!wget https://github.com/BusyCod/4th_SBS/raw/main/model/model_xgb.json
model_xgb = joblib.load('model_xgb.pkl')

--2024-03-02 01:19:02--  https://github.com/BusyCod/4th_SBS/raw/main/model/model_xgb.json
github.com (github.com) 해석 중... 20.200.245.247
다음으로 연결 중: github.com (github.com)|20.200.245.247|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 302 Found
위치: https://raw.githubusercontent.com/BusyCod/4th_SBS/main/model/model_xgb.json [따라감]
--2024-03-02 01:19:03--  https://raw.githubusercontent.com/BusyCod/4th_SBS/main/model/model_xgb.json
raw.githubusercontent.com (raw.githubusercontent.com) 해석 중... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
다음으로 연결 중: raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 7453842 (7.1M) [text/plain]
저장 위치: `model_xgb.json.1'

model_xgb.json.1    100%[===================>]   7.11M  4.79MB/s    /  1.5s    

2024-03-02 01:19:05 (4.79 MB/s) - `model_xgb.json.1' 저장함 [7453842/7453842]



3.2.4. LGBM 모델

In [33]:
# # 모델 학습 및 평가지표 생성 함수
# def objective_xgb(trial, train_x, train_y, val_x, val_y):

#     # 파라미터 지정
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 10, 3000),
#         'boosting_type' : trial.suggest_categorical('boosting_type' , ['gbdt', 'dart']),
#         'colsample_bytree' : trial.suggest_float("colsample_bytree", 0.01, 1),
#         'learning_rate' : trial.suggest_float("learning_rate", 1e-8, 1e-1),
#         'reg_alpha' : trial.suggest_float("reg_alpha", 0.01, 1),
#         'reg_lambda' : trial.suggest_float("reg_lambda", 0.01, 1),
#         'max_depth': trial.suggest_int('max_depth', 1, 100, step=1, log=False), 
#         "n_jobs" : -1,
#         "subsample": trial.suggest_float("subsample", 0.3, 1.0),
#     }

#     # 모델 선언
#     model = LGBMClassifier(**params, random_state = 1015)

#     # 모델 학습 후 예측
#     model.fit(train_x , train_y)
#     pred_y = model.predict(val_x)

#     # F1_score 점수 반환
#     return f1_score(val_y, pred_y)

# # 최적 파라미터 검색 시작
# study_lgc = optuna.create_study(study_name='LGC', direction='maximize', sampler=TPESampler(seed=1015))
# study_lgc.optimize(lambda trial: objective_xgb(trial, x_train, y_train, x_val, y_val), n_trials=30)

# # 최종 F1_score 및 파라미터 출력
# print(study_lgc.best_trial.value)
# print(study_lgc.best_trial.params)

# # 모델 저장
# model_lgc = LGBMClassifier(**study_lgc.best_trial.params, random_state=1015)
# model_lgc.fit(x_train, y_train)
# joblib.dump(model_lgc, 'model_lgc.pkl')

# # << 최종 결과 >>
# # F1_score : 0.7114147909967846
# # Parmas : {'n_estimators': 2913, 'boosting_type': 'gbdt', 'colsample_bytree': 0.37352886894460535, 'learning_rate': 0.010418548716904723, 'reg_alpha': 0.39154260998894846, 'reg_lambda': 0.01881102242870187, 'max_depth': 32, 'subsample': 0.5308595092829225}

[I 2024-03-02 01:19:19,694] A new study created in memory with name: LGC
[I 2024-03-02 01:19:36,495] Trial 0 finished with value: 0.6979125640015753 and parameters: {'n_estimators': 1886, 'boosting_type': 'dart', 'colsample_bytree': 0.6518663155601779, 'learning_rate': 0.08201024761388385, 'reg_alpha': 0.8677769005438556, 'reg_lambda': 0.7923090561279403, 'max_depth': 12, 'subsample': 0.8954702138951065}. Best is trial 0 with value: 0.6979125640015753.
[I 2024-03-02 01:19:38,388] Trial 1 finished with value: 0.682850430696946 and parameters: {'n_estimators': 1461, 'boosting_type': 'gbdt', 'colsample_bytree': 0.8602633063462287, 'learning_rate': 0.00519094734038069, 'reg_alpha': 0.9510967870145884, 'reg_lambda': 0.33571910291091156, 'max_depth': 70, 'subsample': 0.4327953078725317}. Best is trial 0 with value: 0.6979125640015753.
[I 2024-03-02 01:19:47,552] Trial 2 finished with value: 0.703305455993628 and parameters: {'n_estimators': 1207, 'boosting_type': 'dart', 'colsample_bytree': 

0.7114147909967846
{'n_estimators': 2913, 'boosting_type': 'gbdt', 'colsample_bytree': 0.37352886894460535, 'learning_rate': 0.010418548716904723, 'reg_alpha': 0.39154260998894846, 'reg_lambda': 0.01881102242870187, 'max_depth': 32, 'subsample': 0.5308595092829225}


In [38]:
# 위 코드에서 저장한 모델 가져오기
!wget https://github.com/BusyCod/4th_SBS/raw/main/model/model_lgc.pkl
model_lgc = joblib.load('model_lgc.pkl')

--2024-03-02 01:23:58--  https://github.com/BusyCod/4th_SBS/raw/main/model/model_lgc.pkl
github.com (github.com) 해석 중... 20.200.245.247
다음으로 연결 중: github.com (github.com)|20.200.245.247|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 302 Found
위치: https://raw.githubusercontent.com/BusyCod/4th_SBS/main/model/model_lgc.pkl [따라감]
--2024-03-02 01:23:58--  https://raw.githubusercontent.com/BusyCod/4th_SBS/main/model/model_lgc.pkl
raw.githubusercontent.com (raw.githubusercontent.com) 해석 중... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
다음으로 연결 중: raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 9868343 (9.4M) [application/octet-stream]
저장 위치: `model_lgc.pkl'

model_lgc.pkl       100%[===================>]   9.41M  5.60MB/s    /  1.7s    

2024-03-02 01:24:00 (5.60 MB/s) - `model_lgc.pkl' 저장함 [9868343/9868343]



3.2.5. RNN 모델

In [42]:
# 모델 시드 지정
# tf.random.set_seed(1015)      
initializer = tf.keras.initializers.GlorotUniform(seed=1015)

# 모델 레이어 구성
model_rnn = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape = (x_train_ohe.shape[1], )),
    tf.keras.layers.Dense(128, activation = 'relu', kernel_initializer = initializer),
    tf.keras.layers.Dropout(.8),
    tf.keras.layers.Dense(64, activation = 'relu', kernel_initializer = initializer),
    tf.keras.layers.Dropout(.8),
    tf.keras.layers.Dense(32, activation = 'relu', kernel_initializer = initializer),
    tf.keras.layers.Dropout(.8),
    tf.keras.layers.Dense(1, activation = 'sigmoid', kernel_initializer = initializer)
])

# 모델 컴파일 선언
model_rnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
initial_learning_rate = 0.0005
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay( initial_learning_rate, decay_steps=1, decay_rate=0.99, staircase=True)
optimizer = tf.keras.optimizers.legacy.RMSprop(learning_rate=lr_schedule)

# 모델 저장
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('./model_rnn.h5', save_best_only=True)

# 조기 정지 설정
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

# 모델 학습
r = model_rnn.fit(x_train_ohe, y_train,
              validation_data = (x_val_ohe, y_val),
              callbacks = [early_stopping_cb, checkpoint_cb],
              epochs = 100,
              batch_size = 256)

Epoch 1/100
29/29 [==============================] - 1s 14ms/step - loss: 0.7554 - accuracy: 0.5084 - val_loss: 0.6775 - val_accuracy: 0.9096
Epoch 2/100
29/29 [==============================] - 0s 6ms/step - loss: 0.6939 - accuracy: 0.5403 - val_loss: 0.6788 - val_accuracy: 0.9253
Epoch 3/100
 1/29 [>.............................] - ETA: 0s - loss: 0.6766 - accuracy: 0.6055

/Users/my_macbook_pro/anaconda3/envs/my_project/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 0s 7ms/step - loss: 0.6775 - accuracy: 0.5729 - val_loss: 0.6742 - val_accuracy: 0.9070
Epoch 4/100
29/29 [==============================] - 0s 6ms/step - loss: 0.6621 - accuracy: 0.6086 - val_loss: 0.6606 - val_accuracy: 0.8952
Epoch 5/100
29/29 [==============================] - 0s 7ms/step - loss: 0.6314 - accuracy: 0.6502 - val_loss: 0.6276 - val_accuracy: 0.8996
Epoch 6/100
29/29 [==============================] - 0s 6ms/step - loss: 0.5976 - accuracy: 0.6914 - val_loss: 0.5851 - val_accuracy: 0.9083
Epoch 7/100
29/29 [==============================] - 0s 6ms/step - loss: 0.5464 - accuracy: 0.7397 - val_loss: 0.4968 - val_accuracy: 0.9345
Epoch 8/100
29/29 [==============================] - 0s 6ms/step - loss: 0.5099 - accuracy: 0.7688 - val_loss: 0.4359 - val_accuracy: 0.9341
Epoch 9/100
29/29 [==============================] - 0s 6ms/step - loss: 0.4629 - accuracy: 0.8005 - val_loss: 0.3811 - val_accuracy: 0.9230
Epoch 10/100
29/29 [=====

In [46]:
# 위 코드에서 저장한 모델 가져오기
!wget https://github.com/BusyCod/4th_SBS/raw/main/model/model_rnn.h5
model_rnn = tf.keras.models.load_model('./model_rnn.h5')

--2024-03-02 01:26:40--  https://github.com/BusyCod/4th_SBS/raw/main/model/model_rnn.h5
github.com (github.com) 해석 중... 20.200.245.247
다음으로 연결 중: github.com (github.com)|20.200.245.247|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 302 Found
위치: https://raw.githubusercontent.com/BusyCod/4th_SBS/main/model/model_rnn.h5 [따라감]
--2024-03-02 01:26:41--  https://raw.githubusercontent.com/BusyCod/4th_SBS/main/model/model_rnn.h5
raw.githubusercontent.com (raw.githubusercontent.com) 해석 중... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
다음으로 연결 중: raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 2759080 (2.6M) [application/octet-stream]
저장 위치: `model_rnn.h5'

model_rnn.h5        100%[===================>]   2.63M  2.59MB/s    /  1.0s    

2024-03-02 01:26:43 (2.59 MB/s) - `model_rnn.h5' 저장함 [2759080/2759080]



3.2.6. Logistic 모델

In [50]:
# # 모델 학습 및 평가지표 생성 함수
# def objective_xgb(trial, train_x, train_y, val_x, val_y):

#     # 파라미터 지정
#     params = {
#         'tol' : trial.suggest_float('tol' , 1e-6 , 1e-3),
#         'C' : trial.suggest_float("C", 1e-3, 100),
#         'solver' : trial.suggest_categorical('solver' , ['liblinear']),
#         'max_iter' : trial.suggest_int('max_iter' , 100, 1000),
#     }

#     # 모델 선언
#     model = LogisticRegression(**params, random_state = 1015)

#     # 모델 학습 후 예측
#     model.fit(train_x , train_y)
#     pred_y = model.predict(val_x)

#     # F1_score 점수 반환
#     return f1_score(val_y, pred_y)

# # 최적 파라미터 검색 시작
# study_lgr = optuna.create_study(study_name='LGR', direction='maximize', sampler=TPESampler(seed=1015))
# study_lgr.optimize(lambda trial: objective_xgb(trial, x_train_ohe, y_train, x_val_ohe, y_val), n_trials=30)

# # 최종 F1_score 및 파라미터 출력
# print(study_lgr.best_trial.value)
# print(study_lgr.best_trial.params)

# # 모델 저장
# model_lgr = LogisticRegression(**study_lgr.best_trial.params, random_state=1015)
# model_lgr.fit(x_train_ohe, y_train)
# joblib.dump(model_lgr, 'model_lgr.pkl')

# # << 최종 결과 >>
# # F1_score : 0.6959648421893727
# # Parmas : {'tol': 0.0004155486221347932, 'C': 12.931787849099347, 'solver': 'liblinear', 'max_iter': 451}

[I 2024-03-02 01:28:09,639] A new study created in memory with name: LGR
[I 2024-03-02 01:28:09,903] Trial 0 finished with value: 0.6951316839584996 and parameters: {'tol': 0.0006278839568025838, 'C': 13.938512211342067, 'solver': 'liblinear', 'max_iter': 656}. Best is trial 0 with value: 0.6951316839584996.
[I 2024-03-02 01:28:10,175] Trial 1 finished with value: 0.6880697582243362 and parameters: {'tol': 0.0006487014638834524, 'C': 82.01042571245029, 'solver': 'liblinear', 'max_iter': 880}. Best is trial 0 with value: 0.6951316839584996.
[I 2024-03-02 01:28:10,406] Trial 2 finished with value: 0.6951999999999999 and parameters: {'tol': 0.0007904209566381945, 'C': 11.76313025133564, 'solver': 'liblinear', 'max_iter': 866}. Best is trial 2 with value: 0.6951999999999999.
[I 2024-03-02 01:28:10,656] Trial 3 finished with value: 0.6872525732383215 and parameters: {'tol': 0.0004856783756824099, 'C': 91.65481316689844, 'solver': 'liblinear', 'max_iter': 429}. Best is trial 2 with value: 0.

0.6959648421893727
{'tol': 0.0004155486221347932, 'C': 12.931787849099347, 'solver': 'liblinear', 'max_iter': 451}


In [56]:
# 위 코드에서 저장한 모델 가져오기
!wget https://github.com/BusyCod/4th_SBS/raw/main/model/model_lgr.pkl
model_lgr = joblib.load('model_lgr.pkl')

--2024-03-02 01:30:12--  https://github.com/BusyCod/4th_SBS/raw/main/model/model_lgr.pkl
github.com (github.com) 해석 중... 20.200.245.247
다음으로 연결 중: github.com (github.com)|20.200.245.247|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 302 Found
위치: https://raw.githubusercontent.com/BusyCod/4th_SBS/main/model/model_lgr.pkl [따라감]
--2024-03-02 01:30:13--  https://raw.githubusercontent.com/BusyCod/4th_SBS/main/model/model_lgr.pkl
raw.githubusercontent.com (raw.githubusercontent.com) 해석 중... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
다음으로 연결 중: raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 45419 (44K) [application/octet-stream]
저장 위치: `model_lgr.pkl'

model_lgr.pkl       100%[===================>]  44.35K  --.-KB/s    /  0.1s    

2024-03-02 01:30:13 (347 KB/s) - `model_lgr.pkl' 저장함 [45419/45419]



### 3.3. 모델 앙상블 ( 보팅 )

3.2.1. 학습 데이터로 앙상블 모델 확인

In [61]:
# 각 모델별 proba 계산

pred_prob_cat = model_cat.predict_proba(x_val)          # CatBoost 모델
pred_prob_xgb = model_xgb.predict_proba(x_val)          # XGBoost 모델
pred_prob_rfc = model_rfc.predict_proba(x_val)          # RandomForest 모델
pred_prob_lgc = model_lgc.predict_proba(x_val)          # LGBMClassifier 모델
pred_prob_lgr = model_lgr.predict_proba(x_val_ohe)      # Logistic 모델
pred_prob_rnn = model_rnn.predict(x_val_ohe)             # RNN 모델

# 모델 앙상블 - Soft Voting 진행
num_model, threshold = 6, .5        # 모델 갯수, 임계치 설정
pred_prob_sum = pred_prob_xgb[:,[1]].flatten() + pred_prob_cat[:,[1]].flatten() + pred_prob_lgc[:,[1]].flatten() + pred_prob_rnn.flatten() + pred_prob_lgr[:,[1]].flatten() + pred_prob_lgc[:,[1]].flatten()     # 확률값 합치기
pred_prob = pred_prob_sum / num_model       # 확률값의 평균 구하기
y_pred = [1 if x >= threshold else 0 for x in pred_prob]

# 평가 지표 확인
print(classification_report(y_val, y_pred))

349/349 [==============================] - 0s 628us/step
              precision    recall  f1-score   support

           0       1.00      0.94      0.97     10232
           1       0.58      0.96      0.72       924

    accuracy                           0.94     11156
   macro avg       0.79      0.95      0.84     11156
weighted avg       0.96      0.94      0.95     11156



3.2.2. 최종 제출 데이터 학습 및 제출

In [62]:
# 각 모델별 proba 계산

pred_prob_cat_sub = model_cat.predict_proba(df_test_prepare)          # CatBoost 모델
pred_prob_xgb_sub = model_xgb.predict_proba(df_test_prepare)          # XGBoost 모델
pred_prob_rfc_sub = model_rfc.predict_proba(df_test_prepare)          # RandomForest 모델
pred_prob_lgc_sub = model_lgc.predict_proba(df_test_prepare)          # LGBMClassifier 모델
pred_prob_lgr_sub = model_lgr.predict_proba(df_test_prepare_ohe)      # Logistic 모델
pred_prob_rnn_sub = model_rnn.predict(df_test_prepare_ohe)             # RNN 모델

165/165 [==============================] - 0s 1ms/step


In [82]:
# 모델 앙상블 - Soft Voting 진행
num_model, threshold = 6, .499     # 모델 갯수, 임계치 설정
pred_prob_sum = pred_prob_xgb_sub[:,[1]].flatten() + pred_prob_cat_sub[:,[1]].flatten() + pred_prob_lgc_sub[:,[1]].flatten() + pred_prob_rnn_sub.flatten() + pred_prob_lgr_sub[:,[1]].flatten() +  pred_prob_rfc_sub[:,[1]].flatten()            # 확률값 합치기
pred_prob = pred_prob_sum / num_model       # 확률값의 평균 구하기
y_pred = [1 if x >= threshold else 0 for x in pred_prob]

In [64]:
# 제출 데이터와 합치기
df_submmit = pd.read_csv('./submission.csv')
df_submmit['is_converted'] = y_pred

In [65]:
# 제출 파일 저장
df_submmit.to_csv("submission.csv", index=False)

In [66]:
Counter(y_pred)

Counter({0: 3420, 1: 1851})

In [78]:
pred_prob[3878]

0.4895392849976871

In [73]:
pred_prob_rfc_sub[3878]

array([0.50227273, 0.49772727])

In [74]:
pred_prob_cat_sub[3878]


array([0.70779145, 0.29220855])

In [75]:
pred_prob_rnn_sub[3878]

array([0.5344925], dtype=float32)

In [93]:
pred_prob_xgb_sub[3878]

array([0.18605632, 0.8139437 ], dtype=float32)

In [77]:
pred_prob_lgc_sub[3878]

array([0.36506957, 0.63493043])

In [92]:
pred_prob_lgr_sub[3878]

array([0.83606672, 0.16393328])

In [ ]:
!pip list

In [ ]:
# pred_prob_lgc_sub, pred_prob_xgb_sub

In [88]:
# 모델 앙상블 - Soft Voting 진행
num_model, threshold = 6, .5    # 모델 갯수, 임계치 설정
pred_prob_sum = [np.array([round(x, 3) for x in pred_prob_xgb_sub[:,[1]].flatten()]) + 
                np.array([round(x, 3) for x in pred_prob_cat_sub[:,[1]].flatten()]) + 
                np.array([round(x, 3) for x in  pred_prob_lgc_sub[:,[1]].flatten()]) + 
                np.array([round(x, 3) for x in  pred_prob_rnn_sub.flatten()]) + 
                np.array([round(x, 3) for x in  pred_prob_lgr_sub[:,[1]].flatten()]) +  
                np.array([round(x, 3) for x in  pred_prob_rfc_sub[:,[1]].flatten()]) ]           # 확률값 합치기
pred_prob = pred_prob_sum[0] / num_model       # 확률값의 평균 구하기
y_pred_ = [1 if x >= threshold else 0 for x in pred_prob]

In [89]:
Counter(y_pred_)

Counter({0: 3420, 1: 1851})

In [90]:
sum(y_pred == np.array(y_pred_))

5271

In [91]:
y_pred_[3878]

0